In [3]:
import pandas as pd
from MyCreds.mycreds import IowaData
import pymssql
import h3
from sodapy import Socrata # < Unmaintained as of August 31, 2022, just in time for this project
import numpy as np
from datetime import datetime, timedelta

# Config Globals
KEY = IowaData.api_key
SECRET = IowaData.api_key_secret
TOKEN = IowaData.app_token

liquor_sales_data_id = "m3tr-qhgy"

In [3]:
socrata_df.dtypes

invoice_line_no        object
date                   object
store                  object
name                   object
address                object
city                   object
zipcode                object
store_location         object
county_number          object
county                 object
category               object
category_name          object
vendor_no              object
vendor_name            object
itemno                 object
im_desc                object
pack                   object
bottle_volume_ml       object
state_bottle_cost      object
state_bottle_retail    object
sale_bottles           object
sale_dollars           object
sale_liters            object
sale_gallons           object
dtype: object

In [2]:
client = Socrata("data.iowa.gov", app_token=TOKEN)
results = client.get_all("m3tr-qhgy", where="(date between '2022-01-01' and '2022-01-31')")
socrata_df = pd.DataFrame.from_records(results)
socrata_df

,invoice_line_no,date,store,name,address,city,zipcode,store_location,county_number,county,...,itemno,im_desc,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
0,INV-43468500052,2022-01-02T00:00:00.000,2190,"Central City Liquor, Inc.",1460 2ND AVE,Des Moines,50314,"{'type': 'Point', 'coordinates': [-93.619788, ...",77,POLK,...,5362,John Barr Reserve,12,750,11.67,17.51,3,52.53,2.25,0.59
1,INV-43468400001,2022-01-02T00:00:00.000,2536,HY-VEE / PLEASANT HILL,4815 Maple Drive,Pleasant Hill,50317,"{'type': 'Point', 'coordinates': [-93.519491, ...",77,Polk,...,34001,Absolut Swedish Vodka 80prf Mini,10,50,7.92,11.88,10,118.80,0.50,0.13
2,INV-43468400006,2022-01-02T00:00:00.000,2536,HY-VEE / PLEASANT HILL,4815 Maple Drive,Pleasant Hill,50317,"{'type': 'Point', 'coordinates': [-93.519491, ...",77,Polk,...,39825,Western Son Blueberry,12,750,9.01,13.52,12,162.24,9.00,2.37
3,INV-43468400011,2022-01-02T00:00:00.000,2536,HY-VEE / PLEASANT HILL,4815 Maple Drive,Pleasant Hill,50317,"{'type': 'Point', 'coordinates': [-93.519491, ...",77,Polk,...,25616,Seagrams 7 Crown PET Flask,12,750,7.50,11.25,12,135.00,9.00,2.37
4,INV-43468500039,2022-01-02T00:00:00.000,2190,"Central City Liquor, Inc.",1460 2ND AVE,Des Moines,50314,"{'type': 'Point', 'coordinates': [-93.619788, ...",77,POLK,...,88533,Mi Campo Reposado,12,750,12.40,18.60,2,37.20,1.50,0.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182590,INV-44254200005,2022-01-31T00:00:00.000,3773,Benz Distributing,501 7th Ave SE,Cedar Rapids,52401,"{'type': 'Point', 'coordinates': [-91.659875, ...",57,LINN,...,86390,Montezuma Triple Sec,12,1000,2.13,3.20,48,153.60,48.00,12.68
182591,INV-44267400054,2022-01-31T00:00:00.000,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14th St,Des Moines,50320,"{'type': 'Point', 'coordinates': [-93.596756, ...",77,POLK,...,5347,Johnnie Walker Red,12,1000,17.10,25.65,12,307.80,12.00,3.17
182592,INV-44270400014,2022-01-31T00:00:00.000,2200,Sac Liquor Store,619 E Main St,Sac City,50583,"{'type': 'Point', 'coordinates': [-94.974011, ...",81,SAC,...,10834,Crown Royal Black,24,375,8.25,12.38,4,49.52,1.50,0.39
182593,INV-44286800002,2022-01-31T00:00:00.000,5597,Liquor Tobacco & Gas,1401 Central Ave,Dubuque,52001,"{'type': 'Point', 'coordinates': [-90.66783600...",31,DUBUQUE,...,89626,Patron Reposado,12,750,29.00,43.50,12,522.00,9.00,2.37


In [8]:
with open('etl_log.txt', 'a') as f:
    f.write(f"{datetime.now()}, {},{}\n")

In [ ]:
date_between = "(date between '2022-01-01' and '2022-01-31')"

In [8]:
def get_last_date_of_month(year, month):
    """Return the last date of the month.

    Args:
        year (int): Year, i.e. 2022
        month (int): Month, i.e. 1 for January

    Returns:
        date (datetime): Last date of the current month
    """

    if month == 12:
        last_date = datetime(year, month, 31)
    else:
        last_date = datetime(year, month + 1, 1) + timedelta(days=-1)

    return last_date.strftime("%Y-%m-%d")

In [9]:
def log(run_start, time_range, successful=1, error='NaN'):
    run_time = datetime.now() - run_start
    with open('etl_log.csv', 'a') as f:
        f.write(f"{datetime.now()}, {run_time}, {time_range[0]}, {time_range[1]}, {successful}, {error}\n")

In [10]:
def clean_dataframe(df, h3_resolution=8):
    coords_df = pd.DataFrame(df['store_location'].to_dict()).T
    coords_df['coordinates'] = np.where(coords_df['coordinates'].isnull(), pd.Series([[]]*len(coords_df)), coords_df['coordinates']) # fill na with empty list
    df = df.merge(pd.DataFrame(coords_df['coordinates'].to_list(), columns=['long', 'lat']), left_index=True, right_index=True)


    def lat_long_to_h3(row):
        return h3.geo_to_h3(lat=row.lat,lng=row.long,resolution = h3_resolution)

    df['hex_id'] = df.apply(lat_long_to_h3, axis=1)
    df['date'] = pd.to_datetime(df['date'])

    int_cols = ['store', 'county_number', 'category', 'vendor_no', 'itemno', 'pack', 'sale_bottles', ]
    float_cols = ['state_bottle_cost', 'state_bottle_retail', 'sale_dollars', 'sale_liters', 'sale_gallons']

    df[int_cols] = df[int_cols].astype('Int64')
    df[float_cols] = df[float_cols].astype(float)
    df.drop('store_location', axis=1, inplace=True)

    return df

In [11]:
def load(df):
    server = IowaData.azure_server
    user = IowaData.azure_uid
    password = IowaData.azure_pwd

    insert_query = f"""INSERT INTO invoices.temptable({', '.join([*df.columns])}) VALUES ({', '.join(['%s' for col in df.columns])})"""
    write_data = tuple(map(tuple, df.fillna(0).values))

    with pymssql.connect(server, user, password, "iowa_liquor") as cnxn:
        with cnxn.cursor() as cur:
            cur.executemany(insert_query, write_data)
            cnxn.commit()

In [ ]:
date_intervals = [[f"{datetime(y, m, 1):%Y-%m-%d}", f"{get_last_date_of_month(y, m)}"] for y in range(2012, datetime.now().year + 1) for m in range(1, 13)]

client = Socrata("data.iowa.gov", app_token=TOKEN)
success = 1

for interval in date_intervals[:1]:
    run_start_time = datetime.now()
    try:
        date_between = f"(date between '{interval[0]}' and '{interval[1]}')"
        results = client.get_all(liquor_sales_data_id, where=date_between)
        socrata_df = pd.DataFrame.from_records(results)
        socrata_df = clean_dataframe(socrata_df)
        load(socrata_df)
        log(run_start_time, interval)

    except Exception as e:
        log(run_start_time, interval, successful=0, error=e)


socrata_df

In [67]:
socrata_df[socrata_df['zipcode'] == '712-2']

,invoice_line_no,date,store,name,address,city,zipcode,county_number,county,category,...,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons,long,lat,hex_id
10218,S03437000013,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1022100,...,1000,12.50,18.74,12,224.88,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
10472,S03437000007,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1011100,...,1000,7.50,11.24,12,134.88,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
10908,S03437000002,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1081600,...,1000,11.34,17.01,12,204.12,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
11173,S03437000004,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1012100,...,1750,30.50,45.74,6,274.44,10.5,2.77,-95.602969,41.854728,88260c4ebbfffff
12485,S03437000006,2012-01-04,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1012100,...,1750,10.45,15.67,12,188.04,21.0,5.55,-95.602969,41.854728,88260c4ebbfffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118694,S03758800015,2012-01-25,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1081315,...,750,6.94,10.41,2,20.82,1.5,0.40,-95.602969,41.854728,88260c4ebbfffff
119354,S03758800003,2012-01-25,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1062300,...,1000,9.77,14.66,12,175.92,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
119357,S03758800007,2012-01-25,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1031080,...,1000,3.97,5.95,12,71.40,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff
119430,S03758800011,2012-01-25,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1011200,...,1000,11.54,17.31,12,207.72,12.0,3.17,-95.602969,41.854728,88260c4ebbfffff


In [54]:
socrata_df[socrata_df['zipcode'] == '712-2']

,invoice_line_no,date,store,name,address,city,zipcode,county_number,county,category,...,im_desc,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons,store_location
10218,S03437000013,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1022100,...,Jose Cuervo Especial Reposado Tequila,12,1000,12.50,18.74,12,224.88,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
10472,S03437000007,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1011100,...,Seagrams 7 Crown Bl Whiskey,12,1000,7.50,11.24,12,134.88,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
10908,S03437000002,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1081600,...,Southern Comfort,12,1000,11.34,17.01,12,204.12,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
11173,S03437000004,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1012100,...,Crown Royal Canadian Whisky,6,1750,30.50,45.74,6,274.44,10.50,2.77,"{'type': 'Point', 'coordinates': [-95.602969, ..."
12485,S03437000006,2012-01-04T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1012100,...,Black Velvet,6,1750,10.45,15.67,12,188.04,21.00,5.55,"{'type': 'Point', 'coordinates': [-95.602969, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119542,S03758800007,2012-01-25T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1031080,...,Barton Vodka,12,1000,3.97,5.95,12,71.40,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
119615,S03758800011,2012-01-25T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1011200,...,Jim Beam,12,1000,11.54,17.31,12,207.72,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
120027,S03758800003,2012-01-25T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1062300,...,Malibu Coconut Rum,12,1000,9.77,14.66,12,175.92,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."
120030,S03758800007,2012-01-25T00:00:00.000,4307,Crossroads Wine and Liquor,117 IOWA AVE,DUNLAP,712-2,43,Harrison,1031080,...,Barton Vodka,12,1000,3.97,5.95,12,71.40,12.00,3.17,"{'type': 'Point', 'coordinates': [-95.602969, ..."


In [42]:
pd.read_csv('etl_log.csv')

,time_stamp,run_time_min,start_date,end_date,successful,error
0,2022-09-09 22:28:50.609779,0:00:55.578335,2012-01-01,2012-01-31,1,
1,2022-09-09 22:30:50.809604,0:00:57.416926,2012-01-01,2012-01-31,1,NaN


In [ ]:
# example

# insert_query = """INSERT INTO dbo.temptable(CHECK_TIME, DEVICE, METRIC, VALUE, TOWER, LOCATION, ANOMALY, ANOMALY_SCORE, ANOMALY_SEVERITY)
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
# write_data = tuple(map(tuple, data_frame.values))
# cursor.executemany(insert_query, write_data)
# con.commit()
# cursor.close()
# con.close()


server = IowaData.azure_server
user = IowaData.azure_uid
password = IowaData.azure_pwd

insert_query = f"""INSERT INTO invoices.temptable({', '.join([*socrata_df.columns])}) VALUES ({', '.join(['%s' for col in socrata_df.columns])})"""
write_data = tuple(map(tuple, socrata_df.fillna(0).values))

with pymssql.connect(server, user, password, "iowa_liquor") as cnxn:
    with cnxn.cursor() as cur:
        cur.executemany(insert_query, write_data)
        cnxn.commit()

In [7]:
socrata_df

NameError: name 'socrata_df' is not defined

In [73]:
', '.join(['%s' for col in socrata_df.columns])

'%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s'

In [74]:
len(['%s' for col in socrata_df.columns])

27

In [75]:
len(socrata_df.columns)

27

In [87]:
insert_query = f"""INSERT INTO dbo.temptable({', '.join([*socrata_df.columns])})
            VALUES ({', '.join(['%s' for col in socrata_df.columns])})"""
insert_query


'INSERT INTO dbo.temptable(invoice_line_no, date, store, name, address, city, zipcode, county_number, county, category, category_name, vendor_no, vendor_name, itemno, im_desc, pack, bottle_volume_ml, state_bottle_cost, state_bottle_retail, sale_bottles, sale_dollars, sale_liters, sale_gallons, store_location, long, lat, hex_id)\n            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [9]:
f"({', '.join([*socrata_df.columns])})"

'(invoice_line_no, date, store, name, address, city, zipcode, county_number, county, category, category_name, vendor_no, vendor_name, itemno, im_desc, pack, bottle_volume_ml, state_bottle_cost, state_bottle_retail, sale_bottles, sale_dollars, sale_liters, sale_gallons, long, lat, hex_id)'

In [83]:
(*socrata_df.columns)

SyntaxError: can't use starred expression here (988345262.py, line 1)

In [32]:
write_data = tuple(map(tuple, socrata_df.values))
write_data

(('S03407900038',
  Timestamp('2012-01-03 00:00:00'),
  2549,
  'Hy-Vee Food Store / Indianola',
  '910 N JEFFERSON',
  'INDIANOLA',
  '50125',
  91,
  'Warren',
  1031080,
  'VODKA 80 PROOF',
  260,
  'Diageo Americas',
  37418,
  'Popov Vodka 80 Prf',
  6,
  '1750',
  8.49,
  12.74,
  12,
  152.88,
  21.0,
  5.55,
  nan,
  nan,
  '0'),
 ('S03411000067',
  Timestamp('2012-01-03 00:00:00'),
  2625,
  'Hy-Vee Wine and Spirits #2',
  '3301 W KIMBERLY RD',
  'DAVENPORT',
  '52804',
  82,
  'Scott',
  1011200,
  'STRAIGHT BOURBON WHISKIES',
  65,
  'Jim Beam Brands',
  27544,
  'Red Stag By Jim Beam',
  12,
  '750',
  9.97,
  14.95,
  12,
  179.4,
  9.0,
  2.38,
  nan,
  nan,
  '0'),
 ('S03411000030',
  Timestamp('2012-01-03 00:00:00'),
  2625,
  'Hy-Vee Wine and Spirits #2',
  '3301 W KIMBERLY RD',
  'DAVENPORT',
  '52804',
  82,
  'Scott',
  1062310,
  'SPICED RUM',
  260,
  'Diageo Americas',
  43337,
  'Captain Morgan Spiced Rum',
  12,
  '1000',
  10.99,
  16.49,
  12,
  197.88,
  12.

In [25]:
insert_query = f"""INSERT INTO invoices.temptable({', '.join([*socrata_df.columns])}) VALUES ({', '.join(['%s' for col in socrata_df.columns])})"""
insert_query

'INSERT INTO invoices.temptable(invoice_line_no, date, store, name, address, city, zipcode, county_number, county, category, category_name, vendor_no, vendor_name, itemno, im_desc, pack, bottle_volume_ml, state_bottle_cost, state_bottle_retail, sale_bottles, sale_dollars, sale_liters, sale_gallons, long, lat, hex_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [15]:
[*map(tuple, socrata_df.values)]

[('S03407900038',
  Timestamp('2012-01-03 00:00:00'),
  2549,
  'Hy-Vee Food Store / Indianola',
  '910 N JEFFERSON',
  'INDIANOLA',
  '50125',
  91,
  'Warren',
  1031080,
  'VODKA 80 PROOF',
  260,
  'Diageo Americas',
  37418,
  'Popov Vodka 80 Prf',
  6,
  '1750',
  8.49,
  12.74,
  12,
  152.88,
  21.0,
  5.55,
  nan,
  nan,
  '0'),
 ('S03411000067',
  Timestamp('2012-01-03 00:00:00'),
  2625,
  'Hy-Vee Wine and Spirits #2',
  '3301 W KIMBERLY RD',
  'DAVENPORT',
  '52804',
  82,
  'Scott',
  1011200,
  'STRAIGHT BOURBON WHISKIES',
  65,
  'Jim Beam Brands',
  27544,
  'Red Stag By Jim Beam',
  12,
  '750',
  9.97,
  14.95,
  12,
  179.4,
  9.0,
  2.38,
  nan,
  nan,
  '0'),
 ('S03411000030',
  Timestamp('2012-01-03 00:00:00'),
  2625,
  'Hy-Vee Wine and Spirits #2',
  '3301 W KIMBERLY RD',
  'DAVENPORT',
  '52804',
  82,
  'Scott',
  1062310,
  'SPICED RUM',
  260,
  'Diageo Americas',
  43337,
  'Captain Morgan Spiced Rum',
  12,
  '1000',
  10.99,
  16.49,
  12,
  197.88,
  12.

In [18]:
socrata_df.to_records(index=False)

rec.array([('S03407900038', '2012-01-03T00:00:00.000000000', 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37418, 'Popov Vodka 80 Prf', 6, '1750',  8.49, 12.74, 12, 152.88, 21.  , 5.55,        nan,       nan, '0'),
           ('S03411000067', '2012-01-03T00:00:00.000000000', 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750',  9.97, 14.95, 12, 179.4 ,  9.  , 2.38,        nan,       nan, '0'),
           ('S03411000030', '2012-01-03T00:00:00.000000000', 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 12, 197.88, 12.  , 3.17,        nan,       nan, '0'),
           ...,
           ('S03846900020', 

In [24]:
for col in socrata_df.columns:
    print(col)

invoice_line_no
date
store
name
address
city
zipcode
county_number
county
category
category_name
vendor_no
vendor_name
itemno
im_desc
pack
bottle_volume_ml
state_bottle_cost
state_bottle_retail
sale_bottles
sale_dollars
sale_liters
sale_gallons
long
lat
hex_id


In [46]:
socrata_df['hex_id'].str.len().max()

15

In [5]:
server = IowaData.azure_server
user = IowaData.azure_uid
password = IowaData.azure_pwd

st = datetime.now()
sql = "select * from invoices.temptable"

with pymssql.connect(server, user, password, "iowa_liquor") as cnxn:
    df_sql = pd.read_sql(sql, cnxn)
et = datetime.now()
print(f"Run time: {et - st}")
df_sql

Run time: 0:00:00.888834


/Users/michaelmcmanus/.conda/envs/iowa/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,invoice_line_no,date,store,name,address,city,zipcode,county_number,county,category,...,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons,long,lat,hex_id


In [2]:
from pandas import Timestamp

In [3]:
lst = [('S03407900038', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37418, 'Popov Vodka 80 Prf', 6, '1750', 8.49, 12.74, 12, 152.88, 21.0, 5.55, 0.0, 0.0, '0'), ('S03411000067', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750', 9.97, 14.95, 12, 179.4, 9.0, 2.38, 0.0, 0.0, '0'), ('S03411000030', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 12, 197.88, 12.0, 3.17, 0.0, 0.0, '0'), ('S03410300045', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34422, 'Grey Goose Vodka', 6, '1000', 21.77, 32.66, 2, 65.32, 2.0, 0.53, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03400700021', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1081400, 'TRIPLE SEC', 434, 'Luxco-St Louis', 86507, 'Paramount Triple Sec', 12, '1000', 3.75, 5.62, 12, 67.44, 12.0, 3.17, 0.0, 0.0, '0'), ('S03396300007', Timestamp('2012-01-03 00:00:00'), 4041, 'Mandm Pantry', '2344 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43136, 'Bacardi Limon', 12, '750', 8.33, 12.49, 4, 49.96, 3.0, 0.79, -90.213015, 41.821777, '88275b7897fffff'), ('S03396800017', Timestamp('2012-01-03 00:00:00'), 2445, "Ruback's Food Center", '504 SOUTH HIGHWAY', 'OAKLAND', '51560', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37997, 'Smirnoff Vodka 80 Prf', 12, '1000', 8.49, 12.74, 2, 25.48, 2.0, 0.53, 0.0, 0.0, '0'), ('S03400000002', Timestamp('2012-01-03 00:00:00'), 2459, 'Reinhart Foods', '200 STATE PO BOX 98', 'GUTHRIE CENTER', '50115', 39, 'Guthrie', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 12, 295.68, 12.0, 3.17, 0.0, 0.0, '0'), ('S03389000022', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 2, 49.28, 2.0, 0.53, -93.315977, 40.757158, '88267252cdfffff'), ('S03410300082', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24456, 'Kessler Blend Whiskey', 12, '750', 5.51, 8.26, 4, 33.04, 3.0, 0.79, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03411100022', Timestamp('2012-01-03 00:00:00'), 3631, 'Audubon Food Pride', '104 MARKET ST PO BOX 206', 'AUDUBON', '50025', 5, 'Audubon', 1011200, 'STRAIGHT BOURBON WHISKIES', 259, 'Heaven Hill Brands', 73715, 'Evan Williams Cherry Reserve', 12, '750', 7.94, 11.91, 2, 23.82, 1.5, 0.4, -94.93625, 41.723057, '88260c388dfffff'), ('S03400300011', Timestamp('2012-01-03 00:00:00'), 3461, 'Hometown Foods / Stuart', '611 S DIVISION ST', 'STUART', '50250', 1, 'Adair', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54446, 'Paramount Apricot Flavored Brandy', 12, '750', 5.21, 7.82, 4, 31.28, 3.0, 0.79, -94.318443, 41.49759900000001, '88260dcc2bfffff'), ('S03388100138', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69945, 'Rumple Minze Peppermint Schnapps Liqueur', 12, '375', 6.5, 9.74, 4, 38.96, 1.5, 0.4, -95.879662, 41.261925, '882608c93bfffff'), ('S03392900093', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 305, 'MHW Ltd', 34244, 'Chopin Vodka', 6, '750', 15.83, 23.74, 2, 47.48, 1.5, 0.4, 0.0, 0.0, '0'), ('S03388700007', Timestamp('2012-01-03 00:00:00'), 2367, "Spencer's Grocery", '101 SOUTH BURNETT', 'SHENANDOAH', '51601', 73, 'Page', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36907, 'Mccormick Vodka PET', 12, '1000', 3.98, 5.97, 12, 71.64, 12.0, 3.17, -95.381935, 40.765934, '8826080d9dfffff'), ('S03410700044', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82866, 'Dekuyper Watermelon Pucker', 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -94.853591, 42.064155, '88260c16a9fffff'), ('S03383800086', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1081220, 'DARK CREME DE CACAO', 434, 'Luxco-St Louis', 78456, 'Paramount Creme De Cacao/dark', 12, '750', 4.44, 6.66, 1, 6.66, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03388500036', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37994, 'Smirnoff Vodka 80 Prf', 24, '375', 4.48, 6.72, 24, 161.28, 9.0, 2.38, -90.668138, 42.504959, '88275a2119fffff'), ('S03395700041', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1051120, 'CHERRY BRANDIES', 434, 'Luxco-St Louis', 55504, 'Paramount Cherry Brandy', 24, '375', 3.34, 5.0, 3, 15.0, 1.12, 0.3, -95.147741, 43.14521, '88262a1925fffff'), ('S03401000035', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35913, "Five O'clock Vodka", 48, '200', 1.1, 1.65, 3, 4.95, 0.6, 0.16, -90.683356, 42.52717, '88275a215dfffff'), ('S03409700120', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1022100, 'TEQUILA', 65, 'Jim Beam Brands', 89786, 'Sauza Gold', 12, '750', 8.4, 12.59, 4, 50.36, 3.0, 0.79, -93.737447, 41.62941, '88260d802bfffff'), ('S03411200013', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34820, 'Svedka Vodka', 12, '750', 7.97, 11.95, 12, 143.4, 9.0, 2.38, -93.697731, 41.665172, '88260d8231fffff'), ('S03385900025', Timestamp('2012-01-03 00:00:00'), 4356, 'Kum & Go #396 / Nevada', '136 LINCOLN HWY', 'NEVADA', '50201', 85, 'Story', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 38006, 'Smirnoff Vodka Traveller', 12, '750', 7.59, 11.56, 3, 34.68, 2.25, 0.59, -93.458003, 42.022725, '88262b235bfffff'), ('S03385300085', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11771, 'Black Velvet Mini', 10, '600', 4.96, 7.44, 1, 7.44, 0.6, 0.16, -91.136655, 40.814666, '882670aacdfffff'), ('S03395700029', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41694, 'Uv Blue (raspberry) Vodka', 12, '1000', 7.25, 10.87, 3, 32.61, 3.0, 0.79, -95.147741, 43.14521, '88262a1925fffff'), ('S03405900055', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69657, "Dr. Mcgillicuddy's Mentholmint", 12, '1000', 11.0, 16.5, 24, 396.0, 24.0, 6.34, -90.48065, 41.529655, '882675c361fffff'), ('S03410700013', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11347, 'Seagrams V.o. Bl Canadian Whisky', 12, '1000', 9.49, 14.24, 12, 170.88, 12.0, 3.17, -94.853591, 42.064155, '88260c16a9fffff'), ('S03407100003', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43336, 'Captain Morgan Original Spiced', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03412100043', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 37935, 'Skol Vodka Traveler', 12, '750', 3.64, 5.45, 12, 65.4, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03388800005', Timestamp('2012-01-03 00:00:00'), 3811, 'Wal-Mart 1431 / Keokuk', '300 NORTH PARK DR', 'KEOKUK', '52632', 56, 'Lee', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76486, 'Paramount Amaretto', 12, '750', 3.37, 5.06, 12, 60.72, 9.0, 2.38, -91.407263, 40.418589, '88267028d9fffff'), ('S03398600035', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82611, 'Dekuyper Hot Damn!', 12, '750', 6.3, 9.45, 6, 56.7, 4.5, 1.19, -91.343544, 40.626988, '8826703131fffff'), ('S03385300004', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1022100, 'TEQUILA', 395, 'Proximo', 89197, 'Jose Cuervo Especial Reposado Tequila', 12, '1000', 12.5, 18.74, 12, 224.88, 12.0, 3.17, -91.136655, 40.814666, '882670aacdfffff'), ('S03410900008', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24456, 'Kessler Blend Whiskey', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03387400007', Timestamp('2012-01-03 00:00:00'), 3642, "Fraise Foods / Sonny's Supermarket", '327  5TH ST', 'WEST POINT', '52656', 56, 'Lee', 1022100, 'TEQUILA', 395, 'Proximo', 89197, 'Jose Cuervo Especial Reposado Tequila', 12, '1000', 12.5, 18.74, 12, 224.88, 12.0, 3.17, -91.452491, 40.717867, '88267005ebfffff'), ('S03402600044', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15296, 'Windsor Canadian Traveler', 12, '750', 4.2, 6.3, 4, 25.2, 3.0, 0.79, 0.0, 0.0, '0'), ('S03408300190', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1042100, 'IMPORTED DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 28086, 'Beefeater Gin', 12, '750', 10.39, 15.59, 2, 31.18, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03395800163', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1022100, 'TEQUILA', 395, 'Proximo', 89194, 'Jose Cuervo Especial Reposado Flask', 24, '375', 5.49, 8.24, 3, 24.72, 1.12, 0.3, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03383900015', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36307, 'Hawkeye Vodka', 12, '1000', 3.85, 5.78, 12, 69.36, 12.0, 3.17, -91.403752, 40.41538, '8826702f65fffff'), ('S03408300073', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1051010, 'AMERICAN GRAPE BRANDIES', 85, 'Brown-Forman Corporation', 52806, 'Korbel Brandy', 12, '750', 5.69, 8.54, 3, 25.62, 2.25, 0.59, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03411000130', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41715, 'Smirnoff Cranberry Vodka', 12, '750', 7.59, 11.56, 2, 23.12, 1.5, 0.4, 0.0, 0.0, '0'), ('S03385800017', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 12, 268.08, 12.0, 3.17, -92.258152, 42.809882, '8826291267fffff'), ('S03385500041', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1042100, 'IMPORTED DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 28086, 'Beefeater Gin', 12, '750', 10.39, 15.59, 2, 31.18, 1.5, 0.4, -93.21693, 42.741728, '88262b9605fffff'), ('S03409700032', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37418, 'Popov Vodka 80 Prf', 6, '1750', 8.49, 12.74, 6, 76.44, 10.5, 2.77, -93.737447, 41.62941, '88260d802bfffff'), ('S03393400161', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28231, 'Bombay Sapphire Gin Mini', 10, '600', 16.0, 24.0, 1, 24.0, 0.6, 0.16, -90.741064, 42.516633, '88275a212dfffff'), ('S03398600008', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1012100, 'CANADIAN WHISKIES', 85, 'Brown-Forman Corporation', 12478, 'Canadian Mist Canadian Why Pet', 6, '1750', 10.5, 15.75, 6, 94.5, 10.5, 2.77, -91.343544, 40.626988, '8826703131fffff'), ('S03389000004', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81204, 'Paramount Peppermint Schnapps', 24, '375', 2.27, 3.41, 12, 40.92, 4.5, 1.19, -93.315977, 40.757158, '88267252cdfffff'), ('S03404300001', Timestamp('2012-01-03 00:00:00'), 4538, 'K & S Grocery', '402 NORTH MAIN ST', 'CLARKSVILLE', '50619', 12, 'Butler', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 12407, 'Canadian Ltd Whisky', 12, '1000', 5.33, 7.99, 24, 191.76, 24.0, 6.34, -92.667833, 42.787434, '882629cdc1fffff'), ('S03403500008', Timestamp('2012-01-03 00:00:00'), 4156, 'Fareway Stores #412 / Oelwein', '102  2ND AVE SE', 'OELWEIN', '50662', 33, 'Fayette', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69611, "Dr. Mcgillicuddy's Apple Pie Schnapps", 12, '750', 8.32, 12.48, 12, 149.76, 9.0, 2.38, -91.910502, 42.677024, '882629b807fffff'), ('S03410900052', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03408300320', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58866, 'Jose Cuervo Authentic Strawberry Margarita', 12, '750', 4.62, 6.92, 2, 13.84, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03409700029', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1081200, 'CREAM LIQUEURS', 461, 'Campari(skyy)', 68126, "Carolan's Irish Cream Liqueur", 12, '750', 7.69, 11.54, 12, 138.48, 9.0, 2.38, -93.737447, 41.62941, '88260d802bfffff'), ('S03398800048', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 65066, 'Goldschlager Cinnamon Schnapps', 12, '750', 12.0, 17.99, 2, 35.98, 1.5, 0.4, -93.573845, 41.604312, '88260d95b3fffff'), ('S03388500089', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34821, 'Svedka Vodka', 6, '1750', 13.89, 20.84, 2, 41.68, 3.5, 0.92, -90.668138, 42.504959, '88275a2119fffff'), ('S03389300029', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69656, "Dr. Mcgillicuddy's Mentholmint", 12, '750', 8.66, 12.99, 6, 77.94, 4.5, 1.19, -90.609747, 41.589369, '882675d959fffff'), ('S03400700039', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1081365, 'TROPICAL FRUIT SCHNAPPS', 421, 'Sazerac Co., Inc.', 83907, 'Maui Blue Hawaiian Schnapps', 12, '1000', 4.54, 6.81, 12, 81.72, 12.0, 3.17, 0.0, 0.0, '0'), ('S03406300025', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1081300, 'PEPPERMINT SCHNAPPS', 65, 'Jim Beam Brands', 80687, 'Dekuyper Blustery Peppermint Burst Schnapps', 12, '1000', 3.86, 5.78, 12, 69.36, 12.0, 3.17, -91.281344, 42.108275, '882674c6a9fffff'), ('S03396800007', Timestamp('2012-01-03 00:00:00'), 2445, "Ruback's Food Center", '504 SOUTH HIGHWAY', 'OAKLAND', '51560', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25604, 'Seagrams 7 Crown Bl Whiskey', 24, '375', 3.34, 5.0, 4, 20.0, 1.5, 0.4, 0.0, 0.0, '0'), ('S03383800050', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1062300, 'FLAVORED RUM', 260, 'Diageo Americas', 43226, 'Captain Morgan Parrot Bay Passion Fruit', 12, '750', 7.49, 11.23, 1, 11.23, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03405800002', Timestamp('2012-01-03 00:00:00'), 3761, "Cheese's Food Center", '200 N WALNUT ST', 'LENOX', '50851', 87, 'Taylor', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -94.560455, 40.882645, '882608a6e9fffff'), ('S03383800025', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1032080, 'IMPORTED VODKA', 395, 'Proximo', 34935, 'Three Olives Vodka', 12, '750', 11.16, 16.74, 1, 16.74, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03395800130', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 18, 270.0, 13.5, 3.57, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03403400047', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52579, 'E & J Vsop Superior Reserve Mini', 12, '500', 5.91, 8.87, 1, 8.87, 0.5, 0.13, -93.599362, 41.59878, '88260d8743fffff'), ('S03403000015', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86887, 'Southern Comfort', 12, '1000', 11.34, 17.01, 3, 51.03, 3.0, 0.79, -94.480204, 43.39966, '88262a82cbfffff'), ('S03338000005', Timestamp('2012-01-03 00:00:00'), 4590, "Casey's General Store #3045 / Cedar", '601, MAIN ST', 'CEDAR FALLS', '50613', 7, 'Black Hawk', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86886, 'Southern Comfort', 12, '750', 9.36, 14.04, 12, 168.48, 9.0, 2.38, -92.445512, 42.532963, '8826290ab3fffff'), ('S03397500088', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43316, 'Captain Morgan Private Stock Rum', 12, '750', 11.45, 17.18, 2, 34.36, 1.5, 0.4, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03391300059', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41698, 'Uv Ivory (vanilla) Vodka', 12, '750', 6.22, 9.33, 1, 9.33, 0.75, 0.2, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03410400017', Timestamp('2012-01-03 00:00:00'), 4582, 'Pronto / Spirit Lake', '2402, 17th ST', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5346, 'Johnnie Walker Red', 12, '750', 13.78, 20.67, 2, 41.34, 1.5, 0.4, -95.126585, 43.42282, '88262ac235fffff'), ('S03395800190', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32233, 'Seagrams Extra Dry Gin', 48, '200', 2.04, 3.06, 6, 18.36, 1.2, 0.32, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03383600024', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36306, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 12, 58.56, 9.0, 2.38, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03387200005', Timestamp('2012-01-03 00:00:00'), 4296, 'Circle B Market', '111 MAIN ST', 'KEOSAUQUA', '52565', 89, 'Van Buren', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25607, 'Seagrams 7 Crown Bl Whiskey', 12, '1000', 7.5, 11.24, 12, 134.88, 12.0, 3.17, -91.962466, 40.7294, '882670458bfffff'), ('S03411200130', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1031100, '100 PROOF VODKA', 260, 'Diageo Americas', 39866, 'Smirnoff Vodka 100 Prf', 12, '750', 8.65, 12.97, 3, 38.91, 2.25, 0.59, -93.697731, 41.665172, '88260d8231fffff'), ('S03398500010', Timestamp('2012-01-03 00:00:00'), 4249, 'Fareway Stores #048 / Clarinda', '115 ESSIE DAVISON DR', 'CLARINDA', '51632', 73, 'Page', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42716, 'Malibu Coconut Rum', 12, '750', 6.9, 10.35, 3, 31.05, 2.25, 0.59, -95.040293, 40.724008, '8826083189fffff'), ('S03408200024', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43285, 'Captain Morgan Original Spiced Rum Pet', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03406400049', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37346, 'Phillips Vodka', 12, '750', 3.48, 5.23, 2, 10.46, 1.5, 0.4, -90.766126, 41.509119, '88267434edfffff'), ('S03396000006', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25608, 'Seagrams 7 Crown Bl Whiskey', 6, '1750', 10.76, 16.4, 6, 98.4, 10.5, 2.77, -90.705328, 42.491862, '88275a2131fffff'), ('S03406600010', Timestamp('2012-01-03 00:00:00'), 4250, 'Fareway Stores #882 / Eldora', '1232 14TH ST', 'ELDORA', '50627', 42, 'Hardin', 1081340, 'ROOT BEER SCHNAPPS', 65, 'Jim Beam Brands', 82826, "Dekuyper Ragin' Root Beer Schnapps", 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -93.10077, 42.361055, '88262ba259fffff'), ('S03408100029', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 20248, 'Old Crow', 6, '1750', 8.92, 13.38, 6, 80.28, 10.5, 2.77, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03390300012', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10627, 'Canadian Club Whisky', 12, '1000', 9.34, 14.01, 12, 168.12, 12.0, 3.17, -95.854809, 41.262316, '882608c9cdfffff'), ('S03402600021', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32233, 'Seagrams Extra Dry Gin', 48, '200', 2.04, 3.06, 2, 6.12, 0.4, 0.11, 0.0, 0.0, '0'), ('S03408600087', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 2, 20.6, 3.5, 0.92, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03398800017', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76487, 'Paramount Amaretto', 12, '1000', 4.04, 6.06, 12, 72.72, 12.0, 3.17, -93.573845, 41.604312, '88260d95b3fffff'), ('S03399800006', Timestamp('2012-01-03 00:00:00'), 3970, 'The Liquor Cabinet, Inc.', '835 6 ST, STE 1', 'NEVADA', '50201', 85, 'Story', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35317, 'Barton Vodka', 12, '1000', 3.97, 5.95, 12, 71.4, 12.0, 3.17, -93.452277, 42.019292, '88262b222dfffff'), ('S03382700025', Timestamp('2012-01-03 00:00:00'), 2607, 'Hy-Vee Wine and Spirits / Shenandoah', '520 SO FREMONT', 'SHENANDOAH', '51601', 73, 'Page', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41592, 'Smirnoff Lime Vodka', 12, '750', 7.59, 11.56, 2, 23.12, 1.5, 0.4, -95.385111, 40.761736, '8826080d95fffff'), ('S03401400026', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41707, 'Phillips Lime Flavored Vodka', 12, '1000', 5.78, 8.67, 12, 104.04, 12.0, 3.17, -95.126786, 43.416433, '88262ac237fffff'), ('S03396600069', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 3, 48.99, 2.25, 0.59, -93.378772, 43.142868, '8826283b29fffff'), ('S03389000010', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11771, 'Black Velvet Mini', 10, '600', 4.96, 7.44, 1, 7.44, 0.6, 0.16, -93.315977, 40.757158, '88267252cdfffff'), ('S03408800053', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1081010, 'AMERICAN AMARETTO', 421, 'Sazerac Co., Inc.', 71886, 'Amaretto E Dolce', 12, '750', 3.42, 5.13, 3, 15.39, 2.25, 0.59, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03406400003', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36304, 'Hawkeye Vodka', 24, '375', 2.06, 3.09, 1, 3.09, 0.38, 0.1, -90.766126, 41.509119, '88267434edfffff'), ('S03396600019', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 24, 395.76, 24.0, 6.34, -93.378772, 43.142868, '8826283b29fffff'), ('S03401000055', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76486, 'Paramount Amaretto', 12, '750', 3.37, 5.06, 1, 5.06, 0.75, 0.2, -90.683356, 42.52717, '88275a215dfffff'), ('S03388400006', Timestamp('2012-01-03 00:00:00'), 3833, 'Wal-Mart 3394 / Atlantic', '1905 EAST 7TH ST', 'ATLANTIC', '50022', 15, 'Cass', 1022100, 'TEQUILA', 434, 'Luxco-St Louis', 89386, 'Juarez Tequila Gold', 12, '750', 6.29, 9.43, 12, 113.16, 9.0, 2.38, -94.98571, 41.403853, '8826089095fffff'), ('S03403200027', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27551, 'Red Stag By Jim Beam', 6, '1750', 22.05, 33.07, 1, 33.07, 1.75, 0.46, -94.218, 43.29355, '8826284833fffff'), ('S03390300035', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45277, 'Paramount White Rum', 12, '1000', 4.33, 6.5, 12, 78.0, 12.0, 3.17, -95.854809, 41.262316, '882608c9cdfffff'), ('S03388500050', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 65066, 'Goldschlager Cinnamon Schnapps', 12, '750', 12.0, 17.99, 3, 53.97, 2.25, 0.59, -90.668138, 42.504959, '88275a2119fffff'), ('S03398400038', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86887, 'Southern Comfort', 12, '1000', 11.34, 17.01, 2, 34.02, 2.0, 0.53, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03389400022', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 18, 458.64, 31.5, 8.32, -95.909384, 41.242291, '88260e26d5fffff'), ('S03410900121', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43334, 'Captain Morgan Spiced Rum', 24, '375', 4.3, 6.45, 4, 25.8, 1.5, 0.4, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03396400072', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34036, 'Absolut Pears', 12, '750', 10.89, 16.33, 1, 16.33, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03392900087', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 277, 'Stoli Group', 34871, 'Stolichnaya Razberi Vodka 75 Proof', 12, '1000', 16.18, 24.27, 3, 72.81, 3.0, 0.79, 0.0, 0.0, '0'), ('S03389000019', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1011200, 'STRAIGHT BOURBON WHISKIES', 461, 'Campari(skyy)', 22157, 'Wild Turkey 101', 12, '1000', 14.99, 22.49, 4, 89.96, 4.0, 1.06, -93.315977, 40.757158, '88267252cdfffff'), ('S03402900034', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1012200, 'SCOTCH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 4725, 'Chivas Regal 12yr', 12, '375', 8.67, 13.0, 1, 13.0, 0.38, 0.1, -94.990031, 41.403856, '8826089095fffff'), ('S03391000013', Timestamp('2012-01-03 00:00:00'), 3654, 'The Beverage Shop / Belmond', '207 E MAIN ST', 'BELMOND', '50421', 99, 'Wright', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82847, 'Dekuyper Peachtree', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -93.615247, 42.845901, '8826286587fffff'), ('S03396600097', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 1, 10.62, 1.0, 0.26, -93.378772, 43.142868, '8826283b29fffff'), ('S03398800027', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37414, 'Popov Vodka 80 Prf', 24, '375', 2.82, 4.23, 3, 12.69, 1.12, 0.3, -93.573845, 41.604312, '88260d95b3fffff'), ('S03388300016', Timestamp('2012-01-03 00:00:00'), 4039, 'No Frills Supermarkets / Council Blu', '1817 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45246, 'Paramount Gold Rum', 12, '750', 3.85, 5.77, 12, 69.24, 9.0, 2.38, -95.870993, 41.261863, '882608c917fffff'), ('S03397500003', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 12, 180.0, 9.0, 2.38, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03408800073', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82611, 'Dekuyper Hot Damn!', 12, '750', 6.3, 9.45, 5, 47.25, 3.75, 0.99, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03390200054', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31656, 'Paramount Gin', 12, '750', 3.39, 5.09, 1, 5.09, 0.75, 0.2, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03408600058', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25604, 'Seagrams 7 Crown Bl Whiskey', 24, '375', 3.34, 5.0, 2, 10.0, 0.75, 0.2, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03401000086', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24466, 'Kessler Blend Whiskey Traveler', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -90.683356, 42.52717, '88275a215dfffff'), ('S03402500025', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65195, 'Hpnotiq', 6, '750', 11.68, 17.52, 2, 35.04, 1.5, 0.4, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03388500087', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 277, 'Stoli Group', 34870, 'Stoli Razberi (raspberry)', 12, '750', 12.24, 18.37, 3, 55.11, 2.25, 0.59, -90.668138, 42.504959, '88275a2119fffff'), ('S03408000038', Timestamp('2012-01-03 00:00:00'), 3622, 'Wal-Mart 1415 / Spirit Lake', '2600 HWY 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 63755, 'Tgi Fridays Mudslide', 6, '1750', 7.94, 11.91, 6, 71.46, 10.5, 2.77, -95.126535, 43.416452, '88262ac237fffff'), ('S03390300005', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43121, 'Bacardi Superior Rum Mini', 12, '500', 4.74, 7.11, 12, 85.32, 6.0, 1.59, -95.854809, 41.262316, '882608c9cdfffff'), ('S03391300125', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43024, 'Admiral Nelson Spiced Rum', 24, '375', 2.74, 4.11, 2, 8.22, 0.75, 0.2, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03397600083', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81207, 'Paramount Peppermint Schnapps', 12, '1000', 4.05, 6.08, 12, 72.96, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03386500021', Timestamp('2012-01-03 00:00:00'), 2548, 'Hy-Vee Food Store / Altoona', '100 8TH STREET SW', 'ALTOONA', '50009', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 434, 'Luxco-St Louis', 35699, 'Pearl Cucumber Vodka', 12, '750', 7.85, 11.78, 3, 35.34, 2.25, 0.59, -93.465301, 41.644041, '88260d94d9fffff'), ('S03398800030', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52579, 'E & J Vsop Superior Reserve Mini', 12, '500', 5.91, 8.87, 1, 8.87, 0.5, 0.13, -93.573845, 41.604312, '88260d95b3fffff'), ('S03408200037', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1081330, 'PEACH SCHNAPPS', 434, 'Luxco-St Louis', 84456, 'Paramount Peach Schnapps', 12, '750', 3.97, 5.95, 12, 71.4, 9.0, 2.38, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03396600134', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 4, 89.36, 4.0, 1.06, -93.378772, 43.142868, '8826283b29fffff'), ('S03389400027', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 12, 295.68, 12.0, 3.17, -95.909384, 41.242291, '88260e26d5fffff'), ('S03408300183', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 3, 31.86, 3.0, 0.79, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03412100212', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15244, 'Windsor Canadian Supreme Whisky', 24, '375', 2.89, 4.34, 4, 17.36, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03397000030', Timestamp('2012-01-03 00:00:00'), 3994, 'Great Spirits Wine & Liquor Co.', '72 NORTH HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82881, 'Dekuyper Hot Damn! 100 Proof', 12, '750', 6.77, 10.15, 3, 30.45, 2.25, 0.59, -95.127395, 43.368989000000006, '88262ac0e9fffff'), ('S03392900052', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 24, 254.88, 24.0, 6.34, 0.0, 0.0, '0'), ('S03385300027', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23826, 'Five Star', 12, '750', 3.35, 5.02, 12, 60.24, 9.0, 2.38, -91.136655, 40.814666, '882670aacdfffff'), ('S03385300047', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82618, 'Dekuyper Hot Damn! Pet', 6, '1750', 10.5, 15.75, 6, 94.5, 10.5, 2.77, -91.136655, 40.814666, '882670aacdfffff'), ('S03407100028', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1022100, 'TEQUILA', 395, 'Proximo', 89197, 'Jose Cuervo Especial Reposado Tequila', 12, '1000', 12.5, 18.74, 1, 18.74, 1.0, 0.26, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03403900027', Timestamp('2012-01-03 00:00:00'), 2628, 'Hy-Vee Food Store / Eldora', '1616 EDGINGTON', 'ELDORA', '50627', 42, 'Hardin', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58868, 'Jose Cuervo Authentic Strawberry Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, -93.103764, 42.360631, '88262ba25dfffff'), ('S03383900020', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42687, 'Malibu Black', 12, '750', 7.4, 11.6, 12, 139.2, 9.0, 2.38, -91.403752, 40.41538, '8826702f65fffff'), ('S03407800001', Timestamp('2012-01-03 00:00:00'), 4181, 'Puff N Snuff', '1101 RHOMBERG AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43285, 'Captain Morgan Original Spiced Rum Pet', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -90.660269, 42.519502, '88275a215bfffff'), ('S03403800016', Timestamp('2012-01-03 00:00:00'), 3774, 'Eagle Country Market / Dubuque', '1800 ELM ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69636, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -90.665763, 42.511105, '88275a2153fffff'), ('S03388200004', Timestamp('2012-01-03 00:00:00'), 4106, 'Hartig Drug Co. / JFK Rd / Dubuque', '2255 JOHN F KENNEDY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 12, 195.96, 9.0, 2.38, -90.72051800000001, 42.511352, '88275a212bfffff'), ('S03407900013', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11348, 'Seagrams V.o. Bl Canadian Whisky Pet', 6, '1750', 15.42, 23.13, 6, 138.78, 10.5, 2.77, 0.0, 0.0, '0'), ('S03386600036', Timestamp('2012-01-03 00:00:00'), 3647, 'Wal-Mart 2889 / Clinton', '2715 SOUTH 25TH ST', 'CLINTON', '52732', 23, 'Clinton', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 38008, 'Smirnoff Vodka 80 Prf Pet', 6, '1750', 13.98, 20.97, 6, 125.82, 10.5, 2.77, -90.245005, 41.818874, '88275b78b7fffff'), ('S03385300039', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58866, 'Jose Cuervo Authentic Strawberry Margarita', 12, '750', 4.62, 6.92, 12, 83.04, 9.0, 2.38, -91.136655, 40.814666, '882670aacdfffff'), ('S03409700034', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11358, 'Crown Royal Cask No. 16', 6, '750', 34.96, 52.44, 2, 104.88, 1.5, 0.4, -93.737447, 41.62941, '88260d802bfffff'), ('S03388100022', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36886, 'Mccormick Vodka', 12, '750', 3.3, 4.96, 12, 59.52, 9.0, 2.38, -95.879662, 41.261925, '882608c93bfffff'), ('S03408600009', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43127, 'Bacardi Superior Rum', 12, '1000', 9.43, 14.14, 12, 169.68, 12.0, 3.17, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03407900063', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1012210, 'SINGLE MALT SCOTCH', 370, 'Pernod Ricard USA/Austin Nichols', 5061, 'Glenlivet 15 Year French Oak', 6, '750', 26.4, 39.6, 2, 79.2, 1.5, 0.4, 0.0, 0.0, '0'), ('S03390400005', Timestamp('2012-01-03 00:00:00'), 3738, 'Hartig Drug Company #4 / Dubuque', '2225 CENTRAL AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1081600, 'WHISKEY LIQUEUR', 260, 'Diageo Americas', 86666, 'Jeremiah Weed 100prf Bourbon Liqueur', 12, '750', 6.43, 9.64, 12, 115.68, 9.0, 2.38, -90.671531, 42.51384, '88275a2153fffff'), ('S03383800076', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 260, 'Diageo Americas', 66636, 'Romana Sambuca Italian Liquore', 12, '750', 12.5, 18.74, 1, 18.74, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03407100018', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1012210, 'SINGLE MALT SCOTCH', 65, 'Jim Beam Brands', 5606, 'McClelland Islay', 12, '750', 13.01, 19.51, 1, 19.51, 0.75, 0.2, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03392100012', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52594, 'E & J Vs Brandy', 24, '375', 2.93, 4.4, 6, 26.4, 2.25, 0.59, -90.204748, 41.827845, '88275b7891fffff'), ('S03402300005', Timestamp('2012-01-03 00:00:00'), 4456, 'Kum & Go #53 / Iowa City', '955 MORMAN TREK BLVD', 'IOWA CITY', '52246', 52, 'Johnson', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11788, 'Black Velvet', 6, '1750', 10.45, 15.67, 6, 94.02, 10.5, 2.77, -91.574075, 41.649946, '8826747addfffff'), ('S03408600031', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1041200, 'AMERICAN SLOE GINS', 434, 'Luxco-St Louis', 33716, 'Paramount Sloe Gin', 12, '750', 3.84, 5.75, 1, 5.75, 0.75, 0.2, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03406800001', Timestamp('2012-01-03 00:00:00'), 4010, "Newman's Thriftway", '900 S LOCUST ST', 'GLENWOOD', '51534', 65, 'Mills', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43126, 'Bacardi Superior Rum', 12, '750', 7.53, 11.3, 12, 135.6, 9.0, 2.38, -95.745944, 41.038205, '8826085555fffff'), ('S03387700010', Timestamp('2012-01-03 00:00:00'), 3013, "Keith's Foods", '207 E LOCUST ST', 'BLOOMFIELD', '52537', 26, 'Davis', 1011100, 'BLENDED WHISKIES', 434, 'Luxco-St Louis', 24156, 'Hawkeye Blend Whiskey', 12, '750', 3.35, 5.03, 12, 60.36, 9.0, 2.38, -92.412847, 40.752691, '88267284d5fffff'), ('S03395500016', Timestamp('2012-01-03 00:00:00'), 3976, 'North English Family Food', '119 NORTH MAIN ST', 'NORTH ENGLISH', '52316', 48, 'Iowa', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54056, 'Arrow Apricot Flav Brandy', 12, '750', 5.0, 7.5, 1, 7.5, 0.75, 0.2, -92.076452, 41.515209, '882676a0adfffff'), ('S03385700002', Timestamp('2012-01-03 00:00:00'), 2238, 'Adventureland Inn', '3200 Adventureland Dr', 'ALTOONA', '50009', 77, 'Polk', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 3, 67.02, 3.0, 0.79, -93.49924000000001, 41.658513, '88260d94cdfffff'), ('S03388800004', Timestamp('2012-01-03 00:00:00'), 3811, 'Wal-Mart 1431 / Keokuk', '300 NORTH PARK DR', 'KEOKUK', '52632', 56, 'Lee', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 63755, 'Tgi Fridays Mudslide', 6, '1750', 7.94, 11.91, 6, 71.46, 10.5, 2.77, -91.407263, 40.418589, '88267028d9fffff'), ('S03401000033', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52594, 'E & J Vs Brandy', 24, '375', 2.93, 4.4, 4, 17.6, 1.5, 0.4, -90.683356, 42.52717, '88275a215dfffff'), ('S03396400047', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26828, 'Jack Daniels Old #7 Black Lbl', 6, '1750', 26.67, 40.01, 12, 480.12, 21.0, 5.55, -95.8792, 41.238092, '882608c9e7fffff'), ('S03408800004', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 12, 123.6, 21.0, 5.55, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03386100005', Timestamp('2012-01-03 00:00:00'), 4083, 'Fareway Stores #909 / Ankeny', '109 SE ORALABOR RD', 'ANKENY', '50021', 77, 'Polk', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 6, 90.0, 4.5, 1.19, -93.59949600000002, 41.702811, '88260d954bfffff'), ('S03402100057', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28233, 'Bombay Sapphire Gin', 12, '1000', 17.61, 26.41, 2, 52.82, 2.0, 0.53, -92.475403, 42.712263, '8826291801fffff'), ('S03397300101', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43116, 'Bacardi O', 12, '750', 8.33, 12.49, 4, 49.96, 3.0, 0.79, -91.044008, 41.427202, '8826742f4bfffff'), ('S03394000027', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19067, 'Jim Beam', 12, '1000', 11.54, 17.31, 12, 207.72, 12.0, 3.17, -95.902023, 41.257663, '882608c937fffff'), ('S03408400051', Timestamp('2012-01-03 00:00:00'), 2650, 'Hy-Vee Wine and Spirits / Harlan', '1808 23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48106, 'Hennessy Vs Cognac', 12, '750', 17.06, 25.59, 3, 76.77, 2.25, 0.59, -95.339881, 41.650658, '88260c60e9fffff'), ('S03398600064', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1022100, 'TEQUILA', 395, 'Proximo', 89191, 'Jose Cuervo Especial Reposado Tequila Mini', 12, '500', 11.5, 17.25, 1, 17.25, 0.5, 0.13, -91.343544, 40.626988, '8826703131fffff'), ('S03410300018', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 4, 98.56, 4.0, 1.06, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03397300052', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43121, 'Bacardi Superior Rum Mini', 12, '500', 4.74, 7.11, 1, 7.11, 0.5, 0.13, -91.044008, 41.427202, '8826742f4bfffff'), ('S03392100006', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48103, 'Hennessy Vs Cognac', 48, '200', 5.0, 7.49, 4, 29.96, 0.8, 0.21, -90.204748, 41.827845, '88275b7891fffff'), ('S03391300128', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43334, 'Captain Morgan Spiced Rum', 24, '375', 4.3, 6.45, 2, 12.9, 0.75, 0.2, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03383800070', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 380, 'Phillips Beverage Company', 63959, 'Uv Blue Raspberry Lemonade Pet', 6, '1750', 8.25, 12.38, 1, 12.38, 1.75, 0.46, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03395300004', Timestamp('2012-01-03 00:00:00'), 4030, 'Buy Rite Foods', '201 MARKET ST', 'LAKE PARK', '51347', 30, 'Dickinson', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -95.321385, 43.454741, '88262ad9dbfffff'), ('S03412100114', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1031200, 'VODKA FLAVORED', 421, 'Sazerac Co., Inc.', 41209, 'Firefly Sweet Tea Vodka', 12, '750', 10.0, 15.0, 2, 30.0, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03385300006', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081400, 'TRIPLE SEC', 305, 'MHW Ltd', 85987, 'Bols Triple Sec', 12, '1000', 3.85, 5.78, 12, 69.36, 12.0, 3.17, -91.136655, 40.814666, '882670aacdfffff'), ('S03401600043', Timestamp('2012-01-03 00:00:00'), 2522, 'Hy-Vee Wine and Spirits / Spirit Lak', 'HWY 9 & 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 12, 268.08, 12.0, 3.17, 0.0, 0.0, '0'), ('S03395800067', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 380, 'Phillips Beverage Company', 63959, 'Uv Blue Raspberry Lemonade Pet', 6, '1750', 8.25, 12.38, 6, 74.28, 10.5, 2.77, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03395800143', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24458, 'Kessler Blend Whiskey', 6, '1750', 11.01, 16.52, 150, 2478.0, 262.5, 69.35, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03410900060', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 65, 'Jim Beam Brands', 43848, 'Ronrico Silver Label Rum (pr)', 6, '1750', 7.35, 11.02, 6, 66.12, 10.5, 2.77, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03400700026', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57120, "Chi-Chi's Long Island Iced Tea", 6, '1750', 5.99, 9.24, 6, 55.44, 10.5, 2.77, 0.0, 0.0, '0'), ('S03411200110', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1012200, 'SCOTCH WHISKIES', 35, 'Bacardi U.S.A., Inc.', 4876, 'Dewars 12yr', 12, '750', 18.66, 27.99, 2, 55.98, 1.5, 0.4, -93.697731, 41.665172, '88260d8231fffff'), ('S03392900076', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1012210, 'SINGLE MALT SCOTCH', 461, 'Campari(skyy)', 5620, 'McClelland Speyside Scotch', 12, '750', 13.01, 19.51, 2, 39.02, 1.5, 0.4, 0.0, 0.0, '0'), ('S03390300014', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15247, 'Windsor Canadian', 12, '1000', 6.55, 9.83, 12, 117.96, 12.0, 3.17, -95.854809, 41.262316, '882608c9cdfffff'), ('S03390200039', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53218, 'Paul Masson Grande Amber Brandy', 6, '1750', 11.99, 17.99, 1, 17.99, 1.75, 0.46, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03385800060', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1081400, 'TRIPLE SEC', 434, 'Luxco-St Louis', 86251, 'Juarez Triple Sec', 12, '1000', 2.21, 3.31, 2, 6.62, 2.0, 0.53, -92.258152, 42.809882, '8826291267fffff'), ('S03408300202', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86886, 'Southern Comfort', 12, '750', 9.36, 14.04, 4, 56.16, 3.0, 0.79, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03411200059', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43121, 'Bacardi Superior Rum Mini', 12, '500', 4.74, 7.11, 1, 7.11, 0.5, 0.13, -93.697731, 41.665172, '88260d8231fffff'), ('S03404500037', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55086, 'Paramount Blackberry Brandy', 12, '750', 5.21, 7.82, 4, 31.28, 3.0, 0.79, -95.83539, 41.279189, '882608c829fffff'), ('S03395800090', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35916, "Five O'clock Vodka", 12, '750', 3.31, 4.96, 24, 119.04, 18.0, 4.76, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03387400008', Timestamp('2012-01-03 00:00:00'), 3642, "Fraise Foods / Sonny's Supermarket", '327  5TH ST', 'WEST POINT', '52656', 56, 'Lee', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19067, 'Jim Beam', 12, '1000', 11.54, 17.31, 12, 207.72, 12.0, 3.17, -91.452491, 40.717867, '88267005ebfffff'), ('S03411500050', Timestamp('2012-01-03 00:00:00'), 3700, 'Avoca Liquor', '158 SOUTH ELM', 'AVOCA', '51521', 78, 'Pottawattamie', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43137, 'Bacardi Limon', 12, '1000', 10.24, 15.35, 3, 46.05, 3.0, 0.79, -95.339412, 41.476306, '88260c6c8bfffff'), ('S03398800040', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43156, 'Bacardi 151 Proof Rum', 12, '750', 12.66, 18.99, 2, 37.98, 1.5, 0.4, -93.573845, 41.604312, '88260d95b3fffff'), ('S03403300006', Timestamp('2012-01-03 00:00:00'), 4566, 'Kum & Go #508 / Cedar Rapids', '30 FOUR OAKS DR SW', 'CEDAR RAPIDS', '52404', 57, 'Linn', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11788, 'Black Velvet', 6, '1750', 10.45, 15.67, 6, 94.02, 10.5, 2.77, -91.658073, 41.923446, '8826745503fffff'), ('S03397800007', Timestamp('2012-01-03 00:00:00'), 4421, 'KUM & GO #1 / HAMPTON', '609 CENTRAL AVE W', 'HAMPTON', '50441', 35, 'Franklin', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82846, 'Dekuyper Luscious Peachtree Schnapps', 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -93.21508, 42.741741, '88262b9605fffff'), ('S03392300067', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1081400, 'TRIPLE SEC', 434, 'Luxco-St Louis', 86507, 'Paramount Triple Sec', 12, '1000', 3.75, 5.62, 12, 67.44, 12.0, 3.17, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03371300010', Timestamp('2012-01-03 00:00:00'), 4443, 'Palo Mini Mart', '1204 1ST ST', 'PALO', '52324', 57, 'Linn', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21597, 'Ten High', 12, '1000', 4.93, 7.4, 12, 88.8, 12.0, 3.17, -91.795808, 42.070316, '8826745323fffff'), ('S03392900015', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 80577, 'Arrow Peppermint Schnapps', 12, '1000', 4.03, 6.05, 12, 72.6, 12.0, 3.17, 0.0, 0.0, '0'), ('S03394500024', Timestamp('2012-01-03 00:00:00'), 3443, 'Super Saver Iv', '1141 N BROADWAY', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 421, 'Sazerac Co., Inc.', 16516, 'Ancient Age Bourbon', 12, '750', 6.0, 8.99, 12, 107.88, 9.0, 2.38, -95.836515, 41.270824, '882608c823fffff'), ('S03406400025', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43036, 'Bacardi Gold Rum', 12, '750', 7.53, 11.3, 3, 33.9, 2.25, 0.59, -90.766126, 41.509119, '88267434edfffff'), ('S03411000137', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43156, 'Bacardi 151 Proof Rum', 12, '750', 12.66, 18.99, 4, 75.96, 3.0, 0.79, 0.0, 0.0, '0'), ('S03410400009', Timestamp('2012-01-03 00:00:00'), 4582, 'Pronto / Spirit Lake', '2402, 17th ST', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1022100, 'TEQUILA', 969, 'Aveniu Brands', 87946, 'Kah Blanco Tequila', 6, '750', 24.83, 37.25, 2, 74.5, 1.5, 0.4, -95.126585, 43.42282, '88262ac235fffff'), ('S03397600137', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36903, 'Mccormick Vodka', 48, '200', 1.12, 1.68, 10, 16.8, 2.0, 0.53, -95.825137, 41.242732, '882608c9d1fffff'), ('S03404800089', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69947, 'Rumple Minze Peppermint Schnapps Liqueur', 12, '1000', 15.0, 22.49, 12, 269.88, 12.0, 3.17, -91.035138, 41.451349, '88267421abfffff'), ('S03397600156', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1012200, 'SCOTCH WHISKIES', 35, 'Bacardi U.S.A., Inc.', 4876, 'Dewars 12yr', 12, '750', 18.66, 27.99, 3, 83.97, 2.25, 0.59, -95.825137, 41.242732, '882608c9d1fffff'), ('S03406400015', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48106, 'Hennessy Vs Cognac', 12, '750', 17.06, 25.59, 2, 51.18, 1.5, 0.4, -90.766126, 41.509119, '88267434edfffff'), ('S03401000071', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41719, 'Smirnoff Green Apple Vodka', 12, '750', 7.59, 11.56, 1, 11.56, 0.75, 0.2, -90.683356, 42.52717, '88275a215dfffff'), ('S03385900027', Timestamp('2012-01-03 00:00:00'), 4356, 'Kum & Go #396 / Nevada', '136 LINCOLN HWY', 'NEVADA', '50201', 85, 'Story', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 3, 32.1, 5.25, 1.39, -93.458003, 42.022725, '88262b235bfffff'), ('S03410900114', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 259, 'Heaven Hill Brands', 35416, "Burnett's Vodka 80 Prf", 12, '750', 4.55, 6.82, 4, 27.28, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03407900059', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 380, 'Phillips Beverage Company', 45419, 'Phillips White Rum', 12, '1000', 4.23, 6.35, 12, 76.2, 12.0, 3.17, 0.0, 0.0, '0'), ('S03398700001', Timestamp('2012-01-03 00:00:00'), 4318, 'Fareway Stores #841 / Red Oak', '811 BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 6, 90.0, 4.5, 1.19, -95.321174, 41.663944, '88260c62a7fffff'), ('S03390300007', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 36, 382.32, 36.0, 9.51, -95.854809, 41.262316, '882608c9cdfffff'), ('S03397300040', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52596, 'E & J Vs', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -91.044008, 41.427202, '8826742f4bfffff'), ('S03396400050', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58835, 'Jose Cuervo Authentic Lime Margarita', 6, '800', 4.62, 6.92, 12, 83.04, 9.6, 2.54, -95.8792, 41.238092, '882608c9e7fffff'), ('S03410300115', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52563, 'E & J Xo Brandy', 12, '750', 7.99, 11.99, 1, 11.99, 0.75, 0.2, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03400100003', Timestamp('2012-01-03 00:00:00'), 4469, 'Sperry One Stop Shop LLC', '516 HWY 141', 'COON RAPIDS', '50058', 14, 'Carroll', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35918, "Five O'clock Vodka", 6, '1750', 6.92, 10.38, 6, 62.28, 10.5, 2.77, -94.678374, 41.865913, '88260cad0dfffff'), ('S03388200028', Timestamp('2012-01-03 00:00:00'), 4106, 'Hartig Drug Co. / JFK Rd / Dubuque', '2255 JOHN F KENNEDY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42712, 'Malibu Coconut Rum Mini', 12, '500', 4.95, 7.43, 1, 7.43, 0.5, 0.13, -90.72051800000001, 42.511352, '88275a212bfffff'), ('S03407900003', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32237, 'Seagrams Extra Dry Gin', 12, '1000', 7.4, 11.1, 12, 133.2, 12.0, 3.17, 0.0, 0.0, '0'), ('S03397600050', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1032080, 'IMPORTED VODKA', 65, 'Jim Beam Brands', 34578, 'Pinnacle Vodka', 6, '1750', 11.43, 17.14, 6, 102.84, 10.5, 2.77, -95.825137, 41.242732, '882608c9d1fffff'), ('S03396400032', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82847, 'Dekuyper Peachtree', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -95.8792, 41.238092, '882608c9e7fffff'), ('S03411200098', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 85526, 'Dekuyper Signature Blue Curacao', 12, '750', 5.25, 7.87, 2, 15.74, 1.5, 0.4, -93.697731, 41.665172, '88260d8231fffff'), ('S03403100005', Timestamp('2012-01-03 00:00:00'), 4462, "SHELTON'S", '113, 1ST ST NW', 'BUFFALO CENTER', '50424', 95, 'Winnebago', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41592, 'Smirnoff Lime Vodka', 12, '750', 7.59, 11.56, 2, 23.12, 1.5, 0.4, -93.947595, 43.386755, '8826284217fffff'), ('S03412100029', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1041100, 'AMERICAN DRY GINS', 125, 'Cedar Ridge Vineyards,LLC', 29812, 'Clearheart Gin', 6, '750', 12.3, 18.45, 6, 110.7, 4.5, 1.19, -90.483701, 41.552338, '882675da95fffff'), ('S03387700002', Timestamp('2012-01-03 00:00:00'), 3013, "Keith's Foods", '207 E LOCUST ST', 'BLOOMFIELD', '52537', 26, 'Davis', 1041100, 'AMERICAN DRY GINS', 205, 'E AND J GALLO WINERY', 31475, 'New Amsterdam Gin', 12, '750', 6.9, 10.35, 12, 124.2, 9.0, 2.38, -92.412847, 40.752691, '88267284d5fffff'), ('S03390400022', Timestamp('2012-01-03 00:00:00'), 3738, 'Hartig Drug Company #4 / Dubuque', '2225 CENTRAL AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34366, 'Fris Danish Vodka', 12, '750', 5.13, 7.69, 2, 15.38, 1.5, 0.4, -90.671531, 42.51384, '88275a2153fffff'), ('S03385500013', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37418, 'Popov Vodka 80 Prf', 6, '1750', 8.49, 12.74, 6, 76.44, 10.5, 2.77, -93.21693, 42.741728, '88262b9605fffff'), ('S03392300029', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42717, 'Malibu Coconut Rum', 12, '1000', 9.77, 14.66, 12, 175.92, 12.0, 3.17, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03388500007', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35917, "Five O'clock Vodka", 12, '1000', 4.01, 6.01, 24, 144.24, 24.0, 6.34, -90.668138, 42.504959, '88275a2119fffff'), ('S03402700017', Timestamp('2012-01-03 00:00:00'), 3907, "Smokin' Joe's #16 Tobacco and Liquor", '1504 PARK AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26820, 'Jack Daniels Old #7 Black Lbl Flat', 24, '375', 6.06, 9.09, 4, 36.36, 1.5, 0.4, -91.030569, 41.442458, '88267421bdfffff'), ('S03389400043', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41797, 'Smirnoff Raspberry Vodka', 12, '1000', 9.2, 13.8, 3, 41.4, 3.0, 0.79, -95.909384, 41.242291, '88260e26d5fffff'), ('S03385700001', Timestamp('2012-01-03 00:00:00'), 2238, 'Adventureland Inn', '3200 Adventureland Dr', 'ALTOONA', '50009', 77, 'Polk', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19067, 'Jim Beam', 12, '1000', 11.54, 17.31, 3, 51.93, 3.0, 0.79, -93.49924000000001, 41.658513, '88260d94cdfffff'), ('S03395500002', Timestamp('2012-01-03 00:00:00'), 3976, 'North English Family Food', '119 NORTH MAIN ST', 'NORTH ENGLISH', '52316', 48, 'Iowa', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81208, 'Paramount Peppermint Schnapps', 6, '1750', 7.08, 10.62, 6, 63.72, 10.5, 2.77, -92.076452, 41.515209, '882676a0adfffff'), ('S03410300032', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1011200, 'STRAIGHT BOURBON WHISKIES', 259, 'Heaven Hill Brands', 17956, 'Evan Williams Str Bourbon', 12, '750', 6.37, 9.56, 3, 28.68, 2.25, 0.59, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03406400019', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1081200, 'CREAM LIQUEURS', 330, 'Gemini Spirits', 80457, "Ryan's Cream Liqueur", 12, '1000', 7.22, 10.83, 1, 10.83, 1.0, 0.26, -90.766126, 41.509119, '88267434edfffff'), ('S03410300123', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43126, 'Bacardi Superior Rum', 12, '750', 7.53, 11.3, 4, 45.2, 3.0, 0.79, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03402700012', Timestamp('2012-01-03 00:00:00'), 3907, "Smokin' Joe's #16 Tobacco and Liquor", '1504 PARK AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65253, 'Jagermeister Liqueur', 48, '200', 3.47, 5.2, 2, 10.4, 0.4, 0.11, -91.030569, 41.442458, '88267421bdfffff'), ('S03398500004', Timestamp('2012-01-03 00:00:00'), 4249, 'Fareway Stores #048 / Clarinda', '115 ESSIE DAVISON DR', 'CLARINDA', '51632', 73, 'Page', 1071100, 'AMERICAN COCKTAILS', 65, 'Jim Beam Brands', 77395, 'Skinnygirl White Cranberry Cosmo', 12, '750', 7.69, 11.53, 3, 34.59, 2.25, 0.59, -95.040293, 40.724008, '8826083189fffff'), ('S03386600015', Timestamp('2012-01-03 00:00:00'), 3647, 'Wal-Mart 2889 / Clinton', '2715 SOUTH 25TH ST', 'CLINTON', '52732', 23, 'Clinton', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43036, 'Bacardi Gold Rum', 12, '750', 7.53, 11.3, 12, 135.6, 9.0, 2.38, -90.245005, 41.818874, '88275b78b7fffff'), ('S03388000025', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1051110, 'BLACKBERRY BRANDIES', 380, 'Phillips Beverage Company', 55106, 'Phillips Blackberry Flav Brandy', 12, '750', 4.75, 7.13, 12, 85.56, 9.0, 2.38, -93.633306, 43.261538, '88262808e7fffff'), ('S03397600134', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48101, 'Hennessy Vs Cognac Minis', 10, '600', 23.6, 35.4, 1, 35.4, 0.6, 0.16, -95.825137, 41.242732, '882608c9d1fffff'), ('S03397100007', Timestamp('2012-01-03 00:00:00'), 4306, 'Northside One Stop / Hampton', '1208  4TH ST NE', 'HAMPTON', '50441', 35, 'Franklin', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41696, 'Uv Green (apple) Vodka', 12, '750', 6.22, 9.33, 2, 18.66, 1.5, 0.4, -93.202452, 42.753045, '88262b9647fffff'), ('S03404600015', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37417, 'Popov Vodka 80 Prf', 12, '1000', 5.0, 7.49, 12, 89.88, 12.0, 3.17, -95.027238, 40.739231, '88260831cbfffff'), ('S03411000082', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43036, 'Bacardi Gold Rum', 12, '750', 7.53, 11.3, 4, 45.2, 3.0, 0.79, 0.0, 0.0, '0'), ('S03396400148', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15626, 'Jameson', 12, '750', 13.46, 20.2, 1, 20.2, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03411200133', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1031200, 'VODKA FLAVORED', 259, 'Heaven Hill Brands', 41927, "Burnett's Fruit Punch", 12, '750', 3.88, 6.15, 3, 18.45, 2.25, 0.59, -93.697731, 41.665172, '88260d8231fffff'), ('S03395800231', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19476, "Maker's Mark", 12, '750', 15.74, 23.61, 3, 70.83, 2.25, 0.59, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03385800058', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54448, 'Paramount Apricot Flavored Brandy', 6, '1750', 10.75, 16.12, 2, 32.24, 3.5, 0.92, -92.258152, 42.809882, '8826291267fffff'), ('S03412100027', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 125, 'Cedar Ridge Vineyards,LLC', 46113, 'Clearheart Rum', 6, '750', 11.3, 16.95, 6, 101.7, 4.5, 1.19, -90.483701, 41.552338, '882675da95fffff'), ('S03389000048', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69634, "Dr. Mcgillicuddy's Cherry Schnapps", 24, '375', 5.33, 7.99, 2, 15.98, 0.75, 0.2, -93.315977, 40.757158, '88267252cdfffff'), ('S03395800184', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86883, 'Southern Comfort', 48, '200', 2.5, 3.76, 5, 18.8, 1.0, 0.26, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03408300229', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1022100, 'TEQUILA', 395, 'Proximo', 87408, 'Jose Cuervo Especial Silver', 12, '750', 10.49, 15.73, 6, 94.38, 4.5, 1.19, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03408800031', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19066, 'Jim Beam', 12, '750', 8.42, 13.41, 12, 160.92, 9.0, 2.38, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03385300118', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1022100, 'TEQUILA', 330, 'Gemini Spirits', 89496, 'Margaritaville Gold Tequila', 12, '750', 6.01, 9.52, 4, 38.08, 3.0, 0.79, -91.136655, 40.814666, '882670aacdfffff'), ('S03411000098', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1071100, 'AMERICAN COCKTAILS', 380, 'Phillips Beverage Company', 76767, 'Phillips Long Island Iced Tea', 12, '1000', 4.28, 6.41, 6, 38.46, 6.0, 1.59, 0.0, 0.0, '0'), ('S03397600124', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 12, 195.96, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03395700023', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5347, 'Johnnie Walker Red Label Scotch', 12, '1000', 16.99, 25.49, 2, 50.98, 2.0, 0.53, -95.147741, 43.14521, '88262a1925fffff'), ('S03388900012', Timestamp('2012-01-03 00:00:00'), 4540, "Casey's General Store #2915 / Bellev", '1104 N RIVERVIEW ST', 'BELLEVUE', '52031', 49, 'Jackson', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11290, 'Crown Royal Canadian Whisky Mini', 10, '300', 5.99, 8.99, 2, 17.98, 0.6, 0.16, -90.427844, 42.268432, '88275b5ac1fffff'), ('S03393100003', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43331, 'Captain Morgan Spiced Rum Mini', 12, '500', 4.97, 7.25, 12, 87.0, 6.0, 1.59, -95.855054, 41.261673, '882608c9cdfffff'), ('S03406000001', Timestamp('2012-01-03 00:00:00'), 4188, 'Main St. Market, Inc.', '882 MAIN ST', 'HOLY CROSS', '52053', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11777, 'Black Velvet', 12, '1000', 6.63, 9.94, 24, 238.56, 24.0, 6.34, -90.995943, 42.600941, '88275a7469fffff'), ('S03411200076', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5326, 'Johnnie Walker Black', 12, '750', 20.0, 29.99, 1, 29.99, 0.75, 0.2, -93.697731, 41.665172, '88260d8231fffff'), ('S03403700001', Timestamp('2012-01-03 00:00:00'), 4547, 'Fareway Stores #902 / Hampton', '309 CENTRAL AVE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -93.211676, 42.741748, '88262b9605fffff'), ('S03393100049', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15296, 'Windsor Canadian Traveler', 12, '750', 4.2, 6.3, 12, 75.6, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03387200003', Timestamp('2012-01-03 00:00:00'), 4296, 'Circle B Market', '111 MAIN ST', 'KEOSAUQUA', '52565', 89, 'Van Buren', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -91.962466, 40.7294, '882670458bfffff'), ('S03383200019', Timestamp('2012-01-03 00:00:00'), 3562, 'Wal-Mart 0797 / W Burlington', '324 WEST AGENCY RD', 'WEST BURLINGTON', '52655', 29, 'Des Moines', 1062300, 'FLAVORED RUM', 260, 'Diageo Americas', 43282, 'Captain Morgan Parrot Bay Pineapple', 12, '750', 7.49, 11.23, 12, 134.76, 9.0, 2.38, -91.158407, 40.813125, '882670aa17fffff'), ('S03410900050', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 434, 'Luxco-St Louis', 41500, 'Hawkeye Blue Raspberry Vodka', 12, '1000', 5.0, 7.5, 12, 90.0, 12.0, 3.17, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03395800034', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1041100, 'AMERICAN DRY GINS', 297, 'Laird And Company', 29997, "Five O'clock Gin", 12, '1000', 4.06, 6.09, 12, 73.08, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03397000005', Timestamp('2012-01-03 00:00:00'), 3994, 'Great Spirits Wine & Liquor Co.', '72 NORTH HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1012200, 'SCOTCH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 9428, 'Passport Scotch', 6, '1750', 12.56, 17.79, 2, 35.58, 3.5, 0.92, -95.127395, 43.368989000000006, '88262ac0e9fffff'), ('S03408700006', Timestamp('2012-01-03 00:00:00'), 4066, 'Xpress Liquor', '1201 NORTH JEFFERSON STE 100', 'INDIANOLA', '50125', 91, 'Warren', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, 0.0, 0.0, '0'), ('S03389400058', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012210, 'SINGLE MALT SCOTCH', 240, 'WILLIAM GRANT AND SONS, INC.', 5006, 'Glenfiddich 12yr Special Reserve', 12, '750', 21.17, 31.75, 1, 31.75, 0.75, 0.2, -95.909384, 41.242291, '88260e26d5fffff'), ('S03394700018', Timestamp('2012-01-03 00:00:00'), 3959, 'Hartig Drug Company #8 / University', '1600 UNIVERSITY AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86916, 'Southern Comfort 100 Prf', 12, '750', 11.65, 17.47, 3, 52.41, 2.25, 0.59, -90.688027, 42.498279, '88275a2115fffff'), ('S03408100057', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43127, 'Bacardi Superior Rum', 12, '1000', 9.43, 14.14, 4, 56.56, 4.0, 1.06, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03384700016', Timestamp('2012-01-03 00:00:00'), 4138, 'Fareway Stores #657 / Indianola', '1309 W 2ND', 'INDIANOLA', '50125', 91, 'Warren', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43136, 'Bacardi Limon', 12, '750', 8.33, 12.49, 3, 37.47, 2.25, 0.59, -93.579252, 41.356616, '88260db8e9fffff'), ('S03408100075', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23827, 'Five Star', 12, '1000', 4.36, 6.53, 2, 13.06, 2.0, 0.53, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03383600018', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81206, 'Paramount Peppermint Schnapps', 12, '750', 3.7, 5.55, 12, 66.6, 9.0, 2.38, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03404500017', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24451, 'Kessler Blend Whiskey Mini', 12, '500', 5.23, 7.84, 1, 7.84, 0.5, 0.13, -95.83539, 41.279189, '882608c829fffff'), ('S03401400002', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11296, 'Crown Royal', 12, '750', 14.25, 21.37, 12, 256.44, 9.0, 2.38, -95.126786, 43.416433, '88262ac237fffff'), ('S03411200099', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15644, 'Jameson', 24, '375', 5.86, 8.79, 2, 17.58, 0.75, 0.2, -93.697731, 41.665172, '88260d8231fffff'), ('S03388100038', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82787, 'Dekuyper Buttershots', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -95.879662, 41.261925, '882608c93bfffff'), ('S03395700050', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42712, 'Malibu Coconut Rum Mini', 12, '500', 4.95, 7.43, 1, 7.43, 0.5, 0.13, -95.147741, 43.14521, '88262a1925fffff'), ('S03398400035', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1041200, 'AMERICAN SLOE GINS', 434, 'Luxco-St Louis', 33717, 'Paramount Sloe Gin', 12, '1000', 4.9, 7.36, 2, 14.72, 2.0, 0.53, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03390400009', Timestamp('2012-01-03 00:00:00'), 3738, 'Hartig Drug Company #4 / Dubuque', '2225 CENTRAL AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 434, 'Luxco-St Louis', 24158, 'Hawkeye Blend Whiskey', 6, '1750', 7.38, 11.07, 12, 132.84, 21.0, 5.55, -90.671531, 42.51384, '88275a2153fffff'), ('S03402900065', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43136, 'Bacardi Limon', 12, '750', 8.33, 12.49, 2, 24.98, 1.5, 0.4, -94.990031, 41.403856, '8826089095fffff'), ('S03395200066', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41360, 'Uv Pink Lemonade Vodka', 12, '750', 6.22, 9.33, 5, 46.65, 3.75, 0.99, -91.110752, 40.801889, '882670a137fffff'), ('S03395200090', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 20246, 'Old Crow', 12, '750', 4.19, 6.29, 6, 37.74, 4.5, 1.19, -91.110752, 40.801889, '882670a137fffff'), ('S03410900023', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45245, 'Paramount Gold Rum Traveler', 12, '750', 3.85, 5.77, 12, 69.24, 9.0, 2.38, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03407800003', Timestamp('2012-01-03 00:00:00'), 4181, 'Puff N Snuff', '1101 RHOMBERG AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37994, 'Smirnoff Vodka 80 Prf', 24, '375', 4.48, 6.72, 24, 161.28, 9.0, 2.38, -90.660269, 42.519502, '88275a215bfffff'), ('S03389600022', Timestamp('2012-01-03 00:00:00'), 3632, 'Wal-Mart 2004 / Dubuque', '4200 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1022100, 'TEQUILA', 395, 'Proximo', 87408, 'Jose Cuervo Especial Silver', 12, '750', 10.49, 15.73, 12, 188.76, 9.0, 2.38, -90.736955, 42.489041, '88275a2ac1fffff'), ('S03393400086', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 421, 'Sazerac Co., Inc.', 12888, 'Rich & Rare Canadian Whisky', 6, '1750', 8.83, 13.25, 6, 79.5, 10.5, 2.77, -90.741064, 42.516633, '88275a212dfffff'), ('S03408300284', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54444, 'Paramount Apricot Flavored Brandy', 24, '375', 3.34, 5.0, 1, 5.0, 0.38, 0.1, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03389000007', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1022100, 'TEQUILA', 395, 'Proximo', 89193, 'Jose Cuervo Especial Reposado Tequila', 48, '200', 3.0, 4.49, 3, 13.47, 0.6, 0.16, -93.315977, 40.757158, '88267252cdfffff'), ('S03405700031', Timestamp('2012-01-03 00:00:00'), 3651, 'Wal-Mart 1491 / Indianola', '1500 NORTH JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45276, 'Paramount White Rum', 12, '750', 3.85, 5.77, 12, 69.24, 9.0, 2.38, 0.0, 0.0, '0'), ('S03390200017', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11773, 'Black Velvet', 48, '200', 1.56, 2.34, 1, 2.34, 0.2, 0.05, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03398200014', Timestamp('2012-01-03 00:00:00'), 2633, 'Hy-Vee #3 / BDI / Des Moines', '3221 SE 14TH ST', 'DES MOINES', '50320', 77, 'Polk', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 73533, 'Dekuyper 03', 12, '750', 12.07, 18.1, 2, 36.2, 1.5, 0.4, -93.596754, 41.554101, '88260d846dfffff'), ('S03407300003', Timestamp('2012-01-03 00:00:00'), 4029, 'The Store', '204 S MAIN ST', 'COLESBURG', '52035', 28, 'Delaware', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35917, "Five O'clock Vodka", 12, '1000', 4.01, 6.01, 12, 72.12, 12.0, 3.17, -91.368032, 42.958986, '88275a425bfffff'), ('S03396000001', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -90.705328, 42.491862, '88275a2131fffff'), ('S03396400060', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68037, "Bailey's Original Irish Cream", 12, '1000', 16.5, 24.74, 24, 593.76, 24.0, 6.34, -95.8792, 41.238092, '882608c9e7fffff'), ('S03408300031', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1012200, 'SCOTCH WHISKIES', 55, 'Sazerac North America', 8208, 'House Of Stuart', 6, '1750', 9.62, 14.43, 6, 86.58, 10.5, 2.77, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03406300033', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69638, "Dr. Mcgillicuddy's Cherry Schnapps", 6, '1750', 13.33, 19.99, 18, 359.82, 31.5, 8.32, -91.281344, 42.108275, '882674c6a9fffff'), ('S03402100045', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54444, 'Paramount Apricot Flavored Brandy', 24, '375', 3.34, 5.0, 12, 60.0, 4.5, 1.19, -92.475403, 42.712263, '8826291801fffff'), ('S03392300058', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26824, 'Jack Daniels Old #7 Black Lbl Square', 24, '375', 6.06, 9.09, 24, 218.16, 9.0, 2.38, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03388900011', Timestamp('2012-01-03 00:00:00'), 4540, "Casey's General Store #2915 / Bellev", '1104 N RIVERVIEW ST', 'BELLEVUE', '52031', 49, 'Jackson', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65254, 'Jagermeister Liqueur', 24, '375', 6.23, 9.34, 4, 37.36, 1.5, 0.4, -90.427844, 42.268432, '88275b5ac1fffff'), ('S03393100020', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35916, "Five O'clock Vodka", 12, '750', 3.31, 4.96, 12, 59.52, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03396600023', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76487, 'Paramount Amaretto', 12, '1000', 4.04, 6.06, 12, 72.72, 12.0, 3.17, -93.378772, 43.142868, '8826283b29fffff'), ('S03397300025', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1071100, 'AMERICAN COCKTAILS', 434, 'Luxco-St Louis', 63528, "Salvador's Margarita Pet", 6, '1750', 5.91, 8.86, 6, 53.16, 10.5, 2.77, -91.044008, 41.427202, '8826742f4bfffff'), ('S03384600021', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 67006, 'Midori Melon Liqueur', 12, '750', 10.97, 16.46, 12, 197.52, 9.0, 2.38, -93.619787, 41.60566, '88260d8769fffff'), ('S03406900012', Timestamp('2012-01-03 00:00:00'), 4114, 'After 5 Somewhere', '704 W 7TH ST PO BOX 372', 'ATLANTIC', '50022', 15, 'Cass', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82867, 'Dekuyper Watermelon Pucker', 12, '1000', 7.35, 11.02, 4, 44.08, 4.0, 1.06, -95.023116, 41.40406200000001, '88260890a3fffff'), ('S03404800043', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57144, "Chi-Chi's White Russian", 6, '1750', 5.99, 9.24, 6, 55.44, 10.5, 2.77, -91.035138, 41.451349, '88267421abfffff'), ('S03384700007', Timestamp('2012-01-03 00:00:00'), 4138, 'Fareway Stores #657 / Indianola', '1309 W 2ND', 'INDIANOLA', '50125', 91, 'Warren', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37996, 'Smirnoff Vodka 80 Prf', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -93.579252, 41.356616, '88260db8e9fffff'), ('S03390900081', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 4, 65.32, 3.0, 0.79, 0.0, 0.0, '0'), ('S03403900033', Timestamp('2012-01-03 00:00:00'), 2628, 'Hy-Vee Food Store / Eldora', '1616 EDGINGTON', 'ELDORA', '50627', 42, 'Hardin', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34820, 'Svedka Vodka', 12, '750', 7.97, 11.95, 3, 35.85, 2.25, 0.59, -93.103764, 42.360631, '88262ba25dfffff'), ('S03400100004', Timestamp('2012-01-03 00:00:00'), 4469, 'Sperry One Stop Shop LLC', '516 HWY 141', 'COON RAPIDS', '50058', 14, 'Carroll', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57148, "Chi-Chi's Margarita W/tequila", 6, '1750', 5.99, 9.24, 6, 55.44, 10.5, 2.77, -94.678374, 41.865913, '88260cad0dfffff'), ('S03383800057', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1062300, 'FLAVORED RUM', 65, 'Jim Beam Brands', 44520, 'Cruzan Vanilla Rum', 12, '750', 6.38, 9.57, 1, 9.57, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03382300009', Timestamp('2012-01-03 00:00:00'), 4049, 'No Frills Supermarkets / Glenwood', '423 SHARP ST', 'GLENWOOD', '51534', 65, 'Mills', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 30, 309.0, 52.5, 13.87, -95.742987, 41.04635, '882608555dfffff'), ('S03405900042', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1062100, 'JAMAICA RUM', 260, 'Diageo Americas', 42167, "Myers's Rum Original Dark", 12, '1000', 11.95, 17.93, 12, 215.16, 12.0, 3.17, -90.48065, 41.529655, '882675c361fffff'), ('S03410900153', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 65, 'Jim Beam Brands', 77395, 'Skinnygirl White Cranberry Cosmo', 12, '750', 7.69, 11.53, 4, 46.12, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03394500035', Timestamp('2012-01-03 00:00:00'), 3443, 'Super Saver Iv', '1141 N BROADWAY', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42718, 'Malibu Coconut Rum', 6, '1750', 14.9, 22.35, 6, 134.1, 10.5, 2.77, -95.836515, 41.270824, '882608c823fffff'), ('S03397300017', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 12408, 'Canadian Ltd Whisky', 6, '1750', 8.99, 13.49, 6, 80.94, 10.5, 2.77, -91.044008, 41.427202, '8826742f4bfffff'), ('S03387800088', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1052010, 'IMPORTED GRAPE BRANDIES', 389, 'REMY COINTREAU USA         .', 49189, 'Remy Martin Vsop Cognac', 24, '200', 5.41, 8.12, 1, 8.12, 0.2, 0.05, -91.387797, 40.400038, '8826702f29fffff'), ('S03395700021', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11298, 'Crown Royal Canadian Whisky', 6, '1750', 30.5, 45.74, 1, 45.74, 1.75, 0.46, -95.147741, 43.14521, '88262a1925fffff'), ('S03405400038', Timestamp('2012-01-03 00:00:00'), 3816, 'Swils', '200 E OAK ST', 'RED OAK', '51566', 69, 'Montgomery', 1032200, 'IMPORTED VODKA - MISC', 260, 'Diageo Americas', 34449, 'Ketel One Citroen', 12, '750', 13.0, 19.49, 2, 38.98, 1.5, 0.4, -95.230032, 41.016691, '882608152bfffff'), ('S03395800247', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1012300, 'IRISH WHISKIES', 240, 'WILLIAM GRANT AND SONS, INC.', 15940, 'Tullamore Dew Irish Whisky', 12, '750', 12.37, 18.56, 2, 37.12, 1.5, 0.4, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03392900071', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82867, 'Dekuyper Watermelon Pucker', 12, '1000', 7.35, 11.02, 3, 33.06, 3.0, 0.79, 0.0, 0.0, '0'), ('S03390200030', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69945, 'Rumple Minze Peppermint Schnapps Liqueur', 12, '375', 6.5, 9.74, 3, 29.22, 1.12, 0.3, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03384500010', Timestamp('2012-01-03 00:00:00'), 3482, 'Kick The Can', '525 N MAPLE ST', 'CARROLL', '51401', 14, 'Carroll', 1011200, 'STRAIGHT BOURBON WHISKIES', 421, 'Sazerac Co., Inc.', 16516, 'Ancient Age Bourbon', 12, '750', 6.0, 8.99, 2, 17.98, 1.5, 0.4, -94.861667, 42.063255, '88260c16a9fffff'), ('S03385300041', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58838, 'Jose Cuervo Authentic Lime Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, -91.136655, 40.814666, '882670aacdfffff'), ('S03387800084', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1081380, 'MISCELLANEOUS SCHNAPPS', 65, 'Jim Beam Brands', 82627, 'Dekuyper Cherry Pucker', 12, '1000', 7.35, 11.02, 3, 33.06, 3.0, 0.79, -91.387797, 40.400038, '8826702f29fffff'), ('S03393900073', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53213, 'Paul Masson Grande Amber Brandy', 24, '200', 1.77, 2.66, 4, 10.64, 0.8, 0.21, -95.873073, 41.261866, '882608c917fffff'), ('S03408000022', Timestamp('2012-01-03 00:00:00'), 3622, 'Wal-Mart 1415 / Spirit Lake', '2600 HWY 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41704, 'Uv Red (cherry) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -95.126535, 43.416452, '88262ac237fffff'), ('S03410900073', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19226, 'Knob Creek', 6, '750', 19.49, 29.24, 2, 58.48, 1.5, 0.4, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03395200089', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24458, 'Kessler Blend Whiskey', 6, '1750', 11.01, 16.52, 3, 49.56, 5.25, 1.39, -91.110752, 40.801889, '882670a137fffff'), ('S03384600024', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34029, 'Absolut Citron (lemon Vodka)', 12, '1000', 14.89, 22.34, 12, 268.08, 12.0, 3.17, -93.619787, 41.60566, '88260d8769fffff'), ('S03401000064', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37426, 'Popov Vodka 80 Prf Traveler', 12, '750', 4.1, 6.15, 1, 6.15, 0.75, 0.2, -90.683356, 42.52717, '88275a215dfffff'), ('S03408400050', Timestamp('2012-01-03 00:00:00'), 2650, 'Hy-Vee Wine and Spirits / Harlan', '1808 23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 260, 'Diageo Americas', 66636, 'Romana Sambuca Italian Liquore', 12, '750', 12.5, 18.74, 3, 56.22, 2.25, 0.59, -95.339881, 41.650658, '88260c60e9fffff'), ('S03401400005', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -95.126786, 43.416433, '88262ac237fffff'), ('S03383800038', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 301, 'Fifth Generation Inc.', 38176, 'Titos Vodka', 12, '750', 9.64, 14.46, 1, 14.46, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03397600047', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37336, 'Uv Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03389600016', Timestamp('2012-01-03 00:00:00'), 3632, 'Wal-Mart 2004 / Dubuque', '4200 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42718, 'Malibu Coconut Rum', 6, '1750', 14.9, 22.35, 6, 134.1, 10.5, 2.77, -90.736955, 42.489041, '88275a2ac1fffff'), ('S03396400037', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15247, 'Windsor Canadian', 12, '1000', 6.55, 9.83, 12, 117.96, 12.0, 3.17, -95.8792, 41.238092, '882608c9e7fffff'), ('S03391900014', Timestamp('2012-01-03 00:00:00'), 4113, 'Tobacco Hut #6 / Council', '3134 MANAWA CENTRE DR', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64867, 'Fireball Cinnamon Whiskey', 12, '1000', 11.0, 16.5, 12, 198.0, 12.0, 3.17, -95.847514, 41.228712, '882608c983fffff'), ('S03411500009', Timestamp('2012-01-03 00:00:00'), 3700, 'Avoca Liquor', '158 SOUTH ELM', 'AVOCA', '51521', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37417, 'Popov Vodka 80 Prf', 12, '1000', 5.0, 7.49, 12, 89.88, 12.0, 3.17, -95.339412, 41.476306, '88260c6c8bfffff'), ('S03394800006', Timestamp('2012-01-03 00:00:00'), 3880, 'Target Store T-2454 / Council Bluffs', '3804 METRO DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42716, 'Malibu Coconut Rum', 12, '750', 6.9, 10.35, 12, 124.2, 9.0, 2.38, -95.837595, 41.220979, '882608c995fffff'), ('S03402900016', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1101100, 'AMERICAN ALCOHOL', 434, 'Luxco-St Louis', 41846, 'Everclear Alcohol', 12, '750', 8.02, 12.03, 12, 144.36, 9.0, 2.38, -94.990031, 41.403856, '8826089095fffff'), ('S03410300121', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 1, 15.0, 0.75, 0.2, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03412100167', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1012200, 'SCOTCH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 4716, 'Chivas Regal 12 Yr', 12, '750', 17.7, 26.55, 2, 53.1, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03401100013', Timestamp('2012-01-03 00:00:00'), 4239, 'Fareway Stores #850 / Spirit Lake', '1906 KEOKUK', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031080, 'VODKA 80 PROOF', 259, 'Heaven Hill Brands', 35418, "Burnett's Vodka 80 Prf", 6, '1750', 8.43, 13.15, 6, 78.9, 10.5, 2.77, -95.106219, 43.42157, '88262ac23bfffff'), ('S03382700023', Timestamp('2012-01-03 00:00:00'), 2607, 'Hy-Vee Wine and Spirits / Shenandoah', '520 SO FREMONT', 'SHENANDOAH', '51601', 73, 'Page', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24458, 'Kessler Blend Whiskey', 6, '1750', 11.01, 16.52, 6, 99.12, 10.5, 2.77, -95.385111, 40.761736, '8826080d95fffff'), ('S03402100090', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1011200, 'STRAIGHT BOURBON WHISKIES', 461, 'Campari(skyy)', 22151, 'Wild Turkey 101 Mini', 12, '500', 8.52, 12.79, 1, 12.79, 0.5, 0.13, -92.475403, 42.712263, '8826291801fffff'), ('S03403300010', Timestamp('2012-01-03 00:00:00'), 4566, 'Kum & Go #508 / Cedar Rapids', '30 FOUR OAKS DR SW', 'CEDAR RAPIDS', '52404', 57, 'Linn', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37991, 'Smirnoff Vodka 80 Prf Mini', 12, '500', 6.97, 10.45, 2, 20.9, 1.0, 0.26, -91.658073, 41.923446, '8826745503fffff'), ('S03377200004', Timestamp('2012-01-03 00:00:00'), 4438, 'Kum & Go #214  Ames', '111 DUFF AVE', 'AMES', '50010', 85, 'Story', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52596, 'E & J Vs', 12, '750', 5.51, 8.26, 24, 198.24, 18.0, 4.76, -93.610396, 42.023592, '88262b38adfffff'), ('S03398200010', Timestamp('2012-01-03 00:00:00'), 2633, 'Hy-Vee #3 / BDI / Des Moines', '3221 SE 14TH ST', 'DES MOINES', '50320', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 434, 'Luxco-St Louis', 34561, 'Pearl Coconut Vodka', 12, '750', 7.85, 11.78, 1, 11.78, 0.75, 0.2, -93.596754, 41.554101, '88260d846dfffff'), ('S03392800004', Timestamp('2012-01-03 00:00:00'), 4258, 'Fareway Stores #009 / Burlington', '920 LAWRENCE DR', 'BURLINGTON', '52601', 29, 'Des Moines', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35916, "Five O'clock Vodka", 12, '750', 3.31, 4.96, 12, 59.52, 9.0, 2.38, -91.140051, 40.799097, '882670aac7fffff'), ('S03406900028', Timestamp('2012-01-03 00:00:00'), 4114, 'After 5 Somewhere', '704 W 7TH ST PO BOX 372', 'ATLANTIC', '50022', 15, 'Cass', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68036, "Bailey's Original Irish Cream", 12, '750', 12.78, 19.18, 4, 76.72, 3.0, 0.79, -95.023116, 41.40406200000001, '88260890a3fffff'), ('S03408800062', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35918, "Five O'clock Vodka", 6, '1750', 6.92, 10.38, 2, 20.76, 3.5, 0.92, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03397500062', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1012100, 'CANADIAN WHISKIES', 434, 'Luxco-St Louis', 13036, 'Canadian Reserve Whisky', 12, '750', 4.25, 6.38, 12, 76.56, 9.0, 2.38, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03410300023', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11297, 'Crown Royal Canadian Whisky', 12, '1000', 17.46, 26.19, 2, 52.38, 2.0, 0.53, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03383600017', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69637, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '1000', 11.0, 16.5, 24, 396.0, 24.0, 6.34, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03390800014', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35946, "Fleischmann's Royal Vodka", 12, '750', 3.44, 5.15, 12, 61.8, 9.0, 2.38, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03411200151', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 115, 'Constellation Wine Company, Inc.', 34839, 'Svedka Citron Vodka', 12, '750', 7.97, 11.95, 2, 23.9, 1.5, 0.4, -93.697731, 41.665172, '88260d8231fffff'), ('S03397500001', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 389, 'REMY COINTREAU USA         .', 64776, 'Cointreau Liqueur', 12, '750', 23.31, 34.96, 12, 419.52, 9.0, 2.38, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03388000015', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15248, 'Windsor Canadian Pet', 6, '1750', 8.92, 13.38, 18, 240.84, 31.5, 8.32, -93.633306, 43.261538, '88262808e7fffff'), ('S03410900030', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58835, 'Jose Cuervo Authentic Lime Margarita', 6, '800', 4.62, 6.92, 6, 41.52, 4.8, 1.27, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03410900041', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86886, 'Southern Comfort', 12, '750', 9.36, 14.04, 12, 168.48, 9.0, 2.38, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03410900005', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15248, 'Windsor Canadian Pet', 6, '1750', 8.92, 13.38, 6, 80.28, 10.5, 2.77, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03411200033', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19064, 'Jim Beam', 24, '375', 4.53, 6.8, 1, 6.8, 0.38, 0.1, -93.697731, 41.665172, '88260d8231fffff'), ('S03383400026', Timestamp('2012-01-03 00:00:00'), 3488, "Bender's Foods", '400 S SECOND', 'BELLEVUE', '52031', 49, 'Jackson', 1062300, 'FLAVORED RUM', 65, 'Jim Beam Brands', 44499, 'Cruzan Mango Rum', 12, '750', 6.38, 9.57, 4, 38.28, 3.0, 0.79, -90.42292, 42.255449, '88275b5ad5fffff'), ('S03409900007', Timestamp('2012-01-03 00:00:00'), 4387, 'KUM & GO #117 / SPIRIT LAKE', '300 18TH ST', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69636, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -95.094324, 43.422645, '88262ac217fffff'), ('S03404500019', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75208, 'Kinky Liqueur Mini', 6, '500', 4.67, 7.0, 3, 21.0, 1.5, 0.4, -95.83539, 41.279189, '882608c829fffff'), ('S03398600004', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69611, "Dr. Mcgillicuddy's Apple Pie Schnapps", 12, '750', 8.32, 12.48, 12, 149.76, 9.0, 2.38, -91.343544, 40.626988, '8826703131fffff'), ('S03403200044', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1081200, 'CREAM LIQUEURS', 330, 'Gemini Spirits', 80457, "Ryan's Cream Liqueur", 12, '1000', 7.22, 10.83, 1, 10.83, 1.0, 0.26, -94.218, 43.29355, '8826284833fffff'), ('S03407400044', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88296, 'Patron Tequila Silver', 12, '750', 26.08, 39.13, 3, 117.39, 2.25, 0.59, -93.736517, 41.659057, '88260d8045fffff'), ('S03403000017', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 4, 98.56, 4.0, 1.06, -94.480204, 43.39966, '88262a82cbfffff'), ('S03406400017', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1041100, 'AMERICAN DRY GINS', 297, 'Laird And Company', 29997, "Five O'clock Gin", 12, '1000', 4.06, 6.09, 1, 6.09, 1.0, 0.26, -90.766126, 41.509119, '88267434edfffff'), ('S03397600045', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03412100090', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24458, 'Kessler Blend Whiskey', 6, '1750', 11.01, 16.52, 24, 396.48, 42.0, 11.1, -90.483701, 41.552338, '882675da95fffff'), ('S03399000003', Timestamp('2012-01-03 00:00:00'), 3762, 'Wine and Spirits Gallery', '7690 HICKMAN RD', 'WINDSOR HEIGHTS', '50322', 77, 'Polk', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26821, 'Jack Daniels Old #7 Black Lbl Mini', 12, '500', 8.62, 12.94, 1, 12.94, 0.5, 0.13, 0.0, 0.0, '0'), ('S03405200030', Timestamp('2012-01-03 00:00:00'), 2954, "Dahl's / Ingersoll", '3425 INGERSOLL', 'DES MOINES', '50300', 77, 'Polk', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31656, 'Paramount Gin', 12, '750', 3.39, 5.09, 12, 61.08, 9.0, 2.38, -93.663337, 41.586313, '88260d80d7fffff'), ('S03404800114', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 67006, 'Midori Melon Liqueur', 12, '750', 10.97, 16.46, 2, 32.92, 1.5, 0.4, -91.035138, 41.451349, '88267421abfffff'), ('S03371500005', Timestamp('2012-01-03 00:00:00'), 4572, 'Grundy Center Family Foods', '1400, G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11771, 'Black Velvet Mini', 10, '600', 4.96, 7.44, 1, 7.44, 0.6, 0.16, -92.782283, 42.36157, '8826297835fffff'), ('S03392100056', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36904, 'Mccormick Vodka Pet', 24, '375', 1.76, 2.63, 6, 15.78, 2.25, 0.59, -90.204748, 41.827845, '88275b7891fffff'), ('S03385900033', Timestamp('2012-01-03 00:00:00'), 4356, 'Kum & Go #396 / Nevada', '136 LINCOLN HWY', 'NEVADA', '50201', 85, 'Story', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11771, 'Black Velvet Mini', 10, '600', 4.96, 7.44, 1, 7.44, 0.6, 0.16, -93.458003, 42.022725, '88262b235bfffff'), ('S03406700013', Timestamp('2012-01-03 00:00:00'), 3849, 'Hartig Drug Co #6 / Dyersville', '711  16TH AVE SE', 'DYERSVILLE', '52040', 31, 'Dubuque', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69638, "Dr. Mcgillicuddy's Cherry Schnapps", 6, '1750', 13.33, 19.99, 2, 39.98, 3.5, 0.92, -91.116087, 42.470285, '88275a654dfffff'), ('S03402500026', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 4, 37.32, 3.0, 0.79, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03382700015', Timestamp('2012-01-03 00:00:00'), 2607, 'Hy-Vee Wine and Spirits / Shenandoah', '520 SO FREMONT', 'SHENANDOAH', '51601', 73, 'Page', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37426, 'Popov Vodka 80 Prf Traveler', 12, '750', 4.1, 6.15, 12, 73.8, 9.0, 2.38, -95.385111, 40.761736, '8826080d95fffff'), ('S03385500021', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42717, 'Malibu Coconut Rum', 12, '1000', 9.77, 14.66, 12, 175.92, 12.0, 3.17, -93.21693, 42.741728, '88262b9605fffff'), ('S03396400021', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42717, 'Malibu Coconut Rum', 12, '1000', 9.77, 14.66, 24, 351.84, 24.0, 6.34, -95.8792, 41.238092, '882608c9e7fffff'), ('S03397000034', Timestamp('2012-01-03 00:00:00'), 3994, 'Great Spirits Wine & Liquor Co.', '72 NORTH HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11366, 'Crown Royal Special Reserve', 12, '750', 24.3, 36.44, 2, 72.88, 1.5, 0.4, -95.127395, 43.368989000000006, '88262ac0e9fffff'), ('S03395800171', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15677, 'Powers Irish Whiskey', 12, '1000', 11.24, 16.86, 2, 33.72, 2.0, 0.53, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03385300088', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 42002, 'Uv Cake Vodka', 6, '500', 29.3, 43.95, 2, 87.9, 1.0, 0.26, -91.136655, 40.814666, '882670aacdfffff'), ('S03395000004', Timestamp('2012-01-03 00:00:00'), 4165, 'I-35 Spirits / Ankeny', '113 SE DELAWARE AVE STE 101', 'ANKENY', '50023', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37997, 'Smirnoff Vodka 80 Prf', 12, '1000', 8.49, 12.74, 3, 38.22, 3.0, 0.79, -93.581093, 41.731416, '88260d9705fffff'), ('S03397600112', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43386, 'Castillo Silver Rum', 12, '750', 4.16, 6.24, 12, 74.88, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03408200018', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 330, 'Gemini Spirits', 62420, 'Margaritaville Skinny Margarita', 6, '1750', 7.01, 10.52, 6, 63.12, 10.5, 2.77, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03383700013', Timestamp('2012-01-03 00:00:00'), 3934, 'Wal-Mart 1509 / Maquoketa', '103 E CARLISLE', 'MAQUOKETA', '52060', 49, 'Jackson', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 6, 152.88, 10.5, 2.77, -90.66292, 42.04869, '8826748693fffff'), ('S03412100025', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86812, 'Southern Comfort Fiery Pepper', 12, '750', 9.36, 14.04, 12, 168.48, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03395500033', Timestamp('2012-01-03 00:00:00'), 3976, 'North English Family Food', '119 NORTH MAIN ST', 'NORTH ENGLISH', '52316', 48, 'Iowa', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68036, "Bailey's Original Irish Cream", 12, '750', 12.78, 19.18, 1, 19.18, 0.75, 0.2, -92.076452, 41.515209, '882676a0adfffff'), ('S03396400074', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 2, 32.66, 1.5, 0.4, -95.8792, 41.238092, '882608c9e7fffff'), ('S03387800033', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1081010, 'AMERICAN AMARETTO', 421, 'Sazerac Co., Inc.', 71886, 'Amaretto E Dolce', 12, '750', 3.42, 5.13, 12, 61.56, 9.0, 2.38, -91.387797, 40.400038, '8826702f29fffff'), ('S03411000048', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19067, 'Jim Beam', 12, '1000', 11.54, 17.31, 24, 415.44, 24.0, 6.34, 0.0, 0.0, '0'), ('S03387800040', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82867, 'Dekuyper Watermelon Pucker', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -91.387797, 40.400038, '8826702f29fffff'), ('S03410700102', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10624, 'Canadian Club Whisky', 24, '375', 4.05, 6.07, 1, 6.07, 0.38, 0.1, -94.853591, 42.064155, '88260c16a9fffff'), ('S03407600003', Timestamp('2012-01-03 00:00:00'), 3465, 'Hometown Foods / Panora', '601 E MAIN', 'PANORA', '50216', 39, 'Guthrie', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31658, 'Paramount Gin', 6, '1750', 7.1, 10.65, 6, 63.9, 10.5, 2.77, -94.357201, 41.69182, '88260dd89bfffff'), ('S03404800122', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1022100, 'TEQUILA', 395, 'Proximo', 87408, 'Jose Cuervo Especial Silver', 12, '750', 10.49, 15.73, 6, 94.38, 4.5, 1.19, -91.035138, 41.451349, '88267421abfffff'), ('S03408100004', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1022100, 'TEQUILA', 85, 'Brown-Forman Corporation', 89278, 'El Jimador Reposado Tequila', 12, '750', 10.63, 15.95, 12, 191.4, 9.0, 2.38, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03396400078', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82606, 'Dekuyper Sour Apple Pucker', 12, '750', 6.3, 9.45, 1, 9.45, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03371300009', Timestamp('2012-01-03 00:00:00'), 4443, 'Palo Mini Mart', '1204 1ST ST', 'PALO', '52324', 57, 'Linn', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81206, 'Paramount Peppermint Schnapps', 12, '750', 3.7, 5.55, 12, 66.6, 9.0, 2.38, -91.795808, 42.070316, '8826745323fffff'), ('S03411000112', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37991, 'Smirnoff Vodka 80 Prf Mini', 12, '500', 6.97, 10.45, 1, 10.45, 0.5, 0.13, 0.0, 0.0, '0'), ('S03381400003', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28233, 'Bombay Sapphire Gin', 12, '1000', 17.61, 26.41, 72, 1901.52, 72.0, 19.02, -90.527081, 41.559731, '882675daa5fffff'), ('S03395800128', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68037, "Bailey's Original Irish Cream", 12, '1000', 16.5, 24.74, 12, 296.88, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03397500032', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41694, 'Uv Blue (raspberry) Vodka', 12, '1000', 7.25, 10.87, 12, 130.44, 12.0, 3.17, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03397500108', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 65065, 'Goldschlager Cinnamon Schnapps', 12, '375', 6.5, 9.74, 2, 19.48, 0.75, 0.2, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03402900015', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36305, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 12, 58.56, 9.0, 2.38, -94.990031, 41.403856, '8826089095fffff'), ('S03389300013', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43285, 'Captain Morgan Original Spiced Rum Pet', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -90.609747, 41.589369, '882675d959fffff'), ('S03397500097', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1081200, 'CREAM LIQUEURS', 300, 'Mccormick Distilling Company', 65204, 'Tequila Rose Liqueur Mini', 6, '500', 6.63, 9.95, 1, 9.95, 0.5, 0.13, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03394400011', Timestamp('2012-01-03 00:00:00'), 4334, 'Fareway Stores #114 / Dubuque', '2050 JOHN F KENNEDY RD', 'DUBUQUE', '52001', 31, 'Dubuque', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10006, 'Scoresby Rare Scotch', 12, '750', 5.13, 7.69, 3, 23.07, 2.25, 0.59, 0.0, 0.0, '0'), ('S03383700016', Timestamp('2012-01-03 00:00:00'), 3934, 'Wal-Mart 1509 / Maquoketa', '103 E CARLISLE', 'MAQUOKETA', '52060', 49, 'Jackson', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82846, 'Dekuyper Luscious Peachtree Schnapps', 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -90.66292, 42.04869, '8826748693fffff'), ('S03338000004', Timestamp('2012-01-03 00:00:00'), 4590, "Casey's General Store #3045 / Cedar", '601, MAIN ST', 'CEDAR FALLS', '50613', 7, 'Black Hawk', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36306, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 12, 58.56, 9.0, 2.38, -92.445512, 42.532963, '8826290ab3fffff'), ('S03403000026', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11777, 'Black Velvet', 12, '1000', 6.63, 9.94, 3, 29.82, 3.0, 0.79, -94.480204, 43.39966, '88262a82cbfffff'), ('S03390400029', Timestamp('2012-01-03 00:00:00'), 3738, 'Hartig Drug Company #4 / Dubuque', '2225 CENTRAL AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 395, 'Proximo', 34935, 'Three Olives Vodka', 12, '750', 11.16, 16.74, 2, 33.48, 1.5, 0.4, -90.671531, 42.51384, '88275a2153fffff'), ('S03397600076', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67527, 'Kahlua Coffee Liqueur', 12, '1000', 14.81, 22.22, 12, 266.64, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03398800031', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5346, 'Johnnie Walker Red', 12, '750', 13.78, 20.67, 2, 41.34, 1.5, 0.4, -93.573845, 41.604312, '88260d95b3fffff'), ('S03408800055', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1081230, 'WHITE CREME DE CACAO', 434, 'Luxco-St Louis', 78866, 'Paramount Creme De Cacao/white', 12, '750', 4.44, 6.66, 3, 19.98, 2.25, 0.59, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03397500076', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1011300, 'TENNESSEE WHISKIES', 260, 'Diageo Americas', 26656, 'George Dickel #12', 12, '750', 12.23, 18.35, 4, 73.4, 3.0, 0.79, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03391300091', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1011200, 'STRAIGHT BOURBON WHISKIES', 421, 'Sazerac Co., Inc.', 16517, 'Ancient Age Bourbon', 12, '1000', 7.5, 11.25, 3, 33.75, 3.0, 0.79, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03412100157', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26586, 'Gentleman Jack', 12, '750', 18.44, 27.66, 3, 82.98, 2.25, 0.59, -90.483701, 41.552338, '882675da95fffff'), ('S03393400102', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1062310, 'SPICED RUM', 322, 'Prestige Wine and Spirits Group', 77318, 'Rondiaz Rum Spiced', 6, '1750', 10.0, 15.0, 6, 90.0, 10.5, 2.77, -90.741064, 42.516633, '88275a212dfffff'), ('S03371300015', Timestamp('2012-01-03 00:00:00'), 4443, 'Palo Mini Mart', '1204 1ST ST', 'PALO', '52324', 57, 'Linn', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55087, 'Paramount Blackberry Brandy', 12, '1000', 6.21, 9.32, 12, 111.84, 12.0, 3.17, -91.795808, 42.070316, '8826745323fffff'), ('S03385700004', Timestamp('2012-01-03 00:00:00'), 2238, 'Adventureland Inn', '3200 Adventureland Dr', 'ALTOONA', '50009', 77, 'Polk', 1041100, 'AMERICAN DRY GINS', 380, 'Phillips Beverage Company', 31719, 'Phillips Gin', 12, '1000', 4.07, 6.1, 2, 12.2, 2.0, 0.53, -93.49924000000001, 41.658513, '88260d94cdfffff'), ('S03392300073', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19061, 'Jim Beam Mini', 12, '500', 6.83, 10.24, 12, 122.88, 6.0, 1.59, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03391300060', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41715, 'Smirnoff Cranberry Vodka', 12, '750', 7.59, 11.56, 1, 11.56, 0.75, 0.2, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03401600025', Timestamp('2012-01-03 00:00:00'), 2522, 'Hy-Vee Wine and Spirits / Spirit Lak', 'HWY 9 & 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43387, 'Castillo Silver Rum', 12, '1000', 4.89, 7.33, 12, 87.96, 12.0, 3.17, 0.0, 0.0, '0'), ('S03397500046', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11297, 'Crown Royal Canadian Whisky', 12, '1000', 17.46, 26.19, 24, 628.56, 24.0, 6.34, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03392500040', Timestamp('2012-01-03 00:00:00'), 3908, 'County Market #214 / Fort Madison', '11802 AVENUE H', 'FORT MADISON', '52627', 56, 'Lee', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26828, 'Jack Daniels Old #7 Black Lbl', 6, '1750', 26.67, 40.01, 6, 240.06, 10.5, 2.77, 0.0, 0.0, '0'), ('S03404400016', Timestamp('2012-01-03 00:00:00'), 3016, "Super Valu / bender's", '200 SOUTH STATE ST', 'DENVER', '50622', 9, 'Bremer', 1031080, 'VODKA 80 PROOF', 322, 'Prestige Wine and Spirits Group', 37146, 'Opulent Vodka', 6, '750', 10.03, 15.05, 3, 45.15, 2.25, 0.59, -92.33724, 42.672008, '882629034bfffff'), ('S03385800046', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 25878, 'Sunny Brook Blend Whiskey', 6, '1750', 7.87, 11.8, 2, 23.6, 3.5, 0.92, -92.258152, 42.809882, '8826291267fffff'), ('S03382700013', Timestamp('2012-01-03 00:00:00'), 2607, 'Hy-Vee Wine and Spirits / Shenandoah', '520 SO FREMONT', 'SHENANDOAH', '51601', 73, 'Page', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36886, 'Mccormick Vodka', 12, '750', 3.3, 4.96, 12, 59.52, 9.0, 2.38, -95.385111, 40.761736, '8826080d95fffff'), ('S03395700067', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75208, 'Kinky Liqueur Mini', 6, '500', 4.67, 7.0, 3, 21.0, 1.5, 0.4, -95.147741, 43.14521, '88262a1925fffff'), ('S03408600091', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28233, 'Bombay Sapphire Gin', 12, '1000', 17.61, 26.41, 1, 26.41, 1.0, 0.26, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03400300017', Timestamp('2012-01-03 00:00:00'), 3461, 'Hometown Foods / Stuart', '611 S DIVISION ST', 'STUART', '50250', 1, 'Adair', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32236, 'Seagrams Extra Dry Gin', 12, '750', 5.9, 8.85, 2, 17.7, 1.5, 0.4, -94.318443, 41.49759900000001, '88260dcc2bfffff'), ('S03395800222', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081380, 'MISCELLANEOUS SCHNAPPS', 55, 'Sazerac North America', 84170, '99 Bananas', 12, '750', 9.5, 14.25, 3, 42.75, 2.25, 0.59, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03407900058', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42717, 'Malibu Coconut Rum', 12, '1000', 9.77, 14.66, 12, 175.92, 12.0, 3.17, 0.0, 0.0, '0'), ('S03392100022', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11296, 'Crown Royal', 12, '750', 14.25, 21.37, 1, 21.37, 0.75, 0.2, -90.204748, 41.827845, '88275b7891fffff'), ('S03401600093', Timestamp('2012-01-03 00:00:00'), 2522, 'Hy-Vee Wine and Spirits / Spirit Lak', 'HWY 9 & 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88291, 'Patron Tequila Silver Mini', 10, '300', 20.3, 30.45, 1, 30.45, 0.3, 0.08, 0.0, 0.0, '0'), ('S03387200021', Timestamp('2012-01-03 00:00:00'), 4296, 'Circle B Market', '111 MAIN ST', 'KEOSAUQUA', '52565', 89, 'Van Buren', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34433, 'Grey Goose Vodka', 12, '750', 17.97, 26.96, 2, 53.92, 1.5, 0.4, -91.962466, 40.7294, '882670458bfffff'), ('S03397800006', Timestamp('2012-01-03 00:00:00'), 4421, 'KUM & GO #1 / HAMPTON', '609 CENTRAL AVE W', 'HAMPTON', '50441', 35, 'Franklin', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82606, 'Dekuyper Sour Apple Pucker', 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -93.21508, 42.741741, '88262b9605fffff'), ('S03405500010', Timestamp('2012-01-03 00:00:00'), 4203, 'Liquor and Tobacco Outlet /', '620 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1062300, 'FLAVORED RUM', 260, 'Diageo Americas', 43415, 'Captain Morgan Lime Bite', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -92.47898, 42.725694, '8826291847fffff'), ('S03409600006', Timestamp('2012-01-03 00:00:00'), 4313, 'Fareway Stores #940 / Atlantic', '301 POPLAR ST', 'ATLANTIC', '50022', 15, 'Cass', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76486, 'Paramount Amaretto', 12, '750', 3.37, 5.06, 12, 60.72, 9.0, 2.38, -95.014379, 41.407927, '88260890bdfffff'), ('S03402100046', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55084, 'Paramount Blackberry Brandy', 24, '375', 3.34, 5.0, 12, 60.0, 4.5, 1.19, -92.475403, 42.712263, '8826291801fffff'), ('S03382300011', Timestamp('2012-01-03 00:00:00'), 4049, 'No Frills Supermarkets / Glenwood', '423 SHARP ST', 'GLENWOOD', '51534', 65, 'Mills', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 12408, 'Canadian Ltd Whisky', 6, '1750', 8.99, 13.49, 18, 242.82, 31.5, 8.32, -95.742987, 41.04635, '882608555dfffff'), ('S03410300141', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34421, 'Grey Goose Vodka Mini', 10, '600', 20.0, 30.0, 1, 30.0, 0.6, 0.16, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03388100070', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1071100, 'AMERICAN COCKTAILS', 370, 'Pernod Ricard USA/Austin Nichols', 62061, 'Kahlua Frozen Mudslide', 6, '1750', 6.46, 10.19, 6, 61.14, 10.5, 2.77, -95.879662, 41.261925, '882608c93bfffff'), ('S03388200019', Timestamp('2012-01-03 00:00:00'), 4106, 'Hartig Drug Co. / JFK Rd / Dubuque', '2255 JOHN F KENNEDY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45278, 'Paramount White Rum', 6, '1750', 7.58, 11.37, 3, 34.11, 5.25, 1.39, -90.72051800000001, 42.511352, '88275a212bfffff'), ('S03410900082', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 461, 'Campari(skyy)', 37986, 'Skyy Vodka', 12, '750', 10.01, 15.01, 4, 60.04, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03410900046', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35948, "Fleischmann's Royal Vodka 80 Prf", 6, '1750', 7.14, 10.7, 48, 513.6, 84.0, 22.19, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03407100012', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 1, 16.33, 0.75, 0.2, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03404500030', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1062310, 'SPICED RUM', 395, 'Proximo', 46504, 'Kraken Black Spiced Rum', 12, '750', 10.0, 14.99, 3, 44.97, 2.25, 0.59, -95.83539, 41.279189, '882608c829fffff'), ('S03392100032', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1011200, 'STRAIGHT BOURBON WHISKIES', 461, 'Campari(skyy)', 77776, 'Wild Turkey American Honey', 12, '750', 10.5, 15.75, 1, 15.75, 0.75, 0.2, -90.204748, 41.827845, '88275b7891fffff'), ('S03384000034', Timestamp('2012-01-03 00:00:00'), 2327, 'West Side Spirits', '801 HULL ST', 'CORNING', '50841', 2, 'Adams', 1081220, 'DARK CREME DE CACAO', 434, 'Luxco-St Louis', 78456, 'Paramount Creme De Cacao/dark', 12, '750', 4.44, 6.66, 3, 19.98, 2.25, 0.59, -94.74369200000001, 40.990378, '882608a233fffff'), ('S03387400014', Timestamp('2012-01-03 00:00:00'), 3642, "Fraise Foods / Sonny's Supermarket", '327  5TH ST', 'WEST POINT', '52656', 56, 'Lee', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 12, 305.76, 21.0, 5.55, -91.452491, 40.717867, '88267005ebfffff'), ('S03404500014', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82787, 'Dekuyper Buttershots', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -95.83539, 41.279189, '882608c829fffff'), ('S03402100048', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1012200, 'SCOTCH WHISKIES', 305, 'MHW Ltd', 4797, 'Cutty Sark Scotch', 12, '1000', 15.16, 22.74, 2, 45.48, 2.0, 0.53, -92.475403, 42.712263, '8826291801fffff'), ('S03411200067', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82613, 'Dekuyper Hot Damn!', 24, '375', 3.06, 4.59, 1, 4.59, 0.38, 0.1, -93.697731, 41.665172, '88260d8231fffff'), ('S03385800002', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25607, 'Seagrams 7 Crown Bl Whiskey', 12, '1000', 7.5, 11.24, 12, 134.88, 12.0, 3.17, -92.258152, 42.809882, '8826291267fffff'), ('S03403300005', Timestamp('2012-01-03 00:00:00'), 4566, 'Kum & Go #508 / Cedar Rapids', '30 FOUR OAKS DR SW', 'CEDAR RAPIDS', '52404', 57, 'Linn', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86886, 'Southern Comfort', 12, '750', 9.36, 14.04, 12, 168.48, 9.0, 2.38, -91.658073, 41.923446, '8826745503fffff'), ('S03395800216', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81204, 'Paramount Peppermint Schnapps', 24, '375', 2.27, 3.41, 6, 20.46, 2.25, 0.59, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03402700008', Timestamp('2012-01-03 00:00:00'), 3907, "Smokin' Joe's #16 Tobacco and Liquor", '1504 PARK AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82613, 'Dekuyper Hot Damn!', 24, '375', 3.06, 4.59, 1, 4.59, 0.38, 0.1, -91.030569, 41.442458, '88267421bdfffff'), ('S03405200018', Timestamp('2012-01-03 00:00:00'), 2954, "Dahl's / Ingersoll", '3425 INGERSOLL', 'DES MOINES', '50300', 77, 'Polk', 1042100, 'IMPORTED DRY GINS', 260, 'Diageo Americas', 28866, 'Tanqueray Gin', 12, '750', 12.0, 17.99, 12, 215.88, 9.0, 2.38, -93.663337, 41.586313, '88260d80d7fffff'), ('S03382700012', Timestamp('2012-01-03 00:00:00'), 2607, 'Hy-Vee Wine and Spirits / Shenandoah', '520 SO FREMONT', 'SHENANDOAH', '51601', 73, 'Page', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43335, 'Captain Morgan Spiced Rum Pet', 6, '1750', 16.99, 25.48, 6, 152.88, 10.5, 2.77, -95.385111, 40.761736, '8826080d95fffff'), ('S03405900035', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1032200, 'IMPORTED VODKA - MISC', 277, 'Stoli Group', 34881, 'Stolichnaya Vanil Vodka 75 Proof', 12, '1000', 16.18, 24.27, 12, 291.24, 12.0, 3.17, -90.48065, 41.529655, '882675c361fffff'), ('S03401000008', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43031, 'Bacardi Gold Rum Mini', 12, '500', 4.74, 7.11, 1, 7.11, 0.5, 0.13, -90.683356, 42.52717, '88275a215dfffff'), ('S03388100115', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86881, 'Southern Comfort Mini', 12, '500', 4.7, 7.06, 2, 14.12, 1.0, 0.26, -95.879662, 41.261925, '882608c93bfffff'), ('S03389900031', Timestamp('2012-01-03 00:00:00'), 2532, 'Hy-Vee Food Store #2 / Des Moines', '2540 E EUCLID', 'DES MOINES', '50317', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65194, 'Hpnotiq', 12, '375', 6.35, 9.52, 12, 114.24, 4.5, 1.19, -93.569452, 41.627838, '88260d9581fffff'), ('S03409700088', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 330, 'Gemini Spirits', 62420, 'Margaritaville Skinny Margarita', 6, '1750', 7.01, 10.52, 2, 21.04, 3.5, 0.92, -93.737447, 41.62941, '88260d802bfffff'), ('S03393100041', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24466, 'Kessler Blend Whiskey Traveler', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03396400111', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1041100, 'AMERICAN DRY GINS', 55, 'Sazerac North America', 29288, 'Barton Gin', 6, '1750', 6.57, 9.85, 3, 29.55, 5.25, 1.39, -95.8792, 41.238092, '882608c9e7fffff'), ('S03408200076', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68306, 'Emmets Irish Cream Liqueur', 12, '750', 6.0, 8.99, 3, 26.97, 2.25, 0.59, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03407400034', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1022100, 'TEQUILA', 395, 'Proximo', 89196, 'Jose Cuervo Especial Reposado Tequila', 12, '750', 10.49, 15.73, 3, 47.19, 2.25, 0.59, -93.736517, 41.659057, '88260d8045fffff'), ('S03411100007', Timestamp('2012-01-03 00:00:00'), 3631, 'Audubon Food Pride', '104 MARKET ST PO BOX 206', 'AUDUBON', '50025', 5, 'Audubon', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11296, 'Crown Royal', 12, '750', 14.25, 21.37, 12, 256.44, 9.0, 2.38, -94.93625, 41.723057, '88260c388dfffff'), ('S03392500007', Timestamp('2012-01-03 00:00:00'), 3908, 'County Market #214 / Fort Madison', '11802 AVENUE H', 'FORT MADISON', '52627', 56, 'Lee', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58860, 'Jose Cuervo Mango Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, 0.0, 0.0, '0'), ('S03410700006', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76487, 'Paramount Amaretto', 12, '1000', 4.04, 6.06, 24, 145.44, 24.0, 6.34, -94.853591, 42.064155, '88260c16a9fffff'), ('S03412100158', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11347, 'Seagrams V.o. Bl Canadian Whisky', 12, '1000', 9.49, 14.24, 2, 28.48, 2.0, 0.53, -90.483701, 41.552338, '882675da95fffff'), ('S03390800018', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 35, 'Bacardi U.S.A., Inc.', 57092, 'Bacardi Classic Cocktails Strawberry Daiquiri', 6, '1750', 10.0, 15.0, 6, 90.0, 10.5, 2.77, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03395200030', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53214, 'Paul Masson Grande Amber Brandy', 24, '375', 3.05, 4.57, 10, 45.7, 3.75, 0.99, -91.110752, 40.801889, '882670a137fffff'), ('S03390800029', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1012100, 'CANADIAN WHISKIES', 434, 'Luxco-St Louis', 13036, 'Canadian Reserve Whisky', 12, '750', 4.25, 6.38, 12, 76.56, 9.0, 2.38, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03385500059', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1012200, 'SCOTCH WHISKIES', 240, 'WILLIAM GRANT AND SONS, INC.', 5106, "Grant's Blended Scotch", 12, '750', 10.41, 15.62, 2, 31.24, 1.5, 0.4, -93.21693, 42.741728, '88262b9605fffff'), ('S03410700119', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68034, "Bailey's Original Irish Cream", 12, '375', 6.75, 10.12, 2, 20.24, 0.75, 0.2, -94.853591, 42.064155, '88260c16a9fffff'), ('S03393200012', Timestamp('2012-01-03 00:00:00'), 3895, "Tom's Market and Meats", '550 DIVISION ST', 'BURLINGTON', '52601', 29, 'Des Moines', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35918, "Five O'clock Vodka", 6, '1750', 6.92, 10.38, 6, 62.28, 10.5, 2.77, -91.106472, 40.807275000000004, '882670a131fffff'), ('S03389900027', Timestamp('2012-01-03 00:00:00'), 2532, 'Hy-Vee Food Store #2 / Des Moines', '2540 E EUCLID', 'DES MOINES', '50317', 77, 'Polk', 1041200, 'AMERICAN SLOE GINS', 434, 'Luxco-St Louis', 33716, 'Paramount Sloe Gin', 12, '750', 3.84, 5.75, 12, 69.0, 9.0, 2.38, -93.569452, 41.627838, '88260d9581fffff'), ('S03393400125', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 40976, 'Smirnoff Cherry Vodka', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03398400016', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 434, 'Luxco-St Louis', 76526, 'Paramount Melon Liqueur', 12, '750', 5.04, 7.56, 2, 15.12, 1.5, 0.4, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03396600082', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69946, 'Rumple Minze Peppermint Schnapps Liqueur', 12, '750', 12.0, 17.99, 2, 35.98, 1.5, 0.4, -93.378772, 43.142868, '8826283b29fffff'), ('S03390900006', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58838, 'Jose Cuervo Authentic Lime Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, 0.0, 0.0, '0'), ('S03384800009', Timestamp('2012-01-03 00:00:00'), 4395, 'KUM & GO #133 / ELLSWORTH', '1111 IA HIGHWAY 175', 'ELLSWORTH', '50075', 40, 'Hamilton', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43331, 'Captain Morgan Spiced Rum Mini', 12, '500', 4.97, 7.25, 1, 7.25, 0.5, 0.13, -93.572941, 42.309754, '88262b1587fffff'), ('S03406000006', Timestamp('2012-01-03 00:00:00'), 4188, 'Main St. Market, Inc.', '882 MAIN ST', 'HOLY CROSS', '52053', 31, 'Dubuque', 1041100, 'AMERICAN DRY GINS', 297, 'Laird And Company', 29997, "Five O'clock Gin", 12, '1000', 4.06, 6.09, 12, 73.08, 12.0, 3.17, -90.995943, 42.600941, '88275a7469fffff'), ('S03397100018', Timestamp('2012-01-03 00:00:00'), 4306, 'Northside One Stop / Hampton', '1208  4TH ST NE', 'HAMPTON', '50441', 35, 'Franklin', 1012100, 'CANADIAN WHISKIES', 85, 'Brown-Forman Corporation', 12476, 'Canadian Mist Whiskey Pet', 12, '750', 5.44, 8.15, 4, 32.6, 3.0, 0.79, -93.202452, 42.753045, '88262b9647fffff'), ('S03404500020', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65191, 'Hpnotiq Mini', 8, '600', 12.44, 18.66, 1, 18.66, 0.6, 0.16, -95.83539, 41.279189, '882608c829fffff'), ('S03401000018', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43136, 'Bacardi Limon', 12, '750', 8.33, 12.49, 1, 12.49, 0.75, 0.2, -90.683356, 42.52717, '88275a215dfffff'), ('S03408000017', Timestamp('2012-01-03 00:00:00'), 3622, 'Wal-Mart 1415 / Spirit Lake', '2600 HWY 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34821, 'Svedka Vodka', 6, '1750', 13.89, 20.84, 6, 125.04, 10.5, 2.77, -95.126535, 43.416452, '88262ac237fffff'), ('S03408800039', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11776, 'Black Velvet', 12, '750', 4.83, 7.44, 36, 267.84, 27.0, 7.13, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03406000003', Timestamp('2012-01-03 00:00:00'), 4188, 'Main St. Market, Inc.', '882 MAIN ST', 'HOLY CROSS', '52053', 31, 'Dubuque', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 24, 395.76, 24.0, 6.34, -90.995943, 42.600941, '88275a7469fffff'), ('S03407400018', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1081010, 'AMERICAN AMARETTO', 55, 'Sazerac North America', 73986, 'Di Amore Amaretto', 12, '750', 6.0, 8.99, 4, 35.96, 3.0, 0.79, -93.736517, 41.659057, '88260d8045fffff'), ('S03403400031', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1011100, 'BLENDED WHISKIES', 434, 'Luxco-St Louis', 24157, 'Hawkeye Blend Whiskey', 12, '1000', 4.29, 6.44, 2, 12.88, 2.0, 0.53, -93.599362, 41.59878, '88260d8743fffff'), ('S03388300002', Timestamp('2012-01-03 00:00:00'), 4039, 'No Frills Supermarkets / Council Blu', '1817 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19476, "Maker's Mark", 12, '750', 15.74, 23.61, 12, 283.32, 9.0, 2.38, -95.870993, 41.261863, '882608c917fffff'), ('S03398600026', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 77089, 'Southern Comfort Lime', 12, '750', 9.36, 14.04, 2, 28.08, 1.5, 0.4, -91.343544, 40.626988, '8826703131fffff'), ('S03389600018', Timestamp('2012-01-03 00:00:00'), 3632, 'Wal-Mart 2004 / Dubuque', '4200 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43336, 'Captain Morgan Original Spiced', 12, '750', 8.5, 12.74, 36, 458.64, 27.0, 7.13, -90.736955, 42.489041, '88275a2ac1fffff'), ('S03409700014', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -93.737447, 41.62941, '88260d802bfffff'), ('S03398600006', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81208, 'Paramount Peppermint Schnapps', 6, '1750', 7.08, 10.62, 6, 63.72, 10.5, 2.77, -91.343544, 40.626988, '8826703131fffff'), ('S03396100017', Timestamp('2012-01-03 00:00:00'), 2656, 'Hy-Vee Food Store / Corning', '300  10TH ST', 'CORNING', '51632', 22, 'Clayton', 1031080, 'VODKA 80 PROOF', 421, 'Sazerac Co., Inc.', 36976, 'Nikolai Vodka', 12, '750', 3.26, 4.88, 3, 14.64, 2.25, 0.59, -94.731809, 40.991861, '882608a217fffff'), ('S03408800052', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53211, 'Paul Masson Vs Brandy Mini', 10, '600', 5.28, 7.92, 4, 31.68, 2.4, 0.63, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03381400048', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1012210, 'SINGLE MALT SCOTCH', 240, 'WILLIAM GRANT AND SONS, INC.', 5012, 'Glenfiddich 18 Yr Ancient Reserve', 12, '750', 36.5, 54.75, 12, 657.0, 9.0, 2.38, -90.527081, 41.559731, '882675daa5fffff'), ('S03398700002', Timestamp('2012-01-03 00:00:00'), 4318, 'Fareway Stores #841 / Red Oak', '811 BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -95.321174, 41.663944, '88260c62a7fffff'), ('S03404100040', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1081312, 'BUTTERSCOTCH SCHNAPPS', 380, 'Phillips Beverage Company', 83363, 'Ice Hole Butterscotch Schnapps', 12, '750', 5.15, 7.73, 6, 46.38, 4.5, 1.19, -90.993859, 42.3036, '8826749ac1fffff'), ('S03400400001', Timestamp('2012-01-03 00:00:00'), 2655, 'Hy-Vee Food Store / Clarinda', '1200 S 16TH ST', 'CLARINDA', '51632', 73, 'Page', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -95.037849, 40.726808, '8826083189fffff'), ('S03390200025', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11293, 'Crown Royal Canadian Whisky', 48, '200', 4.47, 6.7, 1, 6.7, 0.2, 0.05, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03394000029', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69941, 'Rumple Minze Peppermint Schnapps Liqueur Mini', 8, '800', 20.12, 30.17, 2, 60.34, 1.6, 0.42, -95.902023, 41.257663, '882608c937fffff'), ('S03388300011', Timestamp('2012-01-03 00:00:00'), 4039, 'No Frills Supermarkets / Council Blu', '1817 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37346, 'Phillips Vodka', 12, '750', 3.48, 5.23, 12, 62.76, 9.0, 2.38, -95.870993, 41.261863, '882608c917fffff'), ('S03394000026', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 24, 591.36, 24.0, 6.34, -95.902023, 41.257663, '882608c937fffff'), ('S03404700005', Timestamp('2012-01-03 00:00:00'), 3972, 'Hometown Foods / Hubbard', '210 MAPLE ST BOX 295', 'HUBBARD', '50122', 42, 'Hardin', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81406, 'Thunder 101 Peppermint Schnapps', 12, '750', 6.43, 9.64, 12, 115.68, 9.0, 2.38, -93.302481, 42.305571, '88262baa53fffff'), ('S03411500040', Timestamp('2012-01-03 00:00:00'), 3700, 'Avoca Liquor', '158 SOUTH ELM', 'AVOCA', '51521', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 421, 'Sazerac Co., Inc.', 16517, 'Ancient Age Bourbon', 12, '1000', 7.5, 11.25, 2, 22.5, 2.0, 0.53, -95.339412, 41.476306, '88260c6c8bfffff'), ('S03391000004', Timestamp('2012-01-03 00:00:00'), 3654, 'The Beverage Shop / Belmond', '207 E MAIN ST', 'BELMOND', '50421', 99, 'Wright', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21598, 'Ten High', 6, '1750', 9.09, 13.64, 6, 81.84, 10.5, 2.77, -93.615247, 42.845901, '8826286587fffff'), ('S03402900032', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1081015, 'IMPORTED AMARETTO', 130, 'Disaronno International LLC', 64135, 'Disaronno Amaretto', 12, '375', 6.51, 9.77, 2, 19.54, 0.75, 0.2, -94.990031, 41.403856, '8826089095fffff'), ('S03410900074', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 421, 'Sazerac Co., Inc.', 21480, 'Benchmark Old No. 8', 12, '750', 6.37, 9.56, 2, 19.12, 1.5, 0.4, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03410300027', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10006, 'Scoresby Rare Scotch', 12, '750', 5.13, 7.69, 1, 7.69, 0.75, 0.2, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03407700019', Timestamp('2012-01-03 00:00:00'), 4602, 'Cheap Smokes / Beer City', '58231 190th ST', 'PACIFIC JUNCTION', '51561', 65, 'Mills', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75208, 'Kinky Liqueur Mini', 6, '500', 4.67, 7.0, 1, 7.0, 0.5, 0.13, -95.822983, 41.041739, '8826084255fffff'), ('S03398800045', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81206, 'Paramount Peppermint Schnapps', 12, '750', 3.7, 5.55, 2, 11.1, 1.5, 0.4, -93.573845, 41.604312, '88260d95b3fffff'), ('S03412100116', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 35773, 'Pinnacle Orange Whipped', 12, '750', 7.87, 11.81, 2, 23.62, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03393100030', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43025, 'Admiral Nelson Spiced Rum Traveler', 12, '750', 5.77, 8.66, 12, 103.92, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03402600055', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 65, 'Jim Beam Brands', 43848, 'Ronrico Silver Label Rum (pr)', 6, '1750', 7.35, 11.02, 1, 11.02, 1.75, 0.46, 0.0, 0.0, '0'), ('S03403400006', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53213, 'Paul Masson Grande Amber Brandy', 24, '200', 1.77, 2.66, 24, 63.84, 4.8, 1.27, -93.599362, 41.59878, '88260d8743fffff'), ('S03386600017', Timestamp('2012-01-03 00:00:00'), 3647, 'Wal-Mart 2889 / Clinton', '2715 SOUTH 25TH ST', 'CLINTON', '52732', 23, 'Clinton', 1022100, 'TEQUILA', 395, 'Proximo', 89196, 'Jose Cuervo Especial Reposado Tequila', 12, '750', 10.49, 15.73, 12, 188.76, 9.0, 2.38, -90.245005, 41.818874, '88275b78b7fffff'), ('S03411200042', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 35, 'Bacardi U.S.A., Inc.', 56846, 'Bacardi Party Drinks Rum Island Iced Tea', 6, '1750', 5.98, 9.47, 1, 9.47, 1.75, 0.46, -93.697731, 41.665172, '88260d8231fffff'), ('S03406300051', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25608, 'Seagrams 7 Crown Bl Whiskey', 6, '1750', 10.76, 16.4, 1, 16.4, 1.75, 0.46, -91.281344, 42.108275, '882674c6a9fffff'), ('S03410300031', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11347, 'Seagrams V.o. Bl Canadian Whisky', 12, '1000', 9.49, 14.24, 1, 14.24, 1.0, 0.26, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03388500098', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 260, 'Diageo Americas', 34456, 'Ketel One Imported Vodka', 12, '750', 13.0, 19.49, 4, 77.96, 3.0, 0.79, -90.668138, 42.504959, '88275a2119fffff'), ('S03393600031', Timestamp('2012-01-03 00:00:00'), 4040, 'No Frills Supermarkets / Council Blu', '1801 VALLEY VIEW DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43331, 'Captain Morgan Spiced Rum Mini', 12, '500', 4.97, 7.25, 1, 7.25, 0.5, 0.13, -95.81877500000002, 41.24394, '882608c9d9fffff'), ('S03401400032', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1041100, 'AMERICAN DRY GINS', 205, 'E AND J GALLO WINERY', 31474, 'New Amsterdam Gin', 24, '375', 3.4, 5.1, 4, 20.4, 1.5, 0.4, -95.126786, 43.416433, '88262ac237fffff'), ('S03403100007', Timestamp('2012-01-03 00:00:00'), 4462, "SHELTON'S", '113, 1ST ST NW', 'BUFFALO CENTER', '50424', 95, 'Winnebago', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10006, 'Scoresby Rare Scotch', 12, '750', 5.13, 7.69, 2, 15.38, 1.5, 0.4, -93.947595, 43.386755, '8826284217fffff'), ('S03399600001', Timestamp('2012-01-03 00:00:00'), 4229, 'Fareway Stores #790 / Harlan', '1910  23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67526, 'Kahlua Coffee Liqueur', 12, '750', 11.61, 17.42, 12, 209.04, 9.0, 2.38, -95.339845, 41.650132, '88260c60e9fffff'), ('S03395800161', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65194, 'Hpnotiq', 12, '375', 6.35, 9.52, 2, 19.04, 0.75, 0.2, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03404100055', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 971, 'Hood River Distillers, Inc.', 14192, 'Pendleton Canadian Whisky', 12, '750', 13.1, 19.65, 6, 117.9, 4.5, 1.19, -90.993859, 42.3036, '8826749ac1fffff'), ('S03389300045', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 86670, "Jack Daniel's Tennessee Honey", 12, '750', 13.01, 19.51, 3, 58.53, 2.25, 0.59, -90.609747, 41.589369, '882675d959fffff'), ('S03411000066', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1081340, 'ROOT BEER SCHNAPPS', 380, 'Phillips Beverage Company', 84617, 'Phillips Root Beer Schnapps', 12, '1000', 4.89, 7.33, 24, 175.92, 24.0, 6.34, 0.0, 0.0, '0'), ('S03391300137', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57148, "Chi-Chi's Margarita W/tequila", 6, '1750', 5.99, 9.24, 2, 18.48, 3.5, 0.92, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03385300068', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 22784, "Beam's 8 Star Bl Whiskey", 24, '375', 1.9, 2.85, 6, 17.1, 2.25, 0.59, -91.136655, 40.814666, '882670aacdfffff'), ('S03403000016', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86888, 'Southern Comfort PET', 6, '1750', 18.14, 27.21, 1, 27.21, 1.75, 0.46, -94.480204, 43.39966, '88262a82cbfffff'), ('S03387200018', Timestamp('2012-01-03 00:00:00'), 4296, 'Circle B Market', '111 MAIN ST', 'KEOSAUQUA', '52565', 89, 'Van Buren', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43334, 'Captain Morgan Spiced Rum', 24, '375', 4.3, 6.45, 3, 19.35, 1.12, 0.3, -91.962466, 40.7294, '882670458bfffff'), ('S03385300044', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69637, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '1000', 11.0, 16.5, 12, 198.0, 12.0, 3.17, -91.136655, 40.814666, '882670aacdfffff'), ('S03396800015', Timestamp('2012-01-03 00:00:00'), 2445, "Ruback's Food Center", '504 SOUTH HIGHWAY', 'OAKLAND', '51560', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41694, 'Uv Blue (raspberry) Vodka', 12, '1000', 7.25, 10.87, 4, 43.48, 4.0, 1.06, 0.0, 0.0, '0'), ('S03385800014', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 37938, 'Skol Vodka', 6, '1750', 7.1, 10.65, 6, 63.9, 10.5, 2.77, -92.258152, 42.809882, '8826291267fffff'), ('S03396400129', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 63894, 'Tgif On The Rocks White Russian', 12, '750', 3.82, 5.73, 1, 5.73, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03398600057', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45276, 'Paramount White Rum', 12, '750', 3.85, 5.77, 2, 11.54, 1.5, 0.4, -91.343544, 40.626988, '8826703131fffff'), ('S03407100031', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1081380, 'MISCELLANEOUS SCHNAPPS', 65, 'Jim Beam Brands', 73516, 'Dekuyper Cactus Juice Schnapps', 12, '750', 6.3, 9.45, 1, 9.45, 0.75, 0.2, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03381300002', Timestamp('2012-01-03 00:00:00'), 3944, "Sam's Club 4973 / Dubuque", '4400 ASBURY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25608, 'Seagrams 7 Crown Bl Whiskey', 6, '1750', 10.76, 16.4, 48, 787.2, 84.0, 22.19, -90.737821, 42.515289, '88275a212dfffff'), ('S03408200087', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1031100, '100 PROOF VODKA', 260, 'Diageo Americas', 39868, 'Smirnoff Vodka 100 Prf', 6, '1750', 15.5, 23.24, 2, 46.48, 3.5, 0.92, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03387200025', Timestamp('2012-01-03 00:00:00'), 4296, 'Circle B Market', '111 MAIN ST', 'KEOSAUQUA', '52565', 89, 'Van Buren', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37996, 'Smirnoff Vodka 80 Prf', 12, '750', 7.59, 11.56, 3, 34.68, 2.25, 0.59, -91.962466, 40.7294, '882670458bfffff'), ('S03397600026', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25608, 'Seagrams 7 Crown Bl Whiskey', 6, '1750', 10.76, 16.4, 6, 98.4, 10.5, 2.77, -95.825137, 41.242732, '882608c9d1fffff'), ('S03396500020', Timestamp('2012-01-03 00:00:00'), 3587, 'Save A Lot  #482', '2122 MAIN ST', 'KEOKUK', '52632', 56, 'Lee', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69658, "Dr. Mcgillicuddy's Mentholmint", 6, '1750', 13.33, 19.99, 6, 119.94, 10.5, 2.77, -91.39688800000002, 40.408821, '8826702f67fffff'), ('S03383500012', Timestamp('2012-01-03 00:00:00'), 3756, 'Main Street Liquors / Manning', '316 MAIN ST', 'MANNING', '51455', 14, 'Carroll', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43128, 'Bacardi Superior Rum', 6, '1750', 14.67, 22.0, 1, 22.0, 1.75, 0.46, -95.065122, 41.908573, '88260c0525fffff'), ('S03389400015', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1052010, 'IMPORTED GRAPE BRANDIES', 65, 'Jim Beam Brands', 47786, 'Courvoisier Vs Cognac', 12, '750', 14.17, 21.25, 12, 255.0, 9.0, 2.38, -95.909384, 41.242291, '88260e26d5fffff'), ('S03389900035', Timestamp('2012-01-03 00:00:00'), 2532, 'Hy-Vee Food Store #2 / Des Moines', '2540 E EUCLID', 'DES MOINES', '50317', 77, 'Polk', 1022100, 'TEQUILA', 395, 'Proximo', 89212, 'Jose Cuervo Black Medallion', 12, '750', 12.5, 18.74, 3, 56.22, 2.25, 0.59, -93.569452, 41.627838, '88260d9581fffff'), ('S03408600050', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1011200, 'STRAIGHT BOURBON WHISKIES', 125, 'Cedar Ridge Vineyards,LLC', 17206, 'Cedar Ridge Bourbon', 6, '750', 17.6, 26.4, 1, 26.4, 0.75, 0.2, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03384500007', Timestamp('2012-01-03 00:00:00'), 3482, 'Kick The Can', '525 N MAPLE ST', 'CARROLL', '51401', 14, 'Carroll', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 37938, 'Skol Vodka', 6, '1750', 7.1, 10.65, 6, 63.9, 10.5, 2.77, -94.861667, 42.063255, '88260c16a9fffff'), ('S03408800074', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82786, 'Dekuyper Buttershots Schnapps', 12, '750', 6.3, 9.45, 3, 28.35, 2.25, 0.59, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03412100166', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1012210, 'SINGLE MALT SCOTCH', 240, 'WILLIAM GRANT AND SONS, INC.', 5012, 'Glenfiddich 18 Yr Ancient Reserve', 12, '750', 36.5, 54.75, 1, 54.75, 0.75, 0.2, -90.483701, 41.552338, '882675da95fffff'), ('S03391300073', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55088, 'Paramount Blackberry Brandy', 6, '1750', 10.75, 16.12, 1, 16.12, 1.75, 0.46, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03371500023', Timestamp('2012-01-03 00:00:00'), 4572, 'Grundy Center Family Foods', '1400, G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 12406, 'Canadian Ltd Whisky', 12, '750', 3.75, 5.87, 2, 11.74, 1.5, 0.4, -92.782283, 42.36157, '8826297835fffff'), ('S03398700007', Timestamp('2012-01-03 00:00:00'), 4318, 'Fareway Stores #841 / Red Oak', '811 BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25608, 'Seagrams 7 Crown Bl Whiskey', 6, '1750', 10.76, 16.4, 6, 98.4, 10.5, 2.77, -95.321174, 41.663944, '88260c62a7fffff'), ('S03408300278', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1051100, 'APRICOT BRANDIES', 259, 'Heaven Hill Brands', 52314, 'Christian Bros Brandy', 24, '375', 2.93, 4.4, 6, 26.4, 2.25, 0.59, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03391900013', Timestamp('2012-01-03 00:00:00'), 4113, 'Tobacco Hut #6 / Council', '3134 MANAWA CENTRE DR', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11347, 'Seagrams V.o. Bl Canadian Whisky', 12, '1000', 9.49, 14.24, 12, 170.88, 12.0, 3.17, -95.847514, 41.228712, '882608c983fffff'), ('S03389000005', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11777, 'Black Velvet', 12, '1000', 6.63, 9.94, 12, 119.28, 12.0, 3.17, -93.315977, 40.757158, '88267252cdfffff'), ('S03400700073', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5347, 'Johnnie Walker Red Label Scotch', 12, '1000', 16.99, 25.49, 4, 101.96, 4.0, 1.06, 0.0, 0.0, '0'), ('S03410300103', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15246, 'Windsor Canadian', 12, '750', 4.2, 6.3, 2, 12.6, 1.5, 0.4, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03397300063', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65254, 'Jagermeister Liqueur', 24, '375', 6.23, 9.34, 6, 56.04, 2.25, 0.59, -91.044008, 41.427202, '8826742f4bfffff'), ('S03411100032', Timestamp('2012-01-03 00:00:00'), 3631, 'Audubon Food Pride', '104 MARKET ST PO BOX 206', 'AUDUBON', '50025', 5, 'Audubon', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69667, "Dr. Mcgillicuddy's Vanilla", 12, '1000', 11.0, 16.5, 2, 33.0, 2.0, 0.53, -94.93625, 41.723057, '88260c388dfffff'), ('S03395800123', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69667, "Dr. Mcgillicuddy's Vanilla", 12, '1000', 11.0, 16.5, 12, 198.0, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03397600150', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 89626, 'Patron Reposado Tequila', 12, '750', 27.0, 40.5, 2, 81.0, 1.5, 0.4, -95.825137, 41.242732, '882608c9d1fffff'), ('S03396400001', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82607, 'Dekuyper Sour Apple', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -95.8792, 41.238092, '882608c9e7fffff'), ('S03389300018', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88294, 'Patron Silver Tequila', 12, '375', 13.33, 20.0, 3, 60.0, 1.12, 0.3, -90.609747, 41.589369, '882675d959fffff'), ('S03395700042', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43128, 'Bacardi Superior Rum', 6, '1750', 14.67, 22.0, 1, 22.0, 1.75, 0.46, -95.147741, 43.14521, '88262a1925fffff'), ('S03389000074', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42699, 'Malibu Mango Rum', 12, '750', 6.9, 10.35, 1, 10.35, 0.75, 0.2, -93.315977, 40.757158, '88267252cdfffff'), ('S03386600003', Timestamp('2012-01-03 00:00:00'), 3647, 'Wal-Mart 2889 / Clinton', '2715 SOUTH 25TH ST', 'CLINTON', '52732', 23, 'Clinton', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25606, 'Seagrams 7 Crown Bl Whiskey', 12, '750', 6.14, 9.21, 12, 110.52, 9.0, 2.38, -90.245005, 41.818874, '88275b78b7fffff'), ('S03395500014', Timestamp('2012-01-03 00:00:00'), 3976, 'North English Family Food', '119 NORTH MAIN ST', 'NORTH ENGLISH', '52316', 48, 'Iowa', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43124, 'Bacardi Superior Rum', 24, '375', 4.3, 6.45, 2, 12.9, 0.75, 0.2, -92.076452, 41.515209, '882676a0adfffff'), ('S03408200028', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43136, 'Bacardi Limon', 12, '750', 8.33, 12.49, 12, 149.88, 9.0, 2.38, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03371500010', Timestamp('2012-01-03 00:00:00'), 4572, 'Grundy Center Family Foods', '1400, G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1022100, 'TEQUILA', 421, 'Sazerac Co., Inc.', 89916, 'Tortilla Gold Tequila', 12, '750', 6.34, 9.52, 2, 19.04, 1.5, 0.4, -92.782283, 42.36157, '8826297835fffff'), ('S03403500028', Timestamp('2012-01-03 00:00:00'), 4156, 'Fareway Stores #412 / Oelwein', '102  2ND AVE SE', 'OELWEIN', '50662', 33, 'Fayette', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43244, 'Captain Morgan 100 Proof Spiced Rum', 12, '750', 10.0, 14.99, 3, 44.97, 2.25, 0.59, -91.910502, 42.677024, '882629b807fffff'), ('S03405700036', Timestamp('2012-01-03 00:00:00'), 3651, 'Wal-Mart 1491 / Indianola', '1500 NORTH JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41922, 'Smirnoff Peach Vodka', 12, '750', 7.59, 11.56, 24, 277.44, 18.0, 4.76, 0.0, 0.0, '0'), ('S03408300163', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 12407, 'Canadian Ltd Whisky', 12, '1000', 5.33, 7.99, 6, 47.94, 6.0, 1.59, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03385500032', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26586, 'Gentleman Jack', 12, '750', 18.44, 27.66, 2, 55.32, 1.5, 0.4, -93.21693, 42.741728, '88262b9605fffff'), ('S03396000071', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58875, 'Jose Cuervo Golden Margarita', 6, '1750', 10.0, 15.24, 6, 91.44, 10.5, 2.77, -90.705328, 42.491862, '88275a2131fffff'), ('S03389400025', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 461, 'Campari(skyy)', 22157, 'Wild Turkey 101', 12, '1000', 14.99, 22.49, 12, 269.88, 12.0, 3.17, -95.909384, 41.242291, '88260e26d5fffff'), ('S03409700105', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1081312, 'BUTTERSCOTCH SCHNAPPS', 434, 'Luxco-St Louis', 81966, "Arrow Mcdale's Butterscotch Schnapps", 12, '750', 3.97, 5.95, 4, 23.8, 3.0, 0.79, -93.737447, 41.62941, '88260d802bfffff'), ('S03383200020', Timestamp('2012-01-03 00:00:00'), 3562, 'Wal-Mart 0797 / W Burlington', '324 WEST AGENCY RD', 'WEST BURLINGTON', '52655', 29, 'Des Moines', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82846, 'Dekuyper Luscious Peachtree Schnapps', 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -91.158407, 40.813125, '882670aa17fffff'), ('S03383900033', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1022100, 'TEQUILA', 330, 'Gemini Spirits', 89496, 'Margaritaville Gold Tequila', 12, '750', 6.01, 9.52, 12, 114.24, 9.0, 2.38, -91.403752, 40.41538, '8826702f65fffff'), ('S03408800063', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41704, 'Uv Red (cherry) Vodka', 12, '750', 6.22, 9.33, 2, 18.66, 1.5, 0.4, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03391100017', Timestamp('2012-01-03 00:00:00'), 4172, 'Shamrock Spirits', '1204 WATER ST PO BOX 177', 'ALDEN', '50006', 42, 'Hardin', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55086, 'Paramount Blackberry Brandy', 12, '750', 5.21, 7.82, 3, 23.46, 2.25, 0.59, -93.376271, 42.520423, '88262b8ed9fffff'), ('S03388200032', Timestamp('2012-01-03 00:00:00'), 4106, 'Hartig Drug Co. / JFK Rd / Dubuque', '2255 JOHN F KENNEDY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86884, 'Southern Comfort', 24, '375', 4.54, 6.8, 1, 6.8, 0.38, 0.1, -90.72051800000001, 42.511352, '88275a212bfffff'), ('S03388500034', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32234, 'Seagrams Extra Dry Gin', 24, '375', 3.4, 5.1, 24, 122.4, 9.0, 2.38, -90.668138, 42.504959, '88275a2119fffff'), ('S03406300070', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1051140, 'PEACH BRANDIES', 434, 'Luxco-St Louis', 56206, 'Paramount Peach Flavored Brandy', 12, '750', 5.21, 7.82, 2, 15.64, 1.5, 0.4, -91.281344, 42.108275, '882674c6a9fffff'), ('S03405200038', Timestamp('2012-01-03 00:00:00'), 2954, "Dahl's / Ingersoll", '3425 INGERSOLL', 'DES MOINES', '50300', 77, 'Polk', 1012210, 'SINGLE MALT SCOTCH', 240, 'WILLIAM GRANT AND SONS, INC.', 5014, 'Glenfiddich 15 Yr Solera Reserva', 12, '750', 28.33, 42.94, 2, 85.88, 1.5, 0.4, -93.663337, 41.586313, '88260d80d7fffff'), ('S03396400007', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 48, 791.52, 48.0, 12.68, -95.8792, 41.238092, '882608c9e7fffff'), ('S03388100141', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 80574, 'Arrow Peppermint Schnapps', 24, '375', 2.25, 3.38, 6, 20.28, 2.25, 0.59, -95.879662, 41.261925, '882608c93bfffff'), ('S03393400172', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55084, 'Paramount Blackberry Brandy', 24, '375', 3.34, 5.0, 3, 15.0, 1.12, 0.3, -90.741064, 42.516633, '88275a212dfffff'), ('S03408200039', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1081200, 'CREAM LIQUEURS', 461, 'Campari(skyy)', 68126, "Carolan's Irish Cream Liqueur", 12, '750', 7.69, 11.54, 12, 138.48, 9.0, 2.38, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03383200023', Timestamp('2012-01-03 00:00:00'), 3562, 'Wal-Mart 0797 / W Burlington', '324 WEST AGENCY RD', 'WEST BURLINGTON', '52655', 29, 'Des Moines', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58873, 'Jose Cuervo Authentic Watermelon Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, -91.158407, 40.813125, '882670aa17fffff'), ('S03385300103', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69666, "Dr. Mcgillicuddy's Vanilla", 12, '750', 8.66, 12.99, 6, 77.94, 4.5, 1.19, -91.136655, 40.814666, '882670aacdfffff'), ('S03396400178', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081200, 'CREAM LIQUEURS', 330, 'Gemini Spirits', 80457, "Ryan's Cream Liqueur", 12, '1000', 7.22, 10.83, 2, 21.66, 2.0, 0.53, -95.8792, 41.238092, '882608c9e7fffff'), ('S03396400053', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86887, 'Southern Comfort', 12, '1000', 11.34, 17.01, 12, 204.12, 12.0, 3.17, -95.8792, 41.238092, '882608c9e7fffff'), ('S03393400064', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1012200, 'SCOTCH WHISKIES', 55, 'Sazerac North America', 8828, "Lauder's", 6, '1750', 11.18, 16.76, 6, 100.56, 10.5, 2.77, -90.741064, 42.516633, '88275a212dfffff'), ('S03386600027', Timestamp('2012-01-03 00:00:00'), 3647, 'Wal-Mart 2889 / Clinton', '2715 SOUTH 25TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69666, "Dr. Mcgillicuddy's Vanilla", 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -90.245005, 41.818874, '88275b78b7fffff'), ('S03385300107', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081030, 'COFFEE LIQUEURS', 65, 'Jim Beam Brands', 67556, 'Kamora Coffee Liqueur', 12, '750', 6.29, 9.44, 6, 56.64, 4.5, 1.19, -91.136655, 40.814666, '882670aacdfffff'), ('S03371300012', Timestamp('2012-01-03 00:00:00'), 4443, 'Palo Mini Mart', '1204 1ST ST', 'PALO', '52324', 57, 'Linn', 1012100, 'CANADIAN WHISKIES', 85, 'Brown-Forman Corporation', 12478, 'Canadian Mist Canadian Why Pet', 6, '1750', 10.5, 15.75, 6, 94.5, 10.5, 2.77, -91.795808, 42.070316, '8826745323fffff'), ('S03390900051', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65257, 'Jagermeister Liqueur', 12, '1000', 14.93, 22.39, 2, 44.78, 2.0, 0.53, 0.0, 0.0, '0'), ('S03410700065', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 12, 128.4, 21.0, 5.55, -94.853591, 42.064155, '88260c16a9fffff'), ('S03398800041', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28236, 'Bombay Sapphire Gin', 12, '750', 13.63, 20.45, 2, 40.9, 1.5, 0.4, -93.573845, 41.604312, '88260d95b3fffff'), ('S03408300041', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23826, 'Five Star', 12, '750', 3.35, 5.02, 12, 60.24, 9.0, 2.38, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03390900048', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 63894, 'Tgif On The Rocks White Russian', 12, '750', 3.82, 5.73, 2, 11.46, 1.5, 0.4, 0.0, 0.0, '0'), ('S03385300026', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750', 9.97, 14.95, 12, 179.4, 9.0, 2.38, -91.136655, 40.814666, '882670aacdfffff'), ('S03392300024', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 434, 'Luxco-St Louis', 24157, 'Hawkeye Blend Whiskey', 12, '1000', 4.29, 6.44, 12, 77.28, 12.0, 3.17, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03396400024', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36907, 'Mccormick Vodka PET', 12, '1000', 3.98, 5.97, 24, 143.28, 24.0, 6.34, -95.8792, 41.238092, '882608c9e7fffff'), ('S03399000001', Timestamp('2012-01-03 00:00:00'), 3762, 'Wine and Spirits Gallery', '7690 HICKMAN RD', 'WINDSOR HEIGHTS', '50322', 77, 'Polk', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34422, 'Grey Goose Vodka', 6, '1000', 21.77, 32.66, 6, 195.96, 6.0, 1.59, 0.0, 0.0, '0'), ('S03402100095', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1081600, 'WHISKEY LIQUEUR', 260, 'Diageo Americas', 67264, 'Yukon Jack Canadian Liqueur', 24, '375', 4.69, 7.03, 6, 42.18, 2.25, 0.59, -92.475403, 42.712263, '8826291801fffff'), ('S03390200009', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43031, 'Bacardi Gold Rum Mini', 12, '500', 4.74, 7.11, 2, 14.22, 1.0, 0.26, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03396100011', Timestamp('2012-01-03 00:00:00'), 2656, 'Hy-Vee Food Store / Corning', '300  10TH ST', 'CORNING', '51632', 22, 'Clayton', 1022100, 'TEQUILA', 421, 'Sazerac Co., Inc.', 88766, 'Tortilla White Tequila', 12, '750', 6.34, 9.52, 3, 28.56, 2.25, 0.59, -94.731809, 40.991861, '882608a217fffff'), ('S03405900066', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42718, 'Malibu Coconut Rum', 6, '1750', 14.9, 22.35, 1, 22.35, 1.75, 0.46, -90.48065, 41.529655, '882675c361fffff'), ('S03397100035', Timestamp('2012-01-03 00:00:00'), 4306, 'Northside One Stop / Hampton', '1208  4TH ST NE', 'HAMPTON', '50441', 35, 'Franklin', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69636, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '750', 8.66, 12.99, 3, 38.97, 2.25, 0.59, -93.202452, 42.753045, '88262b9647fffff'), ('S03408600059', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76487, 'Paramount Amaretto', 12, '1000', 4.04, 6.06, 2, 12.12, 2.0, 0.53, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03390900059', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82600, 'Dekuyper Sour Apple Pucker Pet', 6, '1750', 11.02, 16.53, 2, 33.06, 3.5, 0.92, 0.0, 0.0, '0'), ('S03404600023', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 10836, 'Crown Royal Black', 12, '750', 16.47, 24.7, 3, 74.1, 2.25, 0.59, -95.027238, 40.739231, '88260831cbfffff'), ('S03404800050', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1081340, 'ROOT BEER SCHNAPPS', 380, 'Phillips Beverage Company', 84617, 'Phillips Root Beer Schnapps', 12, '1000', 4.89, 7.33, 12, 87.96, 12.0, 3.17, -91.035138, 41.451349, '88267421abfffff'), ('S03408300316', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57144, "Chi-Chi's White Russian", 6, '1750', 5.99, 9.24, 1, 9.24, 1.75, 0.46, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03399400014', Timestamp('2012-01-03 00:00:00'), 3563, 'Wal-Mart 0559 / Muscatine', '3003 NORTH HWY 61', 'MUSCATINE', '52761', 70, 'Muscatine', 1081400, 'TRIPLE SEC', 65, 'Jim Beam Brands', 86112, 'Dekuyper Triple Sec', 12, '1000', 2.21, 3.32, 12, 39.84, 12.0, 3.17, -91.025913, 41.45529, '8826742185fffff'), ('S03389300032', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 54646, 'Arrow Blackberry Flav Brandy', 12, '750', 5.0, 7.5, 3, 22.5, 2.25, 0.59, -90.609747, 41.589369, '882675d959fffff'), ('S03408200059', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36306, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 12, 58.56, 9.0, 2.38, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03396600014', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36307, 'Hawkeye Vodka', 12, '1000', 3.85, 5.78, 48, 277.44, 48.0, 12.68, -93.378772, 43.142868, '8826283b29fffff'), ('S03404000001', Timestamp('2012-01-03 00:00:00'), 3841, 'Hartig Drug Company #2 / Locust', '157 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -90.665439, 42.495308, '88275a211bfffff'), ('S03398800038', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1031100, '100 PROOF VODKA', 260, 'Diageo Americas', 39666, 'Popov Vodka 100 Prf', 12, '750', 5.07, 7.61, 2, 15.22, 1.5, 0.4, -93.573845, 41.604312, '88260d95b3fffff'), ('S03400300014', Timestamp('2012-01-03 00:00:00'), 3461, 'Hometown Foods / Stuart', '611 S DIVISION ST', 'STUART', '50250', 1, 'Adair', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82846, 'Dekuyper Luscious Peachtree Schnapps', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -94.318443, 41.49759900000001, '88260dcc2bfffff'), ('S03383900051', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69636, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -91.403752, 40.41538, '8826702f65fffff'), ('S03406300003', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 12, 268.08, 12.0, 3.17, -91.281344, 42.108275, '882674c6a9fffff'), ('S03392900004', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58868, 'Jose Cuervo Authentic Strawberry Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, 0.0, 0.0, '0'), ('S03410900011', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24466, 'Kessler Blend Whiskey Traveler', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03411000094', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 987, 'Stellar Importing Company, LLC', 64016, 'Achaia Clauss Ouzo Greek Liqueur', 12, '750', 9.16, 13.74, 4, 54.96, 3.0, 0.79, 0.0, 0.0, '0'), ('S03403400004', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32234, 'Seagrams Extra Dry Gin', 24, '375', 3.4, 5.1, 24, 122.4, 9.0, 2.38, -93.599362, 41.59878, '88260d8743fffff'), ('S03398600048', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 35626, 'Pinnacle Whipped', 12, '750', 7.87, 11.81, 4, 47.24, 3.0, 0.79, -91.343544, 40.626988, '8826703131fffff'), ('S03392300053', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35948, "Fleischmann's Royal Vodka 80 Prf", 6, '1750', 7.14, 10.7, 6, 64.2, 10.5, 2.77, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03410900029', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 58801, 'Captain Morgan Long Island Iced Tea', 6, '1750', 10.5, 15.74, 6, 94.44, 10.5, 2.77, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03407200031', Timestamp('2012-01-03 00:00:00'), 4146, 'Fareway Stores #561 / Waverly', '220 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58838, 'Jose Cuervo Authentic Lime Margarita', 6, '1750', 7.7, 11.8, 3, 35.4, 5.25, 1.39, -92.473476, 42.725748, '8826291847fffff'), ('S03381400015', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32237, 'Seagrams Extra Dry Gin', 12, '1000', 7.4, 11.1, 48, 532.8, 48.0, 12.68, -90.527081, 41.559731, '882675daa5fffff'), ('S03384700009', Timestamp('2012-01-03 00:00:00'), 4138, 'Fareway Stores #657 / Indianola', '1309 W 2ND', 'INDIANOLA', '50125', 91, 'Warren', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36908, 'Mccormick Vodka Pet', 6, '1750', 7.46, 11.19, 6, 67.14, 10.5, 2.77, -93.579252, 41.356616, '88260db8e9fffff'), ('S03392100005', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43038, 'Bacardi Gold Rum', 6, '1750', 14.67, 22.0, 1, 22.0, 1.75, 0.46, -90.204748, 41.827845, '88275b7891fffff'), ('S03397000037', Timestamp('2012-01-03 00:00:00'), 3994, 'Great Spirits Wine & Liquor Co.', '72 NORTH HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1081010, 'AMERICAN AMARETTO', 55, 'Sazerac North America', 73986, 'Di Amore Amaretto', 12, '750', 6.0, 8.99, 1, 8.99, 0.75, 0.2, -95.127395, 43.368989000000006, '88262ac0e9fffff'), ('S03406700007', Timestamp('2012-01-03 00:00:00'), 3849, 'Hartig Drug Co #6 / Dyersville', '711  16TH AVE SE', 'DYERSVILLE', '52040', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 3, 27.99, 2.25, 0.59, -91.116087, 42.470285, '88275a654dfffff'), ('S03384000011', Timestamp('2012-01-03 00:00:00'), 2327, 'West Side Spirits', '801 HULL ST', 'CORNING', '50841', 2, 'Adams', 1011200, 'STRAIGHT BOURBON WHISKIES', 297, 'Laird And Company', 18408, 'G & W Bourbon', 6, '1750', 9.14, 13.71, 2, 27.42, 3.5, 0.92, -94.74369200000001, 40.990378, '882608a233fffff'), ('S03412100122', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28236, 'Bombay Sapphire Gin', 12, '750', 13.63, 20.45, 2, 40.9, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03388500032', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69946, 'Rumple Minze Peppermint Schnapps Liqueur', 12, '750', 12.0, 17.99, 12, 215.88, 9.0, 2.38, -90.668138, 42.504959, '88275a2119fffff'), ('S03406300105', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82610, 'Dekuyper Hot Damn!', 12, '1000', 7.35, 11.02, 3, 33.06, 3.0, 0.79, -91.281344, 42.108275, '882674c6a9fffff'), ('S03412100074', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82787, 'Dekuyper Buttershots', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -90.483701, 41.552338, '882675da95fffff'), ('S03388500045', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081350, 'SPEARMINT SCHNAPPS', 434, 'Luxco-St Louis', 82186, 'Arrow Spearmint Schnapps', 12, '750', 3.97, 5.95, 1, 5.95, 0.75, 0.2, -90.668138, 42.504959, '88275a2119fffff'), ('S03400900005', Timestamp('2012-01-03 00:00:00'), 3803, 'Strawberry Foods and Deli', '128 W MISSION', 'STRAWBERRY POINT', '52076', 22, 'Clayton', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26828, 'Jack Daniels Old #7 Black Lbl', 6, '1750', 26.67, 40.01, 6, 240.06, 10.5, 2.77, -91.535736, 42.683731, '882629b66dfffff'), ('S03396300001', Timestamp('2012-01-03 00:00:00'), 4041, 'Mandm Pantry', '2344 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36904, 'Mccormick Vodka Pet', 24, '375', 1.76, 2.63, 24, 63.12, 9.0, 2.38, -90.213015, 41.821777, '88275b7897fffff'), ('S03389000028', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19068, 'Jim Beam', 6, '1750', 18.37, 27.55, 1, 27.55, 1.75, 0.46, -93.315977, 40.757158, '88267252cdfffff'), ('S03409700119', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1022100, 'TEQUILA', 85, 'Brown-Forman Corporation', 89278, 'El Jimador Reposado Tequila', 12, '750', 10.63, 15.95, 4, 63.8, 3.0, 0.79, -93.737447, 41.62941, '88260d802bfffff'), ('S03390300071', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081030, 'COFFEE LIQUEURS', 421, 'Sazerac Co., Inc.', 67586, 'Kapali Coffee Liqueur', 12, '750', 5.72, 8.58, 1, 8.58, 0.75, 0.2, -95.854809, 41.262316, '882608c9cdfffff'), ('S03393900042', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 85, 'Brown-Forman Corporation', 12467, 'Canadian Mist Whisky', 12, '1000', 6.7, 10.05, 1, 10.05, 1.0, 0.26, -95.873073, 41.261866, '882608c917fffff'), ('S03403900002', Timestamp('2012-01-03 00:00:00'), 2628, 'Hy-Vee Food Store / Eldora', '1616 EDGINGTON', 'ELDORA', '50627', 42, 'Hardin', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36304, 'Hawkeye Vodka', 24, '375', 2.06, 3.09, 24, 74.16, 9.0, 2.38, -93.103764, 42.360631, '88262ba25dfffff'), ('S03390400019', Timestamp('2012-01-03 00:00:00'), 3738, 'Hartig Drug Company #4 / Dubuque', '2225 CENTRAL AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58876, 'Jose Cuervo Golden Margarita', 12, '750', 6.42, 9.62, 2, 19.24, 1.5, 0.4, -90.671531, 42.51384, '88275a2153fffff'), ('S03393400117', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031100, '100 PROOF VODKA', 434, 'Luxco-St Louis', 36301, 'Hawkeye Vodka 80 Prf Mini', 12, '500', 3.73, 5.59, 12, 67.08, 6.0, 1.59, -90.741064, 42.516633, '88275a212dfffff'), ('S03393800005', Timestamp('2012-01-03 00:00:00'), 3629, 'Wal-Mart 1965 / Council Bluffs', '3200 MANAWA DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31658, 'Paramount Gin', 6, '1750', 7.1, 10.65, 6, 63.9, 10.5, 2.77, 0.0, 0.0, '0'), ('S03396400149', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5346, 'Johnnie Walker Red', 12, '750', 13.78, 20.67, 1, 20.67, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03398400030', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43285, 'Captain Morgan Original Spiced Rum Pet', 12, '750', 8.5, 12.74, 4, 50.96, 3.0, 0.79, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03409700055', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 35626, 'Pinnacle Whipped', 12, '750', 7.87, 11.81, 3, 35.43, 2.25, 0.59, -93.737447, 41.62941, '88260d802bfffff'), ('S03387800071', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41696, 'Uv Green (apple) Vodka', 12, '750', 6.22, 9.33, 3, 27.99, 2.25, 0.59, -91.387797, 40.400038, '8826702f29fffff'), ('S03412100096', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58875, 'Jose Cuervo Golden Margarita', 6, '1750', 10.0, 15.24, 6, 91.44, 10.5, 2.77, -90.483701, 41.552338, '882675da95fffff'), ('S03381400074', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1032200, 'IMPORTED VODKA - MISC', 277, 'Stoli Group', 34870, 'Stoli Razberi (raspberry)', 12, '750', 12.24, 18.37, 24, 440.88, 18.0, 4.76, -90.527081, 41.559731, '882675daa5fffff'), ('S03398800016', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11297, 'Crown Royal Canadian Whisky', 12, '1000', 17.46, 26.19, 12, 314.28, 12.0, 3.17, -93.573845, 41.604312, '88260d95b3fffff'), ('S03407700017', Timestamp('2012-01-03 00:00:00'), 4602, 'Cheap Smokes / Beer City', '58231 190th ST', 'PACIFIC JUNCTION', '51561', 65, 'Mills', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43331, 'Captain Morgan Spiced Rum Mini', 12, '500', 4.97, 7.25, 1, 7.25, 0.5, 0.13, -95.822983, 41.041739, '8826084255fffff'), ('S03393100016', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 35626, 'Pinnacle Whipped', 12, '750', 7.87, 11.81, 12, 141.72, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03389000058', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69657, "Dr. Mcgillicuddy's Mentholmint", 12, '1000', 11.0, 16.5, 1, 16.5, 1.0, 0.26, -93.315977, 40.757158, '88267252cdfffff'), ('S03393500008', Timestamp('2012-01-03 00:00:00'), 4341, "Mj's Grocery", '524 MAIN ST', 'GRISWOLD', '51535', 15, 'Cass', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35946, "Fleischmann's Royal Vodka", 12, '750', 3.44, 5.15, 3, 15.45, 2.25, 0.59, -95.14161, 41.234743, '8826088f47fffff'), ('S03388100010', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 42009, 'UV Cake Vodka', 6, '1750', 10.63, 15.95, 6, 95.7, 10.5, 2.77, -95.879662, 41.261925, '882608c93bfffff'), ('S03382400004', Timestamp('2012-01-03 00:00:00'), 4314, 'Nodaway Valley Market', '304  4TH ST', 'FONTANELLE', '50846', 1, 'Adair', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -94.562813, 41.289069, '88260d41cdfffff'), ('S03406100022', Timestamp('2012-01-03 00:00:00'), 4509, "A J'S LIQUOR II", '2515 CHAMBERLAIN', 'AMES', '50010', 85, 'Story', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41906, 'Ursus Green Apple Vodka', 12, '750', 5.48, 8.22, 4, 32.88, 3.0, 0.79, -93.650838, 42.021461, '88262b3983fffff'), ('S03408300144', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 40956, 'Smirnoff Strawberry Vodka', 12, '750', 7.59, 11.56, 3, 34.68, 2.25, 0.59, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03383400010', Timestamp('2012-01-03 00:00:00'), 3488, "Bender's Foods", '400 S SECOND', 'BELLEVUE', '52031', 49, 'Jackson', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 12, 295.68, 12.0, 3.17, -90.42292, 42.255449, '88275b5ad5fffff'), ('S03390300066', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43316, 'Captain Morgan Private Stock Rum', 12, '750', 11.45, 17.18, 2, 34.36, 1.5, 0.4, -95.854809, 41.262316, '882608c9cdfffff'), ('S03410300061', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67527, 'Kahlua Coffee Liqueur', 12, '1000', 14.81, 22.22, 2, 44.44, 2.0, 0.53, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03396600032', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69638, "Dr. Mcgillicuddy's Cherry Schnapps", 6, '1750', 13.33, 19.99, 6, 119.94, 10.5, 2.77, -93.378772, 43.142868, '8826283b29fffff'), ('S03404800138', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37991, 'Smirnoff Vodka 80 Prf Mini', 12, '500', 6.97, 10.45, 1, 10.45, 0.5, 0.13, -91.035138, 41.451349, '88267421abfffff'), ('S03396400030', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 37934, 'Skol Vodka', 24, '375', 1.74, 2.6, 24, 62.4, 9.0, 2.38, -95.8792, 41.238092, '882608c9e7fffff'), ('S03402100066', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1031200, 'VODKA FLAVORED', 461, 'Campari(skyy)', 40810, 'Skyy Infusions Passion Fruit', 12, '750', 10.01, 15.01, 2, 30.02, 1.5, 0.4, -92.475403, 42.712263, '8826291801fffff'), ('S03393400016', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1081210, 'CREME DE ALMOND', 350, 'Luxco-Cleveland', 80396, 'Paramount Creme De Almond', 12, '750', 4.43, 6.65, 12, 79.8, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03404500002', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 63755, 'Tgi Fridays Mudslide', 6, '1750', 7.94, 11.91, 6, 71.46, 10.5, 2.77, -95.83539, 41.279189, '882608c829fffff'), ('S03407900060', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 58801, 'Captain Morgan Long Island Iced Tea', 6, '1750', 10.5, 15.74, 6, 94.44, 10.5, 2.77, 0.0, 0.0, '0'), ('S03408200094', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1022100, 'TEQUILA', 85, 'Brown-Forman Corporation', 3676, 'Herradura Blanco Silver 6x6 Tequila', 12, '750', 19.94, 29.91, 2, 59.82, 1.5, 0.4, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03408100026', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1012100, 'CANADIAN WHISKIES', 85, 'Brown-Forman Corporation', 12478, 'Canadian Mist Canadian Why Pet', 6, '1750', 10.5, 15.75, 6, 94.5, 10.5, 2.77, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03395800150', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88291, 'Patron Tequila Silver Mini', 10, '300', 20.3, 30.45, 1, 30.45, 0.3, 0.08, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03390200024', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26820, 'Jack Daniels Old #7 Black Lbl Flat', 24, '375', 6.06, 9.09, 1, 9.09, 0.38, 0.1, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03409700109', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1081340, 'ROOT BEER SCHNAPPS', 65, 'Jim Beam Brands', 82826, "Dekuyper Ragin' Root Beer Schnapps", 12, '750', 6.3, 9.45, 3, 28.35, 2.25, 0.59, -93.737447, 41.62941, '88260d802bfffff'), ('S03406000010', Timestamp('2012-01-03 00:00:00'), 4188, 'Main St. Market, Inc.', '882 MAIN ST', 'HOLY CROSS', '52053', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 24, 254.88, 24.0, 6.34, -90.995943, 42.600941, '88275a7469fffff'), ('S03406400045', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1032200, 'IMPORTED VODKA - MISC', 395, 'Proximo', 34886, 'Three Olives Berry Vodka', 12, '750', 11.16, 16.74, 2, 33.48, 1.5, 0.4, -90.766126, 41.509119, '88267434edfffff'), ('S03398600056', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45277, 'Paramount White Rum', 12, '1000', 4.33, 6.5, 2, 13.0, 2.0, 0.53, -91.343544, 40.626988, '8826703131fffff'), ('S03382400006', Timestamp('2012-01-03 00:00:00'), 4314, 'Nodaway Valley Market', '304  4TH ST', 'FONTANELLE', '50846', 1, 'Adair', 1012100, 'CANADIAN WHISKIES', 434, 'Luxco-St Louis', 13037, 'Canadian Reserve Whisky', 12, '1000', 5.31, 7.97, 12, 95.64, 12.0, 3.17, -94.562813, 41.289069, '88260d41cdfffff'), ('S03407800017', Timestamp('2012-01-03 00:00:00'), 4181, 'Puff N Snuff', '1101 RHOMBERG AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52596, 'E & J Vs', 12, '750', 5.51, 8.26, 1, 8.26, 0.75, 0.2, -90.660269, 42.519502, '88275a215bfffff'), ('S03410900064', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1012210, 'SINGLE MALT SCOTCH', 370, 'Pernod Ricard USA/Austin Nichols', 5086, 'Glenlivet 16yr Nadurra', 6, '750', 31.4, 47.1, 1, 47.1, 0.75, 0.2, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03390200023', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26823, 'Jack Daniels Old #7 Black Lbl', 48, '200', 3.69, 5.54, 2, 11.08, 0.4, 0.11, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03408300086', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081030, 'COFFEE LIQUEURS', 259, 'Heaven Hill Brands', 67426, 'Copa De Oro Mexican Coffee', 12, '750', 5.29, 7.94, 3, 23.82, 2.25, 0.59, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03383400020', Timestamp('2012-01-03 00:00:00'), 3488, "Bender's Foods", '400 S SECOND', 'BELLEVUE', '52031', 49, 'Jackson', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 4, 37.32, 3.0, 0.79, -90.42292, 42.255449, '88275b5ad5fffff'), ('S03410300094', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 11936, 'Canadian Ltd Whisky Convenience Pack', 12, '750', 3.75, 5.87, 3, 17.61, 2.25, 0.59, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03390300059', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1042100, 'IMPORTED DRY GINS', 260, 'Diageo Americas', 28867, 'Tanqueray Gin', 12, '1000', 14.99, 22.48, 1, 22.48, 1.0, 0.26, -95.854809, 41.262316, '882608c9cdfffff'), ('S03408600093', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43037, 'Bacardi Gold Rum', 12, '1000', 9.43, 14.14, 2, 28.28, 2.0, 0.53, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03383600022', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42716, 'Malibu Coconut Rum', 12, '750', 6.9, 10.35, 12, 124.2, 9.0, 2.38, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03410900035', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68039, "Bailey's Original Irish Cream", 4, '2400', 27.7, 41.55, 4, 166.2, 9.6, 2.54, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03390900019', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, 0.0, 0.0, '0'), ('S03390300039', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82787, 'Dekuyper Buttershots', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -95.854809, 41.262316, '882608c9cdfffff'), ('S03404800072', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 42009, 'UV Cake Vodka', 6, '1750', 10.63, 15.95, 6, 95.7, 10.5, 2.77, -91.035138, 41.451349, '88267421abfffff'), ('S03404700016', Timestamp('2012-01-03 00:00:00'), 3972, 'Hometown Foods / Hubbard', '210 MAPLE ST BOX 295', 'HUBBARD', '50122', 42, 'Hardin', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23828, 'Five Star', 6, '1750', 7.36, 11.04, 2, 22.08, 3.5, 0.92, -93.302481, 42.305571, '88262baa53fffff'), ('S03395800137', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25606, 'Seagrams 7 Crown Bl Whiskey', 12, '750', 6.14, 9.21, 12, 110.52, 9.0, 2.38, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03395200114', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82618, 'Dekuyper Hot Damn! Pet', 6, '1750', 10.5, 15.75, 3, 47.25, 5.25, 1.39, -91.110752, 40.801889, '882670a137fffff'), ('S03393100021', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 421, 'Sazerac Co., Inc.', 36976, 'Nikolai Vodka', 12, '750', 3.26, 4.88, 12, 58.56, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03402500018', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 73596, 'Dekuyper Michigan Cherry Liqueur', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03397000044', Timestamp('2012-01-03 00:00:00'), 3994, 'Great Spirits Wine & Liquor Co.', '72 NORTH HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54056, 'Arrow Apricot Flav Brandy', 12, '750', 5.0, 7.5, 1, 7.5, 0.75, 0.2, -95.127395, 43.368989000000006, '88262ac0e9fffff'), ('S03395200102', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21595, 'Ten High Traveler', 12, '750', 4.0, 5.99, 2, 11.98, 1.5, 0.4, -91.110752, 40.801889, '882670a137fffff'), ('S03395200100', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37346, 'Phillips Vodka', 12, '750', 3.48, 5.23, 3, 15.69, 2.25, 0.59, -91.110752, 40.801889, '882670a137fffff'), ('S03398800036', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 63894, 'Tgif On The Rocks White Russian', 12, '750', 3.82, 5.73, 2, 11.46, 1.5, 0.4, -93.573845, 41.604312, '88260d95b3fffff'), ('S03410900059', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 18, 458.64, 31.5, 8.32, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03403600014', Timestamp('2012-01-03 00:00:00'), 3744, 'Payless Foods / Dyersville', '733 16TH AVE SE', 'DYERSVILLE', '52040', 31, 'Dubuque', 1081312, 'BUTTERSCOTCH SCHNAPPS', 380, 'Phillips Beverage Company', 83363, 'Ice Hole Butterscotch Schnapps', 12, '750', 5.15, 7.73, 3, 23.19, 2.25, 0.59, -91.115769, 42.470204, '88275a654dfffff'), ('S03410900106', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34061, 'Absolut Ruby Red', 12, '750', 10.89, 16.33, 4, 65.32, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03408300228', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081400, 'TRIPLE SEC', 434, 'Luxco-St Louis', 86506, 'Paramount Triple Sec', 12, '750', 3.33, 4.99, 6, 29.94, 4.5, 1.19, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03383800072', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 461, 'Campari(skyy)', 64996, 'Frangelico Liqueur', 12, '750', 13.01, 19.52, 1, 19.52, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03383400012', Timestamp('2012-01-03 00:00:00'), 3488, "Bender's Foods", '400 S SECOND', 'BELLEVUE', '52031', 49, 'Jackson', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86887, 'Southern Comfort', 12, '1000', 11.34, 17.01, 12, 204.12, 12.0, 3.17, -90.42292, 42.255449, '88275b5ad5fffff'), ('S03397600053', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1022100, 'TEQUILA', 330, 'Gemini Spirits', 89496, 'Margaritaville Gold Tequila', 12, '750', 6.01, 9.52, 12, 114.24, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03403000014', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86884, 'Southern Comfort', 24, '375', 4.54, 6.8, 4, 27.2, 1.5, 0.4, -94.480204, 43.39966, '88262a82cbfffff'), ('S03408200099', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26824, 'Jack Daniels Old #7 Black Lbl Square', 24, '375', 6.06, 9.09, 6, 54.54, 2.25, 0.59, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03397300072', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43316, 'Captain Morgan Private Stock Rum', 12, '750', 11.45, 17.18, 3, 51.54, 2.25, 0.59, -91.044008, 41.427202, '8826742f4bfffff'), ('S03387700013', Timestamp('2012-01-03 00:00:00'), 3013, "Keith's Foods", '207 E LOCUST ST', 'BLOOMFIELD', '52537', 26, 'Davis', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 46350, 'Hawkeye Light Rum', 6, '1750', 7.56, 11.33, 6, 67.98, 10.5, 2.77, -92.412847, 40.752691, '88267284d5fffff'), ('S03404500007', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1012100, 'CANADIAN WHISKIES', 434, 'Luxco-St Louis', 13037, 'Canadian Reserve Whisky', 12, '1000', 5.31, 7.97, 12, 95.64, 12.0, 3.17, -95.83539, 41.279189, '882608c829fffff'), ('S03397500005', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68037, "Bailey's Original Irish Cream", 12, '1000', 16.5, 24.74, 12, 296.88, 12.0, 3.17, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03404800125', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 4626, "Buchanan's 12 Yr Deluxe Scotch", 12, '750', 20.0, 29.99, 1, 29.99, 0.75, 0.2, -91.035138, 41.451349, '88267421abfffff'), ('S03406000011', Timestamp('2012-01-03 00:00:00'), 4188, 'Main St. Market, Inc.', '882 MAIN ST', 'HOLY CROSS', '52053', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25607, 'Seagrams 7 Crown Bl Whiskey', 12, '1000', 7.5, 11.24, 12, 134.88, 12.0, 3.17, -90.995943, 42.600941, '88275a7469fffff'), ('S03390900034', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 3, 73.92, 3.0, 0.79, 0.0, 0.0, '0'), ('S03408800096', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28236, 'Bombay Sapphire Gin', 12, '750', 13.63, 20.45, 6, 122.7, 4.5, 1.19, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03405800011', Timestamp('2012-01-03 00:00:00'), 3761, "Cheese's Food Center", '200 N WALNUT ST', 'LENOX', '50851', 87, 'Taylor', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82866, 'Dekuyper Watermelon Pucker', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -94.560455, 40.882645, '882608a6e9fffff'), ('S03386600023', Timestamp('2012-01-03 00:00:00'), 3647, 'Wal-Mart 2889 / Clinton', '2715 SOUTH 25TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82606, 'Dekuyper Sour Apple Pucker', 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -90.245005, 41.818874, '88275b78b7fffff'), ('S03384700004', Timestamp('2012-01-03 00:00:00'), 4138, 'Fareway Stores #657 / Indianola', '1309 W 2ND', 'INDIANOLA', '50125', 91, 'Warren', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11296, 'Crown Royal', 12, '750', 14.25, 21.37, 12, 256.44, 9.0, 2.38, -93.579252, 41.356616, '88260db8e9fffff'), ('S03388300019', Timestamp('2012-01-03 00:00:00'), 4039, 'No Frills Supermarkets / Council Blu', '1817 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 30, 309.0, 52.5, 13.87, -95.870993, 41.261863, '882608c917fffff'), ('S03410300139', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37991, 'Smirnoff Vodka 80 Prf Mini', 12, '500', 6.97, 10.45, 1, 10.45, 0.5, 0.13, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03406100011', Timestamp('2012-01-03 00:00:00'), 4509, "A J'S LIQUOR II", '2515 CHAMBERLAIN', 'AMES', '50010', 85, 'Story', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15626, 'Jameson', 12, '750', 13.46, 20.2, 2, 40.4, 1.5, 0.4, -93.650838, 42.021461, '88262b3983fffff'), ('S03407600006', Timestamp('2012-01-03 00:00:00'), 3465, 'Hometown Foods / Panora', '601 E MAIN', 'PANORA', '50216', 39, 'Guthrie', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 6, 97.98, 4.5, 1.19, -94.357201, 41.69182, '88260dd89bfffff'), ('S03383500016', Timestamp('2012-01-03 00:00:00'), 3756, 'Main Street Liquors / Manning', '316 MAIN ST', 'MANNING', '51455', 14, 'Carroll', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67524, 'Kahlua Coffee Liqueur', 24, '375', 6.43, 9.65, 2, 19.3, 0.75, 0.2, -95.065122, 41.908573, '88260c0525fffff'), ('S03396000085', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69667, "Dr. Mcgillicuddy's Vanilla", 12, '1000', 11.0, 16.5, 12, 198.0, 12.0, 3.17, -90.705328, 42.491862, '88275a2131fffff'), ('S03397600069', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37346, 'Phillips Vodka', 12, '750', 3.48, 5.23, 12, 62.76, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03385500060', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1012200, 'SCOTCH WHISKIES', 55, 'Sazerac North America', 8827, "Lauder's", 12, '1000', 6.13, 9.2, 4, 36.8, 4.0, 1.06, -93.21693, 42.741728, '88262b9605fffff'), ('S03391000007', Timestamp('2012-01-03 00:00:00'), 3654, 'The Beverage Shop / Belmond', '207 E MAIN ST', 'BELMOND', '50421', 99, 'Wright', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 12, 295.68, 12.0, 3.17, -93.615247, 42.845901, '8826286587fffff'), ('S03390800009', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1101100, 'AMERICAN ALCOHOL', 434, 'Luxco-St Louis', 41846, 'Everclear Alcohol', 12, '750', 8.02, 12.03, 12, 144.36, 9.0, 2.38, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03396300008', Timestamp('2012-01-03 00:00:00'), 4041, 'Mandm Pantry', '2344 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43124, 'Bacardi Superior Rum', 24, '375', 4.3, 6.45, 6, 38.7, 2.25, 0.59, -90.213015, 41.821777, '88275b7897fffff'), ('S03394800002', Timestamp('2012-01-03 00:00:00'), 3880, 'Target Store T-2454 / Council Bluffs', '3804 METRO DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19476, "Maker's Mark", 12, '750', 15.74, 23.61, 12, 283.32, 9.0, 2.38, -95.837595, 41.220979, '882608c995fffff'), ('S03410700032', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25606, 'Seagrams 7 Crown Bl Whiskey', 12, '750', 6.14, 9.21, 12, 110.52, 9.0, 2.38, -94.853591, 42.064155, '88260c16a9fffff'), ('S03395800200', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1032080, 'IMPORTED VODKA', 190, 'Firestarter Spirits Inc.', 34709, 'Firestarter Vodka', 6, '750', 11.71, 17.57, 3, 52.71, 2.25, 0.59, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03411500003', Timestamp('2012-01-03 00:00:00'), 3700, 'Avoca Liquor', '158 SOUTH ELM', 'AVOCA', '51521', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11788, 'Black Velvet', 6, '1750', 10.45, 15.67, 6, 94.02, 10.5, 2.77, -95.339412, 41.476306, '88260c6c8bfffff'), ('S03398600041', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 6, 97.98, 4.5, 1.19, -91.343544, 40.626988, '8826703131fffff'), ('S03396400017', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 24, 416.88, 18.0, 4.76, -95.8792, 41.238092, '882608c9e7fffff'), ('S03397300047', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 80571, 'Arrow Peppermint Schnapps Mini', 12, '500', 4.54, 6.81, 1, 6.81, 0.5, 0.13, -91.044008, 41.427202, '8826742f4bfffff'), ('S03398600047', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34820, 'Svedka Vodka', 12, '750', 7.97, 11.95, 2, 23.9, 1.5, 0.4, -91.343544, 40.626988, '8826703131fffff'), ('S03395800144', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 24, 254.88, 24.0, 6.34, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03397600061', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081400, 'TRIPLE SEC', 370, 'Pernod Ricard USA/Austin Nichols', 86190, 'Hiram Walker Triple Sec', 12, '1000', 2.33, 3.49, 12, 41.88, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03403100003', Timestamp('2012-01-03 00:00:00'), 4462, "SHELTON'S", '113, 1ST ST NW', 'BUFFALO CENTER', '50424', 95, 'Winnebago', 1062310, 'SPICED RUM', 322, 'Prestige Wine and Spirits Group', 77318, 'Rondiaz Rum Spiced', 6, '1750', 10.0, 15.0, 6, 90.0, 10.5, 2.77, -93.947595, 43.386755, '8826284217fffff'), ('S03383400013', Timestamp('2012-01-03 00:00:00'), 3488, "Bender's Foods", '400 S SECOND', 'BELLEVUE', '52031', 49, 'Jackson', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 6, 152.88, 10.5, 2.77, -90.42292, 42.255449, '88275b5ad5fffff'), ('S03406100009', Timestamp('2012-01-03 00:00:00'), 4509, "A J'S LIQUOR II", '2515 CHAMBERLAIN', 'AMES', '50010', 85, 'Story', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34820, 'Svedka Vodka', 12, '750', 7.97, 11.95, 12, 143.4, 9.0, 2.38, -93.650838, 42.021461, '88262b3983fffff'), ('S03401000005', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34821, 'Svedka Vodka', 6, '1750', 13.89, 20.84, 6, 125.04, 10.5, 2.77, -90.683356, 42.52717, '88275a215dfffff'), ('S03387400004', Timestamp('2012-01-03 00:00:00'), 3642, "Fraise Foods / Sonny's Supermarket", '327  5TH ST', 'WEST POINT', '52656', 56, 'Lee', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86888, 'Southern Comfort PET', 6, '1750', 18.14, 27.21, 6, 163.26, 10.5, 2.77, -91.452491, 40.717867, '88267005ebfffff'), ('S03411000019', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1041100, 'AMERICAN DRY GINS', 306, 'Mississippi River Distilling Co.', 31990, 'River Rose Gin', 6, '750', 16.17, 24.25, 6, 145.5, 4.5, 1.19, 0.0, 0.0, '0'), ('S03403300008', Timestamp('2012-01-03 00:00:00'), 4566, 'Kum & Go #508 / Cedar Rapids', '30 FOUR OAKS DR SW', 'CEDAR RAPIDS', '52404', 57, 'Linn', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 12, 208.44, 9.0, 2.38, -91.658073, 41.923446, '8826745503fffff'), ('S03408100056', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 2, 21.24, 2.0, 0.53, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03405900052', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67527, 'Kahlua Coffee Liqueur', 12, '1000', 14.81, 22.22, 12, 266.64, 12.0, 3.17, -90.48065, 41.529655, '882675c361fffff'), ('S03411100035', Timestamp('2012-01-03 00:00:00'), 3631, 'Audubon Food Pride', '104 MARKET ST PO BOX 206', 'AUDUBON', '50025', 5, 'Audubon', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 80096, 'Dekuyper Signature Creme De Banana', 12, '750', 5.25, 7.87, 2, 15.74, 1.5, 0.4, -94.93625, 41.723057, '88260c388dfffff'), ('S03406300024', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82867, 'Dekuyper Watermelon Pucker', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -91.281344, 42.108275, '882674c6a9fffff'), ('S03395200079', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54056, 'Arrow Apricot Flav Brandy', 12, '750', 5.0, 7.5, 3, 22.5, 2.25, 0.59, -91.110752, 40.801889, '882670a137fffff'), ('S03386600024', Timestamp('2012-01-03 00:00:00'), 3647, 'Wal-Mart 2889 / Clinton', '2715 SOUTH 25TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081200, 'CREAM LIQUEURS', 330, 'Gemini Spirits', 80456, "Ryan's Cream Liqueur", 12, '750', 6.69, 10.04, 12, 120.48, 9.0, 2.38, -90.245005, 41.818874, '88275b78b7fffff'), ('S03407400042', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23826, 'Five Star', 12, '750', 3.35, 5.02, 3, 15.06, 2.25, 0.59, -93.736517, 41.659057, '88260d8045fffff'), ('S03402500021', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1081210, 'CREME DE ALMOND', 350, 'Luxco-Cleveland', 80396, 'Paramount Creme De Almond', 12, '750', 4.43, 6.65, 3, 19.95, 2.25, 0.59, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03410300055', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82867, 'Dekuyper Watermelon Pucker', 12, '1000', 7.35, 11.02, 2, 22.04, 2.0, 0.53, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03410700078', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1011100, 'BLENDED WHISKIES', 125, 'Cedar Ridge Vineyards,LLC', 27512, "Griff's Cowboy Whiskey", 6, '750', 15.55, 23.32, 4, 93.28, 3.0, 0.79, -94.853591, 42.064155, '88260c16a9fffff'), ('S03396000092', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86887, 'Southern Comfort', 12, '1000', 11.34, 17.01, 84, 1428.84, 84.0, 22.19, -90.705328, 42.491862, '88275a2131fffff'), ('S03389100004', Timestamp('2012-01-03 00:00:00'), 4457, 'KUM & GO #422 / IOWA CITY', '513 S RIVERSIDE DR', 'IOWA CITY', '52240', 52, 'Johnson', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45276, 'Paramount White Rum', 12, '750', 3.85, 5.77, 12, 69.24, 9.0, 2.38, -91.540402, 41.653301, '88267471e9fffff'), ('S03411600005', Timestamp('2012-01-03 00:00:00'), 4001, 'Wal-Mart 1787 / Carroll', '2014 KITTY HAWK AVE', 'CARROLL', '51401', 14, 'Carroll', 1101100, 'AMERICAN ALCOHOL', 434, 'Luxco-St Louis', 41846, 'Everclear Alcohol', 12, '750', 8.02, 12.03, 12, 144.36, 9.0, 2.38, -94.893501, 42.081312, '88260c144bfffff'), ('S03388500097', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75208, 'Kinky Liqueur Mini', 6, '500', 4.67, 7.0, 2, 14.0, 1.0, 0.26, -90.668138, 42.504959, '88275a2119fffff'), ('S03384700020', Timestamp('2012-01-03 00:00:00'), 4138, 'Fareway Stores #657 / Indianola', '1309 W 2ND', 'INDIANOLA', '50125', 91, 'Warren', 1051010, 'AMERICAN GRAPE BRANDIES', 259, 'Heaven Hill Brands', 52316, 'Christian Bros Brandy', 12, '750', 5.51, 8.26, 4, 33.04, 3.0, 0.79, -93.579252, 41.356616, '88260db8e9fffff'), ('S03397300022', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37348, 'Phillips Vodka', 6, '1750', 7.31, 10.97, 6, 65.82, 10.5, 2.77, -91.044008, 41.427202, '8826742f4bfffff'), ('S03392100004', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 305, 'MHW Ltd', 64000, 'Absente', 12, '750', 21.67, 32.5, 2, 65.0, 1.5, 0.4, -90.204748, 41.827845, '88275b7891fffff'), ('S03406700012', Timestamp('2012-01-03 00:00:00'), 3849, 'Hartig Drug Co #6 / Dyersville', '711  16TH AVE SE', 'DYERSVILLE', '52040', 31, 'Dubuque', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82786, 'Dekuyper Buttershots Schnapps', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -91.116087, 42.470285, '88275a654dfffff'), ('S03394000019', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 461, 'Campari(skyy)', 37987, 'Skyy Vodka', 12, '1000', 12.34, 18.52, 24, 444.48, 24.0, 6.34, -95.902023, 41.257663, '882608c937fffff'), ('S03397100013', Timestamp('2012-01-03 00:00:00'), 4306, 'Northside One Stop / Hampton', '1208  4TH ST NE', 'HAMPTON', '50441', 35, 'Franklin', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43316, 'Captain Morgan Private Stock Rum', 12, '750', 11.45, 17.18, 2, 34.36, 1.5, 0.4, -93.202452, 42.753045, '88262b9647fffff'), ('S03395800095', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081317, 'GRAPE SCHNAPPS', 65, 'Jim Beam Brands', 82637, 'Dekuyper Grape Pucker', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03398800043', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1041100, 'AMERICAN DRY GINS', 55, 'Sazerac North America', 30056, "Fleischmann's Gin", 12, '750', 3.25, 4.88, 2, 9.76, 1.5, 0.4, -93.573845, 41.604312, '88260d95b3fffff'), ('S03407700021', Timestamp('2012-01-03 00:00:00'), 4602, 'Cheap Smokes / Beer City', '58231 190th ST', 'PACIFIC JUNCTION', '51561', 65, 'Mills', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41076, 'Jeremiah Weed Sweet Tea Vodka', 12, '750', 9.46, 14.2, 2, 28.4, 1.5, 0.4, -95.822983, 41.041739, '8826084255fffff'), ('S03395800082', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41698, 'Uv Ivory (vanilla) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03402100032', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58875, 'Jose Cuervo Golden Margarita', 6, '1750', 10.0, 15.24, 6, 91.44, 10.5, 2.77, -92.475403, 42.712263, '8826291801fffff'), ('S03402500006', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11776, 'Black Velvet', 12, '750', 4.83, 7.44, 12, 89.28, 9.0, 2.38, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03388500082', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37991, 'Smirnoff Vodka 80 Prf Mini', 12, '500', 6.97, 10.45, 2, 20.9, 1.0, 0.26, -90.668138, 42.504959, '88275a2119fffff'), ('S03408300199', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31658, 'Paramount Gin', 6, '1750', 7.1, 10.65, 1, 10.65, 1.75, 0.46, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03395200092', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43025, 'Admiral Nelson Spiced Rum Traveler', 12, '750', 5.77, 8.66, 5, 43.3, 3.75, 0.99, -91.110752, 40.801889, '882670a137fffff'), ('S03408000034', Timestamp('2012-01-03 00:00:00'), 3622, 'Wal-Mart 1415 / Spirit Lake', '2600 HWY 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 12, 208.44, 9.0, 2.38, -95.126535, 43.416452, '88262ac237fffff'), ('S03395800057', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57144, "Chi-Chi's White Russian", 6, '1750', 5.99, 9.24, 6, 55.44, 10.5, 2.77, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03395800104', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081230, 'WHITE CREME DE CACAO', 434, 'Luxco-St Louis', 78616, 'Arrow Creme de Cacao White', 12, '750', 4.39, 6.58, 12, 78.96, 9.0, 2.38, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03405900070', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1062300, 'FLAVORED RUM', 260, 'Diageo Americas', 43226, 'Captain Morgan Parrot Bay Passion Fruit', 12, '750', 7.49, 11.23, 2, 22.46, 1.5, 0.4, -90.48065, 41.529655, '882675c361fffff'), ('S03392300085', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11366, 'Crown Royal Special Reserve', 12, '750', 24.3, 36.44, 1, 36.44, 0.75, 0.2, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03404800018', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37997, 'Smirnoff Vodka 80 Prf', 12, '1000', 8.49, 12.74, 12, 152.88, 12.0, 3.17, -91.035138, 41.451349, '88267421abfffff'), ('S03383900018', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35317, 'Barton Vodka', 12, '1000', 3.97, 5.95, 24, 142.8, 24.0, 6.34, -91.403752, 40.41538, '8826702f65fffff'), ('S03388500072', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34116, 'Absolut Mandrin', 12, '750', 10.89, 16.33, 1, 16.33, 0.75, 0.2, -90.668138, 42.504959, '88275a2119fffff'), ('S03385300040', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58835, 'Jose Cuervo Authentic Lime Margarita', 6, '800', 4.62, 6.92, 6, 41.52, 4.8, 1.27, -91.136655, 40.814666, '882670aacdfffff'), ('S03408200013', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58866, 'Jose Cuervo Authentic Strawberry Margarita', 12, '750', 4.62, 6.92, 12, 83.04, 9.0, 2.38, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03411200068', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 987, 'Stellar Importing Company, LLC', 64016, 'Achaia Clauss Ouzo Greek Liqueur', 12, '750', 9.16, 13.74, 1, 13.74, 0.75, 0.2, -93.697731, 41.665172, '88260d8231fffff'), ('S03405900024', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1081400, 'TRIPLE SEC', 434, 'Luxco-St Louis', 86251, 'Juarez Triple Sec', 12, '1000', 2.21, 3.31, 24, 79.44, 24.0, 6.34, -90.48065, 41.529655, '882675c361fffff'), ('S03388100120', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 240, 'WILLIAM GRANT AND SONS, INC.', 45884, 'Sailor Jerry Spiced Navy Rum', 24, '375', 4.53, 6.8, 2, 13.6, 0.75, 0.2, -95.879662, 41.261925, '882608c93bfffff'), ('S03394400002', Timestamp('2012-01-03 00:00:00'), 4334, 'Fareway Stores #114 / Dubuque', '2050 JOHN F KENNEDY RD', 'DUBUQUE', '52001', 31, 'Dubuque', 1101100, 'AMERICAN ALCOHOL', 434, 'Luxco-St Louis', 41846, 'Everclear Alcohol', 12, '750', 8.02, 12.03, 12, 144.36, 9.0, 2.38, 0.0, 0.0, '0'), ('S03383600010', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03390800006', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41696, 'Uv Green (apple) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03404700012', Timestamp('2012-01-03 00:00:00'), 3972, 'Hometown Foods / Hubbard', '210 MAPLE ST BOX 295', 'HUBBARD', '50122', 42, 'Hardin', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24456, 'Kessler Blend Whiskey', 12, '750', 5.51, 8.26, 2, 16.52, 1.5, 0.4, -93.302481, 42.305571, '88262baa53fffff'), ('S03389600008', Timestamp('2012-01-03 00:00:00'), 3632, 'Wal-Mart 2004 / Dubuque', '4200 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34433, 'Grey Goose Vodka', 12, '750', 17.97, 26.96, 12, 323.52, 9.0, 2.38, -90.736955, 42.489041, '88275a2ac1fffff'), ('S03396400068', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25608, 'Seagrams 7 Crown Bl Whiskey', 6, '1750', 10.76, 16.4, 12, 196.8, 21.0, 5.55, -95.8792, 41.238092, '882608c9e7fffff'), ('S03407200022', Timestamp('2012-01-03 00:00:00'), 4146, 'Fareway Stores #561 / Waverly', '220 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41698, 'Uv Ivory (vanilla) Vodka', 12, '750', 6.22, 9.33, 4, 37.32, 3.0, 0.79, -92.473476, 42.725748, '8826291847fffff'), ('S03407900073', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26821, 'Jack Daniels Old #7 Black Lbl Mini', 12, '500', 8.62, 12.94, 1, 12.94, 0.5, 0.13, 0.0, 0.0, '0'), ('S03390900017', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35918, "Five O'clock Vodka", 6, '1750', 6.92, 10.38, 6, 62.28, 10.5, 2.77, 0.0, 0.0, '0'), ('S03396000013', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11788, 'Black Velvet', 6, '1750', 10.45, 15.67, 12, 188.04, 21.0, 5.55, -90.705328, 42.491862, '88275a2131fffff'), ('S03393900064', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37994, 'Smirnoff Vodka 80 Prf', 24, '375', 4.48, 6.72, 6, 40.32, 2.25, 0.59, -95.873073, 41.261866, '882608c917fffff'), ('S03391300038', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1011200, 'STRAIGHT BOURBON WHISKIES', 260, 'Diageo Americas', 27508, 'Jeremiah Weed Cherry Mash', 12, '750', 8.92, 13.38, 1, 13.38, 0.75, 0.2, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03408400036', Timestamp('2012-01-03 00:00:00'), 2650, 'Hy-Vee Wine and Spirits / Harlan', '1808 23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10632, 'Canadian Club Reserve Triple Aged', 12, '750', 8.92, 13.38, 2, 26.76, 1.5, 0.4, -95.339881, 41.650658, '88260c60e9fffff'), ('S03397600086', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 67006, 'Midori Melon Liqueur', 12, '750', 10.97, 16.46, 12, 197.52, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03393400034', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35948, "Fleischmann's Royal Vodka 80 Prf", 6, '1750', 7.14, 10.7, 90, 963.0, 157.5, 41.61, -90.741064, 42.516633, '88275a212dfffff'), ('S03395800191', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32234, 'Seagrams Extra Dry Gin', 24, '375', 3.4, 5.1, 8, 40.8, 3.0, 0.79, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03398600051', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1031200, 'VODKA FLAVORED', 65, 'Jim Beam Brands', 40841, 'Pucker Vodka Wild Cherry Tease', 12, '750', 9.99, 14.99, 2, 29.98, 1.5, 0.4, -91.343544, 40.626988, '8826703131fffff'), ('S03412100032', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32235, 'Seagrams Extra Dry Gin Pet', 12, '750', 5.9, 8.85, 12, 106.2, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03404800063', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69636, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -91.035138, 41.451349, '88267421abfffff'), ('S03391100002', Timestamp('2012-01-03 00:00:00'), 4172, 'Shamrock Spirits', '1204 WATER ST PO BOX 177', 'ALDEN', '50006', 42, 'Hardin', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11777, 'Black Velvet', 12, '1000', 6.63, 9.94, 24, 238.56, 24.0, 6.34, -93.376271, 42.520423, '88262b8ed9fffff'), ('S03405400011', Timestamp('2012-01-03 00:00:00'), 3816, 'Swils', '200 E OAK ST', 'RED OAK', '51566', 69, 'Montgomery', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35918, "Five O'clock Vodka", 6, '1750', 6.92, 10.38, 6, 62.28, 10.5, 2.77, -95.230032, 41.016691, '882608152bfffff'), ('S03391500002', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58838, 'Jose Cuervo Authentic Lime Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, -95.879662, 41.261925, '882608c93bfffff'), ('S03411000103', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32236, 'Seagrams Extra Dry Gin', 12, '750', 5.9, 8.85, 4, 35.4, 3.0, 0.79, 0.0, 0.0, '0'), ('S03395200068', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1031080, 'VODKA 80 PROOF', 205, 'E AND J GALLO WINERY', 36969, 'New Amsterdam Vodka', 12, '750', 6.9, 10.35, 7, 72.45, 5.25, 1.39, -91.110752, 40.801889, '882670a137fffff'), ('S03385300076', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68034, "Bailey's Original Irish Cream", 12, '375', 6.75, 10.12, 2, 20.24, 0.75, 0.2, -91.136655, 40.814666, '882670aacdfffff'), ('S03406400009', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1071100, 'AMERICAN COCKTAILS', 65, 'Jim Beam Brands', 63355, 'Skinnygirl Margarita', 12, '750', 7.69, 11.53, 1, 11.53, 0.75, 0.2, -90.766126, 41.509119, '88267434edfffff'), ('S03395200082', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1081200, 'CREAM LIQUEURS', 461, 'Campari(skyy)', 68126, "Carolan's Irish Cream Liqueur", 12, '750', 7.69, 11.54, 2, 23.08, 1.5, 0.4, -91.110752, 40.801889, '882670a137fffff'), ('S03400700064', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 10836, 'Crown Royal Black', 12, '750', 16.47, 24.7, 4, 98.8, 3.0, 0.79, 0.0, 0.0, '0'), ('S03383700014', Timestamp('2012-01-03 00:00:00'), 3934, 'Wal-Mart 1509 / Maquoketa', '103 E CARLISLE', 'MAQUOKETA', '52060', 49, 'Jackson', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43336, 'Captain Morgan Original Spiced', 12, '750', 8.5, 12.74, 24, 305.76, 18.0, 4.76, -90.66292, 42.04869, '8826748693fffff'), ('S03411100028', Timestamp('2012-01-03 00:00:00'), 3631, 'Audubon Food Pride', '104 MARKET ST PO BOX 206', 'AUDUBON', '50025', 5, 'Audubon', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 2, 44.68, 2.0, 0.53, -94.93625, 41.723057, '88260c388dfffff'), ('S03405500003', Timestamp('2012-01-03 00:00:00'), 4203, 'Liquor and Tobacco Outlet /', '620 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1011200, 'STRAIGHT BOURBON WHISKIES', 461, 'Campari(skyy)', 77776, 'Wild Turkey American Honey', 12, '750', 10.5, 15.75, 12, 189.0, 9.0, 2.38, -92.47898, 42.725694, '8826291847fffff'), ('S03387300009', Timestamp('2012-01-03 00:00:00'), 3549, 'Quicker Liquor Store', '1414 48TH ST', 'FORT MADISON', '52627', 56, 'Lee', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35914, "Five O'clock Vodka", 24, '375', 1.84, 2.75, 24, 66.0, 9.0, 2.38, -91.373211, 40.62422600000001, '8826703ac5fffff'), ('S03396500003', Timestamp('2012-01-03 00:00:00'), 3587, 'Save A Lot  #482', '2122 MAIN ST', 'KEOKUK', '52632', 56, 'Lee', 1022100, 'TEQUILA', 434, 'Luxco-St Louis', 87937, 'Juarez Tequila Silver', 12, '1000', 6.67, 10.0, 12, 120.0, 12.0, 3.17, -91.39688800000002, 40.408821, '8826702f67fffff'), ('S03390300033', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 36, 593.64, 36.0, 9.51, -95.854809, 41.262316, '882608c9cdfffff'), ('S03409700091', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1081030, 'COFFEE LIQUEURS', 259, 'Heaven Hill Brands', 67426, 'Copa De Oro Mexican Coffee', 12, '750', 5.29, 7.94, 6, 47.64, 4.5, 1.19, -93.737447, 41.62941, '88260d802bfffff'), ('S03392900028', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42717, 'Malibu Coconut Rum', 12, '1000', 9.77, 14.66, 12, 175.92, 12.0, 3.17, 0.0, 0.0, '0'), ('S03391300138', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1071100, 'AMERICAN COCKTAILS', 421, 'Sazerac Co., Inc.', 59231, 'Ice Box Mudslide', 6, '1750', 7.01, 10.52, 2, 21.04, 3.5, 0.92, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03397000021', Timestamp('2012-01-03 00:00:00'), 3994, 'Great Spirits Wine & Liquor Co.', '72 NORTH HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1011200, 'STRAIGHT BOURBON WHISKIES', 85, 'Brown-Forman Corporation', 17830, 'Early Times 36 Mo Ky Str Bourbon Pet', 6, '1750', 10.88, 16.32, 1, 16.32, 1.75, 0.46, -95.127395, 43.368989000000006, '88262ac0e9fffff'), ('S03395800026', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32236, 'Seagrams Extra Dry Gin', 12, '750', 5.9, 8.85, 12, 106.2, 9.0, 2.38, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03387700016', Timestamp('2012-01-03 00:00:00'), 3013, "Keith's Foods", '207 E LOCUST ST', 'BLOOMFIELD', '52537', 26, 'Davis', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65195, 'Hpnotiq', 6, '750', 11.68, 17.52, 1, 17.52, 0.75, 0.2, -92.412847, 40.752691, '88267284d5fffff'), ('S03405200041', Timestamp('2012-01-03 00:00:00'), 2954, "Dahl's / Ingersoll", '3425 INGERSOLL', 'DES MOINES', '50300', 77, 'Polk', 1062100, 'JAMAICA RUM', 260, 'Diageo Americas', 42166, "Myers's Rum Original Dark", 12, '750', 9.49, 14.24, 5, 71.2, 3.75, 0.99, -93.663337, 41.586313, '88260d80d7fffff'), ('S03401100008', Timestamp('2012-01-03 00:00:00'), 4239, 'Fareway Stores #850 / Spirit Lake', '1906 KEOKUK', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15248, 'Windsor Canadian Pet', 6, '1750', 8.92, 13.38, 6, 80.28, 10.5, 2.77, -95.106219, 43.42157, '88262ac23bfffff'), ('S03407100005', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1031080, 'VODKA 80 PROOF', 421, 'Sazerac Co., Inc.', 36978, 'Nikolai Vodka', 6, '1750', 6.87, 10.3, 6, 61.8, 10.5, 2.77, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03406800010', Timestamp('2012-01-03 00:00:00'), 4010, "Newman's Thriftway", '900 S LOCUST ST', 'GLENWOOD', '51534', 65, 'Mills', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81206, 'Paramount Peppermint Schnapps', 12, '750', 3.7, 5.55, 12, 66.6, 9.0, 2.38, -95.745944, 41.038205, '8826085555fffff'), ('S03392900003', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57129, "Chi-Chi's Mexican Mudslide", 6, '1750', 5.99, 9.24, 6, 55.44, 10.5, 2.77, 0.0, 0.0, '0'), ('S03388500054', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 12406, 'Canadian Ltd Whisky', 12, '750', 3.75, 5.87, 2, 11.74, 1.5, 0.4, -90.668138, 42.504959, '88275a2119fffff'), ('S03381300008', Timestamp('2012-01-03 00:00:00'), 3944, "Sam's Club 4973 / Dubuque", '4400 ASBURY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35317, 'Barton Vodka', 12, '1000', 3.97, 5.95, 12, 71.4, 12.0, 3.17, -90.737821, 42.515289, '88275a212dfffff'), ('S03388500048', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43124, 'Bacardi Superior Rum', 24, '375', 4.3, 6.45, 6, 38.7, 2.25, 0.59, -90.668138, 42.504959, '88275a2119fffff'), ('S03385900011', Timestamp('2012-01-03 00:00:00'), 4356, 'Kum & Go #396 / Nevada', '136 LINCOLN HWY', 'NEVADA', '50201', 85, 'Story', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26820, 'Jack Daniels Old #7 Black Lbl Flat', 24, '375', 6.06, 9.09, 3, 27.27, 1.12, 0.3, -93.458003, 42.022725, '88262b235bfffff'), ('S03393900044', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23626, 'Five Star Pet', 12, '750', 3.35, 5.02, 7, 35.14, 5.25, 1.39, -95.873073, 41.261866, '882608c917fffff'), ('S03389900011', Timestamp('2012-01-03 00:00:00'), 2532, 'Hy-Vee Food Store #2 / Des Moines', '2540 E EUCLID', 'DES MOINES', '50317', 77, 'Polk', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43051, 'Bacardi Dragon Berry', 12, '750', 8.33, 12.49, 12, 149.88, 9.0, 2.38, -93.569452, 41.627838, '88260d9581fffff'), ('S03388500096', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43051, 'Bacardi Dragon Berry', 12, '750', 8.33, 12.49, 4, 49.96, 3.0, 0.79, -90.668138, 42.504959, '88275a2119fffff'), ('S03392100051', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65254, 'Jagermeister Liqueur', 24, '375', 6.23, 9.34, 2, 18.68, 0.75, 0.2, -90.204748, 41.827845, '88275b7891fffff'), ('S03395700057', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82613, 'Dekuyper Hot Damn!', 24, '375', 3.06, 4.59, 4, 18.36, 1.5, 0.4, -95.147741, 43.14521, '88262a1925fffff'), ('S03408800027', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 36188, "Gordon's Vodka 80 Prf - Pet", 6, '1750', 9.68, 14.52, 6, 87.12, 10.5, 2.77, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03393600009', Timestamp('2012-01-03 00:00:00'), 4040, 'No Frills Supermarkets / Council Blu', '1801 VALLEY VIEW DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36907, 'Mccormick Vodka PET', 12, '1000', 3.98, 5.97, 12, 71.64, 12.0, 3.17, -95.81877500000002, 41.24394, '882608c9d9fffff'), ('S03386600032', Timestamp('2012-01-03 00:00:00'), 3647, 'Wal-Mart 2889 / Clinton', '2715 SOUTH 25TH ST', 'CLINTON', '52732', 23, 'Clinton', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24458, 'Kessler Blend Whiskey', 6, '1750', 11.01, 16.52, 6, 99.12, 10.5, 2.77, -90.245005, 41.818874, '88275b78b7fffff'), ('S03390300034', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42717, 'Malibu Coconut Rum', 12, '1000', 9.77, 14.66, 24, 351.84, 24.0, 6.34, -95.854809, 41.262316, '882608c9cdfffff'), ('S03395800206', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1032200, 'IMPORTED VODKA - MISC', 115, 'Constellation Wine Company, Inc.', 34812, 'Svedka Raspberry Vodka', 12, '750', 7.97, 11.95, 4, 47.8, 3.0, 0.79, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03390900047', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 63792, 'Tgif On The Rocks Pina Colada', 12, '750', 3.82, 5.73, 2, 11.46, 1.5, 0.4, 0.0, 0.0, '0'), ('S03395200109', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45246, 'Paramount Gold Rum', 12, '750', 3.85, 5.77, 3, 17.31, 2.25, 0.59, -91.110752, 40.801889, '882670a137fffff'), ('S03388000082', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 85, 'Brown-Forman Corporation', 64676, 'Chambord Liqueur Royale De France', 12, '750', 18.56, 27.84, 2, 55.68, 1.5, 0.4, -93.633306, 43.261538, '88262808e7fffff'), ('S03388000061', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -93.633306, 43.261538, '88262808e7fffff'), ('S03397300038', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43336, 'Captain Morgan Original Spiced', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -91.044008, 41.427202, '8826742f4bfffff'), ('S03397600161', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1032200, 'IMPORTED VODKA - MISC', 35, 'Bacardi U.S.A., Inc.', 34436, "Grey Goose Vodka L'orange", 6, '750', 17.97, 26.96, 6, 161.76, 4.5, 1.19, -95.825137, 41.242732, '882608c9d1fffff'), ('S03408600033', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 461, 'Campari(skyy)', 67192, 'X Rated Fusion Liqueur', 6, '750', 14.66, 21.99, 1, 21.99, 0.75, 0.2, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03409700077', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43197, 'Bacardi Torched Cherry', 12, '750', 8.33, 12.49, 3, 37.47, 2.25, 0.59, -93.737447, 41.62941, '88260d802bfffff'), ('S03392500029', Timestamp('2012-01-03 00:00:00'), 3908, 'County Market #214 / Fort Madison', '11802 AVENUE H', 'FORT MADISON', '52627', 56, 'Lee', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37996, 'Smirnoff Vodka 80 Prf', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, 0.0, 0.0, '0'), ('S03402900054', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86916, 'Southern Comfort 100 Prf', 12, '750', 11.65, 17.47, 2, 34.94, 1.5, 0.4, -94.990031, 41.403856, '8826089095fffff'), ('S03393900029', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1051110, 'BLACKBERRY BRANDIES', 55, 'Sazerac North America', 55066, 'Mr. Boston Blackberry Brandy', 12, '750', 4.12, 6.37, 1, 6.37, 0.75, 0.2, -95.873073, 41.261866, '882608c917fffff'), ('S03408800043', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68039, "Bailey's Original Irish Cream", 4, '2400', 27.7, 41.55, 4, 166.2, 9.6, 2.54, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03396300011', Timestamp('2012-01-03 00:00:00'), 4041, 'Mandm Pantry', '2344 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43126, 'Bacardi Superior Rum', 12, '750', 7.53, 11.3, 3, 33.9, 2.25, 0.59, -90.213015, 41.821777, '88275b7897fffff'), ('S03383900077', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 44258, 'Blackheart Premium Spiced Rum', 12, '750', 7.94, 11.91, 4, 47.64, 3.0, 0.79, -91.403752, 40.41538, '8826702f65fffff'), ('S03395800234', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1011200, 'STRAIGHT BOURBON WHISKIES', 259, 'Heaven Hill Brands', 73715, 'Evan Williams Cherry Reserve', 12, '750', 7.94, 11.91, 2, 23.82, 1.5, 0.4, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03389300030', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1081330, 'PEACH SCHNAPPS', 434, 'Luxco-St Louis', 82126, 'Arrow Peach Schnapps', 12, '750', 3.97, 5.95, 3, 17.85, 2.25, 0.59, -90.609747, 41.589369, '882675d959fffff'), ('S03387800007', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19068, 'Jim Beam', 6, '1750', 18.37, 27.55, 6, 165.3, 10.5, 2.77, -91.387797, 40.400038, '8826702f29fffff'), ('S03391300012', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35917, "Five O'clock Vodka", 12, '1000', 4.01, 6.01, 24, 144.24, 24.0, 6.34, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03382700029', Timestamp('2012-01-03 00:00:00'), 2607, 'Hy-Vee Wine and Spirits / Shenandoah', '520 SO FREMONT', 'SHENANDOAH', '51601', 73, 'Page', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19486, "Maker's 46", 6, '750', 20.47, 30.7, 1, 30.7, 0.75, 0.2, -95.385111, 40.761736, '8826080d95fffff'), ('S03391800015', Timestamp('2012-01-03 00:00:00'), 4480, 'Circle K #6602 / Clinton', '1530 NORTH 2ND ST', 'CLINTON', '52732', 23, 'Clinton', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53213, 'Paul Masson Grande Amber Brandy', 24, '200', 1.77, 2.66, 6, 15.96, 1.2, 0.32, -90.183722, 41.861892, '88275b7ab3fffff'), ('S03397800011', Timestamp('2012-01-03 00:00:00'), 4421, 'KUM & GO #1 / HAMPTON', '609 CENTRAL AVE W', 'HAMPTON', '50441', 35, 'Franklin', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26821, 'Jack Daniels Old #7 Black Lbl Mini', 12, '500', 8.62, 12.94, 2, 25.88, 1.0, 0.26, -93.21508, 42.741741, '88262b9605fffff'), ('S03392300057', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 22784, "Beam's 8 Star Bl Whiskey", 24, '375', 1.9, 2.85, 24, 68.4, 9.0, 2.38, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03409700017', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43126, 'Bacardi Superior Rum', 12, '750', 7.53, 11.3, 12, 135.6, 9.0, 2.38, -93.737447, 41.62941, '88260d802bfffff'), ('S03407900056', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1062310, 'SPICED RUM', 240, 'WILLIAM GRANT AND SONS, INC.', 45886, 'Sailor Jerry Spiced Navy Rum', 12, '750', 7.41, 11.62, 12, 139.44, 9.0, 2.38, 0.0, 0.0, '0'), ('S03411200086', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34026, 'Absolut Peppar', 12, '750', 10.89, 16.33, 1, 16.33, 0.75, 0.2, -93.697731, 41.665172, '88260d8231fffff'), ('S03401000059', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45278, 'Paramount White Rum', 6, '1750', 7.58, 11.37, 2, 22.74, 3.5, 0.92, -90.683356, 42.52717, '88275a215dfffff'), ('S03401600035', Timestamp('2012-01-03 00:00:00'), 2522, 'Hy-Vee Wine and Spirits / Spirit Lak', 'HWY 9 & 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88296, 'Patron Tequila Silver', 12, '750', 26.08, 39.13, 12, 469.56, 9.0, 2.38, 0.0, 0.0, '0'), ('S03388000028', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45247, 'Paramount Gold Rum', 12, '1000', 4.33, 6.5, 12, 78.0, 12.0, 3.17, -93.633306, 43.261538, '88262808e7fffff'), ('S03407700025', Timestamp('2012-01-03 00:00:00'), 4602, 'Cheap Smokes / Beer City', '58231 190th ST', 'PACIFIC JUNCTION', '51561', 65, 'Mills', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 61752, 'Jeremiah Weed Half And Half', 6, '1750', 10.5, 15.74, 2, 31.48, 3.5, 0.92, -95.822983, 41.041739, '8826084255fffff'), ('S03411100011', Timestamp('2012-01-03 00:00:00'), 3631, 'Audubon Food Pride', '104 MARKET ST PO BOX 206', 'AUDUBON', '50025', 5, 'Audubon', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 37938, 'Skol Vodka', 6, '1750', 7.1, 10.65, 6, 63.9, 10.5, 2.77, -94.93625, 41.723057, '88260c388dfffff'), ('S03406300057', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 2, 34.74, 1.5, 0.4, -91.281344, 42.108275, '882674c6a9fffff'), ('S03405800019', Timestamp('2012-01-03 00:00:00'), 3761, "Cheese's Food Center", '200 N WALNUT ST', 'LENOX', '50851', 87, 'Taylor', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36886, 'Mccormick Vodka', 12, '750', 3.3, 4.96, 2, 9.92, 1.5, 0.4, -94.560455, 40.882645, '882608a6e9fffff'), ('S03404100014', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25607, 'Seagrams 7 Crown Bl Whiskey', 12, '1000', 7.5, 11.24, 12, 134.88, 12.0, 3.17, -90.993859, 42.3036, '8826749ac1fffff'), ('S03402900009', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 13388, 'Northern Light', 6, '1750', 8.32, 12.48, 6, 74.88, 10.5, 2.77, -94.990031, 41.403856, '8826089095fffff'), ('S03395200049', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69946, 'Rumple Minze Peppermint Schnapps Liqueur', 12, '750', 12.0, 17.99, 4, 71.96, 3.0, 0.79, -91.110752, 40.801889, '882670a137fffff'), ('S03410700124', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1022100, 'TEQUILA', 395, 'Proximo', 89193, 'Jose Cuervo Especial Reposado Tequila', 48, '200', 3.0, 4.49, 1, 4.49, 0.2, 0.05, -94.853591, 42.064155, '88260c16a9fffff'), ('S03403200013', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88296, 'Patron Tequila Silver', 12, '750', 26.08, 39.13, 12, 469.56, 9.0, 2.38, -94.218, 43.29355, '8826284833fffff'), ('S03411500061', Timestamp('2012-01-03 00:00:00'), 3700, 'Avoca Liquor', '158 SOUTH ELM', 'AVOCA', '51521', 78, 'Pottawattamie', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 73526, 'Dekuyper Melon Liqueur', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -95.339412, 41.476306, '88260c6c8bfffff'), ('S03385800057', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52599, 'E & J Vs Brandy Round', 12, '1000', 6.94, 10.41, 2, 20.82, 2.0, 0.53, -92.258152, 42.809882, '8826291267fffff'), ('S03393900087', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32231, 'Seagrams Extra Dry Gin Mini', 12, '500', 5.44, 8.16, 2, 16.32, 1.0, 0.26, -95.873073, 41.261866, '882608c917fffff'), ('S03410700007', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1012200, 'SCOTCH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 4717, 'Chivas Regal 12 Yr', 12, '1000', 24.47, 36.71, 12, 440.52, 12.0, 3.17, -94.853591, 42.064155, '88260c16a9fffff'), ('S03408100049', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69666, "Dr. Mcgillicuddy's Vanilla", 12, '750', 8.66, 12.99, 1, 12.99, 0.75, 0.2, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03403500014', Timestamp('2012-01-03 00:00:00'), 4156, 'Fareway Stores #412 / Oelwein', '102  2ND AVE SE', 'OELWEIN', '50662', 33, 'Fayette', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -91.910502, 42.677024, '882629b807fffff'), ('S03405200007', Timestamp('2012-01-03 00:00:00'), 2954, "Dahl's / Ingersoll", '3425 INGERSOLL', 'DES MOINES', '50300', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 12, 208.44, 9.0, 2.38, -93.663337, 41.586313, '88260d80d7fffff'), ('S03388300004', Timestamp('2012-01-03 00:00:00'), 4039, 'No Frills Supermarkets / Council Blu', '1817 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11346, 'Seagrams V.o. Bl Canadian Whisky', 12, '750', 7.49, 11.24, 12, 134.88, 9.0, 2.38, -95.870993, 41.261863, '882608c917fffff'), ('S03412100108', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 125, 'Cedar Ridge Vineyards,LLC', 72950, 'Cedar Ridge Lemoncella', 12, '375', 8.0, 12.0, 3, 36.0, 1.12, 0.3, -90.483701, 41.552338, '882675da95fffff'), ('S03406300104', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1041100, 'AMERICAN DRY GINS', 260, 'Diageo Americas', 30317, "Gordon's Gin London Dry", 12, '1000', 6.5, 9.74, 1, 9.74, 1.0, 0.26, -91.281344, 42.108275, '882674c6a9fffff'), ('S03395800226', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081220, 'DARK CREME DE CACAO', 434, 'Luxco-St Louis', 78456, 'Paramount Creme De Cacao/dark', 12, '750', 4.44, 6.66, 5, 33.3, 3.75, 0.99, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03405400035', Timestamp('2012-01-03 00:00:00'), 3816, 'Swils', '200 E OAK ST', 'RED OAK', '51566', 69, 'Montgomery', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 10836, 'Crown Royal Black', 12, '750', 16.47, 24.7, 2, 49.4, 1.5, 0.4, -95.230032, 41.016691, '882608152bfffff'), ('S03397100004', Timestamp('2012-01-03 00:00:00'), 4306, 'Northside One Stop / Hampton', '1208  4TH ST NE', 'HAMPTON', '50441', 35, 'Franklin', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 13388, 'Northern Light', 6, '1750', 8.32, 12.48, 6, 74.88, 10.5, 2.77, -93.202452, 42.753045, '88262b9647fffff'), ('S03389000009', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37413, 'Popov Vodka 80 Prf', 48, '200', 1.53, 2.3, 5, 11.5, 1.0, 0.26, -93.315977, 40.757158, '88267252cdfffff'), ('S03371500016', Timestamp('2012-01-03 00:00:00'), 4572, 'Grundy Center Family Foods', '1400, G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43136, 'Bacardi Limon', 12, '750', 8.33, 12.49, 2, 24.98, 1.5, 0.4, -92.782283, 42.36157, '8826297835fffff'), ('S03410100005', Timestamp('2012-01-03 00:00:00'), 3962, 'Durant Street Country Stop', '618 DURANT ST', 'HARLAN', '51537', 83, 'Shelby', 1011100, 'BLENDED WHISKIES', 300, 'Mccormick Distilling Company', 24706, 'Mccormick Blend Whiskey', 12, '750', 3.36, 5.03, 12, 60.36, 9.0, 2.38, -95.318597, 41.656803, '88260c62a7fffff'), ('S03396000059', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48106, 'Hennessy Vs Cognac', 12, '750', 17.06, 25.59, 12, 307.08, 9.0, 2.38, -90.705328, 42.491862, '88275a2131fffff'), ('S03394700022', Timestamp('2012-01-03 00:00:00'), 3959, 'Hartig Drug Company #8 / University', '1600 UNIVERSITY AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43331, 'Captain Morgan Spiced Rum Mini', 12, '500', 4.97, 7.25, 2, 14.5, 1.0, 0.26, -90.688027, 42.498279, '88275a2115fffff'), ('S03400100011', Timestamp('2012-01-03 00:00:00'), 4469, 'Sperry One Stop Shop LLC', '516 HWY 141', 'COON RAPIDS', '50058', 14, 'Carroll', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81207, 'Paramount Peppermint Schnapps', 12, '1000', 4.05, 6.08, 12, 72.96, 12.0, 3.17, -94.678374, 41.865913, '88260cad0dfffff'), ('S03402900061', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1031090, 'OTHER PROOF VODKA', 260, 'Diageo Americas', 40926, 'Smirnoff Silver Vodka 90 Prf', 12, '750', 7.86, 11.79, 1, 11.79, 0.75, 0.2, -94.990031, 41.403856, '8826089095fffff'), ('S03394400010', Timestamp('2012-01-03 00:00:00'), 4334, 'Fareway Stores #114 / Dubuque', '2050 JOHN F KENNEDY RD', 'DUBUQUE', '52001', 31, 'Dubuque', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10008, 'Scoresby Rare Scotch', 6, '1750', 10.22, 15.33, 3, 45.99, 5.25, 1.39, 0.0, 0.0, '0'), ('S03396600054', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26826, 'Jack Daniels Old #7 Black Lbl', 12, '750', 13.01, 19.51, 3, 58.53, 2.25, 0.59, -93.378772, 43.142868, '8826283b29fffff'), ('S03396600135', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82867, 'Dekuyper Watermelon Pucker', 12, '1000', 7.35, 11.02, 3, 33.06, 3.0, 0.79, -93.378772, 43.142868, '8826283b29fffff'), ('S03410700089', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1081400, 'TRIPLE SEC', 434, 'Luxco-St Louis', 86506, 'Paramount Triple Sec', 12, '750', 3.33, 4.99, 4, 19.96, 3.0, 0.79, -94.853591, 42.064155, '88260c16a9fffff'), ('S03412100131', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52563, 'E & J Xo Brandy', 12, '750', 7.99, 11.99, 3, 35.97, 2.25, 0.59, -90.483701, 41.552338, '882675da95fffff'), ('S03396000017', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1012200, 'SCOTCH WHISKIES', 35, 'Bacardi U.S.A., Inc.', 4867, 'Dewars White Label Scotch', 12, '1000', 16.86, 25.29, 12, 303.48, 12.0, 3.17, -90.705328, 42.491862, '88275a2131fffff'), ('S03392800009', Timestamp('2012-01-03 00:00:00'), 4258, 'Fareway Stores #009 / Burlington', '920 LAWRENCE DR', 'BURLINGTON', '52601', 29, 'Des Moines', 1081312, 'BUTTERSCOTCH SCHNAPPS', 380, 'Phillips Beverage Company', 83363, 'Ice Hole Butterscotch Schnapps', 12, '750', 5.15, 7.73, 4, 30.92, 3.0, 0.79, -91.140051, 40.799097, '882670aac7fffff'), ('S03406400030', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19067, 'Jim Beam', 12, '1000', 11.54, 17.31, 2, 34.62, 2.0, 0.53, -90.766126, 41.509119, '88267434edfffff'), ('S03404500016', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11771, 'Black Velvet Mini', 10, '600', 4.96, 7.44, 1, 7.44, 0.6, 0.16, -95.83539, 41.279189, '882608c829fffff'), ('S03412100113', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1032200, 'IMPORTED VODKA - MISC', 434, 'Luxco-St Louis', 35699, 'Pearl Cucumber Vodka', 12, '750', 7.85, 11.78, 2, 23.56, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03398600060', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 89626, 'Patron Reposado Tequila', 12, '750', 27.0, 40.5, 2, 81.0, 1.5, 0.4, -91.343544, 40.626988, '8826703131fffff'), ('S03400700060', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1022100, 'TEQUILA', 461, 'Campari(skyy)', 89139, 'Cabo Wabo Reposado', 6, '750', 20.76, 31.14, 1, 31.14, 0.75, 0.2, 0.0, 0.0, '0'), ('S03391000002', Timestamp('2012-01-03 00:00:00'), 3654, 'The Beverage Shop / Belmond', '207 E MAIN ST', 'BELMOND', '50421', 99, 'Wright', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35317, 'Barton Vodka', 12, '1000', 3.97, 5.95, 12, 71.4, 12.0, 3.17, -93.615247, 42.845901, '8826286587fffff'), ('S03404800026', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43418, 'Captain Morgan Tattoo', 12, '750', 8.21, 12.31, 12, 147.72, 9.0, 2.38, -91.035138, 41.451349, '88267421abfffff'), ('S03392100065', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69636, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '750', 8.66, 12.99, 4, 51.96, 3.0, 0.79, -90.204748, 41.827845, '88275b7891fffff'), ('S03407900009', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 86670, "Jack Daniel's Tennessee Honey", 12, '750', 13.01, 19.51, 12, 234.12, 9.0, 2.38, 0.0, 0.0, '0'), ('S03396400065', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 12, 268.08, 12.0, 3.17, -95.8792, 41.238092, '882608c9e7fffff'), ('S03400000021', Timestamp('2012-01-03 00:00:00'), 2459, 'Reinhart Foods', '200 STATE PO BOX 98', 'GUTHRIE CENTER', '50115', 39, 'Guthrie', 1081330, 'PEACH SCHNAPPS', 434, 'Luxco-St Louis', 82126, 'Arrow Peach Schnapps', 12, '750', 3.97, 5.95, 2, 11.9, 1.5, 0.4, 0.0, 0.0, '0'), ('S03408100002', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1012100, 'CANADIAN WHISKIES', 434, 'Luxco-St Louis', 13038, 'Canadian Reserve Whisky', 6, '1750', 8.77, 13.16, 6, 78.96, 10.5, 2.77, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03402900020', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11290, 'Crown Royal Canadian Whisky Mini', 10, '300', 5.99, 8.99, 1, 8.99, 0.3, 0.08, -94.990031, 41.403856, '8826089095fffff'), ('S03410300108', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36908, 'Mccormick Vodka Pet', 6, '1750', 7.46, 11.19, 2, 22.38, 3.5, 0.92, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03410600009', Timestamp('2012-01-03 00:00:00'), 4115, 'Forest Market Convenience', '2105 FOREST', 'DES MOINES', '50311', 77, 'Polk', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48103, 'Hennessy Vs Cognac', 48, '200', 5.0, 7.49, 48, 359.52, 9.6, 2.54, -93.645606, 41.604077, '88260d82b3fffff'), ('S03386500001', Timestamp('2012-01-03 00:00:00'), 2548, 'Hy-Vee Food Store / Altoona', '100 8TH STREET SW', 'ALTOONA', '50009', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65229, 'Hpnotiq Harmonie', 6, '750', 11.68, 17.52, 6, 105.12, 4.5, 1.19, -93.465301, 41.644041, '88260d94d9fffff'), ('S03406100015', Timestamp('2012-01-03 00:00:00'), 4509, "A J'S LIQUOR II", '2515 CHAMBERLAIN', 'AMES', '50010', 85, 'Story', 1032080, 'IMPORTED VODKA', 260, 'Diageo Americas', 34456, 'Ketel One Imported Vodka', 12, '750', 13.0, 19.49, 2, 38.98, 1.5, 0.4, -93.650838, 42.021461, '88262b3983fffff'), ('S03402900069', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41796, 'Smirnoff Raspberry Vodka', 12, '750', 7.59, 11.56, 4, 46.24, 3.0, 0.79, -94.990031, 41.403856, '8826089095fffff'), ('S03396800020', Timestamp('2012-01-03 00:00:00'), 2445, "Ruback's Food Center", '504 SOUTH HIGHWAY', 'OAKLAND', '51560', 78, 'Pottawattamie', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67527, 'Kahlua Coffee Liqueur', 12, '1000', 14.81, 22.22, 2, 44.44, 2.0, 0.53, 0.0, 0.0, '0'), ('S03387200028', Timestamp('2012-01-03 00:00:00'), 4296, 'Circle B Market', '111 MAIN ST', 'KEOSAUQUA', '52565', 89, 'Van Buren', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41681, 'Uv Red (cherry) Vodka', 6, '1750', 10.63, 15.95, 2, 31.9, 3.5, 0.92, -91.962466, 40.7294, '882670458bfffff'), ('S03402100031', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58860, 'Jose Cuervo Mango Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, -92.475403, 42.712263, '8826291801fffff'), ('S03388100062', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45278, 'Paramount White Rum', 6, '1750', 7.58, 11.37, 6, 68.22, 10.5, 2.77, -95.879662, 41.261925, '882608c93bfffff'), ('S03395800076', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41692, 'Uv Blue (raspberry) Vodka', 6, '1750', 10.63, 15.95, 6, 95.7, 10.5, 2.77, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03383900019', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42716, 'Malibu Coconut Rum', 12, '750', 6.9, 10.35, 12, 124.2, 9.0, 2.38, -91.403752, 40.41538, '8826702f65fffff'), ('S03404100037', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53216, 'Paul Masson Grande Amber Brandy', 12, '750', 6.29, 9.44, 6, 56.64, 4.5, 1.19, -90.993859, 42.3036, '8826749ac1fffff'), ('S03394500010', Timestamp('2012-01-03 00:00:00'), 3443, 'Super Saver Iv', '1141 N BROADWAY', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1071100, 'AMERICAN COCKTAILS', 35, 'Bacardi U.S.A., Inc.', 56843, 'Bacardi Party Drinks Zombie', 6, '1750', 5.98, 9.47, 6, 56.82, 10.5, 2.77, -95.836515, 41.270824, '882608c823fffff'), ('S03407800020', Timestamp('2012-01-03 00:00:00'), 4181, 'Puff N Snuff', '1101 RHOMBERG AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25604, 'Seagrams 7 Crown Bl Whiskey', 24, '375', 3.34, 5.0, 4, 20.0, 1.5, 0.4, -90.660269, 42.519502, '88275a215bfffff'), ('S03404400018', Timestamp('2012-01-03 00:00:00'), 3016, "Super Valu / bender's", '200 SOUTH STATE ST', 'DENVER', '50622', 9, 'Bremer', 1081010, 'AMERICAN AMARETTO', 55, 'Sazerac North America', 73986, 'Di Amore Amaretto', 12, '750', 6.0, 8.99, 3, 26.97, 2.25, 0.59, -92.33724, 42.672008, '882629034bfffff'), ('S03409700005', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15626, 'Jameson', 12, '750', 13.46, 20.2, 12, 242.4, 9.0, 2.38, -93.737447, 41.62941, '88260d802bfffff'), ('S03393400166', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11358, 'Crown Royal Cask No. 16', 6, '750', 34.96, 52.44, 2, 104.88, 1.5, 0.4, -90.741064, 42.516633, '88275a212dfffff'), ('S03394000021', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35317, 'Barton Vodka', 12, '1000', 3.97, 5.95, 24, 142.8, 24.0, 6.34, -95.902023, 41.257663, '882608c937fffff'), ('S03396600072', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1031200, 'VODKA FLAVORED', 421, 'Sazerac Co., Inc.', 41209, 'Firefly Sweet Tea Vodka', 12, '750', 10.0, 15.0, 2, 30.0, 1.5, 0.4, -93.378772, 43.142868, '8826283b29fffff'), ('S03388300021', Timestamp('2012-01-03 00:00:00'), 4039, 'No Frills Supermarkets / Council Blu', '1817 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1022100, 'TEQUILA', 395, 'Proximo', 89197, 'Jose Cuervo Especial Reposado Tequila', 12, '1000', 12.5, 18.74, 12, 224.88, 12.0, 3.17, -95.870993, 41.261863, '882608c917fffff'), ('S03385300116', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1062300, 'FLAVORED RUM', 260, 'Diageo Americas', 43232, 'Captain Morgan Parrot Bay Key Lime', 12, '750', 7.49, 11.23, 4, 44.92, 3.0, 0.79, -91.136655, 40.814666, '882670aacdfffff'), ('S03395200053', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43128, 'Bacardi Superior Rum', 6, '1750', 14.67, 22.0, 2, 44.0, 3.5, 0.92, -91.110752, 40.801889, '882670a137fffff'), ('S03404600040', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41360, 'Uv Pink Lemonade Vodka', 12, '750', 6.22, 9.33, 2, 18.66, 1.5, 0.4, -95.027238, 40.739231, '88260831cbfffff'), ('S03397600111', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45248, 'Paramount Gold Rum', 6, '1750', 7.58, 11.37, 6, 68.22, 10.5, 2.77, -95.825137, 41.242732, '882608c9d1fffff'), ('S03385800024', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81208, 'Paramount Peppermint Schnapps', 6, '1750', 7.08, 10.62, 6, 63.72, 10.5, 2.77, -92.258152, 42.809882, '8826291267fffff'), ('S03399800022', Timestamp('2012-01-03 00:00:00'), 3970, 'The Liquor Cabinet, Inc.', '835 6 ST, STE 1', 'NEVADA', '50201', 85, 'Story', 1081365, 'TROPICAL FRUIT SCHNAPPS', 421, 'Sazerac Co., Inc.', 83906, 'Maui Blue Hawaiian Schnapps', 12, '750', 3.67, 5.5, 4, 22.0, 3.0, 0.79, -93.452277, 42.019292, '88262b222dfffff'), ('S03402900011', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 12, 180.0, 9.0, 2.38, -94.990031, 41.403856, '8826089095fffff'), ('S03411000140', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1011500, 'STRAIGHT RYE WHISKIES', 421, 'Sazerac Co., Inc.', 27100, 'Sazerac Rye', 6, '750', 15.6, 23.39, 1, 23.39, 0.75, 0.2, 0.0, 0.0, '0'), ('S03393400171', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26824, 'Jack Daniels Old #7 Black Lbl Square', 24, '375', 6.06, 9.09, 6, 54.54, 2.25, 0.59, -90.741064, 42.516633, '88275a212dfffff'), ('S03395800220', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 73526, 'Dekuyper Melon Liqueur', 12, '750', 6.3, 9.45, 3, 28.35, 2.25, 0.59, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03411100010', Timestamp('2012-01-03 00:00:00'), 3631, 'Audubon Food Pride', '104 MARKET ST PO BOX 206', 'AUDUBON', '50025', 5, 'Audubon', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15248, 'Windsor Canadian Pet', 6, '1750', 8.92, 13.38, 6, 80.28, 10.5, 2.77, -94.93625, 41.723057, '88260c388dfffff'), ('S03408100052', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 46351, 'Hawkeye Light Rum', 12, '1000', 4.22, 6.33, 1, 6.33, 1.0, 0.26, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03395800218', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081355, 'STRAWBERRY SCHNAPPS', 65, 'Jim Beam Brands', 82820, 'Dekuyper Strawberry Pucker', 12, '750', 6.3, 9.45, 5, 47.25, 3.75, 0.99, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03388100071', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1022100, 'TEQUILA', 395, 'Proximo', 89197, 'Jose Cuervo Especial Reposado Tequila', 12, '1000', 12.5, 18.74, 12, 224.88, 12.0, 3.17, -95.879662, 41.261925, '882608c93bfffff'), ('S03401600022', Timestamp('2012-01-03 00:00:00'), 2522, 'Hy-Vee Wine and Spirits / Spirit Lak', 'HWY 9 & 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43025, 'Admiral Nelson Spiced Rum Traveler', 12, '750', 5.77, 8.66, 12, 103.92, 9.0, 2.38, 0.0, 0.0, '0'), ('S03384500006', Timestamp('2012-01-03 00:00:00'), 3482, 'Kick The Can', '525 N MAPLE ST', 'CARROLL', '51401', 14, 'Carroll', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -94.861667, 42.063255, '88260c16a9fffff'), ('S03393300005', Timestamp('2012-01-03 00:00:00'), 4366, 'Kum & Go #137 / Tiffin', '101 VILLAGE DR', 'TIFFIN', '52340', 52, 'Johnson', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -91.6656, 41.697221, '8826747a67fffff'), ('S03403000036', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43335, 'Captain Morgan Spiced Rum Pet', 6, '1750', 16.99, 25.48, 2, 50.96, 3.5, 0.92, -94.480204, 43.39966, '88262a82cbfffff'), ('S03410300134', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 80574, 'Arrow Peppermint Schnapps', 24, '375', 2.25, 3.38, 5, 16.9, 1.88, 0.5, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03402600084', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19224, 'Knob Creek Bourbon', 12, '375', 10.25, 15.38, 1, 15.38, 0.38, 0.1, 0.0, 0.0, '0'), ('S03404700010', Timestamp('2012-01-03 00:00:00'), 3972, 'Hometown Foods / Hubbard', '210 MAPLE ST BOX 295', 'HUBBARD', '50122', 42, 'Hardin', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26826, 'Jack Daniels Old #7 Black Lbl', 12, '750', 13.01, 19.51, 3, 58.53, 2.25, 0.59, -93.302481, 42.305571, '88262baa53fffff'), ('S03395800204', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34116, 'Absolut Mandrin', 12, '750', 10.89, 16.33, 4, 65.32, 3.0, 0.79, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03408200079', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1012210, 'SINGLE MALT SCOTCH', 240, 'WILLIAM GRANT AND SONS, INC.', 4356, 'Balvenie 12yr Doublewood Single Malt Scotch', 12, '750', 30.41, 45.62, 3, 136.86, 2.25, 0.59, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03401600046', Timestamp('2012-01-03 00:00:00'), 2522, 'Hy-Vee Wine and Spirits / Spirit Lak', 'HWY 9 & 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37418, 'Popov Vodka 80 Prf', 6, '1750', 8.49, 12.74, 6, 76.44, 10.5, 2.77, 0.0, 0.0, '0'), ('S03410900158', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81196, 'Paramount Peppermint Schnapps Traveler', 12, '750', 3.7, 5.55, 4, 22.2, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03410700009', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5347, 'Johnnie Walker Red Label Scotch', 12, '1000', 16.99, 25.49, 12, 305.88, 12.0, 3.17, -94.853591, 42.064155, '88260c16a9fffff'), ('S03393900003', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43127, 'Bacardi Superior Rum', 12, '1000', 9.43, 14.14, 12, 169.68, 12.0, 3.17, -95.873073, 41.261866, '882608c917fffff'), ('S03395700061', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82867, 'Dekuyper Watermelon Pucker', 12, '1000', 7.35, 11.02, 1, 11.02, 1.0, 0.26, -95.147741, 43.14521, '88262a1925fffff'), ('S03406400018', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1081200, 'CREAM LIQUEURS', 300, 'Mccormick Distilling Company', 65199, 'Tequila Rose Liqueur', 12, '1000', 12.0, 17.99, 1, 17.99, 1.0, 0.26, -90.766126, 41.509119, '88267434edfffff'), ('S03405700014', Timestamp('2012-01-03 00:00:00'), 3651, 'Wal-Mart 1491 / Indianola', '1500 NORTH JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43336, 'Captain Morgan Original Spiced', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, 0.0, 0.0, '0'), ('S03408200057', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37346, 'Phillips Vodka', 12, '750', 3.48, 5.23, 12, 62.76, 9.0, 2.38, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03404600014', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11777, 'Black Velvet', 12, '1000', 6.63, 9.94, 12, 119.28, 12.0, 3.17, -95.027238, 40.739231, '88260831cbfffff'), ('S03397300056', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43123, 'Bacardi Superior Rum', 48, '200', 2.5, 3.75, 2, 7.5, 0.4, 0.11, -91.044008, 41.427202, '8826742f4bfffff'), ('S03410300048', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43198, 'Bacardi Torched Cherry', 12, '1000', 10.24, 15.35, 1, 15.35, 1.0, 0.26, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03408400016', Timestamp('2012-01-03 00:00:00'), 2650, 'Hy-Vee Wine and Spirits / Harlan', '1808 23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1051010, 'AMERICAN GRAPE BRANDIES', 259, 'Heaven Hill Brands', 52146, 'Christian Bros Brandy Traveler', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -95.339881, 41.650658, '88260c60e9fffff'), ('S03404800133', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1032200, 'IMPORTED VODKA - MISC', 277, 'Stoli Group', 34881, 'Stolichnaya Vanil Vodka 75 Proof', 12, '1000', 16.18, 24.27, 6, 145.62, 6.0, 1.59, -91.035138, 41.451349, '88267421abfffff'), ('S03412100012', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 40930, 'Smirnoff Grape Vodka', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03390000016', Timestamp('2012-01-03 00:00:00'), 3606, "Scotty's Liquor Store", '804 G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1081380, 'MISCELLANEOUS SCHNAPPS', 65, 'Jim Beam Brands', 82627, 'Dekuyper Cherry Pucker', 12, '1000', 7.35, 11.02, 2, 22.04, 2.0, 0.53, -92.77421500000001, 42.361514, '8826297831fffff'), ('S03395200040', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34433, 'Grey Goose Vodka', 12, '750', 17.97, 26.96, 4, 107.84, 3.0, 0.79, -91.110752, 40.801889, '882670a137fffff'), ('S03383800082', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1081380, 'MISCELLANEOUS SCHNAPPS', 65, 'Jim Beam Brands', 73516, 'Dekuyper Cactus Juice Schnapps', 12, '750', 6.3, 9.45, 1, 9.45, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff')]
[('S03394500034', Timestamp('2012-01-03 00:00:00'), 3443, 'Super Saver Iv', '1141 N BROADWAY', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 37938, 'Skol Vodka', 6, '1750', 7.1, 10.65, 6, 63.9, 10.5, 2.77, -95.836515, 41.270824, '882608c823fffff'), ('S03388700010', Timestamp('2012-01-03 00:00:00'), 2367, "Spencer's Grocery", '101 SOUTH BURNETT', 'SHENANDOAH', '51601', 73, 'Page', 1081365, 'TROPICAL FRUIT SCHNAPPS', 421, 'Sazerac Co., Inc.', 83906, 'Maui Blue Hawaiian Schnapps', 12, '750', 3.67, 5.5, 12, 66.0, 9.0, 2.38, -95.381935, 40.765934, '8826080d9dfffff'), ('S03408600047', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 65066, 'Goldschlager Cinnamon Schnapps', 12, '750', 12.0, 17.99, 3, 53.97, 2.25, 0.59, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03412100159', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 421, 'Sazerac Co., Inc.', 18006, 'Buffalo Trace Bourbon', 12, '750', 11.5, 17.24, 2, 34.48, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03411200144', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 35626, 'Pinnacle Whipped', 12, '750', 7.87, 11.81, 6, 70.86, 4.5, 1.19, -93.697731, 41.665172, '88260d8231fffff'), ('S03403200009', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36307, 'Hawkeye Vodka', 12, '1000', 3.85, 5.78, 12, 69.36, 12.0, 3.17, -94.218, 43.29355, '8826284833fffff'), ('S03389000023', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26828, 'Jack Daniels Old #7 Black Lbl', 6, '1750', 26.67, 40.01, 2, 80.02, 3.5, 0.92, -93.315977, 40.757158, '88267252cdfffff'), ('S03383600027', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 20248, 'Old Crow', 6, '1750', 8.92, 13.38, 6, 80.28, 10.5, 2.77, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03412100061', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43285, 'Captain Morgan Original Spiced Rum Pet', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03402100058', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1042100, 'IMPORTED DRY GINS', 240, 'WILLIAM GRANT AND SONS, INC.', 28625, "Hendrick's Gin", 6, '750', 16.25, 24.38, 2, 48.76, 1.5, 0.4, -92.475403, 42.712263, '8826291801fffff'), ('S03397600010', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10627, 'Canadian Club Whisky', 12, '1000', 9.34, 14.01, 12, 168.12, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03392300052', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35947, "Fleischmann's Royal Vodka", 12, '1000', 3.94, 5.9, 24, 141.6, 24.0, 6.34, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03399100011', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 421, 'Sazerac Co., Inc.', 59037, 'Desert Island Long Island Ice Tea Cocktail', 12, '1000', 4.35, 6.53, 12, 78.36, 12.0, 3.17, -93.619787, 41.60566, '88260d8769fffff'), ('S03390900024', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1012100, 'CANADIAN WHISKIES', 971, 'Hood River Distillers, Inc.', 14192, 'Pendleton Canadian Whisky', 12, '750', 13.1, 19.65, 3, 58.95, 2.25, 0.59, 0.0, 0.0, '0'), ('S03403100017', Timestamp('2012-01-03 00:00:00'), 4462, "SHELTON'S", '113, 1ST ST NW', 'BUFFALO CENTER', '50424', 95, 'Winnebago', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82786, 'Dekuyper Buttershots Schnapps', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -93.947595, 43.386755, '8826284217fffff'), ('S03403900024', Timestamp('2012-01-03 00:00:00'), 2628, 'Hy-Vee Food Store / Eldora', '1616 EDGINGTON', 'ELDORA', '50627', 42, 'Hardin', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 6, 61.8, 10.5, 2.77, -93.103764, 42.360631, '88262ba25dfffff'), ('S03389300034', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1081380, 'MISCELLANEOUS SCHNAPPS', 65, 'Jim Beam Brands', 82626, 'Dekuyper Cherry Pucker Schnapps', 12, '750', 6.3, 9.45, 1, 9.45, 0.75, 0.2, -90.609747, 41.589369, '882675d959fffff'), ('S03406000009', Timestamp('2012-01-03 00:00:00'), 4188, 'Main St. Market, Inc.', '882 MAIN ST', 'HOLY CROSS', '52053', 31, 'Dubuque', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65257, 'Jagermeister Liqueur', 12, '1000', 14.93, 22.39, 12, 268.68, 12.0, 3.17, -90.995943, 42.600941, '88275a7469fffff'), ('S03388800003', Timestamp('2012-01-03 00:00:00'), 3811, 'Wal-Mart 1431 / Keokuk', '300 NORTH PARK DR', 'KEOKUK', '52632', 56, 'Lee', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82846, 'Dekuyper Luscious Peachtree Schnapps', 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -91.407263, 40.418589, '88267028d9fffff'), ('S03405900003', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10627, 'Canadian Club Whisky', 12, '1000', 9.34, 14.01, 12, 168.12, 12.0, 3.17, -90.48065, 41.529655, '882675c361fffff'), ('S03396400075', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43025, 'Admiral Nelson Spiced Rum Traveler', 12, '750', 5.77, 8.66, 5, 43.3, 3.75, 0.99, -95.8792, 41.238092, '882608c9e7fffff'), ('S03393100028', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42716, 'Malibu Coconut Rum', 12, '750', 6.9, 10.35, 12, 124.2, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03396000078', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1081200, 'CREAM LIQUEURS', 300, 'Mccormick Distilling Company', 65199, 'Tequila Rose Liqueur', 12, '1000', 12.0, 17.99, 12, 215.88, 12.0, 3.17, -90.705328, 42.491862, '88275a2131fffff'), ('S03407700011', Timestamp('2012-01-03 00:00:00'), 4602, 'Cheap Smokes / Beer City', '58231 190th ST', 'PACIFIC JUNCTION', '51561', 65, 'Mills', 1031200, 'VODKA FLAVORED', 65, 'Jim Beam Brands', 3118, 'Pucker Vodka 50ml Gravity Feed', 3, '2000', 26.68, 40.02, 1, 40.02, 2.0, 0.53, -95.822983, 41.041739, '8826084255fffff'), ('S03385300045', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76487, 'Paramount Amaretto', 12, '1000', 4.04, 6.06, 12, 72.72, 12.0, 3.17, -91.136655, 40.814666, '882670aacdfffff'), ('S03397300023', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 22786, "Beam's 8 Star Bl Whiskey", 12, '750', 3.25, 4.88, 12, 58.56, 9.0, 2.38, -91.044008, 41.427202, '8826742f4bfffff'), ('S03408000027', Timestamp('2012-01-03 00:00:00'), 3622, 'Wal-Mart 1415 / Spirit Lake', '2600 HWY 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35918, "Five O'clock Vodka", 6, '1750', 6.92, 10.38, 12, 124.56, 21.0, 5.55, -95.126535, 43.416452, '88262ac237fffff'), ('S03393400023', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1101100, 'AMERICAN ALCOHOL', 434, 'Luxco-St Louis', 41846, 'Everclear Alcohol', 12, '750', 8.02, 12.03, 12, 144.36, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03405400010', Timestamp('2012-01-03 00:00:00'), 3816, 'Swils', '200 E OAK ST', 'RED OAK', '51566', 69, 'Montgomery', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35917, "Five O'clock Vodka", 12, '1000', 4.01, 6.01, 12, 72.12, 12.0, 3.17, -95.230032, 41.016691, '882608152bfffff'), ('S03392100079', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1031100, '100 PROOF VODKA', 434, 'Luxco-St Louis', 36301, 'Hawkeye Vodka 80 Prf Mini', 12, '500', 3.73, 5.59, 1, 5.59, 0.5, 0.13, -90.204748, 41.827845, '88275b7891fffff'), ('S03408000029', Timestamp('2012-01-03 00:00:00'), 3622, 'Wal-Mart 1415 / Spirit Lake', '2600 HWY 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 36, 917.28, 63.0, 16.64, -95.126535, 43.416452, '88262ac237fffff'), ('S03410300028', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21597, 'Ten High', 12, '1000', 4.93, 7.4, 1, 7.4, 1.0, 0.26, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03390800021', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58868, 'Jose Cuervo Authentic Strawberry Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03407700022', Timestamp('2012-01-03 00:00:00'), 4602, 'Cheap Smokes / Beer City', '58231 190th ST', 'PACIFIC JUNCTION', '51561', 65, 'Mills', 1011200, 'STRAIGHT BOURBON WHISKIES', 260, 'Diageo Americas', 27508, 'Jeremiah Weed Cherry Mash', 12, '750', 8.92, 13.38, 2, 26.76, 1.5, 0.4, -95.822983, 41.041739, '8826084255fffff'), ('S03393400115', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57148, "Chi-Chi's Margarita W/tequila", 6, '1750', 5.99, 9.24, 6, 55.44, 10.5, 2.77, -90.741064, 42.516633, '88275a212dfffff'), ('S03406200004', Timestamp('2012-01-03 00:00:00'), 4551, "Casey's General Store #2845 / Urband", '201 SUNSET ST', 'URBANA', '52345', 6, 'Benton', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, 0.0, 0.0, '0'), ('S03397300058', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32233, 'Seagrams Extra Dry Gin', 48, '200', 2.04, 3.06, 2, 6.12, 0.4, 0.11, -91.044008, 41.427202, '8826742f4bfffff'), ('S03392900065', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69611, "Dr. Mcgillicuddy's Apple Pie Schnapps", 12, '750', 8.32, 12.48, 4, 49.92, 3.0, 0.79, 0.0, 0.0, '0'), ('S03408300117', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82847, 'Dekuyper Peachtree', 12, '1000', 7.35, 11.02, 5, 55.1, 5.0, 1.32, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03410700057', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1062310, 'SPICED RUM', 434, 'Luxco-St Louis', 44658, 'Lady Bligh Spiced Rum', 6, '1750', 9.68, 14.53, 12, 174.36, 21.0, 5.55, -94.853591, 42.064155, '88260c16a9fffff'), ('S03401400006', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15247, 'Windsor Canadian', 12, '1000', 6.55, 9.83, 12, 117.96, 12.0, 3.17, -95.126786, 43.416433, '88262ac237fffff'), ('S03397500061', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1012100, 'CANADIAN WHISKIES', 434, 'Luxco-St Louis', 13037, 'Canadian Reserve Whisky', 12, '1000', 5.31, 7.97, 36, 286.92, 36.0, 9.51, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03408800097', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28206, 'Bombay Dry Gin', 12, '750', 10.26, 15.39, 3, 46.17, 2.25, 0.59, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03395200048', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 305, 'MHW Ltd', 66050, 'Nuvo Sparkling Liqueur', 12, '375', 9.93, 14.9, 4, 59.6, 1.5, 0.4, -91.110752, 40.801889, '882670a137fffff'), ('S03386500002', Timestamp('2012-01-03 00:00:00'), 2548, 'Hy-Vee Food Store / Altoona', '100 8TH STREET SW', 'ALTOONA', '50009', 77, 'Polk', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 6, 90.0, 4.5, 1.19, -93.465301, 41.644041, '88260d94d9fffff'), ('S03403700014', Timestamp('2012-01-03 00:00:00'), 4547, 'Fareway Stores #902 / Hampton', '309 CENTRAL AVE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 5, 46.65, 3.75, 0.99, -93.211676, 42.741748, '88262b9605fffff'), ('S03396600009', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11777, 'Black Velvet', 12, '1000', 6.63, 9.94, 24, 238.56, 24.0, 6.34, -93.378772, 43.142868, '8826283b29fffff'), ('S03409900004', Timestamp('2012-01-03 00:00:00'), 4387, 'KUM & GO #117 / SPIRIT LAKE', '300 18TH ST', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81206, 'Paramount Peppermint Schnapps', 12, '750', 3.7, 5.55, 12, 66.6, 9.0, 2.38, -95.094324, 43.422645, '88262ac217fffff'), ('S03396400003', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43137, 'Bacardi Limon', 12, '1000', 10.24, 15.35, 24, 368.4, 24.0, 6.34, -95.8792, 41.238092, '882608c9e7fffff'), ('S03384800001', Timestamp('2012-01-03 00:00:00'), 4395, 'KUM & GO #133 / ELLSWORTH', '1111 IA HIGHWAY 175', 'ELLSWORTH', '50075', 40, 'Hamilton', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25606, 'Seagrams 7 Crown Bl Whiskey', 12, '750', 6.14, 9.21, 12, 110.52, 9.0, 2.38, -93.572941, 42.309754, '88262b1587fffff'), ('S03410700039', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1062300, 'FLAVORED RUM', 65, 'Jim Beam Brands', 44520, 'Cruzan Vanilla Rum', 12, '750', 6.38, 9.57, 12, 114.84, 9.0, 2.38, -94.853591, 42.064155, '88260c16a9fffff'), ('S03406300023', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82607, 'Dekuyper Sour Apple', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -91.281344, 42.108275, '882674c6a9fffff'), ('S03384600016', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1032080, 'IMPORTED VODKA', 239, 'Russian Standard Vodka, USA', 35109, 'Russian Standard Original Vodka', 12, '750', 10.89, 16.33, 12, 195.96, 9.0, 2.38, -93.619787, 41.60566, '88260d8769fffff'), ('S03390800001', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34433, 'Grey Goose Vodka', 12, '750', 17.97, 26.96, 12, 323.52, 9.0, 2.38, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03404800119', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42699, 'Malibu Mango Rum', 12, '750', 6.9, 10.35, 4, 41.4, 3.0, 0.79, -91.035138, 41.451349, '88267421abfffff'), ('S03388200012', Timestamp('2012-01-03 00:00:00'), 4106, 'Hartig Drug Co. / JFK Rd / Dubuque', '2255 JOHN F KENNEDY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31658, 'Paramount Gin', 6, '1750', 7.1, 10.65, 1, 10.65, 1.75, 0.46, -90.72051800000001, 42.511352, '88275a212bfffff'), ('S03400700069', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1032080, 'IMPORTED VODKA', 277, 'Stoli Group', 34747, 'Stolichnaya Premium Vodka 80 Proof (lv)', 12, '1000', 16.18, 24.27, 3, 72.81, 3.0, 0.79, 0.0, 0.0, '0'), ('S03396400091', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82787, 'Dekuyper Buttershots', 12, '1000', 7.35, 11.02, 4, 44.08, 4.0, 1.06, -95.8792, 41.238092, '882608c9e7fffff'), ('S03381300006', Timestamp('2012-01-03 00:00:00'), 3944, "Sam's Club 4973 / Dubuque", '4400 ASBURY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10628, 'Canadian Club Whisky', 6, '1750', 15.75, 23.62, 6, 141.72, 10.5, 2.77, -90.737821, 42.515289, '88275a212dfffff'), ('S03404800076', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 22788, "Beam's 8 Star Bl Whiskey", 6, '1750', 7.24, 10.86, 6, 65.16, 10.5, 2.77, -91.035138, 41.451349, '88267421abfffff'), ('S03405900028', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1022100, 'TEQUILA', 434, 'Luxco-St Louis', 87937, 'Juarez Tequila Silver', 12, '1000', 6.67, 10.0, 12, 120.0, 12.0, 3.17, -90.48065, 41.529655, '882675c361fffff'), ('S03410900099', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43197, 'Bacardi Torched Cherry', 12, '750', 8.33, 12.49, 4, 49.96, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03403200034', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1011100, 'BLENDED WHISKIES', 300, 'Mccormick Distilling Company', 24728, 'Mccormick Blend Whiskey, Pet', 6, '1750', 7.41, 11.11, 1, 11.11, 1.75, 0.46, -94.218, 43.29355, '8826284833fffff'), ('S03396400155', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081030, 'COFFEE LIQUEURS', 65, 'Jim Beam Brands', 67557, 'Kamora Coffee Liqueur', 12, '1000', 8.13, 12.2, 1, 12.2, 1.0, 0.26, -95.8792, 41.238092, '882608c9e7fffff'), ('S03412100134', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1052010, 'IMPORTED GRAPE BRANDIES', 389, 'REMY COINTREAU USA         .', 50686, 'St. Remy Vsop Napoleon Brandy', 12, '750', 7.14, 10.72, 2, 21.44, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03387900003', Timestamp('2012-01-03 00:00:00'), 4493, "Casey's General Store #2850 / Cedar", '5655 C ST SW', 'CEDAR RAPIDS', '52404', 57, 'Linn', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37996, 'Smirnoff Vodka 80 Prf', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -91.637306, 41.922505, '8826745519fffff'), ('S03402600012', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26822, 'Jack Daniels Black Label', 48, '100', 1.51, 2.27, 7, 15.89, 0.7, 0.18, 0.0, 0.0, '0'), ('S03394000010', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081300, 'PEPPERMINT SCHNAPPS', 421, 'Sazerac Co., Inc.', 80827, 'Ice 101, Peppermint Schnapps', 12, '1000', 9.87, 14.8, 12, 177.6, 12.0, 3.17, -95.902023, 41.257663, '882608c937fffff'), ('S03405800003', Timestamp('2012-01-03 00:00:00'), 3761, "Cheese's Food Center", '200 N WALNUT ST', 'LENOX', '50851', 87, 'Taylor', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 6, 61.8, 10.5, 2.77, -94.560455, 40.882645, '882608a6e9fffff'), ('S03398400013', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 58106, 'Club Cocktails Brass Monkey', 12, '750', 3.46, 5.19, 2, 10.38, 1.5, 0.4, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03405400042', Timestamp('2012-01-03 00:00:00'), 3816, 'Swils', '200 E OAK ST', 'RED OAK', '51566', 69, 'Montgomery', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 987, 'Stellar Importing Company, LLC', 64016, 'Achaia Clauss Ouzo Greek Liqueur', 12, '750', 9.16, 13.74, 2, 27.48, 1.5, 0.4, -95.230032, 41.016691, '882608152bfffff'), ('S03395200060', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32236, 'Seagrams Extra Dry Gin', 12, '750', 5.9, 8.85, 3, 26.55, 2.25, 0.59, -91.110752, 40.801889, '882670a137fffff'), ('S03408000011', Timestamp('2012-01-03 00:00:00'), 3622, 'Wal-Mart 1415 / Spirit Lake', '2600 HWY 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1012100, 'CANADIAN WHISKIES', 434, 'Luxco-St Louis', 13038, 'Canadian Reserve Whisky', 6, '1750', 8.77, 13.16, 12, 157.92, 21.0, 5.55, -95.126535, 43.416452, '88262ac237fffff'), ('S03383600006', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 6, 152.88, 10.5, 2.77, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03408700012', Timestamp('2012-01-03 00:00:00'), 4066, 'Xpress Liquor', '1201 NORTH JEFFERSON STE 100', 'INDIANOLA', '50125', 91, 'Warren', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55086, 'Paramount Blackberry Brandy', 12, '750', 5.21, 7.82, 12, 93.84, 9.0, 2.38, 0.0, 0.0, '0'), ('S03404700007', Timestamp('2012-01-03 00:00:00'), 3972, 'Hometown Foods / Hubbard', '210 MAPLE ST BOX 295', 'HUBBARD', '50122', 42, 'Hardin', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 24, 256.8, 42.0, 11.1, -93.302481, 42.305571, '88262baa53fffff'), ('S03407900018', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43296, 'Captain Morgan Silver Spiced Rum', 12, '750', 8.0, 11.99, 12, 143.88, 9.0, 2.38, 0.0, 0.0, '0'), ('S03398600038', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1051010, 'AMERICAN GRAPE BRANDIES', 259, 'Heaven Hill Brands', 52318, 'Christian Bros Brandy', 6, '1750', 9.71, 15.32, 1, 15.32, 1.75, 0.46, -91.343544, 40.626988, '8826703131fffff'), ('S03408100038', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 4626, "Buchanan's 12 Yr Deluxe Scotch", 12, '750', 20.0, 29.99, 4, 119.96, 3.0, 0.79, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03393400148', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1012210, 'SINGLE MALT SCOTCH', 370, 'Pernod Ricard USA/Austin Nichols', 4084, 'Aberlour 12 Year Single Malt Scotch', 6, '750', 21.32, 31.97, 2, 63.94, 1.5, 0.4, -90.741064, 42.516633, '88275a212dfffff'), ('S03402700014', Timestamp('2012-01-03 00:00:00'), 3907, "Smokin' Joe's #16 Tobacco and Liquor", '1504 PARK AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11774, 'Black Velvet', 24, '375', 3.07, 4.6, 8, 36.8, 3.0, 0.79, -91.030569, 41.442458, '88267421bdfffff'), ('S03386100001', Timestamp('2012-01-03 00:00:00'), 4083, 'Fareway Stores #909 / Ankeny', '109 SE ORALABOR RD', 'ANKENY', '50021', 77, 'Polk', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19486, "Maker's 46", 6, '750', 20.47, 30.7, 6, 184.2, 4.5, 1.19, -93.59949600000002, 41.702811, '88260d954bfffff'), ('S03387200037', Timestamp('2012-01-03 00:00:00'), 4296, 'Circle B Market', '111 MAIN ST', 'KEOSAUQUA', '52565', 89, 'Van Buren', 1041200, 'AMERICAN SLOE GINS', 434, 'Luxco-St Louis', 33716, 'Paramount Sloe Gin', 12, '750', 3.84, 5.75, 3, 17.25, 2.25, 0.59, -91.962466, 40.7294, '882670458bfffff'), ('S03397500028', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1032080, 'IMPORTED VODKA', 277, 'Stoli Group', 34747, 'Stolichnaya Premium Vodka 80 Proof (lv)', 12, '1000', 16.18, 24.27, 12, 291.24, 12.0, 3.17, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03392100023', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26823, 'Jack Daniels Old #7 Black Lbl', 48, '200', 3.69, 5.54, 4, 22.16, 0.8, 0.21, -90.204748, 41.827845, '88275b7891fffff'), ('S03405300003', Timestamp('2012-01-03 00:00:00'), 4576, 'THE BOONEDOCKS', '228 N HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -95.125928, 43.370941, '88262ac013fffff'), ('S03390900074', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 2, 20.6, 3.5, 0.92, 0.0, 0.0, '0'), ('S03400700028', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1071100, 'AMERICAN COCKTAILS', 35, 'Bacardi U.S.A., Inc.', 56850, 'Bacardi Party Drinks Bahama Mama', 6, '1750', 5.98, 9.47, 12, 113.64, 21.0, 5.55, 0.0, 0.0, '0'), ('S03401400022', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 37938, 'Skol Vodka', 6, '1750', 7.1, 10.65, 18, 191.7, 31.5, 8.32, -95.126786, 43.416433, '88262ac237fffff'), ('S03385300079', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 971, 'Hood River Distillers, Inc.', 65536, 'Lucid Absinthe', 6, '750', 32.5, 48.74, 1, 48.74, 0.75, 0.2, -91.136655, 40.814666, '882670aacdfffff'), ('S03407400001', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -93.736517, 41.659057, '88260d8045fffff'), ('S03395700014', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19096, 'Jim Beam Traveler', 12, '750', 8.42, 13.41, 1, 13.41, 0.75, 0.2, -95.147741, 43.14521, '88262a1925fffff'), ('S03412100120', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 36186, "Gordon's Vodka 80 Prf", 12, '750', 5.26, 7.89, 3, 23.67, 2.25, 0.59, -90.483701, 41.552338, '882675da95fffff'), ('S03388500068', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19066, 'Jim Beam', 12, '750', 8.42, 13.41, 2, 26.82, 1.5, 0.4, -90.668138, 42.504959, '88275a2119fffff'), ('S03407900020', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57125, "Chi-Chi's Gold Margarita", 6, '1750', 5.99, 9.24, 6, 55.44, 10.5, 2.77, 0.0, 0.0, '0'), ('S03389000024', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750', 9.97, 14.95, 2, 29.9, 1.5, 0.4, -93.315977, 40.757158, '88267252cdfffff'), ('S03385200002', Timestamp('2012-01-03 00:00:00'), 4534, 'D & B Foods', '141 W GILLET ST', 'PRESTON', '52069', 49, 'Jackson', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 6, 90.0, 4.5, 1.19, -90.39584400000001, 42.050312, '88275b4e29fffff'), ('S03395700005', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35914, "Five O'clock Vodka", 24, '375', 1.84, 2.75, 24, 66.0, 9.0, 2.38, -95.147741, 43.14521, '88262a1925fffff'), ('S03407100017', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64867, 'Fireball Cinnamon Whiskey', 12, '1000', 11.0, 16.5, 1, 16.5, 1.0, 0.26, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03387800001', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23828, 'Five Star', 6, '1750', 7.36, 11.04, 6, 66.24, 10.5, 2.77, -91.387797, 40.400038, '8826702f29fffff'), ('S03401000078', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41604, 'Uv Grape Vodka', 12, '750', 6.22, 9.33, 1, 9.33, 0.75, 0.2, -90.683356, 42.52717, '88275a215dfffff'), ('S03393500002', Timestamp('2012-01-03 00:00:00'), 4341, "Mj's Grocery", '524 MAIN ST', 'GRISWOLD', '51535', 15, 'Cass', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -95.14161, 41.234743, '8826088f47fffff'), ('S03390300051', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11788, 'Black Velvet', 6, '1750', 10.45, 15.67, 1, 15.67, 1.75, 0.46, -95.854809, 41.262316, '882608c9cdfffff'), ('S03398300007', Timestamp('2012-01-03 00:00:00'), 4543, "Casey's General Store #2924 / Marion", '1100 EAGLE VIEW DR', 'MARION', '52302', 57, 'Linn', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37996, 'Smirnoff Vodka 80 Prf', 12, '750', 7.59, 11.56, 6, 69.36, 4.5, 1.19, -91.553462, 42.037621, '882674cdddfffff'), ('S03408200055', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 205, 'E AND J GALLO WINERY', 36969, 'New Amsterdam Vodka', 12, '750', 6.9, 10.35, 24, 248.4, 18.0, 4.76, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03387800052', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65257, 'Jagermeister Liqueur', 12, '1000', 14.93, 22.39, 12, 268.68, 12.0, 3.17, -91.387797, 40.400038, '8826702f29fffff'), ('S03384600043', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43050, 'Bacardi Dragon Berry', 12, '1000', 10.24, 15.35, 12, 184.2, 12.0, 3.17, -93.619787, 41.60566, '88260d8769fffff'), ('S03388500041', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34001, 'Absolut Swedish Vodka 80 Prf Mini', 10, '600', 7.92, 11.88, 1, 11.88, 0.6, 0.16, -90.668138, 42.504959, '88275a2119fffff'), ('S03389000060', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64867, 'Fireball Cinnamon Whiskey', 12, '1000', 11.0, 16.5, 3, 49.5, 3.0, 0.79, -93.315977, 40.757158, '88267252cdfffff'), ('S03405400033', Timestamp('2012-01-03 00:00:00'), 3816, 'Swils', '200 E OAK ST', 'RED OAK', '51566', 69, 'Montgomery', 1022100, 'TEQUILA', 395, 'Proximo', 89199, 'Jose Cuervo Especial Reposado Square', 12, '375', 5.49, 8.24, 12, 98.88, 4.5, 1.19, -95.230032, 41.016691, '882608152bfffff'), ('S03389000011', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 86668, "Jack Daniel's Tennessee Honey Mini", 12, '500', 8.62, 12.94, 1, 12.94, 0.5, 0.13, -93.315977, 40.757158, '88267252cdfffff'), ('S03389900037', Timestamp('2012-01-03 00:00:00'), 2532, 'Hy-Vee Food Store #2 / Des Moines', '2540 E EUCLID', 'DES MOINES', '50317', 77, 'Polk', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 65065, 'Goldschlager Cinnamon Schnapps', 12, '375', 6.5, 9.74, 3, 29.22, 1.12, 0.3, -93.569452, 41.627838, '88260d9581fffff'), ('S03393900018', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1022100, 'TEQUILA', 395, 'Proximo', 89197, 'Jose Cuervo Especial Reposado Tequila', 12, '1000', 12.5, 18.74, 2, 37.48, 2.0, 0.53, -95.873073, 41.261866, '882608c917fffff'), ('S03383700019', Timestamp('2012-01-03 00:00:00'), 3934, 'Wal-Mart 1509 / Maquoketa', '103 E CARLISLE', 'MAQUOKETA', '52060', 49, 'Jackson', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35916, "Five O'clock Vodka", 12, '750', 3.31, 4.96, 48, 238.08, 36.0, 9.51, -90.66292, 42.04869, '8826748693fffff'), ('S03402600076', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 38006, 'Smirnoff Vodka Traveller', 12, '750', 7.59, 11.56, 2, 23.12, 1.5, 0.4, 0.0, 0.0, '0'), ('S03395200070', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1081600, 'WHISKEY LIQUEUR', 260, 'Diageo Americas', 67267, 'Yukon Jack Canadian Liqueur', 12, '1000', 11.21, 16.81, 3, 50.43, 3.0, 0.79, -91.110752, 40.801889, '882670a137fffff'), ('S03396000080', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82607, 'Dekuyper Sour Apple', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -90.705328, 42.491862, '88275a2131fffff'), ('S03406100007', Timestamp('2012-01-03 00:00:00'), 4509, "A J'S LIQUOR II", '2515 CHAMBERLAIN', 'AMES', '50010', 85, 'Story', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43285, 'Captain Morgan Original Spiced Rum Pet', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -93.650838, 42.021461, '88262b3983fffff'), ('S03405800001', Timestamp('2012-01-03 00:00:00'), 3761, "Cheese's Food Center", '200 N WALNUT ST', 'LENOX', '50851', 87, 'Taylor', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35914, "Five O'clock Vodka", 24, '375', 1.84, 2.75, 24, 66.0, 9.0, 2.38, -94.560455, 40.882645, '882608a6e9fffff'), ('S03396000032', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36307, 'Hawkeye Vodka', 12, '1000', 3.85, 5.78, 24, 138.72, 24.0, 6.34, -90.705328, 42.491862, '88275a2131fffff'), ('S03408300310', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1071100, 'AMERICAN COCKTAILS', 35, 'Bacardi U.S.A., Inc.', 56840, 'Bacardi Party Drinks Hurricane', 6, '1750', 5.98, 9.47, 4, 37.88, 7.0, 1.85, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03396700005', Timestamp('2012-01-03 00:00:00'), 4159, 'Fareway Stores #073 / Council Bluffs', '310 MCKENZIE AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64866, 'Fireball Cinnamon Whiskey', 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -95.81799100000002, 41.280084, '882608c807fffff'), ('S03402900041', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55084, 'Paramount Blackberry Brandy', 24, '375', 3.34, 5.0, 4, 20.0, 1.5, 0.4, -94.990031, 41.403856, '8826089095fffff'), ('S03389300009', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55088, 'Paramount Blackberry Brandy', 6, '1750', 10.75, 16.12, 6, 96.72, 10.5, 2.77, -90.609747, 41.589369, '882675d959fffff'), ('S03403900049', Timestamp('2012-01-03 00:00:00'), 2628, 'Hy-Vee Food Store / Eldora', '1616 EDGINGTON', 'ELDORA', '50627', 42, 'Hardin', 1051120, 'CHERRY BRANDIES', 434, 'Luxco-St Louis', 55246, 'Arrow Wild Cherry Flav Brandy', 12, '750', 5.0, 7.5, 3, 22.5, 2.25, 0.59, -93.103764, 42.360631, '88262ba25dfffff'), ('S03389400010', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1042100, 'IMPORTED DRY GINS', 260, 'Diageo Americas', 28868, 'Tanqueray Gin', 6, '1750', 24.29, 36.44, 6, 218.64, 10.5, 2.77, -95.909384, 41.242291, '88260e26d5fffff'), ('S03402600083', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21236, 'Ridgemont Reserve 1792', 6, '750', 15.6, 23.39, 1, 23.39, 0.75, 0.2, 0.0, 0.0, '0'), ('S03407200039', Timestamp('2012-01-03 00:00:00'), 4146, 'Fareway Stores #561 / Waverly', '220 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31656, 'Paramount Gin', 12, '750', 3.39, 5.09, 3, 15.27, 2.25, 0.59, -92.473476, 42.725748, '8826291847fffff'), ('S03400700077', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 40976, 'Smirnoff Cherry Vodka', 12, '750', 7.59, 11.56, 3, 34.68, 2.25, 0.59, 0.0, 0.0, '0'), ('S03396600119', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 25878, 'Sunny Brook Blend Whiskey', 6, '1750', 7.87, 11.8, 2, 23.6, 3.5, 0.92, -93.378772, 43.142868, '8826283b29fffff'), ('S03406900003', Timestamp('2012-01-03 00:00:00'), 4114, 'After 5 Somewhere', '704 W 7TH ST PO BOX 372', 'ATLANTIC', '50022', 15, 'Cass', 1011100, 'BLENDED WHISKIES', 300, 'Mccormick Distilling Company', 24728, 'Mccormick Blend Whiskey, Pet', 6, '1750', 7.41, 11.11, 6, 66.66, 10.5, 2.77, -95.023116, 41.40406200000001, '88260890a3fffff'), ('S03392900006', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1022100, 'TEQUILA', 434, 'Luxco-St Louis', 89387, 'Juarez Tequila Gold', 12, '1000', 6.67, 10.0, 12, 120.0, 12.0, 3.17, 0.0, 0.0, '0'), ('S03403200041', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 40957, 'Smirnoff Strawberry Vodka', 12, '1000', 9.2, 13.8, 1, 13.8, 1.0, 0.26, -94.218, 43.29355, '8826284833fffff'), ('S03395800187', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25604, 'Seagrams 7 Crown Bl Whiskey', 24, '375', 3.34, 5.0, 8, 40.0, 3.0, 0.79, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03382200018', Timestamp('2012-01-03 00:00:00'), 4584, 'J & C Grocery / Dumont', '611 MAIN ST', 'DUMONT', '50602', 12, 'Butler', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43336, 'Captain Morgan Original Spiced', 12, '750', 8.5, 12.74, 3, 38.22, 2.25, 0.59, -92.973022, 42.750551, '8826295aebfffff'), ('S03391900017', Timestamp('2012-01-03 00:00:00'), 4113, 'Tobacco Hut #6 / Council', '3134 MANAWA CENTRE DR', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 12404, 'Canadian Ltd Whisky', 24, '375', 1.87, 2.8, 6, 16.8, 2.25, 0.59, -95.847514, 41.228712, '882608c983fffff'), ('S03396100005', Timestamp('2012-01-03 00:00:00'), 2656, 'Hy-Vee Food Store / Corning', '300  10TH ST', 'CORNING', '51632', 22, 'Clayton', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37418, 'Popov Vodka 80 Prf', 6, '1750', 8.49, 12.74, 6, 76.44, 10.5, 2.77, -94.731809, 40.991861, '882608a217fffff'), ('S03395800052', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 35, 'Bacardi U.S.A., Inc.', 56843, 'Bacardi Party Drinks Zombie', 6, '1750', 5.98, 9.47, 6, 56.82, 10.5, 2.77, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03406300081', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1051110, 'BLACKBERRY BRANDIES', 55, 'Sazerac North America', 55066, 'Mr. Boston Blackberry Brandy', 12, '750', 4.12, 6.37, 2, 12.74, 1.5, 0.4, -91.281344, 42.108275, '882674c6a9fffff'), ('S03404100029', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -90.993859, 42.3036, '8826749ac1fffff'), ('S03408600086', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 38008, 'Smirnoff Vodka 80 Prf Pet', 6, '1750', 13.98, 20.97, 2, 41.94, 3.5, 0.92, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03399800011', Timestamp('2012-01-03 00:00:00'), 3970, 'The Liquor Cabinet, Inc.', '835 6 ST, STE 1', 'NEVADA', '50201', 85, 'Story', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41796, 'Smirnoff Raspberry Vodka', 12, '750', 7.59, 11.56, 4, 46.24, 3.0, 0.79, -93.452277, 42.019292, '88262b222dfffff'), ('S03407400025', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1042100, 'IMPORTED DRY GINS', 260, 'Diageo Americas', 28868, 'Tanqueray Gin', 6, '1750', 24.29, 36.44, 2, 72.88, 3.5, 0.92, -93.736517, 41.659057, '88260d8045fffff'), ('S03389000017', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15626, 'Jameson', 12, '750', 13.46, 20.2, 1, 20.2, 0.75, 0.2, -93.315977, 40.757158, '88267252cdfffff'), ('S03410700091', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1012210, 'SINGLE MALT SCOTCH', 370, 'Pernod Ricard USA/Austin Nichols', 5061, 'Glenlivet 15 Year French Oak', 6, '750', 26.4, 39.6, 2, 79.2, 1.5, 0.4, -94.853591, 42.064155, '88260c16a9fffff'), ('S03393400025', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41705, 'Uv Red (cherry) Vodka', 12, '1000', 7.25, 10.87, 12, 130.44, 12.0, 3.17, -90.741064, 42.516633, '88275a212dfffff'), ('S03405500016', Timestamp('2012-01-03 00:00:00'), 4203, 'Liquor and Tobacco Outlet /', '620 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82881, 'Dekuyper Hot Damn! 100 Proof', 12, '750', 6.77, 10.15, 3, 30.45, 2.25, 0.59, -92.47898, 42.725694, '8826291847fffff'), ('S03383200016', Timestamp('2012-01-03 00:00:00'), 3562, 'Wal-Mart 0797 / W Burlington', '324 WEST AGENCY RD', 'WEST BURLINGTON', '52655', 29, 'Des Moines', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43285, 'Captain Morgan Original Spiced Rum Pet', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -91.158407, 40.813125, '882670aa17fffff'), ('S03405700021', Timestamp('2012-01-03 00:00:00'), 3651, 'Wal-Mart 1491 / Indianola', '1500 NORTH JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 12, 208.44, 9.0, 2.38, 0.0, 0.0, '0'), ('S03404400009', Timestamp('2012-01-03 00:00:00'), 3016, "Super Valu / bender's", '200 SOUTH STATE ST', 'DENVER', '50622', 9, 'Bremer', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 54646, 'Arrow Blackberry Flav Brandy', 12, '750', 5.0, 7.5, 12, 90.0, 9.0, 2.38, -92.33724, 42.672008, '882629034bfffff'), ('S03404600056', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1081335, 'RASPBERRY SCHNAPPS', 65, 'Jim Beam Brands', 73456, 'Dekuyper Luscious Razzmatazz', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -95.027238, 40.739231, '88260831cbfffff'), ('S03388300025', Timestamp('2012-01-03 00:00:00'), 4039, 'No Frills Supermarkets / Council Blu', '1817 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1022100, 'TEQUILA', 395, 'Proximo', 89199, 'Jose Cuervo Especial Reposado Square', 12, '375', 5.49, 8.24, 12, 98.88, 4.5, 1.19, -95.870993, 41.261863, '882608c917fffff'), ('S03391400009', Timestamp('2012-01-03 00:00:00'), 4272, 'Fareway Stores #407 / Esterville', '1108 CENTRAL AVE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 20248, 'Old Crow', 6, '1750', 8.92, 13.38, 6, 80.28, 10.5, 2.77, -94.829963, 43.402096, '88262a8b47fffff'), ('S03381400056', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41604, 'Uv Grape Vodka', 12, '750', 6.22, 9.33, 24, 223.92, 18.0, 4.76, -90.527081, 41.559731, '882675daa5fffff'), ('S03382700020', Timestamp('2012-01-03 00:00:00'), 2607, 'Hy-Vee Wine and Spirits / Shenandoah', '520 SO FREMONT', 'SHENANDOAH', '51601', 73, 'Page', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15247, 'Windsor Canadian', 12, '1000', 6.55, 9.83, 12, 117.96, 12.0, 3.17, -95.385111, 40.761736, '8826080d95fffff'), ('S03383200002', Timestamp('2012-01-03 00:00:00'), 3562, 'Wal-Mart 0797 / W Burlington', '324 WEST AGENCY RD', 'WEST BURLINGTON', '52655', 29, 'Des Moines', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86916, 'Southern Comfort 100 Prf', 12, '750', 11.65, 17.47, 24, 419.28, 18.0, 4.76, -91.158407, 40.813125, '882670aa17fffff'), ('S03402700003', Timestamp('2012-01-03 00:00:00'), 3907, "Smokin' Joe's #16 Tobacco and Liquor", '1504 PARK AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35917, "Five O'clock Vodka", 12, '1000', 4.01, 6.01, 12, 72.12, 12.0, 3.17, -91.030569, 41.442458, '88267421bdfffff'), ('S03397600113', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 6, 90.0, 4.5, 1.19, -95.825137, 41.242732, '882608c9d1fffff'), ('S03408100064', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1051140, 'PEACH BRANDIES', 434, 'Luxco-St Louis', 56206, 'Paramount Peach Flavored Brandy', 12, '750', 5.21, 7.82, 1, 7.82, 0.75, 0.2, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03392300083', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82610, 'Dekuyper Hot Damn!', 12, '1000', 7.35, 11.02, 3, 33.06, 3.0, 0.79, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03408400062', Timestamp('2012-01-03 00:00:00'), 2650, 'Hy-Vee Wine and Spirits / Harlan', '1808 23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43334, 'Captain Morgan Spiced Rum', 24, '375', 4.3, 6.45, 5, 32.25, 1.88, 0.5, -95.339881, 41.650658, '88260c60e9fffff'), ('S03389000018', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1012100, 'CANADIAN WHISKIES', 971, 'Hood River Distillers, Inc.', 14192, 'Pendleton Canadian Whisky', 12, '750', 13.1, 19.65, 1, 19.65, 0.75, 0.2, -93.315977, 40.757158, '88267252cdfffff'), ('S03391900015', Timestamp('2012-01-03 00:00:00'), 4113, 'Tobacco Hut #6 / Council', '3134 MANAWA CENTRE DR', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69667, "Dr. Mcgillicuddy's Vanilla", 12, '1000', 11.0, 16.5, 12, 198.0, 12.0, 3.17, -95.847514, 41.228712, '882608c983fffff'), ('S03400400024', Timestamp('2012-01-03 00:00:00'), 2655, 'Hy-Vee Food Store / Clarinda', '1200 S 16TH ST', 'CLARINDA', '51632', 73, 'Page', 1062300, 'FLAVORED RUM', 65, 'Jim Beam Brands', 44520, 'Cruzan Vanilla Rum', 12, '750', 6.38, 9.57, 4, 38.28, 3.0, 0.79, -95.037849, 40.726808, '8826083189fffff'), ('S03402900012', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34008, 'Absolut Swedish Vodka 80 Prf', 6, '1750', 19.88, 29.81, 6, 178.86, 10.5, 2.77, -94.990031, 41.403856, '8826089095fffff'), ('S03403400025', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23824, 'Five Star', 24, '375', 1.98, 2.96, 9, 26.64, 3.38, 0.89, -93.599362, 41.59878, '88260d8743fffff'), ('S03393400003', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 12, 208.44, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03384500003', Timestamp('2012-01-03 00:00:00'), 3482, 'Kick The Can', '525 N MAPLE ST', 'CARROLL', '51401', 14, 'Carroll', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41694, 'Uv Blue (raspberry) Vodka', 12, '1000', 7.25, 10.87, 12, 130.44, 12.0, 3.17, -94.861667, 42.063255, '88260c16a9fffff'), ('S03399400007', Timestamp('2012-01-03 00:00:00'), 3563, 'Wal-Mart 0559 / Muscatine', '3003 NORTH HWY 61', 'MUSCATINE', '52761', 70, 'Muscatine', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41696, 'Uv Green (apple) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -91.025913, 41.45529, '8826742185fffff'), ('S03395200012', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52596, 'E & J Vs', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -91.110752, 40.801889, '882670a137fffff'), ('S03405400021', Timestamp('2012-01-03 00:00:00'), 3816, 'Swils', '200 E OAK ST', 'RED OAK', '51566', 69, 'Montgomery', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 65066, 'Goldschlager Cinnamon Schnapps', 12, '750', 12.0, 17.99, 12, 215.88, 9.0, 2.38, -95.230032, 41.016691, '882608152bfffff'), ('S03404600044', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43121, 'Bacardi Superior Rum Mini', 12, '500', 4.74, 7.11, 1, 7.11, 0.5, 0.13, -95.027238, 40.739231, '88260831cbfffff'), ('S03405200031', Timestamp('2012-01-03 00:00:00'), 2954, "Dahl's / Ingersoll", '3425 INGERSOLL', 'DES MOINES', '50300', 77, 'Polk', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 46350, 'Hawkeye Light Rum', 6, '1750', 7.56, 11.33, 6, 67.98, 10.5, 2.77, -93.663337, 41.586313, '88260d80d7fffff'), ('S03385300034', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54446, 'Paramount Apricot Flavored Brandy', 12, '750', 5.21, 7.82, 12, 93.84, 9.0, 2.38, -91.136655, 40.814666, '882670aacdfffff'), ('S03408700016', Timestamp('2012-01-03 00:00:00'), 4066, 'Xpress Liquor', '1201 NORTH JEFFERSON STE 100', 'INDIANOLA', '50125', 91, 'Warren', 1012210, 'SINGLE MALT SCOTCH', 65, 'Jim Beam Brands', 5596, 'Mcclelland Highland', 12, '750', 13.01, 19.51, 3, 58.53, 2.25, 0.59, 0.0, 0.0, '0'), ('S03399000009', Timestamp('2012-01-03 00:00:00'), 3762, 'Wine and Spirits Gallery', '7690 HICKMAN RD', 'WINDSOR HEIGHTS', '50322', 77, 'Polk', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 40945, 'Smirnoff Fluffed Marshallmallow Mini', 12, '500', 5.72, 8.57, 1, 8.57, 0.5, 0.13, 0.0, 0.0, '0'), ('S03391100001', Timestamp('2012-01-03 00:00:00'), 4172, 'Shamrock Spirits', '1204 WATER ST PO BOX 177', 'ALDEN', '50006', 42, 'Hardin', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43336, 'Captain Morgan Original Spiced', 12, '750', 8.5, 12.74, 12, 152.88, 9.0, 2.38, -93.376271, 42.520423, '88262b8ed9fffff'), ('S03396400020', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 36, 382.32, 36.0, 9.51, -95.8792, 41.238092, '882608c9e7fffff'), ('S03388700017', Timestamp('2012-01-03 00:00:00'), 2367, "Spencer's Grocery", '101 SOUTH BURNETT', 'SHENANDOAH', '51601', 73, 'Page', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64866, 'Fireball Cinnamon Whiskey', 12, '750', 8.66, 12.99, 3, 38.97, 2.25, 0.59, -95.381935, 40.765934, '8826080d9dfffff'), ('S03393900037', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 434, 'Luxco-St Louis', 18196, 'Ezra Brooks 50 Mo Ky Sour Mash Bourbon', 12, '750', 6.39, 9.59, 1, 9.59, 0.75, 0.2, -95.873073, 41.261866, '882608c917fffff'), ('S03408600005', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03393600017', Timestamp('2012-01-03 00:00:00'), 4040, 'No Frills Supermarkets / Council Blu', '1801 VALLEY VIEW DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1071100, 'AMERICAN COCKTAILS', 35, 'Bacardi U.S.A., Inc.', 56846, 'Bacardi Party Drinks Rum Island Iced Tea', 6, '1750', 5.98, 9.47, 6, 56.82, 10.5, 2.77, -95.81877500000002, 41.24394, '882608c9d9fffff'), ('S03408200040', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52596, 'E & J Vs', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03393400059', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36305, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 12, 58.56, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03390800037', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11294, 'Crown Royal Canadian Whisky', 24, '375', 7.24, 10.85, 4, 43.4, 1.5, 0.4, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03407200038', Timestamp('2012-01-03 00:00:00'), 4146, 'Fareway Stores #561 / Waverly', '220 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45246, 'Paramount Gold Rum', 12, '750', 3.85, 5.77, 3, 17.31, 2.25, 0.59, -92.473476, 42.725748, '8826291847fffff'), ('S03396300006', Timestamp('2012-01-03 00:00:00'), 4041, 'Mandm Pantry', '2344 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45278, 'Paramount White Rum', 6, '1750', 7.58, 11.37, 6, 68.22, 10.5, 2.77, -90.213015, 41.821777, '88275b7897fffff'), ('S03402700042', Timestamp('2012-01-03 00:00:00'), 3907, "Smokin' Joe's #16 Tobacco and Liquor", '1504 PARK AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43124, 'Bacardi Superior Rum', 24, '375', 4.3, 6.45, 4, 25.8, 1.5, 0.4, -91.030569, 41.442458, '88267421bdfffff'), ('S03404300005', Timestamp('2012-01-03 00:00:00'), 4538, 'K & S Grocery', '402 NORTH MAIN ST', 'CLARKSVILLE', '50619', 12, 'Butler', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41694, 'Uv Blue (raspberry) Vodka', 12, '1000', 7.25, 10.87, 12, 130.44, 12.0, 3.17, -92.667833, 42.787434, '882629cdc1fffff'), ('S03401600053', Timestamp('2012-01-03 00:00:00'), 2522, 'Hy-Vee Wine and Spirits / Spirit Lak', 'HWY 9 & 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750', 9.97, 14.95, 12, 179.4, 9.0, 2.38, 0.0, 0.0, '0'), ('S03408200096', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35914, "Five O'clock Vodka", 24, '375', 1.84, 2.75, 3, 8.25, 1.12, 0.3, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03407400014', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37996, 'Smirnoff Vodka 80 Prf', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -93.736517, 41.659057, '88260d8045fffff'), ('S03400300003', Timestamp('2012-01-03 00:00:00'), 3461, 'Hometown Foods / Stuart', '611 S DIVISION ST', 'STUART', '50250', 1, 'Adair', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 6, 152.88, 10.5, 2.77, -94.318443, 41.49759900000001, '88260dcc2bfffff'), ('S03397600077', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69638, "Dr. Mcgillicuddy's Cherry Schnapps", 6, '1750', 13.33, 19.99, 6, 119.94, 10.5, 2.77, -95.825137, 41.242732, '882608c9d1fffff'), ('S03392300063', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1022100, 'TEQUILA', 395, 'Proximo', 89197, 'Jose Cuervo Especial Reposado Tequila', 12, '1000', 12.5, 18.74, 12, 224.88, 12.0, 3.17, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03403000031', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1051010, 'AMERICAN GRAPE BRANDIES', 85, 'Brown-Forman Corporation', 52808, 'Korbel Brandy', 6, '1750', 11.33, 16.99, 1, 16.99, 1.75, 0.46, -94.480204, 43.39966, '88262a82cbfffff'), ('S03407400020', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1022100, 'TEQUILA', 395, 'Proximo', 87510, '1800 Silver Tequila', 12, '750', 14.5, 21.74, 3, 65.22, 2.25, 0.59, -93.736517, 41.659057, '88260d8045fffff'), ('S03389000052', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 81207, 'Paramount Peppermint Schnapps', 12, '1000', 4.05, 6.08, 2, 12.16, 2.0, 0.53, -93.315977, 40.757158, '88267252cdfffff'), ('S03394300003', Timestamp('2012-01-03 00:00:00'), 4252, 'Fareway Stores #840 / Monticello', '433 N FARLEY ST', 'MONTICELLO', '52310', 53, 'Jones', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43126, 'Bacardi Superior Rum', 12, '750', 7.53, 11.3, 12, 135.6, 9.0, 2.38, -91.191385, 42.241939, '882674d69dfffff'), ('S03402600064', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10008, 'Scoresby Rare Scotch', 6, '1750', 10.22, 15.33, 1, 15.33, 1.75, 0.46, 0.0, 0.0, '0'), ('S03402600049', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43296, 'Captain Morgan Silver Spiced Rum', 12, '750', 8.0, 11.99, 2, 23.98, 1.5, 0.4, 0.0, 0.0, '0'), ('S03394000012', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43334, 'Captain Morgan Spiced Rum', 24, '375', 4.3, 6.45, 24, 154.8, 9.0, 2.38, -95.902023, 41.257663, '882608c937fffff'), ('S03408800038', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86886, 'Southern Comfort', 12, '750', 9.36, 14.04, 12, 168.48, 9.0, 2.38, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03408300334', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 125, 'Cedar Ridge Vineyards,LLC', 72950, 'Cedar Ridge Lemoncella', 12, '375', 8.0, 12.0, 3, 36.0, 1.12, 0.3, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03383900004', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1031080, 'VODKA 80 PROOF', 255, 'Wilson Daniels Ltd.', 37886, 'Seagrams Vodka', 12, '750', 5.38, 8.07, 12, 96.84, 9.0, 2.38, -91.403752, 40.41538, '8826702f65fffff'), ('S03390300070', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67527, 'Kahlua Coffee Liqueur', 12, '1000', 14.81, 22.22, 3, 66.66, 3.0, 0.79, -95.854809, 41.262316, '882608c9cdfffff'), ('S03392300031', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34422, 'Grey Goose Vodka', 6, '1000', 21.77, 32.66, 24, 783.84, 24.0, 6.34, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03404800031', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15246, 'Windsor Canadian', 12, '750', 4.2, 6.3, 12, 75.6, 9.0, 2.38, -91.035138, 41.451349, '88267421abfffff'), ('S03408000001', Timestamp('2012-01-03 00:00:00'), 3622, 'Wal-Mart 1415 / Spirit Lake', '2600 HWY 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -95.126535, 43.416452, '88262ac237fffff'), ('S03400300009', Timestamp('2012-01-03 00:00:00'), 3461, 'Hometown Foods / Stuart', '611 S DIVISION ST', 'STUART', '50250', 1, 'Adair', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55084, 'Paramount Blackberry Brandy', 24, '375', 3.34, 5.0, 6, 30.0, 2.25, 0.59, -94.318443, 41.49759900000001, '88260dcc2bfffff'), ('S03411200022', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 389, 'REMY COINTREAU USA         .', 64776, 'Cointreau Liqueur', 12, '750', 23.31, 34.96, 1, 34.96, 0.75, 0.2, -93.697731, 41.665172, '88260d8231fffff'), ('S03393900013', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081365, 'TROPICAL FRUIT SCHNAPPS', 421, 'Sazerac Co., Inc.', 83907, 'Maui Blue Hawaiian Schnapps', 12, '1000', 4.54, 6.81, 1, 6.81, 1.0, 0.26, -95.873073, 41.261866, '882608c917fffff'), ('S03394700011', Timestamp('2012-01-03 00:00:00'), 3959, 'Hartig Drug Company #8 / University', '1600 UNIVERSITY AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15626, 'Jameson', 12, '750', 13.46, 20.2, 1, 20.2, 0.75, 0.2, -90.688027, 42.498279, '88275a2115fffff'), ('S03392500042', Timestamp('2012-01-03 00:00:00'), 3908, 'County Market #214 / Fort Madison', '11802 AVENUE H', 'FORT MADISON', '52627', 56, 'Lee', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23828, 'Five Star', 6, '1750', 7.36, 11.04, 12, 132.48, 21.0, 5.55, 0.0, 0.0, '0'), ('S03384000003', Timestamp('2012-01-03 00:00:00'), 2327, 'West Side Spirits', '801 HULL ST', 'CORNING', '50841', 2, 'Adams', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55088, 'Paramount Blackberry Brandy', 6, '1750', 10.75, 16.12, 6, 96.72, 10.5, 2.77, -94.74369200000001, 40.990378, '882608a233fffff'), ('S03408600038', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 20248, 'Old Crow', 6, '1750', 8.92, 13.38, 1, 13.38, 1.75, 0.46, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03398200008', Timestamp('2012-01-03 00:00:00'), 2633, 'Hy-Vee #3 / BDI / Des Moines', '3221 SE 14TH ST', 'DES MOINES', '50320', 77, 'Polk', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 74086, 'Godiva Liqueur', 12, '750', 16.42, 24.62, 12, 295.44, 9.0, 2.38, -93.596754, 41.554101, '88260d846dfffff'), ('S03395800162', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34423, 'Grey Goose Vodka', 12, '375', 8.48, 12.72, 3, 38.16, 1.12, 0.3, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03385500030', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58838, 'Jose Cuervo Authentic Lime Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, -93.21693, 42.741728, '88262b9605fffff'), ('S03397600074', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081380, 'MISCELLANEOUS SCHNAPPS', 260, 'Diageo Americas', 67296, 'Yukon Jack Perma Frost Schnapps', 12, '750', 8.54, 12.81, 12, 153.72, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03396400019', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19067, 'Jim Beam', 12, '1000', 11.54, 17.31, 36, 623.16, 36.0, 9.51, -95.8792, 41.238092, '882608c9e7fffff'), ('S03366100005', Timestamp('2012-01-03 00:00:00'), 4488, "Casey's General Store #2762 / Cedar", '20 MILLER AVE SW', 'CEDAR RAPIDS', '52404', 57, 'Linn', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37994, 'Smirnoff Vodka 80 Prf', 24, '375', 4.48, 6.72, 24, 161.28, 9.0, 2.38, -91.657314, 41.924504, '8826745503fffff'), ('S03395800210', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69646, "Dr. Mcgillicuddy's Lemon Drop", 12, '750', 8.66, 12.99, 3, 38.97, 2.25, 0.59, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03388100099', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081240, 'GREEN CREME DE MENTHE', 434, 'Luxco-St Louis', 79026, 'Arrow Creme De Menthe Green', 12, '750', 4.39, 6.58, 2, 13.16, 1.5, 0.4, -95.879662, 41.261925, '882608c93bfffff'), ('S03402900010', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43335, 'Captain Morgan Spiced Rum Pet', 6, '1750', 16.99, 25.48, 6, 152.88, 10.5, 2.77, -94.990031, 41.403856, '8826089095fffff'), ('S03409700115', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1081330, 'PEACH SCHNAPPS', 434, 'Luxco-St Louis', 84456, 'Paramount Peach Schnapps', 12, '750', 3.97, 5.95, 6, 35.7, 4.5, 1.19, -93.737447, 41.62941, '88260d802bfffff'), ('S03391300124', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1101100, 'AMERICAN ALCOHOL', 434, 'Luxco-St Louis', 41846, 'Everclear Alcohol', 12, '750', 8.02, 12.03, 2, 24.06, 1.5, 0.4, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03393600001', Timestamp('2012-01-03 00:00:00'), 4040, 'No Frills Supermarkets / Council Blu', '1801 VALLEY VIEW DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36904, 'Mccormick Vodka Pet', 24, '375', 1.76, 2.63, 24, 63.12, 9.0, 2.38, -95.81877500000002, 41.24394, '882608c9d9fffff'), ('S03371500024', Timestamp('2012-01-03 00:00:00'), 4572, 'Grundy Center Family Foods', '1400, G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21596, 'Ten High', 12, '750', 4.0, 5.99, 2, 11.98, 1.5, 0.4, -92.782283, 42.36157, '8826297835fffff'), ('S03387400015', Timestamp('2012-01-03 00:00:00'), 3642, "Fraise Foods / Sonny's Supermarket", '327  5TH ST', 'WEST POINT', '52656', 56, 'Lee', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 24, 395.76, 24.0, 6.34, -91.452491, 40.717867, '88267005ebfffff'), ('S03394000011', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 240, 'WILLIAM GRANT AND SONS, INC.', 45887, 'Sailor Jerry Spiced Navy Rum', 12, '1000', 10.5, 15.75, 12, 189.0, 12.0, 3.17, -95.902023, 41.257663, '882608c937fffff'), ('S03412100170', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1012300, 'IRISH WHISKIES', 395, 'Proximo', 15777, 'Bushmills Irish Whiskey', 12, '1000', 16.25, 24.37, 2, 48.74, 2.0, 0.53, -90.483701, 41.552338, '882675da95fffff'), ('S03408600064', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1081200, 'CREAM LIQUEURS', 300, 'Mccormick Distilling Company', 68611, "McCormick's Irish Cream", 12, '750', 6.39, 9.59, 1, 9.59, 0.75, 0.2, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03408200042', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1012210, 'SINGLE MALT SCOTCH', 420, 'Moet Hennessy USA, Inc.', 5103, 'Glenmorangie Lasanta', 6, '750', 25.0, 37.5, 6, 225.0, 4.5, 1.19, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03403500026', Timestamp('2012-01-03 00:00:00'), 4156, 'Fareway Stores #412 / Oelwein', '102  2ND AVE SE', 'OELWEIN', '50662', 33, 'Fayette', 1041100, 'AMERICAN DRY GINS', 205, 'E AND J GALLO WINERY', 31475, 'New Amsterdam Gin', 12, '750', 6.9, 10.35, 4, 41.4, 3.0, 0.79, -91.910502, 42.677024, '882629b807fffff'), ('S03411200011', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11786, 'Black Velvet Traveler', 12, '750', 4.83, 7.44, 12, 89.28, 9.0, 2.38, -93.697731, 41.665172, '88260d8231fffff'), ('S03396600081', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43296, 'Captain Morgan Silver Spiced Rum', 12, '750', 8.0, 11.99, 2, 23.98, 1.5, 0.4, -93.378772, 43.142868, '8826283b29fffff'), ('S03382200022', Timestamp('2012-01-03 00:00:00'), 4584, 'J & C Grocery / Dumont', '611 MAIN ST', 'DUMONT', '50602', 12, 'Butler', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52596, 'E & J Vs', 12, '750', 5.51, 8.26, 4, 33.04, 3.0, 0.79, -92.973022, 42.750551, '8826295aebfffff'), ('S03401000084', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35917, "Five O'clock Vodka", 12, '1000', 4.01, 6.01, 3, 18.03, 3.0, 0.79, -90.683356, 42.52717, '88275a215dfffff'), ('S03397600158', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19486, "Maker's 46", 6, '750', 20.47, 30.7, 4, 122.8, 3.0, 0.79, -95.825137, 41.242732, '882608c9d1fffff'), ('S03385900014', Timestamp('2012-01-03 00:00:00'), 4356, 'Kum & Go #396 / Nevada', '136 LINCOLN HWY', 'NEVADA', '50201', 85, 'Story', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31654, 'Paramount Gin', 24, '375', 1.99, 2.98, 3, 8.94, 1.12, 0.3, -93.458003, 42.022725, '88262b235bfffff'), ('S03410900020', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 35626, 'Pinnacle Whipped', 12, '750', 7.87, 11.81, 12, 141.72, 9.0, 2.38, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03388500013', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35946, "Fleischmann's Royal Vodka", 12, '750', 3.44, 5.15, 24, 123.6, 18.0, 4.76, -90.668138, 42.504959, '88275a2119fffff'), ('S03403400023', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65254, 'Jagermeister Liqueur', 24, '375', 6.23, 9.34, 9, 84.06, 3.38, 0.89, -93.599362, 41.59878, '88260d8743fffff'), ('S03409700122', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1032080, 'IMPORTED VODKA', 260, 'Diageo Americas', 34198, 'Ciroc Super Premium Vodka', 6, '750', 17.97, 26.96, 4, 107.84, 3.0, 0.79, -93.737447, 41.62941, '88260d802bfffff'), ('S03410300126', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55088, 'Paramount Blackberry Brandy', 6, '1750', 10.75, 16.12, 1, 16.12, 1.75, 0.46, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03385500046', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82784, 'Dekuyper Buttershots Burst Schnapps', 24, '375', 3.06, 4.59, 5, 22.95, 1.88, 0.5, -93.21693, 42.741728, '88262b9605fffff'), ('S03396500017', Timestamp('2012-01-03 00:00:00'), 3587, 'Save A Lot  #482', '2122 MAIN ST', 'KEOKUK', '52632', 56, 'Lee', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34422, 'Grey Goose Vodka', 6, '1000', 21.77, 32.66, 6, 195.96, 6.0, 1.59, -91.39688800000002, 40.408821, '8826702f67fffff'), ('S03412100102', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10278, "Usher's Green Stripe", 6, '1750', 10.0, 14.99, 6, 89.94, 10.5, 2.77, -90.483701, 41.552338, '882675da95fffff'), ('S03392300040', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43387, 'Castillo Silver Rum', 12, '1000', 4.89, 7.33, 24, 175.92, 24.0, 6.34, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03405900019', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1032080, 'IMPORTED VODKA', 420, 'Moet Hennessy USA, Inc.', 34162, 'Belvedere Vodka', 6, '1000', 20.62, 30.94, 6, 185.64, 6.0, 1.59, -90.48065, 41.529655, '882675c361fffff'), ('S03396400100', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11293, 'Crown Royal Canadian Whisky', 48, '200', 4.47, 6.7, 2, 13.4, 0.4, 0.11, -95.8792, 41.238092, '882608c9e7fffff'), ('S03384000032', Timestamp('2012-01-03 00:00:00'), 2327, 'West Side Spirits', '801 HULL ST', 'CORNING', '50841', 2, 'Adams', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52596, 'E & J Vs', 12, '750', 5.51, 8.26, 3, 24.78, 2.25, 0.59, -94.74369200000001, 40.990378, '882608a233fffff'), ('S03410300025', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15627, 'Jameson', 12, '1000', 17.4, 26.1, 1, 26.1, 1.0, 0.26, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03396400083', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43123, 'Bacardi Superior Rum', 48, '200', 2.5, 3.75, 2, 7.5, 0.4, 0.11, -95.8792, 41.238092, '882608c9e7fffff'), ('S03392300028', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19067, 'Jim Beam', 12, '1000', 11.54, 17.31, 12, 207.72, 12.0, 3.17, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03405800014', Timestamp('2012-01-03 00:00:00'), 3761, "Cheese's Food Center", '200 N WALNUT ST', 'LENOX', '50851', 87, 'Taylor', 1081600, 'WHISKEY LIQUEUR', 260, 'Diageo Americas', 67266, 'Yukon Jack Canadian Liqueur', 12, '750', 8.54, 12.81, 2, 25.62, 1.5, 0.4, -94.560455, 40.882645, '882608a6e9fffff'), ('S03411200155', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1081030, 'COFFEE LIQUEURS', 306, 'Mississippi River Distilling Co.', 74776, 'Iowa Coffee Company Liqueur', 12, '375', 9.08, 13.63, 1, 13.63, 0.38, 0.1, -93.697731, 41.665172, '88260d8231fffff'), ('S03402100099', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64866, 'Fireball Cinnamon Whiskey', 12, '750', 8.66, 12.99, 60, 779.4, 45.0, 11.89, -92.475403, 42.712263, '8826291801fffff'), ('S03403700016', Timestamp('2012-01-03 00:00:00'), 4547, 'Fareway Stores #902 / Hampton', '309 CENTRAL AVE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25606, 'Seagrams 7 Crown Bl Whiskey', 12, '750', 6.14, 9.21, 2, 18.42, 1.5, 0.4, -93.211676, 42.741748, '88262b9605fffff'), ('S03399800014', Timestamp('2012-01-03 00:00:00'), 3970, 'The Liquor Cabinet, Inc.', '835 6 ST, STE 1', 'NEVADA', '50201', 85, 'Story', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34026, 'Absolut Peppar', 12, '750', 10.89, 16.33, 4, 65.32, 3.0, 0.79, -93.452277, 42.019292, '88262b222dfffff'), ('S03395200039', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1032080, 'IMPORTED VODKA', 260, 'Diageo Americas', 34198, 'Ciroc Super Premium Vodka', 6, '750', 17.97, 26.96, 3, 80.88, 2.25, 0.59, -91.110752, 40.801889, '882670a137fffff'), ('S03395200096', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58838, 'Jose Cuervo Authentic Lime Margarita', 6, '1750', 7.7, 11.8, 3, 35.4, 5.25, 1.39, -91.110752, 40.801889, '882670a137fffff'), ('S03388100046', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 12, 295.68, 12.0, 3.17, -95.879662, 41.261925, '882608c93bfffff'), ('S03388000085', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43027, 'Admiral Nelson Spiced Rum', 12, '1000', 7.27, 10.91, 24, 261.84, 24.0, 6.34, -93.633306, 43.261538, '88262808e7fffff'), ('S03400700001', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36904, 'Mccormick Vodka Pet', 24, '375', 1.76, 2.63, 24, 63.12, 9.0, 2.38, 0.0, 0.0, '0'), ('S03410300059', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76487, 'Paramount Amaretto', 12, '1000', 4.04, 6.06, 7, 42.42, 7.0, 1.85, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03403800010', Timestamp('2012-01-03 00:00:00'), 3774, 'Eagle Country Market / Dubuque', '1800 ELM ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35916, "Five O'clock Vodka", 12, '750', 3.31, 4.96, 24, 119.04, 18.0, 4.76, -90.665763, 42.511105, '88275a2153fffff'), ('S03390400025', Timestamp('2012-01-03 00:00:00'), 3738, 'Hartig Drug Company #4 / Dubuque', '2225 CENTRAL AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 63755, 'Tgi Fridays Mudslide', 6, '1750', 7.94, 11.91, 3, 35.73, 5.25, 1.39, -90.671531, 42.51384, '88275a2153fffff'), ('S03393800003', Timestamp('2012-01-03 00:00:00'), 3629, 'Wal-Mart 1965 / Council Bluffs', '3200 MANAWA DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 24, 223.92, 18.0, 4.76, 0.0, 0.0, '0'), ('S03410700068', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 37938, 'Skol Vodka', 6, '1750', 7.1, 10.65, 6, 63.9, 10.5, 2.77, -94.853591, 42.064155, '88260c16a9fffff'), ('S03410900091', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41360, 'Uv Pink Lemonade Vodka', 12, '750', 6.22, 9.33, 4, 37.32, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03408600060', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 375, 'Park Street Imports-Berentzen', 64446, 'Berentzen Apple Liqueur', 12, '750', 10.12, 15.18, 1, 15.18, 0.75, 0.2, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03396600064', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15246, 'Windsor Canadian', 12, '750', 4.2, 6.3, 2, 12.6, 1.5, 0.4, -93.378772, 43.142868, '8826283b29fffff'), ('S03404700011', Timestamp('2012-01-03 00:00:00'), 3972, 'Hometown Foods / Hubbard', '210 MAPLE ST BOX 295', 'HUBBARD', '50122', 42, 'Hardin', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65195, 'Hpnotiq', 6, '750', 11.68, 17.52, 1, 17.52, 0.75, 0.2, -93.302481, 42.305571, '88262baa53fffff'), ('S03385300012', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 36188, "Gordon's Vodka 80 Prf - Pet", 6, '1750', 9.68, 14.52, 6, 87.12, 10.5, 2.77, -91.136655, 40.814666, '882670aacdfffff'), ('S03394400013', Timestamp('2012-01-03 00:00:00'), 4334, 'Fareway Stores #114 / Dubuque', '2050 JOHN F KENNEDY RD', 'DUBUQUE', '52001', 31, 'Dubuque', 1022100, 'TEQUILA', 395, 'Proximo', 89191, 'Jose Cuervo Especial Reposado Tequila Mini', 12, '500', 11.5, 17.25, 2, 34.5, 1.0, 0.26, 0.0, 0.0, '0'), ('S03400300018', Timestamp('2012-01-03 00:00:00'), 3461, 'Hometown Foods / Stuart', '611 S DIVISION ST', 'STUART', '50250', 1, 'Adair', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19066, 'Jim Beam', 12, '750', 8.42, 13.41, 4, 53.64, 3.0, 0.79, -94.318443, 41.49759900000001, '88260dcc2bfffff'), ('S03410200008', Timestamp('2012-01-03 00:00:00'), 3889, 'Wal-Mart 1005 / Waverly', '700  29 ST SW', 'WAVERLY', '50677', 9, 'Bremer', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 12, 128.4, 21.0, 5.55, -92.478899, 42.70032100000001, '8826291807fffff'), ('S03408700018', Timestamp('2012-01-03 00:00:00'), 4066, 'Xpress Liquor', '1201 NORTH JEFFERSON STE 100', 'INDIANOLA', '50125', 91, 'Warren', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82866, 'Dekuyper Watermelon Pucker', 12, '750', 6.3, 9.45, 5, 47.25, 3.75, 0.99, 0.0, 0.0, '0'), ('S03404100053', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 285, 'Charles Jacquin Et. Cie., Inc.', 56957, 'Bartenders Hot Sex', 12, '1000', 5.11, 7.66, 1, 7.66, 1.0, 0.26, -90.993859, 42.3036, '8826749ac1fffff'), ('S03411200030', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5345, 'Johnnie Walker Red', 24, '375', 7.5, 11.24, 1, 11.24, 0.38, 0.1, -93.697731, 41.665172, '88260d8231fffff'), ('S03406700010', Timestamp('2012-01-03 00:00:00'), 3849, 'Hartig Drug Co #6 / Dyersville', '711  16TH AVE SE', 'DYERSVILLE', '52040', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58838, 'Jose Cuervo Authentic Lime Margarita', 6, '1750', 7.7, 11.8, 2, 23.6, 3.5, 0.92, -91.116087, 42.470285, '88275a654dfffff'), ('S03401000038', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23824, 'Five Star', 24, '375', 1.98, 2.96, 6, 17.76, 2.25, 0.59, -90.683356, 42.52717, '88275a215dfffff'), ('S03403500031', Timestamp('2012-01-03 00:00:00'), 4156, 'Fareway Stores #412 / Oelwein', '102  2ND AVE SE', 'OELWEIN', '50662', 33, 'Fayette', 1081365, 'TROPICAL FRUIT SCHNAPPS', 65, 'Jim Beam Brands', 82856, 'Dekuyper Island Punch Pucker', 12, '750', 6.3, 9.45, 4, 37.8, 3.0, 0.79, -91.910502, 42.677024, '882629b807fffff'), ('S03396400061', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 55, 'Sazerac North America', 44217, 'Barton Rum Light', 12, '1000', 3.99, 5.99, 24, 143.76, 24.0, 6.34, -95.8792, 41.238092, '882608c9e7fffff'), ('S03403200054', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26824, 'Jack Daniels Old #7 Black Lbl Square', 24, '375', 6.06, 9.09, 1, 9.09, 0.38, 0.1, -94.218, 43.29355, '8826284833fffff'), ('S03404100022', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -90.993859, 42.3036, '8826749ac1fffff'), ('S03408300157', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1012210, 'SINGLE MALT SCOTCH', 420, 'Moet Hennessy USA, Inc.', 5133, 'Glenmorangie 10 Yr Single Malt Scotch', 6, '750', 19.96, 29.94, 1, 29.94, 0.75, 0.2, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03408200016', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58875, 'Jose Cuervo Golden Margarita', 6, '1750', 10.0, 15.24, 6, 91.44, 10.5, 2.77, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03395700015', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 20248, 'Old Crow', 6, '1750', 8.92, 13.38, 1, 13.38, 1.75, 0.46, -95.147741, 43.14521, '88262a1925fffff'), ('S03393900046', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86885, 'Southern Comfort Pet', 12, '750', 9.36, 14.04, 1, 14.04, 0.75, 0.2, -95.873073, 41.261866, '882608c917fffff'), ('S03396000095', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1081700, 'DISTILLED SPIRITS SPECIALTY', 55, 'Sazerac North America', 76227, 'Montezuma Blue', 12, '1000', 4.16, 6.24, 36, 224.64, 36.0, 9.51, -90.705328, 42.491862, '88275a2131fffff'), ('S03389600001', Timestamp('2012-01-03 00:00:00'), 3632, 'Wal-Mart 2004 / Dubuque', '4200 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11348, 'Seagrams V.o. Bl Canadian Whisky Pet', 6, '1750', 15.42, 23.13, 6, 138.78, 10.5, 2.77, -90.736955, 42.489041, '88275a2ac1fffff'), ('S03404800091', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82607, 'Dekuyper Sour Apple', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -91.035138, 41.451349, '88267421abfffff'), ('S03396000070', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42717, 'Malibu Coconut Rum', 12, '1000', 9.77, 14.66, 24, 351.84, 24.0, 6.34, -90.705328, 42.491862, '88275a2131fffff'), ('S03395200075', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1012100, 'CANADIAN WHISKIES', 434, 'Luxco-St Louis', 13038, 'Canadian Reserve Whisky', 6, '1750', 8.77, 13.16, 2, 26.32, 3.5, 0.92, -91.110752, 40.801889, '882670a137fffff'), ('S03408300257', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34433, 'Grey Goose Vodka', 12, '750', 17.97, 26.96, 2, 53.92, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03410900103', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34030, 'Absolut Citron (lemon Vodka)', 12, '750', 10.89, 16.33, 4, 65.32, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03396500007', Timestamp('2012-01-03 00:00:00'), 3587, 'Save A Lot  #482', '2122 MAIN ST', 'KEOKUK', '52632', 56, 'Lee', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35316, 'Barton Vodka', 12, '750', 3.37, 5.05, 12, 60.6, 9.0, 2.38, -91.39688800000002, 40.408821, '8826702f67fffff'), ('S03389400021', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45277, 'Paramount White Rum', 12, '1000', 4.33, 6.5, 24, 156.0, 24.0, 6.34, -95.909384, 41.242291, '88260e26d5fffff'), ('S03410300043', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34051, 'Absolut Raspberri', 12, '1000', 14.89, 22.34, 1, 22.34, 1.0, 0.26, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03404800042', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57129, "Chi-Chi's Mexican Mudslide", 6, '1750', 5.99, 9.24, 6, 55.44, 10.5, 2.77, -91.035138, 41.451349, '88267421abfffff'), ('S03402900036', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52594, 'E & J Vs Brandy', 24, '375', 2.93, 4.4, 1, 4.4, 0.38, 0.1, -94.990031, 41.403856, '8826089095fffff'), ('S03390800008', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31658, 'Paramount Gin', 6, '1750', 7.1, 10.65, 6, 63.9, 10.5, 2.77, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03402100040', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1081365, 'TROPICAL FRUIT SCHNAPPS', 421, 'Sazerac Co., Inc.', 83906, 'Maui Blue Hawaiian Schnapps', 12, '750', 3.67, 5.5, 12, 66.0, 9.0, 2.38, -92.475403, 42.712263, '8826291801fffff'), ('S03410300040', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41076, 'Jeremiah Weed Sweet Tea Vodka', 12, '750', 9.46, 14.2, 2, 28.4, 1.5, 0.4, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03396400101', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11294, 'Crown Royal Canadian Whisky', 24, '375', 7.24, 10.85, 4, 43.4, 1.5, 0.4, -95.8792, 41.238092, '882608c9e7fffff'), ('S03397600075', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 420, 'Moet Hennessy USA, Inc.', 65127, 'Grand Marnier Cordon Rouge', 12, '1000', 25.54, 38.31, 12, 459.72, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03385300126', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1012210, 'SINGLE MALT SCOTCH', 240, 'WILLIAM GRANT AND SONS, INC.', 5006, 'Glenfiddich 12yr Special Reserve', 12, '750', 21.17, 31.75, 1, 31.75, 0.75, 0.2, -91.136655, 40.814666, '882670aacdfffff'), ('S03401100010', Timestamp('2012-01-03 00:00:00'), 4239, 'Fareway Stores #850 / Spirit Lake', '1906 KEOKUK', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35916, "Five O'clock Vodka", 12, '750', 3.31, 4.96, 12, 59.52, 9.0, 2.38, -95.106219, 43.42157, '88262ac23bfffff'), ('S03395800179', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 59154, '1800 Ultimate Margarita', 6, '1750', 9.5, 14.24, 2, 28.48, 3.5, 0.92, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03410300056', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1081365, 'TROPICAL FRUIT SCHNAPPS', 421, 'Sazerac Co., Inc.', 83907, 'Maui Blue Hawaiian Schnapps', 12, '1000', 4.54, 6.81, 1, 6.81, 1.0, 0.26, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03396200006', Timestamp('2012-01-03 00:00:00'), 3732, 'North Scott Foods', '425 E LECLAIRE RD', 'ELDRIDGE', '52748', 82, 'Scott', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -90.577311, 41.654695000000004, '882674a5bdfffff'), ('S03390900022', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11773, 'Black Velvet', 48, '200', 1.56, 2.34, 3, 7.02, 0.6, 0.16, 0.0, 0.0, '0'), ('S03408200045', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10278, "Usher's Green Stripe", 6, '1750', 10.0, 14.99, 6, 89.94, 10.5, 2.77, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03408300329', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 63894, 'Tgif On The Rocks White Russian', 12, '750', 3.82, 5.73, 2, 11.46, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03388200009', Timestamp('2012-01-03 00:00:00'), 4106, 'Hartig Drug Co. / JFK Rd / Dubuque', '2255 JOHN F KENNEDY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26826, 'Jack Daniels Old #7 Black Lbl', 12, '750', 13.01, 19.51, 4, 78.04, 3.0, 0.79, -90.72051800000001, 42.511352, '88275a212bfffff'), ('S03408300245', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 85, 'Brown-Forman Corporation', 22216, 'Woodford Reserve Bourbon', 6, '750', 19.5, 29.24, 2, 58.48, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03390300025', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41694, 'Uv Blue (raspberry) Vodka', 12, '1000', 7.25, 10.87, 12, 130.44, 12.0, 3.17, -95.854809, 41.262316, '882608c9cdfffff'), ('S03393400081', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 20246, 'Old Crow', 12, '750', 4.19, 6.29, 12, 75.48, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03402100028', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1051110, 'BLACKBERRY BRANDIES', 55, 'Sazerac North America', 55066, 'Mr. Boston Blackberry Brandy', 12, '750', 4.12, 6.37, 12, 76.44, 9.0, 2.38, -92.475403, 42.712263, '8826291801fffff'), ('S03395800153', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75208, 'Kinky Liqueur Mini', 6, '500', 4.67, 7.0, 3, 21.0, 1.5, 0.4, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03395800051', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 58801, 'Captain Morgan Long Island Iced Tea', 6, '1750', 10.5, 15.74, 6, 94.44, 10.5, 2.77, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03397100024', Timestamp('2012-01-03 00:00:00'), 4306, 'Northside One Stop / Hampton', '1208  4TH ST NE', 'HAMPTON', '50441', 35, 'Franklin', 1081200, 'CREAM LIQUEURS', 300, 'Mccormick Distilling Company', 65199, 'Tequila Rose Liqueur', 12, '1000', 12.0, 17.99, 3, 53.97, 3.0, 0.79, -93.202452, 42.753045, '88262b9647fffff'), ('S03396000077', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68037, "Bailey's Original Irish Cream", 12, '1000', 16.5, 24.74, 12, 296.88, 12.0, 3.17, -90.705328, 42.491862, '88275a2131fffff'), ('S03402100029', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55086, 'Paramount Blackberry Brandy', 12, '750', 5.21, 7.82, 12, 93.84, 9.0, 2.38, -92.475403, 42.712263, '8826291801fffff'), ('S03396500016', Timestamp('2012-01-03 00:00:00'), 3587, 'Save A Lot  #482', '2122 MAIN ST', 'KEOKUK', '52632', 56, 'Lee', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 12, 268.08, 12.0, 3.17, -91.39688800000002, 40.408821, '8826702f67fffff'), ('S03404800022', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1081600, 'WHISKEY LIQUEUR', 259, 'Heaven Hill Brands', 86796, 'Southern Host Whiskey Liqueur', 12, '750', 4.83, 7.25, 12, 87.0, 9.0, 2.38, -91.035138, 41.451349, '88267421abfffff'), ('S03390300006', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23826, 'Five Star', 12, '750', 3.35, 5.02, 12, 60.24, 9.0, 2.38, -95.854809, 41.262316, '882608c9cdfffff'), ('S03385300086', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88291, 'Patron Tequila Silver Mini', 10, '300', 20.3, 30.45, 1, 30.45, 0.3, 0.08, -91.136655, 40.814666, '882670aacdfffff'), ('S03406900016', Timestamp('2012-01-03 00:00:00'), 4114, 'After 5 Somewhere', '704 W 7TH ST PO BOX 372', 'ATLANTIC', '50022', 15, 'Cass', 1011200, 'STRAIGHT BOURBON WHISKIES', 461, 'Campari(skyy)', 22155, 'Wild Turkey 101 Round', 12, '375', 5.75, 8.62, 4, 34.48, 1.5, 0.4, -95.023116, 41.40406200000001, '88260890a3fffff'), ('S03388100063', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45276, 'Paramount White Rum', 12, '750', 3.85, 5.77, 12, 69.24, 9.0, 2.38, -95.879662, 41.261925, '882608c93bfffff'), ('S03389300063', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 34189, 'Effen Black Cherry Vodka', 6, '750', 14.17, 21.25, 1, 21.25, 0.75, 0.2, -90.609747, 41.589369, '882675d959fffff'), ('S03403900030', Timestamp('2012-01-03 00:00:00'), 2628, 'Hy-Vee Food Store / Eldora', '1616 EDGINGTON', 'ELDORA', '50627', 42, 'Hardin', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54448, 'Paramount Apricot Flavored Brandy', 6, '1750', 10.75, 16.12, 6, 96.72, 10.5, 2.77, -93.103764, 42.360631, '88262ba25dfffff'), ('S03410200007', Timestamp('2012-01-03 00:00:00'), 3889, 'Wal-Mart 1005 / Waverly', '700  29 ST SW', 'WAVERLY', '50677', 9, 'Bremer', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 12, 305.76, 21.0, 5.55, -92.478899, 42.70032100000001, '8826291807fffff'), ('S03405300012', Timestamp('2012-01-03 00:00:00'), 4576, 'THE BOONEDOCKS', '228 N HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86886, 'Southern Comfort', 12, '750', 9.36, 14.04, 3, 42.12, 2.25, 0.59, -95.125928, 43.370941, '88262ac013fffff'), ('S03395700065', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69947, 'Rumple Minze Peppermint Schnapps Liqueur', 12, '1000', 15.0, 22.49, 2, 44.98, 2.0, 0.53, -95.147741, 43.14521, '88262a1925fffff'), ('S03377200016', Timestamp('2012-01-03 00:00:00'), 4438, 'Kum & Go #214  Ames', '111 DUFF AVE', 'AMES', '50010', 85, 'Story', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25604, 'Seagrams 7 Crown Bl Whiskey', 24, '375', 3.34, 5.0, 6, 30.0, 2.25, 0.59, -93.610396, 42.023592, '88262b38adfffff'), ('S03395700060', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82847, 'Dekuyper Peachtree', 12, '1000', 7.35, 11.02, 3, 33.06, 3.0, 0.79, -95.147741, 43.14521, '88262a1925fffff'), ('S03393100052', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76486, 'Paramount Amaretto', 12, '750', 3.37, 5.06, 12, 60.72, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03391300096', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35913, "Five O'clock Vodka", 48, '200', 1.1, 1.65, 3, 4.95, 0.6, 0.16, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03406100003', Timestamp('2012-01-03 00:00:00'), 4509, "A J'S LIQUOR II", '2515 CHAMBERLAIN', 'AMES', '50010', 85, 'Story', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37996, 'Smirnoff Vodka 80 Prf', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -93.650838, 42.021461, '88262b3983fffff'), ('S03383800062', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1051120, 'CHERRY BRANDIES', 434, 'Luxco-St Louis', 55504, 'Paramount Cherry Brandy', 24, '375', 3.34, 5.0, 1, 5.0, 0.38, 0.1, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03390900060', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69657, "Dr. Mcgillicuddy's Mentholmint", 12, '1000', 11.0, 16.5, 2, 33.0, 2.0, 0.53, 0.0, 0.0, '0'), ('S03395500040', Timestamp('2012-01-03 00:00:00'), 3976, 'North English Family Food', '119 NORTH MAIN ST', 'NORTH ENGLISH', '52316', 48, 'Iowa', 1062300, 'FLAVORED RUM', 298, 'White Rock Distilleries, Inc.', 46105, 'Cabana Bay Wild Cherry Rum', 12, '750', 6.08, 9.12, 1, 9.12, 0.75, 0.2, -92.076452, 41.515209, '882676a0adfffff'), ('S03407800002', Timestamp('2012-01-03 00:00:00'), 4181, 'Puff N Snuff', '1101 RHOMBERG AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35948, "Fleischmann's Royal Vodka 80 Prf", 6, '1750', 7.14, 10.7, 6, 64.2, 10.5, 2.77, -90.660269, 42.519502, '88275a215bfffff'), ('S03408800023', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03390800034', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45274, 'Paramount White Rum', 24, '375', 2.84, 4.25, 3, 12.75, 1.12, 0.3, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03398800012', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1081330, 'PEACH SCHNAPPS', 434, 'Luxco-St Louis', 84456, 'Paramount Peach Schnapps', 12, '750', 3.97, 5.95, 12, 71.4, 9.0, 2.38, -93.573845, 41.604312, '88260d95b3fffff'), ('S03403500012', Timestamp('2012-01-03 00:00:00'), 4156, 'Fareway Stores #412 / Oelwein', '102  2ND AVE SE', 'OELWEIN', '50662', 33, 'Fayette', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86886, 'Southern Comfort', 12, '750', 9.36, 14.04, 12, 168.48, 9.0, 2.38, -91.910502, 42.677024, '882629b807fffff'), ('S03397100016', Timestamp('2012-01-03 00:00:00'), 4306, 'Northside One Stop / Hampton', '1208  4TH ST NE', 'HAMPTON', '50441', 35, 'Franklin', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10625, 'Canadian Club Whisky Pet', 12, '750', 7.34, 11.01, 3, 33.03, 2.25, 0.59, -93.202452, 42.753045, '88262b9647fffff'), ('S03408300102', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081220, 'DARK CREME DE CACAO', 434, 'Luxco-St Louis', 78456, 'Paramount Creme De Cacao/dark', 12, '750', 4.44, 6.66, 3, 19.98, 2.25, 0.59, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03408800015', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1011200, 'STRAIGHT BOURBON WHISKIES', 421, 'Sazerac Co., Inc.', 16516, 'Ancient Age Bourbon', 12, '750', 6.0, 8.99, 12, 107.88, 9.0, 2.38, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03389100003', Timestamp('2012-01-03 00:00:00'), 4457, 'KUM & GO #422 / IOWA CITY', '513 S RIVERSIDE DR', 'IOWA CITY', '52240', 52, 'Johnson', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42716, 'Malibu Coconut Rum', 12, '750', 6.9, 10.35, 12, 124.2, 9.0, 2.38, -91.540402, 41.653301, '88267471e9fffff'), ('S03383800074', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65229, 'Hpnotiq Harmonie', 6, '750', 11.68, 17.52, 1, 17.52, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03412100209', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25604, 'Seagrams 7 Crown Bl Whiskey', 24, '375', 3.34, 5.0, 4, 20.0, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03390900077', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41719, 'Smirnoff Green Apple Vodka', 12, '750', 7.59, 11.56, 2, 23.12, 1.5, 0.4, 0.0, 0.0, '0'), ('S03396000101', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 125, 'Cedar Ridge Vineyards,LLC', 17206, 'Cedar Ridge Bourbon', 6, '750', 17.6, 26.4, 2, 52.8, 1.5, 0.4, -90.705328, 42.491862, '88275a2131fffff'), ('S03402700011', Timestamp('2012-01-03 00:00:00'), 3907, "Smokin' Joe's #16 Tobacco and Liquor", '1504 PARK AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65194, 'Hpnotiq', 12, '375', 6.35, 9.52, 1, 9.52, 0.38, 0.1, -91.030569, 41.442458, '88267421bdfffff'), ('S03411000062', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45277, 'Paramount White Rum', 12, '1000', 4.33, 6.5, 12, 78.0, 12.0, 3.17, 0.0, 0.0, '0'), ('S03384600002', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 395, 'Proximo', 34968, 'Three Olives Chocolate Vodka', 12, '750', 11.16, 16.74, 12, 200.88, 9.0, 2.38, -93.619787, 41.60566, '88260d8769fffff'), ('S03392200011', Timestamp('2012-01-03 00:00:00'), 4578, "Casey's General Store #2592 / Farmin", '201 S 4TH ST', 'FARMINGTON', '52626', 89, 'Van Buren', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -91.740069, 40.639743, '88267070c1fffff'), ('S03395800169', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1042100, 'IMPORTED DRY GINS', 240, 'WILLIAM GRANT AND SONS, INC.', 28625, "Hendrick's Gin", 6, '750', 16.25, 24.38, 3, 73.14, 2.25, 0.59, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03396100008', Timestamp('2012-01-03 00:00:00'), 2656, 'Hy-Vee Food Store / Corning', '300  10TH ST', 'CORNING', '51632', 22, 'Clayton', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26826, 'Jack Daniels Old #7 Black Lbl', 12, '750', 13.01, 19.51, 2, 39.02, 1.5, 0.4, -94.731809, 40.991861, '882608a217fffff'), ('S03412100052', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37347, 'Phillips Vodka', 12, '1000', 4.0, 6.01, 12, 72.12, 12.0, 3.17, -90.483701, 41.552338, '882675da95fffff'), ('S03396400109', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52598, 'E & J Vs', 6, '1750', 11.21, 16.82, 3, 50.46, 5.25, 1.39, -95.8792, 41.238092, '882608c9e7fffff'), ('S03408600090', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1041150, 'FLAVORED GINS', 370, 'Pernod Ricard USA/Austin Nichols', 33266, "Seagram's Orange Twisted Gin", 12, '750', 5.9, 8.85, 1, 8.85, 0.75, 0.2, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03408200012', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58860, 'Jose Cuervo Mango Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03398600016', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37348, 'Phillips Vodka', 6, '1750', 7.31, 10.97, 6, 65.82, 10.5, 2.77, -91.343544, 40.626988, '8826703131fffff'), ('S03410700076', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1062100, 'JAMAICA RUM', 461, 'Campari(skyy)', 42006, 'Appleton Signature Blend', 12, '750', 8.96, 13.45, 24, 322.8, 18.0, 4.76, -94.853591, 42.064155, '88260c16a9fffff'), ('S03387700011', Timestamp('2012-01-03 00:00:00'), 3013, "Keith's Foods", '207 E LOCUST ST', 'BLOOMFIELD', '52537', 26, 'Davis', 1031080, 'VODKA 80 PROOF', 421, 'Sazerac Co., Inc.', 36976, 'Nikolai Vodka', 12, '750', 3.26, 4.88, 12, 58.56, 9.0, 2.38, -92.412847, 40.752691, '88267284d5fffff'), ('S03389400014', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 30, 309.0, 52.5, 13.87, -95.909384, 41.242291, '88260e26d5fffff'), ('S03408100054', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15248, 'Windsor Canadian Pet', 6, '1750', 8.92, 13.38, 1, 13.38, 1.75, 0.46, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03408300101', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 305, 'MHW Ltd', 77570, 'Travis Hasse Apple Pie Liqueur', 12, '750', 7.94, 11.91, 2, 23.82, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03387100002', Timestamp('2012-01-03 00:00:00'), 4132, 'Camanche Food Pride', '908 7TH AVE', 'CAMANCHE', '52730', 23, 'Clinton', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11297, 'Crown Royal Canadian Whisky', 12, '1000', 17.46, 26.19, 12, 314.28, 12.0, 3.17, -90.257535, 41.788751, '88275b6a1dfffff'), ('S03383200010', Timestamp('2012-01-03 00:00:00'), 3562, 'Wal-Mart 0797 / W Burlington', '324 WEST AGENCY RD', 'WEST BURLINGTON', '52655', 29, 'Des Moines', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41592, 'Smirnoff Lime Vodka', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -91.158407, 40.813125, '882670aa17fffff'), ('S03401100006', Timestamp('2012-01-03 00:00:00'), 4239, 'Fareway Stores #850 / Spirit Lake', '1906 KEOKUK', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1042100, 'IMPORTED DRY GINS', 260, 'Diageo Americas', 28868, 'Tanqueray Gin', 6, '1750', 24.29, 36.44, 6, 218.64, 10.5, 2.77, -95.106219, 43.42157, '88262ac23bfffff'), ('S03395800009', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45277, 'Paramount White Rum', 12, '1000', 4.33, 6.5, 12, 78.0, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03404800098', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 305, 'MHW Ltd', 65015, 'Galliano Liqueur', 12, '750', 18.96, 28.44, 2, 56.88, 1.5, 0.4, -91.035138, 41.451349, '88267421abfffff'), ('S03384000020', Timestamp('2012-01-03 00:00:00'), 2327, 'West Side Spirits', '801 HULL ST', 'CORNING', '50841', 2, 'Adams', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 3, 67.02, 3.0, 0.79, -94.74369200000001, 40.990378, '882608a233fffff'), ('S03404000014', Timestamp('2012-01-03 00:00:00'), 3841, 'Hartig Drug Company #2 / Locust', '157 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1041100, 'AMERICAN DRY GINS', 55, 'Sazerac North America', 29288, 'Barton Gin', 6, '1750', 6.57, 9.85, 1, 9.85, 1.75, 0.46, -90.665439, 42.495308, '88275a211bfffff'), ('S03409700083', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48101, 'Hennessy Vs Cognac Minis', 10, '600', 23.6, 35.4, 1, 35.4, 0.6, 0.16, -93.737447, 41.62941, '88260d802bfffff'), ('S03395700033', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32234, 'Seagrams Extra Dry Gin', 24, '375', 3.4, 5.1, 5, 25.5, 1.88, 0.5, -95.147741, 43.14521, '88262a1925fffff'), ('S03409700048', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1041200, 'AMERICAN SLOE GINS', 434, 'Luxco-St Louis', 33716, 'Paramount Sloe Gin', 12, '750', 3.84, 5.75, 3, 17.25, 2.25, 0.59, -93.737447, 41.62941, '88260d802bfffff'), ('S03393400139', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 395, 'Proximo', 34886, 'Three Olives Berry Vodka', 12, '750', 11.16, 16.74, 2, 33.48, 1.5, 0.4, -90.741064, 42.516633, '88275a212dfffff'), ('S03394500041', Timestamp('2012-01-03 00:00:00'), 3443, 'Super Saver Iv', '1141 N BROADWAY', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43037, 'Bacardi Gold Rum', 12, '1000', 9.43, 14.14, 12, 169.68, 12.0, 3.17, -95.836515, 41.270824, '882608c823fffff'), ('S03381400029', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1052010, 'IMPORTED GRAPE BRANDIES', 389, 'REMY COINTREAU USA         .', 50686, 'St. Remy Vsop Napoleon Brandy', 12, '750', 7.14, 10.72, 24, 257.28, 18.0, 4.76, -90.527081, 41.559731, '882675daa5fffff'), ('S03395800196', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54444, 'Paramount Apricot Flavored Brandy', 24, '375', 3.34, 5.0, 3, 15.0, 1.12, 0.3, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03382200020', Timestamp('2012-01-03 00:00:00'), 4584, 'J & C Grocery / Dumont', '611 MAIN ST', 'DUMONT', '50602', 12, 'Butler', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 12406, 'Canadian Ltd Whisky', 12, '750', 3.75, 5.87, 6, 35.22, 4.5, 1.19, -92.973022, 42.750551, '8826295aebfffff'), ('S03408300300', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82604, 'Dekuyper Sour Apple Pucker', 24, '375', 3.06, 4.59, 4, 18.36, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03393200017', Timestamp('2012-01-03 00:00:00'), 3895, "Tom's Market and Meats", '550 DIVISION ST', 'BURLINGTON', '52601', 29, 'Des Moines', 1031100, '100 PROOF VODKA', 260, 'Diageo Americas', 39868, 'Smirnoff Vodka 100 Prf', 6, '1750', 15.5, 23.24, 6, 139.44, 10.5, 2.77, -91.106472, 40.807275000000004, '882670a131fffff'), ('S03390200045', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88296, 'Patron Tequila Silver', 12, '750', 26.08, 39.13, 1, 39.13, 0.75, 0.2, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03395800131', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 86672, "Jack Daniel's Tennessee Honey", 12, '1000', 16.43, 24.64, 12, 295.68, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03386500012', Timestamp('2012-01-03 00:00:00'), 2548, 'Hy-Vee Food Store / Altoona', '100 8TH STREET SW', 'ALTOONA', '50009', 77, 'Polk', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64867, 'Fireball Cinnamon Whiskey', 12, '1000', 11.0, 16.5, 12, 198.0, 12.0, 3.17, -93.465301, 41.644041, '88260d94d9fffff'), ('S03388100031', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67527, 'Kahlua Coffee Liqueur', 12, '1000', 14.81, 22.22, 12, 266.64, 12.0, 3.17, -95.879662, 41.261925, '882608c93bfffff'), ('S03412100147', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 389, 'REMY COINTREAU USA         .', 65896, 'Metaxa Ouzo', 12, '750', 9.24, 13.86, 2, 27.72, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03388300023', Timestamp('2012-01-03 00:00:00'), 4039, 'No Frills Supermarkets / Council Blu', '1817 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48104, 'Hennessy Vs Cognac', 24, '375', 8.38, 12.58, 24, 301.92, 9.0, 2.38, -95.870993, 41.261863, '882608c917fffff'), ('S03384000005', Timestamp('2012-01-03 00:00:00'), 2327, 'West Side Spirits', '801 HULL ST', 'CORNING', '50841', 2, 'Adams', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65253, 'Jagermeister Liqueur', 48, '200', 3.47, 5.2, 3, 15.6, 0.6, 0.16, -94.74369200000001, 40.990378, '882608a233fffff'), ('S03385400001', Timestamp('2012-01-03 00:00:00'), 4404, 'KUM & GO #292 / ANKENY', '1825 N ANKENY BLVD', 'ANKENY', '50021', 77, 'Polk', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19061, 'Jim Beam Mini', 12, '500', 6.83, 10.24, 12, 122.88, 6.0, 1.59, 0.0, 0.0, '0'), ('S03408200031', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43027, 'Admiral Nelson Spiced Rum', 12, '1000', 7.27, 10.91, 12, 130.92, 12.0, 3.17, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03402600004', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36904, 'Mccormick Vodka Pet', 24, '375', 1.76, 2.63, 24, 63.12, 9.0, 2.38, 0.0, 0.0, '0'), ('S03396400040', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1032080, 'IMPORTED VODKA', 260, 'Diageo Americas', 34457, 'Ketel One Imported Vodka', 12, '1000', 16.47, 24.7, 12, 296.4, 12.0, 3.17, -95.8792, 41.238092, '882608c9e7fffff'), ('S03410900089', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 300, 'Mccormick Distilling Company', 41019, '360 Double Chocolate Vodka', 12, '750', 10.64, 15.97, 2, 31.94, 1.5, 0.4, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03390000013', Timestamp('2012-01-03 00:00:00'), 3606, "Scotty's Liquor Store", '804 G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10627, 'Canadian Club Whisky', 12, '1000', 9.34, 14.01, 3, 42.03, 3.0, 0.79, -92.77421500000001, 42.361514, '8826297831fffff'), ('S03390900027', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1012200, 'SCOTCH WHISKIES', 240, 'WILLIAM GRANT AND SONS, INC.', 5106, "Grant's Blended Scotch", 12, '750', 10.41, 15.62, 2, 31.24, 1.5, 0.4, 0.0, 0.0, '0'), ('S03389600023', Timestamp('2012-01-03 00:00:00'), 3632, 'Wal-Mart 2004 / Dubuque', '4200 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58875, 'Jose Cuervo Golden Margarita', 6, '1750', 10.0, 15.24, 6, 91.44, 10.5, 2.77, -90.736955, 42.489041, '88275a2ac1fffff'), ('S03383800040', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41116, 'Jeremiah Weed Peach Sweet Tea', 12, '750', 9.46, 14.2, 1, 14.2, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03397600096', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43126, 'Bacardi Superior Rum', 12, '750', 7.53, 11.3, 12, 135.6, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03388000060', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41705, 'Uv Red (cherry) Vodka', 12, '1000', 7.25, 10.87, 12, 130.44, 12.0, 3.17, -93.633306, 43.261538, '88262808e7fffff'), ('S03381400031', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69657, "Dr. Mcgillicuddy's Mentholmint", 12, '1000', 11.0, 16.5, 24, 396.0, 24.0, 6.34, -90.527081, 41.559731, '882675daa5fffff'), ('S03395200080', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53216, 'Paul Masson Grande Amber Brandy', 12, '750', 6.29, 9.44, 3, 28.32, 2.25, 0.59, -91.110752, 40.801889, '882670a137fffff'), ('S03385800005', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11777, 'Black Velvet', 12, '1000', 6.63, 9.94, 24, 238.56, 24.0, 6.34, -92.258152, 42.809882, '8826291267fffff'), ('S03389300047', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1081600, 'WHISKEY LIQUEUR', 259, 'Heaven Hill Brands', 86796, 'Southern Host Whiskey Liqueur', 12, '750', 4.83, 7.25, 3, 21.75, 2.25, 0.59, -90.609747, 41.589369, '882675d959fffff'), ('S03394100015', Timestamp('2012-01-03 00:00:00'), 3451, 'Barnes Foodland', '622 10TH ST', 'DEWITT', '52742', 23, 'Clinton', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31658, 'Paramount Gin', 6, '1750', 7.1, 10.65, 6, 63.9, 10.5, 2.77, -90.540132, 41.825139, '882674b5e7fffff'), ('S03387400005', Timestamp('2012-01-03 00:00:00'), 3642, "Fraise Foods / Sonny's Supermarket", '327  5TH ST', 'WEST POINT', '52656', 56, 'Lee', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86887, 'Southern Comfort', 12, '1000', 11.34, 17.01, 12, 204.12, 12.0, 3.17, -91.452491, 40.717867, '88267005ebfffff'), ('S03396600129', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1081200, 'CREAM LIQUEURS', 330, 'Gemini Spirits', 80457, "Ryan's Cream Liqueur", 12, '1000', 7.22, 10.83, 3, 32.49, 3.0, 0.79, -93.378772, 43.142868, '8826283b29fffff'), ('S03408300287', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 65, 'Jim Beam Brands', 64795, 'Courvoisier Rose', 12, '750', 13.99, 20.98, 2, 41.96, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03393800004', Timestamp('2012-01-03 00:00:00'), 3629, 'Wal-Mart 1965 / Council Bluffs', '3200 MANAWA DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42716, 'Malibu Coconut Rum', 12, '750', 6.9, 10.35, 24, 248.4, 18.0, 4.76, 0.0, 0.0, '0'), ('S03406700015', Timestamp('2012-01-03 00:00:00'), 3849, 'Hartig Drug Co #6 / Dyersville', '711  16TH AVE SE', 'DYERSVILLE', '52040', 31, 'Dubuque', 1051120, 'CHERRY BRANDIES', 434, 'Luxco-St Louis', 55506, 'Paramount Cherry Flavored Brandy', 12, '750', 5.21, 7.82, 2, 15.64, 1.5, 0.4, -91.116087, 42.470285, '88275a654dfffff'), ('S03395800102', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081200, 'CREAM LIQUEURS', 461, 'Campari(skyy)', 68127, "Carolan's Irish Cream Liqueur", 12, '1000', 9.77, 14.65, 12, 175.8, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03403400048', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19061, 'Jim Beam Mini', 12, '500', 6.83, 10.24, 3, 30.72, 1.5, 0.4, -93.599362, 41.59878, '88260d8743fffff'), ('S03403200053', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1081317, 'GRAPE SCHNAPPS', 65, 'Jim Beam Brands', 82637, 'Dekuyper Grape Pucker', 12, '1000', 7.35, 11.02, 1, 11.02, 1.0, 0.26, -94.218, 43.29355, '8826284833fffff'), ('S03411000041', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69637, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '1000', 11.0, 16.5, 24, 396.0, 24.0, 6.34, 0.0, 0.0, '0'), ('S03411200012', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 12, 195.96, 9.0, 2.38, -93.697731, 41.665172, '88260d8231fffff'), ('S03400400015', Timestamp('2012-01-03 00:00:00'), 2655, 'Hy-Vee Food Store / Clarinda', '1200 S 16TH ST', 'CLARINDA', '51632', 73, 'Page', 1032200, 'IMPORTED VODKA - MISC', 395, 'Proximo', 34995, 'Three Olives Grape Vodka', 12, '750', 11.16, 16.74, 3, 50.22, 2.25, 0.59, -95.037849, 40.726808, '8826083189fffff'), ('S03385200003', Timestamp('2012-01-03 00:00:00'), 4534, 'D & B Foods', '141 W GILLET ST', 'PRESTON', '52069', 49, 'Jackson', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 6, 152.88, 10.5, 2.77, -90.39584400000001, 42.050312, '88275b4e29fffff'), ('S03385800029', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65257, 'Jagermeister Liqueur', 12, '1000', 14.93, 22.39, 12, 268.68, 12.0, 3.17, -92.258152, 42.809882, '8826291267fffff'), ('S03394000044', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1022100, 'TEQUILA', 395, 'Proximo', 89193, 'Jose Cuervo Especial Reposado Tequila', 48, '200', 3.0, 4.49, 6, 26.94, 1.2, 0.32, -95.902023, 41.257663, '882608c937fffff'), ('S03392100057', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36886, 'Mccormick Vodka', 12, '750', 3.3, 4.96, 2, 9.92, 1.5, 0.4, -90.204748, 41.827845, '88275b7891fffff'), ('S03397300078', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 34189, 'Effen Black Cherry Vodka', 6, '750', 14.17, 21.25, 3, 63.75, 2.25, 0.59, -91.044008, 41.427202, '8826742f4bfffff'), ('S03408300024', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 38006, 'Smirnoff Vodka Traveller', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03411000084', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1062300, 'FLAVORED RUM', 65, 'Jim Beam Brands', 45995, 'Calico Jack Whipped', 12, '750', 4.0, 6.5, 2, 13.0, 1.5, 0.4, 0.0, 0.0, '0'), ('S03408800030', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1041100, 'AMERICAN DRY GINS', 259, 'Heaven Hill Brands', 29568, "Burnett's Gin London Dry", 6, '1750', 9.06, 14.09, 6, 84.54, 10.5, 2.77, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03405700025', Timestamp('2012-01-03 00:00:00'), 3651, 'Wal-Mart 1491 / Indianola', '1500 NORTH JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58868, 'Jose Cuervo Authentic Strawberry Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, 0.0, 0.0, '0'), ('S03391300045', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 35626, 'Pinnacle Whipped', 12, '750', 7.87, 11.81, 1, 11.81, 0.75, 0.2, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03397600043', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1022100, 'TEQUILA', 395, 'Proximo', 89197, 'Jose Cuervo Especial Reposado Tequila', 12, '1000', 12.5, 18.74, 12, 224.88, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03408300104', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081230, 'WHITE CREME DE CACAO', 434, 'Luxco-St Louis', 78866, 'Paramount Creme De Cacao/white', 12, '750', 4.44, 6.66, 4, 26.64, 3.0, 0.79, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03390900052', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65258, 'Jagermeister Liqueur', 6, '1750', 25.69, 38.53, 2, 77.06, 3.5, 0.92, 0.0, 0.0, '0'), ('S03394500037', Timestamp('2012-01-03 00:00:00'), 3443, 'Super Saver Iv', '1141 N BROADWAY', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1042100, 'IMPORTED DRY GINS', 260, 'Diageo Americas', 28866, 'Tanqueray Gin', 12, '750', 12.0, 17.99, 12, 215.88, 9.0, 2.38, -95.836515, 41.270824, '882608c823fffff'), ('S03402100015', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42717, 'Malibu Coconut Rum', 12, '1000', 9.77, 14.66, 12, 175.92, 12.0, 3.17, -92.475403, 42.712263, '8826291801fffff'), ('S03410700004', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68036, "Bailey's Original Irish Cream", 12, '750', 12.78, 19.18, 12, 230.16, 9.0, 2.38, -94.853591, 42.064155, '88260c16a9fffff'), ('S03389300006', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53214, 'Paul Masson Grande Amber Brandy', 24, '375', 3.05, 4.57, 24, 109.68, 9.0, 2.38, -90.609747, 41.589369, '882675d959fffff'), ('S03392900033', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43025, 'Admiral Nelson Spiced Rum Traveler', 12, '750', 5.77, 8.66, 12, 103.92, 9.0, 2.38, 0.0, 0.0, '0'), ('S03382200009', Timestamp('2012-01-03 00:00:00'), 4584, 'J & C Grocery / Dumont', '611 MAIN ST', 'DUMONT', '50602', 12, 'Butler', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11788, 'Black Velvet', 6, '1750', 10.45, 15.67, 6, 94.02, 10.5, 2.77, -92.973022, 42.750551, '8826295aebfffff'), ('S03396600105', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1041100, 'AMERICAN DRY GINS', 300, 'Mccormick Distilling Company', 31207, 'McCormick Gin', 12, '1000', 4.0, 6.0, 2, 12.0, 2.0, 0.53, -93.378772, 43.142868, '8826283b29fffff'), ('S03398600015', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36908, 'Mccormick Vodka Pet', 6, '1750', 7.46, 11.19, 6, 67.14, 10.5, 2.77, -91.343544, 40.626988, '8826703131fffff'), ('S03402500029', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1031080, 'VODKA 80 PROOF', 461, 'Campari(skyy)', 37986, 'Skyy Vodka', 12, '750', 10.01, 15.01, 2, 30.02, 1.5, 0.4, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03403700019', Timestamp('2012-01-03 00:00:00'), 4547, 'Fareway Stores #902 / Hampton', '309 CENTRAL AVE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1022100, 'TEQUILA', 421, 'Sazerac Co., Inc.', 89916, 'Tortilla Gold Tequila', 12, '750', 6.34, 9.52, 3, 28.56, 2.25, 0.59, -93.211676, 42.741748, '88262b9605fffff'), ('S03397600133', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11298, 'Crown Royal Canadian Whisky', 6, '1750', 30.5, 45.74, 3, 137.22, 5.25, 1.39, -95.825137, 41.242732, '882608c9d1fffff'), ('S03407500007', Timestamp('2012-01-03 00:00:00'), 4267, 'Main Street Market Of Anita', '735 MAIN ST', 'ANITA', '50020', 15, 'Cass', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82787, 'Dekuyper Buttershots', 12, '1000', 7.35, 11.02, 2, 22.04, 2.0, 0.53, -94.765365, 41.444887, '88260d594dfffff'), ('S03396400134', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 1, 9.33, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03408800019', Timestamp('2012-01-03 00:00:00'), 2959, "Dahl's / Merle Hay", '4343 MERLE HAY ROAD', 'DES MOINES', '50310', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37418, 'Popov Vodka 80 Prf', 6, '1750', 8.49, 12.74, 6, 76.44, 10.5, 2.77, -93.697647, 41.63740800000001, '88260d8053fffff'), ('S03401700011', Timestamp('2012-01-03 00:00:00'), 4350, 'Kum & Go #703 / Marshalltown', '916 E MAIN ST', 'MARSHALLTOWN', '50158', 64, 'Marshall', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88294, 'Patron Silver Tequila', 12, '375', 13.33, 20.0, 3, 60.0, 1.12, 0.3, -92.89647300000001, 42.049238, '882676d189fffff'), ('S03399800001', Timestamp('2012-01-03 00:00:00'), 3970, 'The Liquor Cabinet, Inc.', '835 6 ST, STE 1', 'NEVADA', '50201', 85, 'Story', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11297, 'Crown Royal Canadian Whisky', 12, '1000', 17.46, 26.19, 12, 314.28, 12.0, 3.17, -93.452277, 42.019292, '88262b222dfffff'), ('S03390000012', Timestamp('2012-01-03 00:00:00'), 3606, "Scotty's Liquor Store", '804 G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21595, 'Ten High Traveler', 12, '750', 4.0, 5.99, 3, 17.97, 2.25, 0.59, -92.77421500000001, 42.361514, '8826297831fffff'), ('S03408300112', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081317, 'GRAPE SCHNAPPS', 65, 'Jim Beam Brands', 82636, 'Dekuyper Grape Pucker Schnapps', 12, '750', 6.3, 9.45, 3, 28.35, 2.25, 0.59, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03411000045', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36307, 'Hawkeye Vodka', 12, '1000', 3.85, 5.78, 36, 208.08, 36.0, 9.51, 0.0, 0.0, '0'), ('S03397500075', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19474, "Maker's Mark", 12, '375', 7.61, 11.42, 2, 22.84, 0.75, 0.2, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03394400009', Timestamp('2012-01-03 00:00:00'), 4334, 'Fareway Stores #114 / Dubuque', '2050 JOHN F KENNEDY RD', 'DUBUQUE', '52001', 31, 'Dubuque', 1012200, 'SCOTCH WHISKIES', 35, 'Bacardi U.S.A., Inc.', 4866, 'Dewars White Label Scotch', 12, '750', 13.73, 20.59, 3, 61.77, 2.25, 0.59, 0.0, 0.0, '0'), ('S03396400011', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36306, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 12, 58.56, 9.0, 2.38, -95.8792, 41.238092, '882608c9e7fffff'), ('S03406300018', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45278, 'Paramount White Rum', 6, '1750', 7.58, 11.37, 6, 68.22, 10.5, 2.77, -91.281344, 42.108275, '882674c6a9fffff'), ('S03384700012', Timestamp('2012-01-03 00:00:00'), 4138, 'Fareway Stores #657 / Indianola', '1309 W 2ND', 'INDIANOLA', '50125', 91, 'Warren', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21596, 'Ten High', 12, '750', 4.0, 5.99, 12, 71.88, 9.0, 2.38, -93.579252, 41.356616, '88260db8e9fffff'), ('S03389400019', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43128, 'Bacardi Superior Rum', 6, '1750', 14.67, 22.0, 12, 264.0, 21.0, 5.55, -95.909384, 41.242291, '88260e26d5fffff'), ('S03398600063', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11771, 'Black Velvet Mini', 10, '600', 4.96, 7.44, 1, 7.44, 0.6, 0.16, -91.343544, 40.626988, '8826703131fffff'), ('S03401100011', Timestamp('2012-01-03 00:00:00'), 4239, 'Fareway Stores #850 / Spirit Lake', '1906 KEOKUK', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37418, 'Popov Vodka 80 Prf', 6, '1750', 8.49, 12.74, 6, 76.44, 10.5, 2.77, -95.106219, 43.42157, '88262ac23bfffff'), ('S03397600029', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 461, 'Campari(skyy)', 22121, 'Wild Turkey 81', 12, '750', 10.83, 16.25, 12, 195.0, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03397000022', Timestamp('2012-01-03 00:00:00'), 3994, 'Great Spirits Wine & Liquor Co.', '72 NORTH HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36908, 'Mccormick Vodka Pet', 6, '1750', 7.46, 11.19, 2, 22.38, 3.5, 0.92, -95.127395, 43.368989000000006, '88262ac0e9fffff'), ('S03407400032', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43316, 'Captain Morgan Private Stock Rum', 12, '750', 11.45, 17.18, 3, 51.54, 2.25, 0.59, -93.736517, 41.659057, '88260d8045fffff'), ('S03394700007', Timestamp('2012-01-03 00:00:00'), 3959, 'Hartig Drug Company #8 / University', '1600 UNIVERSITY AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 308, 'Mango Bottling, Inc.', 71529, 'Tooters Ala Bama Slama', 12, '375', 5.0, 7.49, 1, 7.49, 0.38, 0.1, -90.688027, 42.498279, '88275a2115fffff'), ('S03401100004', Timestamp('2012-01-03 00:00:00'), 4239, 'Fareway Stores #850 / Spirit Lake', '1906 KEOKUK', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 38008, 'Smirnoff Vodka 80 Prf Pet', 6, '1750', 13.98, 20.97, 6, 125.82, 10.5, 2.77, -95.106219, 43.42157, '88262ac23bfffff'), ('S03388500046', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43031, 'Bacardi Gold Rum Mini', 12, '500', 4.74, 7.11, 1, 7.11, 0.5, 0.13, -90.668138, 42.504959, '88275a2119fffff'), ('S03396400125', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 88291, 'Patron Tequila Silver Mini', 10, '300', 20.3, 30.45, 2, 60.9, 0.6, 0.16, -95.8792, 41.238092, '882608c9e7fffff'), ('S03410700083', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1081030, 'COFFEE LIQUEURS', 259, 'Heaven Hill Brands', 67426, 'Copa De Oro Mexican Coffee', 12, '750', 5.29, 7.94, 3, 23.82, 2.25, 0.59, -94.853591, 42.064155, '88260c16a9fffff'), ('S03401400034', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34015, 'Absolut Apeach', 12, '1000', 14.89, 22.34, 2, 44.68, 2.0, 0.53, -95.126786, 43.416433, '88262ac237fffff'), ('S03412100026', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86886, 'Southern Comfort', 12, '750', 9.36, 14.04, 12, 168.48, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03397600117', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081700, 'DISTILLED SPIRITS SPECIALTY', 469, 'TY KU, LLC', 67067, 'Ty Ku Super Premium Soju', 6, '750', 16.33, 24.5, 6, 147.0, 4.5, 1.19, -95.825137, 41.242732, '882608c9d1fffff'), ('S03384000007', Timestamp('2012-01-03 00:00:00'), 2327, 'West Side Spirits', '801 HULL ST', 'CORNING', '50841', 2, 'Adams', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 3, 31.86, 3.0, 0.79, -94.74369200000001, 40.990378, '882608a233fffff'), ('S03396400036', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15246, 'Windsor Canadian', 12, '750', 4.2, 6.3, 12, 75.6, 9.0, 2.38, -95.8792, 41.238092, '882608c9e7fffff'), ('S03398400027', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37417, 'Popov Vodka 80 Prf', 12, '1000', 5.0, 7.49, 3, 22.47, 3.0, 0.79, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03397500051', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26826, 'Jack Daniels Old #7 Black Lbl', 12, '750', 13.01, 19.51, 12, 234.12, 9.0, 2.38, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03395200064', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41692, 'Uv Blue (raspberry) Vodka', 6, '1750', 10.63, 15.95, 1, 15.95, 1.75, 0.46, -91.110752, 40.801889, '882670a137fffff'), ('S03383500021', Timestamp('2012-01-03 00:00:00'), 3756, 'Main Street Liquors / Manning', '316 MAIN ST', 'MANNING', '51455', 14, 'Carroll', 1022100, 'TEQUILA', 85, 'Brown-Forman Corporation', 87586, 'El Jimador Silver Tequila', 12, '750', 10.63, 15.95, 1, 15.95, 0.75, 0.2, -95.065122, 41.908573, '88260c0525fffff'), ('S03403000030', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 30528, 'Hawkeye Gin', 6, '1750', 7.0, 10.5, 2, 21.0, 3.5, 0.92, -94.480204, 43.39966, '88262a82cbfffff'), ('S03403400038', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 285, 'Charles Jacquin Et. Cie., Inc.', 56958, 'Bartenders Hot Sex', 6, '1750', 8.98, 13.46, 2, 26.92, 3.5, 0.92, -93.599362, 41.59878, '88260d8743fffff'), ('S03407400005', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28236, 'Bombay Sapphire Gin', 12, '750', 13.63, 20.45, 12, 245.4, 9.0, 2.38, -93.736517, 41.659057, '88260d8045fffff'), ('S03408200010', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58835, 'Jose Cuervo Authentic Lime Margarita', 6, '800', 4.62, 6.92, 6, 41.52, 4.8, 1.27, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03388000051', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1031080, 'VODKA 80 PROOF', 259, 'Heaven Hill Brands', 35418, "Burnett's Vodka 80 Prf", 6, '1750', 8.43, 13.15, 6, 78.9, 10.5, 2.77, -93.633306, 43.261538, '88262808e7fffff'), ('S03392300062', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65257, 'Jagermeister Liqueur', 12, '1000', 14.93, 22.39, 24, 537.36, 24.0, 6.34, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03401000021', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 22788, "Beam's 8 Star Bl Whiskey", 6, '1750', 7.24, 10.86, 1, 10.86, 1.75, 0.46, -90.683356, 42.52717, '88275a215dfffff'), ('S03397300037', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43126, 'Bacardi Superior Rum', 12, '750', 7.53, 11.3, 12, 135.6, 9.0, 2.38, -91.044008, 41.427202, '8826742f4bfffff'), ('S03388200003', Timestamp('2012-01-03 00:00:00'), 4106, 'Hartig Drug Co. / JFK Rd / Dubuque', '2255 JOHN F KENNEDY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -90.72051800000001, 42.511352, '88275a212bfffff'), ('S03388100005', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 461, 'Campari(skyy)', 67192, 'X Rated Fusion Liqueur', 6, '750', 14.66, 21.99, 6, 131.94, 4.5, 1.19, -95.879662, 41.261925, '882608c93bfffff'), ('S03390300027', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 434, 'Luxco-St Louis', 85606, 'Paramount Blue Curacao', 12, '750', 3.24, 4.86, 12, 58.32, 9.0, 2.38, -95.854809, 41.262316, '882608c9cdfffff'), ('S03399100018', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1022100, 'TEQUILA', 395, 'Proximo', 89196, 'Jose Cuervo Especial Reposado Tequila', 12, '750', 10.49, 15.73, 3, 47.19, 2.25, 0.59, -93.619787, 41.60566, '88260d8769fffff'), ('S03385900016', Timestamp('2012-01-03 00:00:00'), 4356, 'Kum & Go #396 / Nevada', '136 LINCOLN HWY', 'NEVADA', '50201', 85, 'Story', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36304, 'Hawkeye Vodka', 24, '375', 2.06, 3.09, 3, 9.27, 1.12, 0.3, -93.458003, 42.022725, '88262b235bfffff'), ('S03385300105', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69611, "Dr. Mcgillicuddy's Apple Pie Schnapps", 12, '750', 8.32, 12.48, 6, 74.88, 4.5, 1.19, -91.136655, 40.814666, '882670aacdfffff'), ('S03402500019', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82786, 'Dekuyper Buttershots Schnapps', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03402700006', Timestamp('2012-01-03 00:00:00'), 3907, "Smokin' Joe's #16 Tobacco and Liquor", '1504 PARK AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43034, 'Bacardi Gold Rum', 24, '375', 4.3, 6.46, 1, 6.46, 0.38, 0.1, -91.030569, 41.442458, '88267421bdfffff'), ('S03395200024', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23826, 'Five Star', 12, '750', 3.35, 5.02, 12, 60.24, 9.0, 2.38, -91.110752, 40.801889, '882670a137fffff'), ('S03411000043', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 65066, 'Goldschlager Cinnamon Schnapps', 12, '750', 12.0, 17.99, 12, 215.88, 9.0, 2.38, 0.0, 0.0, '0'), ('S03408600041', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34422, 'Grey Goose Vodka', 6, '1000', 21.77, 32.66, 3, 97.98, 3.0, 0.79, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03408300250', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1041100, 'AMERICAN DRY GINS', 205, 'E AND J GALLO WINERY', 31474, 'New Amsterdam Gin', 24, '375', 3.4, 5.1, 4, 20.4, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03394100018', Timestamp('2012-01-03 00:00:00'), 3451, 'Barnes Foodland', '622 10TH ST', 'DEWITT', '52742', 23, 'Clinton', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 12, 208.44, 9.0, 2.38, -90.540132, 41.825139, '882674b5e7fffff'), ('S03393900047', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19068, 'Jim Beam', 6, '1750', 18.37, 27.55, 1, 27.55, 1.75, 0.46, -95.873073, 41.261866, '882608c917fffff'), ('S03407900022', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82607, 'Dekuyper Sour Apple', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, 0.0, 0.0, '0'), ('S03394300002', Timestamp('2012-01-03 00:00:00'), 4252, 'Fareway Stores #840 / Monticello', '433 N FARLEY ST', 'MONTICELLO', '52310', 53, 'Jones', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43336, 'Captain Morgan Original Spiced', 12, '750', 8.5, 12.74, 24, 305.76, 18.0, 4.76, -91.191385, 42.241939, '882674d69dfffff'), ('S03396800016', Timestamp('2012-01-03 00:00:00'), 2445, "Ruback's Food Center", '504 SOUTH HIGHWAY', 'OAKLAND', '51560', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 3, 27.99, 2.25, 0.59, 0.0, 0.0, '0'), ('S03397600149', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 89646, 'Patron Anejo', 12, '750', 30.0, 45.0, 2, 90.0, 1.5, 0.4, -95.825137, 41.242732, '882608c9d1fffff'), ('S03410300128', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1051110, 'BLACKBERRY BRANDIES', 55, 'Sazerac North America', 55066, 'Mr. Boston Blackberry Brandy', 12, '750', 4.12, 6.37, 3, 19.11, 2.25, 0.59, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03410900077', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1041100, 'AMERICAN DRY GINS', 260, 'Diageo Americas', 30316, "Gordon's Gin London Dry", 12, '750', 5.0, 7.49, 2, 14.98, 1.5, 0.4, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03383900035', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1071100, 'AMERICAN COCKTAILS', 308, 'Mango Bottling, Inc.', 77545, 'Tooters Apple-Tini Mini', 12, '375', 5.0, 7.49, 12, 89.88, 4.5, 1.19, -91.403752, 40.41538, '8826702f65fffff'), ('S03396500023', Timestamp('2012-01-03 00:00:00'), 3587, 'Save A Lot  #482', '2122 MAIN ST', 'KEOKUK', '52632', 56, 'Lee', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 4, 69.48, 3.0, 0.79, -91.39688800000002, 40.408821, '8826702f67fffff'), ('S03405900067', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43046, 'Bacardi Select Rum', 12, '750', 7.53, 11.3, 2, 22.6, 1.5, 0.4, -90.48065, 41.529655, '882675c361fffff'), ('S03393400038', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03387700014', Timestamp('2012-01-03 00:00:00'), 3013, "Keith's Foods", '207 E LOCUST ST', 'BLOOMFIELD', '52537', 26, 'Davis', 1051120, 'CHERRY BRANDIES', 434, 'Luxco-St Louis', 55246, 'Arrow Wild Cherry Flav Brandy', 12, '750', 5.0, 7.5, 12, 90.0, 9.0, 2.38, -92.412847, 40.752691, '88267284d5fffff'), ('S03396600108', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45247, 'Paramount Gold Rum', 12, '1000', 4.33, 6.5, 3, 19.5, 3.0, 0.79, -93.378772, 43.142868, '8826283b29fffff'), ('S03393300001', Timestamp('2012-01-03 00:00:00'), 4366, 'Kum & Go #137 / Tiffin', '101 VILLAGE DR', 'TIFFIN', '52340', 52, 'Johnson', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -91.6656, 41.697221, '8826747a67fffff'), ('S03395200103', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21596, 'Ten High', 12, '750', 4.0, 5.99, 4, 23.96, 3.0, 0.79, -91.110752, 40.801889, '882670a137fffff'), ('S03411500059', Timestamp('2012-01-03 00:00:00'), 3700, 'Avoca Liquor', '158 SOUTH ELM', 'AVOCA', '51521', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 461, 'Campari(skyy)', 77776, 'Wild Turkey American Honey', 12, '750', 10.5, 15.75, 2, 31.5, 1.5, 0.4, -95.339412, 41.476306, '88260c6c8bfffff'), ('S03388100094', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081380, 'MISCELLANEOUS SCHNAPPS', 260, 'Diageo Americas', 67296, 'Yukon Jack Perma Frost Schnapps', 12, '750', 8.54, 12.81, 3, 38.43, 2.25, 0.59, -95.879662, 41.261925, '882608c93bfffff'), ('S03385900007', Timestamp('2012-01-03 00:00:00'), 4356, 'Kum & Go #396 / Nevada', '136 LINCOLN HWY', 'NEVADA', '50201', 85, 'Story', 1022100, 'TEQUILA', 395, 'Proximo', 89194, 'Jose Cuervo Especial Reposado Flask', 24, '375', 5.49, 8.24, 3, 24.72, 1.12, 0.3, -93.458003, 42.022725, '88262b235bfffff'), ('S03382600001', Timestamp('2012-01-03 00:00:00'), 4317, 'Fareway Stores #941 / Greenfield', '212 SW KENT', 'GREENFIELD', '50849', 1, 'Adair', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11586, 'Black Velvet Reserve', 12, '750', 6.75, 10.12, 12, 121.44, 9.0, 2.38, -94.462719, 41.303167, '88260d42abfffff'), ('S03393400047', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 35626, 'Pinnacle Whipped', 12, '750', 7.87, 11.81, 12, 141.72, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03404800083', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1081370, 'WATERMELON SCHNAPPS', 65, 'Jim Beam Brands', 82867, 'Dekuyper Watermelon Pucker', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -91.035138, 41.451349, '88267421abfffff'), ('S03397300086', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 6, 55.98, 4.5, 1.19, -91.044008, 41.427202, '8826742f4bfffff'), ('S03403400036', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34030, 'Absolut Citron (lemon Vodka)', 12, '750', 10.89, 16.33, 1, 16.33, 0.75, 0.2, -93.599362, 41.59878, '88260d8743fffff'), ('S03404600007', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -95.027238, 40.739231, '88260831cbfffff'), ('S03402900035', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48104, 'Hennessy Vs Cognac', 24, '375', 8.38, 12.58, 1, 12.58, 0.38, 0.1, -94.990031, 41.403856, '8826089095fffff'), ('S03412100207', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19474, "Maker's Mark", 12, '375', 7.61, 11.42, 4, 45.68, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03408400012', Timestamp('2012-01-03 00:00:00'), 2650, 'Hy-Vee Wine and Spirits / Harlan', '1808 23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58868, 'Jose Cuervo Authentic Strawberry Margarita', 6, '1750', 7.7, 11.8, 6, 70.8, 10.5, 2.77, -95.339881, 41.650658, '88260c60e9fffff'), ('S03385300036', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64866, 'Fireball Cinnamon Whiskey', 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -91.136655, 40.814666, '882670aacdfffff'), ('S03385500054', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5289, 'J & B Rare Scotch', 12, '1000', 14.98, 22.47, 2, 44.94, 2.0, 0.53, -93.21693, 42.741728, '88262b9605fffff'), ('S03410700002', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1081200, 'CREAM LIQUEURS', 300, 'Mccormick Distilling Company', 65200, 'Tequila Rose Liqueur', 12, '750', 10.05, 15.07, 12, 180.84, 9.0, 2.38, -94.853591, 42.064155, '88260c16a9fffff'), ('S03402100042', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86887, 'Southern Comfort', 12, '1000', 11.34, 17.01, 12, 204.12, 12.0, 3.17, -92.475403, 42.712263, '8826291801fffff'), ('S03388200007', Timestamp('2012-01-03 00:00:00'), 4106, 'Hartig Drug Co. / JFK Rd / Dubuque', '2255 JOHN F KENNEDY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 22786, "Beam's 8 Star Bl Whiskey", 12, '750', 3.25, 4.88, 4, 19.52, 3.0, 0.79, -90.72051800000001, 42.511352, '88275a212bfffff'), ('S03385300129', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1052010, 'IMPORTED GRAPE BRANDIES', 389, 'REMY COINTREAU USA         .', 50686, 'St. Remy Vsop Napoleon Brandy', 12, '750', 7.14, 10.72, 2, 21.44, 1.5, 0.4, -91.136655, 40.814666, '882670aacdfffff'), ('S03396600073', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41076, 'Jeremiah Weed Sweet Tea Vodka', 12, '750', 9.46, 14.2, 2, 28.4, 1.5, 0.4, -93.378772, 43.142868, '8826283b29fffff'), ('S03411000081', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42716, 'Malibu Coconut Rum', 12, '750', 6.9, 10.35, 4, 41.4, 3.0, 0.79, 0.0, 0.0, '0'), ('S03393400126', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34030, 'Absolut Citron (lemon Vodka)', 12, '750', 10.89, 16.33, 36, 587.88, 27.0, 7.13, -90.741064, 42.516633, '88275a212dfffff'), ('S03395200083', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36886, 'Mccormick Vodka', 12, '750', 3.3, 4.96, 6, 29.76, 4.5, 1.19, -91.110752, 40.801889, '882670a137fffff'), ('S03410700047', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 31656, 'Paramount Gin', 12, '750', 3.39, 5.09, 12, 61.08, 9.0, 2.38, -94.853591, 42.064155, '88260c16a9fffff'), ('S03390300067', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43156, 'Bacardi 151 Proof Rum', 12, '750', 12.66, 18.99, 1, 18.99, 0.75, 0.2, -95.854809, 41.262316, '882608c9cdfffff'), ('S03407200037', Timestamp('2012-01-03 00:00:00'), 4146, 'Fareway Stores #561 / Waverly', '220 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37346, 'Phillips Vodka', 12, '750', 3.48, 5.23, 3, 15.69, 2.25, 0.59, -92.473476, 42.725748, '8826291847fffff'), ('S03412100177', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1022100, 'TEQUILA', 395, 'Proximo', 89191, 'Jose Cuervo Especial Reposado Tequila Mini', 12, '500', 11.5, 17.25, 3, 51.75, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03386500018', Timestamp('2012-01-03 00:00:00'), 2548, 'Hy-Vee Food Store / Altoona', '100 8TH STREET SW', 'ALTOONA', '50009', 77, 'Polk', 1022100, 'TEQUILA', 259, 'Heaven Hill Brands', 89946, 'Two Fingers Gold Tequila', 12, '750', 8.33, 12.49, 12, 149.88, 9.0, 2.38, -93.465301, 41.644041, '88260d94d9fffff'), ('S03389000068', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36305, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 3, 14.64, 2.25, 0.59, -93.315977, 40.757158, '88267252cdfffff'), ('S03404800019', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42717, 'Malibu Coconut Rum', 12, '1000', 9.77, 14.66, 12, 175.92, 12.0, 3.17, -91.035138, 41.451349, '88267421abfffff'), ('S03388100054', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 22788, "Beam's 8 Star Bl Whiskey", 6, '1750', 7.24, 10.86, 6, 65.16, 10.5, 2.77, -95.879662, 41.261925, '882608c93bfffff'), ('S03396200002', Timestamp('2012-01-03 00:00:00'), 3732, 'North Scott Foods', '425 E LECLAIRE RD', 'ELDRIDGE', '52748', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36307, 'Hawkeye Vodka', 12, '1000', 3.85, 5.78, 12, 69.36, 12.0, 3.17, -90.577311, 41.654695000000004, '882674a5bdfffff'), ('S03403600018', Timestamp('2012-01-03 00:00:00'), 3744, 'Payless Foods / Dyersville', '733 16TH AVE SE', 'DYERSVILLE', '52040', 31, 'Dubuque', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5290, 'J & B Rare Scotch', 12, '750', 12.24, 18.36, 2, 36.72, 1.5, 0.4, -91.115769, 42.470204, '88275a654dfffff'), ('S03398600029', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1011200, 'STRAIGHT BOURBON WHISKIES', 461, 'Campari(skyy)', 22156, 'Wild Turkey 101', 12, '750', 12.49, 18.74, 2, 37.48, 1.5, 0.4, -91.343544, 40.626988, '8826703131fffff'), ('S03387800055', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1081340, 'ROOT BEER SCHNAPPS', 380, 'Phillips Beverage Company', 84617, 'Phillips Root Beer Schnapps', 12, '1000', 4.89, 7.33, 12, 87.96, 12.0, 3.17, -91.387797, 40.400038, '8826702f29fffff'), ('S03392200015', Timestamp('2012-01-03 00:00:00'), 4578, "Casey's General Store #2592 / Farmin", '201 S 4TH ST', 'FARMINGTON', '52626', 89, 'Van Buren', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11294, 'Crown Royal Canadian Whisky', 24, '375', 7.24, 10.85, 24, 260.4, 9.0, 2.38, -91.740069, 40.639743, '88267070c1fffff'), ('S03396000064', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43137, 'Bacardi Limon', 12, '1000', 10.24, 15.35, 24, 368.4, 24.0, 6.34, -90.705328, 42.491862, '88275a2131fffff'), ('S03381400019', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67527, 'Kahlua Coffee Liqueur', 12, '1000', 14.81, 22.22, 60, 1333.2, 60.0, 15.85, -90.527081, 41.559731, '882675daa5fffff'), ('S03390200063', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1032200, 'IMPORTED VODKA - MISC', 260, 'Diageo Americas', 64764, 'Ciroc Red Berry Vodka', 6, '750', 17.97, 26.96, 3, 80.88, 2.25, 0.59, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03397500004', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68036, "Bailey's Original Irish Cream", 12, '750', 12.78, 19.18, 12, 230.16, 9.0, 2.38, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03403400007', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1022100, 'TEQUILA', 395, 'Proximo', 89193, 'Jose Cuervo Especial Reposado Tequila', 48, '200', 3.0, 4.49, 48, 215.52, 9.6, 2.54, -93.599362, 41.59878, '88260d8743fffff'), ('S03402100013', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1031080, 'VODKA 80 PROOF', 421, 'Sazerac Co., Inc.', 36978, 'Nikolai Vodka', 6, '1750', 6.87, 10.3, 6, 61.8, 10.5, 2.77, -92.475403, 42.712263, '8826291801fffff'), ('S03410300047', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43137, 'Bacardi Limon', 12, '1000', 10.24, 15.35, 3, 46.05, 3.0, 0.79, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03408300139', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37416, 'Popov Vodka 80 Prf', 12, '750', 4.1, 6.15, 2, 12.3, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03400700076', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41746, 'Smirnoff Citrus Vodka', 12, '750', 7.59, 11.56, 3, 34.68, 2.25, 0.59, 0.0, 0.0, '0'), ('S03395700030', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41704, 'Uv Red (cherry) Vodka', 12, '750', 6.22, 9.33, 1, 9.33, 0.75, 0.2, -95.147741, 43.14521, '88262a1925fffff'), ('S03406600007', Timestamp('2012-01-03 00:00:00'), 4250, 'Fareway Stores #882 / Eldora', '1232 14TH ST', 'ELDORA', '50627', 42, 'Hardin', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19066, 'Jim Beam', 12, '750', 8.42, 13.41, 3, 40.23, 2.25, 0.59, -93.10077, 42.361055, '88262ba259fffff'), ('S03396600144', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26821, 'Jack Daniels Old #7 Black Lbl Mini', 12, '500', 8.62, 12.94, 1, 12.94, 0.5, 0.13, -93.378772, 43.142868, '8826283b29fffff'), ('S03385500035', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1031090, 'OTHER PROOF VODKA', 260, 'Diageo Americas', 40926, 'Smirnoff Silver Vodka 90 Prf', 12, '750', 7.86, 11.79, 3, 35.37, 2.25, 0.59, -93.21693, 42.741728, '88262b9605fffff'), ('S03392300018', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1022100, 'TEQUILA', 65, 'Jim Beam Brands', 89787, 'Sauza Gold', 12, '1000', 11.55, 17.32, 12, 207.84, 12.0, 3.17, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03412100049', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36305, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 12, 58.56, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03383900037', Timestamp('2012-01-03 00:00:00'), 2555, 'Hy-Vee Food Store / Keokuk', '3111 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86885, 'Southern Comfort Pet', 12, '750', 9.36, 14.04, 12, 168.48, 9.0, 2.38, -91.403752, 40.41538, '8826702f65fffff'), ('S03398700005', Timestamp('2012-01-03 00:00:00'), 4318, 'Fareway Stores #841 / Red Oak', '811 BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11296, 'Crown Royal', 12, '750', 14.25, 21.37, 12, 256.44, 9.0, 2.38, -95.321174, 41.663944, '88260c62a7fffff'), ('S03392300022', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19096, 'Jim Beam Traveler', 12, '750', 8.42, 13.41, 12, 160.92, 9.0, 2.38, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03409700031', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35918, "Five O'clock Vodka", 6, '1750', 6.92, 10.38, 6, 62.28, 10.5, 2.77, -93.737447, 41.62941, '88260d802bfffff'), ('S03408400044', Timestamp('2012-01-03 00:00:00'), 2650, 'Hy-Vee Wine and Spirits / Harlan', '1808 23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 25876, 'Sunny Brook Blend Whiskey', 12, '750', 3.67, 5.5, 3, 16.5, 2.25, 0.59, -95.339881, 41.650658, '88260c60e9fffff'), ('S03410900125', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45246, 'Paramount Gold Rum', 12, '750', 3.85, 5.77, 4, 23.08, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03401600050', Timestamp('2012-01-03 00:00:00'), 2522, 'Hy-Vee Wine and Spirits / Spirit Lak', 'HWY 9 & 71', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1032200, 'IMPORTED VODKA - MISC', 260, 'Diageo Americas', 64742, 'Ciroc Coconut Vodka', 6, '750', 17.97, 26.96, 6, 161.76, 4.5, 1.19, 0.0, 0.0, '0'), ('S03398400028', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 11936, 'Canadian Ltd Whisky Convenience Pack', 12, '750', 3.75, 5.87, 5, 29.35, 3.75, 0.99, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03407100019', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 1, 9.33, 0.75, 0.2, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03406300021', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10008, 'Scoresby Rare Scotch', 6, '1750', 10.22, 15.33, 6, 91.98, 10.5, 2.77, -91.281344, 42.108275, '882674c6a9fffff'), ('S03412100200', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23823, 'Five Star', 48, '200', 1.21, 1.81, 6, 10.86, 1.2, 0.32, -90.483701, 41.552338, '882675da95fffff'), ('S03406200002', Timestamp('2012-01-03 00:00:00'), 4551, "Casey's General Store #2845 / Urband", '201 SUNSET ST', 'URBANA', '52345', 6, 'Benton', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19066, 'Jim Beam', 12, '750', 8.42, 13.41, 12, 160.92, 9.0, 2.38, 0.0, 0.0, '0'), ('S03385300033', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52599, 'E & J Vs Brandy Round', 12, '1000', 6.94, 10.41, 12, 124.92, 12.0, 3.17, -91.136655, 40.814666, '882670aacdfffff'), ('S03395700013', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 25878, 'Sunny Brook Blend Whiskey', 6, '1750', 7.87, 11.8, 1, 11.8, 1.75, 0.46, -95.147741, 43.14521, '88262a1925fffff'), ('S03408200109', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10624, 'Canadian Club Whisky', 24, '375', 4.05, 6.07, 3, 18.21, 1.12, 0.3, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03394000046', Timestamp('2012-01-03 00:00:00'), 4309, 'The Party Pantry', '430 S 35TH ST', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012200, 'SCOTCH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 4717, 'Chivas Regal 12 Yr', 12, '1000', 24.47, 36.71, 1, 36.71, 1.0, 0.26, -95.902023, 41.257663, '882608c937fffff'), ('S03408200100', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26823, 'Jack Daniels Old #7 Black Lbl', 48, '200', 3.69, 5.54, 6, 33.24, 1.2, 0.32, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03401400020', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34368, 'Fris Danish Vodka', 6, '1750', 10.25, 15.38, 6, 92.28, 10.5, 2.77, -95.126786, 43.416433, '88262ac237fffff'), ('S03395800037', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 308, 'Mango Bottling, Inc.', 77545, 'Tooters Apple-Tini Mini', 12, '375', 5.0, 7.49, 12, 89.88, 4.5, 1.19, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03395800031', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5347, 'Johnnie Walker Red Label Scotch', 12, '1000', 16.99, 25.49, 12, 305.88, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03405900010', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24457, 'Kessler Blend Whiskey', 12, '1000', 7.08, 10.62, 48, 509.76, 48.0, 12.68, -90.48065, 41.529655, '882675c361fffff'), ('S03408600032', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1081330, 'PEACH SCHNAPPS', 434, 'Luxco-St Louis', 82126, 'Arrow Peach Schnapps', 12, '750', 3.97, 5.95, 1, 5.95, 0.75, 0.2, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03382400003', Timestamp('2012-01-03 00:00:00'), 4314, 'Nodaway Valley Market', '304  4TH ST', 'FONTANELLE', '50846', 1, 'Adair', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11788, 'Black Velvet', 6, '1750', 10.45, 15.67, 6, 94.02, 10.5, 2.77, -94.562813, 41.289069, '88260d41cdfffff'), ('S03402900002', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65256, 'Jagermeister Liqueur', 12, '750', 11.58, 17.37, 12, 208.44, 9.0, 2.38, -94.990031, 41.403856, '8826089095fffff'), ('S03395500005', Timestamp('2012-01-03 00:00:00'), 3976, 'North English Family Food', '119 NORTH MAIN ST', 'NORTH ENGLISH', '52316', 48, 'Iowa', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35918, "Five O'clock Vodka", 6, '1750', 6.92, 10.38, 6, 62.28, 10.5, 2.77, -92.076452, 41.515209, '882676a0adfffff'), ('S03393100018', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41360, 'Uv Pink Lemonade Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03394200007', Timestamp('2012-01-03 00:00:00'), 2845, 'CVS Pharmacy #4816 / Council Bluffs', '545 W WASHINGTON ST', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081600, 'WHISKEY LIQUEUR', 260, 'Diageo Americas', 67266, 'Yukon Jack Canadian Liqueur', 12, '750', 8.54, 12.81, 12, 153.72, 9.0, 2.38, -95.344053, 41.472968, '88260c6c8bfffff'), ('S03389900036', Timestamp('2012-01-03 00:00:00'), 2532, 'Hy-Vee Food Store #2 / Des Moines', '2540 E EUCLID', 'DES MOINES', '50317', 77, 'Polk', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34001, 'Absolut Swedish Vodka 80 Prf Mini', 10, '600', 7.92, 11.88, 1, 11.88, 0.6, 0.16, -93.569452, 41.627838, '88260d9581fffff'), ('S03410300004', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69638, "Dr. Mcgillicuddy's Cherry Schnapps", 6, '1750', 13.33, 19.99, 6, 119.94, 10.5, 2.77, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03411500011', Timestamp('2012-01-03 00:00:00'), 3700, 'Avoca Liquor', '158 SOUTH ELM', 'AVOCA', '51521', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43127, 'Bacardi Superior Rum', 12, '1000', 9.43, 14.14, 12, 169.68, 12.0, 3.17, -95.339412, 41.476306, '88260c6c8bfffff'), ('S03408300293', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 305, 'MHW Ltd', 66050, 'Nuvo Sparkling Liqueur', 12, '375', 9.93, 14.9, 4, 59.6, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03408300230', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1022100, 'TEQUILA', 395, 'Proximo', 87632, 'El Zarco Silver', 12, '750', 7.67, 11.5, 1, 11.5, 0.75, 0.2, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03390800010', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 46350, 'Hawkeye Light Rum', 6, '1750', 7.56, 11.33, 6, 67.98, 10.5, 2.77, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03406100018', Timestamp('2012-01-03 00:00:00'), 4509, "A J'S LIQUOR II", '2515 CHAMBERLAIN', 'AMES', '50010', 85, 'Story', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64868, 'Fireball Cinnamon Whiskey', 6, '1750', 13.33, 19.99, 2, 39.98, 3.5, 0.92, -93.650838, 42.021461, '88262b3983fffff'), ('S03387800079', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1081220, 'DARK CREME DE CACAO', 434, 'Luxco-St Louis', 78036, 'Arrow Creme de Cacao Brown', 12, '750', 4.31, 6.46, 2, 12.92, 1.5, 0.4, -91.387797, 40.400038, '8826702f29fffff'), ('S03404100008', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11297, 'Crown Royal Canadian Whisky', 12, '1000', 17.46, 26.19, 12, 314.28, 12.0, 3.17, -90.993859, 42.3036, '8826749ac1fffff'), ('S03389600003', Timestamp('2012-01-03 00:00:00'), 3632, 'Wal-Mart 2004 / Dubuque', '4200 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15626, 'Jameson', 12, '750', 13.46, 20.2, 12, 242.4, 9.0, 2.38, -90.736955, 42.489041, '88275a2ac1fffff'), ('S03397600035', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 434, 'Luxco-St Louis', 24157, 'Hawkeye Blend Whiskey', 12, '1000', 4.29, 6.44, 12, 77.28, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03408100048', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69611, "Dr. Mcgillicuddy's Apple Pie Schnapps", 12, '750', 8.32, 12.48, 2, 24.96, 1.5, 0.4, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03393900015', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45278, 'Paramount White Rum', 6, '1750', 7.58, 11.37, 2, 22.74, 3.5, 0.92, -95.873073, 41.261866, '882608c917fffff'), ('S03407200032', Timestamp('2012-01-03 00:00:00'), 4146, 'Fareway Stores #561 / Waverly', '220 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57148, "Chi-Chi's Margarita W/tequila", 6, '1750', 5.99, 9.24, 3, 27.72, 5.25, 1.39, -92.473476, 42.725748, '8826291847fffff'), ('S03389900033', Timestamp('2012-01-03 00:00:00'), 2532, 'Hy-Vee Food Store #2 / Des Moines', '2540 E EUCLID', 'DES MOINES', '50317', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 38008, 'Smirnoff Vodka 80 Prf Pet', 6, '1750', 13.98, 20.97, 6, 125.82, 10.5, 2.77, -93.569452, 41.627838, '88260d9581fffff'), ('S03398800020', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43127, 'Bacardi Superior Rum', 12, '1000', 9.43, 14.14, 12, 169.68, 12.0, 3.17, -93.573845, 41.604312, '88260d95b3fffff'), ('S03389000070', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1042100, 'IMPORTED DRY GINS', 260, 'Diageo Americas', 28866, 'Tanqueray Gin', 12, '750', 12.0, 17.99, 1, 17.99, 0.75, 0.2, -93.315977, 40.757158, '88267252cdfffff'), ('S03412100197', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 421, 'Sazerac Co., Inc.', 36974, 'Nikolai Vodka', 24, '375', 1.71, 2.56, 4, 10.24, 1.5, 0.4, -90.483701, 41.552338, '882675da95fffff'), ('S03407100023', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82611, 'Dekuyper Hot Damn!', 12, '750', 6.3, 9.45, 1, 9.45, 0.75, 0.2, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03393900017', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1022100, 'TEQUILA', 434, 'Luxco-St Louis', 89387, 'Juarez Tequila Gold', 12, '1000', 6.67, 10.0, 1, 10.0, 1.0, 0.26, -95.873073, 41.261866, '882608c917fffff'), ('S03402900048', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1081355, 'STRAWBERRY SCHNAPPS', 65, 'Jim Beam Brands', 82820, 'Dekuyper Strawberry Pucker', 12, '750', 6.3, 9.45, 1, 9.45, 0.75, 0.2, -94.990031, 41.403856, '8826089095fffff'), ('S03390200069', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35916, "Five O'clock Vodka", 12, '750', 3.31, 4.96, 1, 4.96, 0.75, 0.2, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03408300290', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65194, 'Hpnotiq', 12, '375', 6.35, 9.52, 1, 9.52, 0.38, 0.1, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03385500048', Timestamp('2012-01-03 00:00:00'), 2460, 'Liquor Barn, Inc.', '721 CENTRAL AVENUE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43334, 'Captain Morgan Spiced Rum', 24, '375', 4.3, 6.45, 6, 38.7, 2.25, 0.59, -93.21693, 42.741728, '88262b9605fffff'), ('S03395200067', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41604, 'Uv Grape Vodka', 12, '750', 6.22, 9.33, 2, 18.66, 1.5, 0.4, -91.110752, 40.801889, '882670a137fffff'), ('S03399100003', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37997, 'Smirnoff Vodka 80 Prf', 12, '1000', 8.49, 12.74, 12, 152.88, 12.0, 3.17, -93.619787, 41.60566, '88260d8769fffff'), ('S03383800044', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41704, 'Uv Red (cherry) Vodka', 12, '750', 6.22, 9.33, 1, 9.33, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03393900011', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1041100, 'AMERICAN DRY GINS', 205, 'E AND J GALLO WINERY', 31474, 'New Amsterdam Gin', 24, '375', 3.4, 5.1, 3, 15.3, 1.12, 0.3, -95.873073, 41.261866, '882608c917fffff'), ('S03410900056', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43335, 'Captain Morgan Spiced Rum Pet', 6, '1750', 16.99, 25.48, 12, 305.76, 21.0, 5.55, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03371500015', Timestamp('2012-01-03 00:00:00'), 4572, 'Grundy Center Family Foods', '1400, G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1032080, 'IMPORTED VODKA', 260, 'Diageo Americas', 34456, 'Ketel One Imported Vodka', 12, '750', 13.0, 19.49, 2, 38.98, 1.5, 0.4, -92.782283, 42.36157, '8826297835fffff'), ('S03402500004', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11296, 'Crown Royal', 12, '750', 14.25, 21.37, 12, 256.44, 9.0, 2.38, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03392100044', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10624, 'Canadian Club Whisky', 24, '375', 4.05, 6.07, 3, 18.21, 1.12, 0.3, -90.204748, 41.827845, '88275b7891fffff'), ('S03408100035', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1011200, 'STRAIGHT BOURBON WHISKIES', 85, 'Brown-Forman Corporation', 17826, 'Early Times 36 Mo Ky Str Bourbon', 12, '750', 4.76, 7.51, 1, 7.51, 0.75, 0.2, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03400000016', Timestamp('2012-01-03 00:00:00'), 2459, 'Reinhart Foods', '200 STATE PO BOX 98', 'GUTHRIE CENTER', '50115', 39, 'Guthrie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41696, 'Uv Green (apple) Vodka', 12, '750', 6.22, 9.33, 3, 27.99, 2.25, 0.59, 0.0, 0.0, '0'), ('S03384500019', Timestamp('2012-01-03 00:00:00'), 3482, 'Kick The Can', '525 N MAPLE ST', 'CARROLL', '51401', 14, 'Carroll', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45247, 'Paramount Gold Rum', 12, '1000', 4.33, 6.5, 2, 13.0, 2.0, 0.53, -94.861667, 42.063255, '88260c16a9fffff'), ('S03397600062', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081400, 'TRIPLE SEC', 65, 'Jim Beam Brands', 86112, 'Dekuyper Triple Sec', 12, '1000', 2.21, 3.32, 12, 39.84, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03393400028', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37336, 'Uv Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03408300091', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68028, "Bailey's Mint Chocolate Irish Cream", 12, '750', 12.78, 19.18, 2, 38.36, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03394100010', Timestamp('2012-01-03 00:00:00'), 3451, 'Barnes Foodland', '622 10TH ST', 'DEWITT', '52742', 23, 'Clinton', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19026, 'Jim Beam Black', 12, '750', 13.12, 19.68, 12, 236.16, 9.0, 2.38, -90.540132, 41.825139, '882674b5e7fffff'), ('S03407400036', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1081300, 'PEPPERMINT SCHNAPPS', 380, 'Phillips Beverage Company', 83370, 'Ice Hole Mint Schnapps', 12, '750', 5.15, 7.73, 3, 23.19, 2.25, 0.59, -93.736517, 41.659057, '88260d8045fffff'), ('S03395800147', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 54647, 'Arrow Blackberry Flav Brandy', 12, '1000', 6.11, 9.17, 12, 110.04, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03386100012', Timestamp('2012-01-03 00:00:00'), 4083, 'Fareway Stores #909 / Ankeny', '109 SE ORALABOR RD', 'ANKENY', '50021', 77, 'Polk', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41715, 'Smirnoff Cranberry Vodka', 12, '750', 7.59, 11.56, 2, 23.12, 1.5, 0.4, -93.59949600000002, 41.702811, '88260d954bfffff'), ('S03402600003', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 6, 90.0, 4.5, 1.19, 0.0, 0.0, '0'), ('S03381300015', Timestamp('2012-01-03 00:00:00'), 3944, "Sam's Club 4973 / Dubuque", '4400 ASBURY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35918, "Five O'clock Vodka", 6, '1750', 6.92, 10.38, 18, 186.84, 31.5, 8.32, -90.737821, 42.515289, '88275a212dfffff'), ('S03390200019', Timestamp('2012-01-03 00:00:00'), 4109, 'Best Trip', '1516 SE 1ST', 'DES MOINES', '50315', 77, 'Polk', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23824, 'Five Star', 24, '375', 1.98, 2.96, 3, 8.88, 1.12, 0.3, -93.613469, 41.57592900000001, '88260d8707fffff'), ('S03402500023', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26586, 'Gentleman Jack', 12, '750', 18.44, 27.66, 2, 55.32, 1.5, 0.4, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03411900008', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1081210, 'CREME DE ALMOND', 350, 'Luxco-Cleveland', 80396, 'Paramount Creme De Almond', 12, '750', 4.43, 6.65, 1, 6.65, 0.75, 0.2, -95.147741, 43.14521, '88262a1925fffff'), ('S03396000083', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69637, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '1000', 11.0, 16.5, 48, 792.0, 48.0, 12.68, -90.705328, 42.491862, '88275a2131fffff'), ('S03392300019', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 420, 'Moet Hennessy USA, Inc.', 34162, 'Belvedere Vodka', 6, '1000', 20.62, 30.94, 6, 185.64, 6.0, 1.59, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03412100203', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11293, 'Crown Royal Canadian Whisky', 48, '200', 4.47, 6.7, 6, 40.2, 1.2, 0.32, -90.483701, 41.552338, '882675da95fffff'), ('S03387800090', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43244, 'Captain Morgan 100 Proof Spiced Rum', 12, '750', 10.0, 14.99, 3, 44.97, 2.25, 0.59, -91.387797, 40.400038, '8826702f29fffff'), ('S03395200042', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 6, 97.98, 4.5, 1.19, -91.110752, 40.801889, '882670a137fffff'), ('S03411000046', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 48, 1182.72, 48.0, 12.68, 0.0, 0.0, '0'), ('S03387200034', Timestamp('2012-01-03 00:00:00'), 4296, 'Circle B Market', '111 MAIN ST', 'KEOSAUQUA', '52565', 89, 'Van Buren', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67540, 'Kahlua French Vanilla Liqueur', 12, '750', 11.61, 17.42, 1, 17.42, 0.75, 0.2, -91.962466, 40.7294, '882670458bfffff'), ('S03397600151', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1022100, 'TEQUILA', 969, 'Aveniu Brands', 89453, 'Kah Anejo', 6, '750', 32.83, 49.25, 2, 98.5, 1.5, 0.4, -95.825137, 41.242732, '882608c9d1fffff'), ('S03412100093', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1011100, 'BLENDED WHISKIES', 434, 'Luxco-St Louis', 24156, 'Hawkeye Blend Whiskey', 12, '750', 3.35, 5.03, 12, 60.36, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03408300219', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43418, 'Captain Morgan Tattoo', 12, '750', 8.21, 12.31, 3, 36.93, 2.25, 0.59, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03385800034', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1081380, 'MISCELLANEOUS SCHNAPPS', 55, 'Sazerac North America', 84170, '99 Bananas', 12, '750', 9.5, 14.25, 2, 28.5, 1.5, 0.4, -92.258152, 42.809882, '8826291267fffff'), ('S03408300062', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43334, 'Captain Morgan Spiced Rum', 24, '375', 4.3, 6.45, 24, 154.8, 9.0, 2.38, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03395200041', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1032200, 'IMPORTED VODKA - MISC', 65, 'Jim Beam Brands', 35626, 'Pinnacle Whipped', 12, '750', 7.87, 11.81, 2, 23.62, 1.5, 0.4, -91.110752, 40.801889, '882670a137fffff'), ('S03404600017', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45277, 'Paramount White Rum', 12, '1000', 4.33, 6.5, 12, 78.0, 12.0, 3.17, -95.027238, 40.739231, '88260831cbfffff'), ('S03383700008', Timestamp('2012-01-03 00:00:00'), 3934, 'Wal-Mart 1509 / Maquoketa', '103 E CARLISLE', 'MAQUOKETA', '52060', 49, 'Jackson', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52596, 'E & J Vs', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -90.66292, 42.04869, '8826748693fffff'), ('S03404800054', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -91.035138, 41.451349, '88267421abfffff'), ('S03386100003', Timestamp('2012-01-03 00:00:00'), 4083, 'Fareway Stores #909 / Ankeny', '109 SE ORALABOR RD', 'ANKENY', '50021', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37996, 'Smirnoff Vodka 80 Prf', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -93.59949600000002, 41.702811, '88260d954bfffff'), ('S03401000032', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11296, 'Crown Royal', 12, '750', 14.25, 21.37, 1, 21.37, 0.75, 0.2, -90.683356, 42.52717, '88275a215dfffff'), ('S03410300075', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1081355, 'STRAWBERRY SCHNAPPS', 65, 'Jim Beam Brands', 82820, 'Dekuyper Strawberry Pucker', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03396400119', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 461, 'Campari(skyy)', 64996, 'Frangelico Liqueur', 12, '750', 13.01, 19.52, 3, 58.56, 2.25, 0.59, -95.8792, 41.238092, '882608c9e7fffff'), ('S03408400054', Timestamp('2012-01-03 00:00:00'), 2650, 'Hy-Vee Wine and Spirits / Harlan', '1808 23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1051110, 'BLACKBERRY BRANDIES', 55, 'Sazerac North America', 55066, 'Mr. Boston Blackberry Brandy', 12, '750', 4.12, 6.37, 2, 12.74, 1.5, 0.4, -95.339881, 41.650658, '88260c60e9fffff'), ('S03407300008', Timestamp('2012-01-03 00:00:00'), 4029, 'The Store', '204 S MAIN ST', 'COLESBURG', '52035', 28, 'Delaware', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69657, "Dr. Mcgillicuddy's Mentholmint", 12, '1000', 11.0, 16.5, 3, 49.5, 3.0, 0.79, -91.368032, 42.958986, '88275a425bfffff'), ('S03395800116', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65258, 'Jagermeister Liqueur', 6, '1750', 25.69, 38.53, 6, 231.18, 10.5, 2.77, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03404100049', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15627, 'Jameson', 12, '1000', 17.4, 26.1, 6, 156.6, 6.0, 1.59, -90.993859, 42.3036, '8826749ac1fffff'), ('S03407900067', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 389, 'REMY COINTREAU USA         .', 64776, 'Cointreau Liqueur', 12, '750', 23.31, 34.96, 2, 69.92, 1.5, 0.4, 0.0, 0.0, '0'), ('S03406100013', Timestamp('2012-01-03 00:00:00'), 4509, "A J'S LIQUOR II", '2515 CHAMBERLAIN', 'AMES', '50010', 85, 'Story', 1031080, 'VODKA 80 PROOF', 306, 'Mississippi River Distilling Co.', 37642, 'River Pilot Vodka', 6, '750', 12.81, 19.21, 2, 38.42, 1.5, 0.4, -93.650838, 42.021461, '88262b3983fffff'), ('S03388100108', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1052010, 'IMPORTED GRAPE BRANDIES', 389, 'REMY COINTREAU USA         .', 49186, 'Remy Martin Vsop Cognac', 12, '750', 20.83, 31.25, 1, 31.25, 0.75, 0.2, -95.879662, 41.261925, '882608c93bfffff'), ('S03390400012', Timestamp('2012-01-03 00:00:00'), 3738, 'Hartig Drug Company #4 / Dubuque', '2225 CENTRAL AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 30526, 'Hawkeye Gin', 12, '750', 3.33, 5.0, 12, 60.0, 9.0, 2.38, -90.671531, 42.51384, '88275a2153fffff'), ('S03383800048', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42676, 'Malibu Passion Fruit Rum', 12, '750', 6.9, 10.35, 1, 10.35, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03395700011', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25608, 'Seagrams 7 Crown Bl Whiskey', 6, '1750', 10.76, 16.4, 2, 32.8, 3.5, 0.92, -95.147741, 43.14521, '88262a1925fffff'), ('S03396400089', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11774, 'Black Velvet', 24, '375', 3.07, 4.6, 1, 4.6, 0.38, 0.1, -95.8792, 41.238092, '882608c9e7fffff'), ('S03405200029', Timestamp('2012-01-03 00:00:00'), 2954, "Dahl's / Ingersoll", '3425 INGERSOLL', 'DES MOINES', '50300', 77, 'Polk', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36886, 'Mccormick Vodka', 12, '750', 3.3, 4.96, 12, 59.52, 9.0, 2.38, -93.663337, 41.586313, '88260d80d7fffff'), ('S03383800079', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67526, 'Kahlua Coffee Liqueur', 12, '750', 11.61, 17.42, 1, 17.42, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03405300007', Timestamp('2012-01-03 00:00:00'), 4576, 'THE BOONEDOCKS', '228 N HWY 71', 'ARNOLDS PARK', '51331', 30, 'Dickinson', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 3, 27.99, 2.25, 0.59, -95.125928, 43.370941, '88262ac013fffff'), ('S03393400033', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35947, "Fleischmann's Royal Vodka", 12, '1000', 3.94, 5.9, 12, 70.8, 12.0, 3.17, -90.741064, 42.516633, '88275a212dfffff'), ('S03400000004', Timestamp('2012-01-03 00:00:00'), 2459, 'Reinhart Foods', '200 STATE PO BOX 98', 'GUTHRIE CENTER', '50115', 39, 'Guthrie', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, 0.0, 0.0, '0'), ('S03406300043', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34029, 'Absolut Citron (lemon Vodka)', 12, '1000', 14.89, 22.34, 1, 22.34, 1.0, 0.26, -91.281344, 42.108275, '882674c6a9fffff'), ('S03390300050', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19477, "Maker's Mark", 12, '1000', 18.89, 28.34, 2, 56.68, 2.0, 0.53, -95.854809, 41.262316, '882608c9cdfffff'), ('S03396400051', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28206, 'Bombay Dry Gin', 12, '750', 10.26, 15.39, 12, 184.68, 9.0, 2.38, -95.8792, 41.238092, '882608c9e7fffff'), ('S03406300032', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1011200, 'STRAIGHT BOURBON WHISKIES', 434, 'Luxco-St Louis', 16987, 'Bourbon Supreme Bourbon', 12, '1000', 4.7, 7.05, 12, 84.6, 12.0, 3.17, -91.281344, 42.108275, '882674c6a9fffff'), ('S03408200015', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58876, 'Jose Cuervo Golden Margarita', 12, '750', 6.42, 9.62, 12, 115.44, 9.0, 2.38, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03386500030', Timestamp('2012-01-03 00:00:00'), 2548, 'Hy-Vee Food Store / Altoona', '100 8TH STREET SW', 'ALTOONA', '50009', 77, 'Polk', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 434, 'Luxco-St Louis', 85606, 'Paramount Blue Curacao', 12, '750', 3.24, 4.86, 3, 14.58, 2.25, 0.59, -93.465301, 41.644041, '88260d94d9fffff'), ('S03392900058', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081300, 'PEPPERMINT SCHNAPPS', 434, 'Luxco-St Louis', 80571, 'Arrow Peppermint Schnapps Mini', 12, '500', 4.54, 6.81, 1, 6.81, 0.5, 0.13, 0.0, 0.0, '0'), ('S03392900037', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34422, 'Grey Goose Vodka', 6, '1000', 21.77, 32.66, 6, 195.96, 6.0, 1.59, 0.0, 0.0, '0'), ('S03411200120', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 330, 'Gemini Spirits', 62400, 'Margaritaville Classic Lime Margarita', 6, '1750', 7.01, 10.52, 2, 21.04, 3.5, 0.92, -93.697731, 41.665172, '88260d8231fffff'), ('S03405700034', Timestamp('2012-01-03 00:00:00'), 3651, 'Wal-Mart 1491 / Indianola', '1500 NORTH JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, 0.0, 0.0, '0'), ('S03406300061', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1081030, 'COFFEE LIQUEURS', 65, 'Jim Beam Brands', 67557, 'Kamora Coffee Liqueur', 12, '1000', 8.13, 12.2, 4, 48.8, 4.0, 1.06, -91.281344, 42.108275, '882674c6a9fffff'), ('S03399800020', Timestamp('2012-01-03 00:00:00'), 3970, 'The Liquor Cabinet, Inc.', '835 6 ST, STE 1', 'NEVADA', '50201', 85, 'Story', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15626, 'Jameson', 12, '750', 13.46, 20.2, 4, 80.8, 3.0, 0.79, -93.452277, 42.019292, '88262b222dfffff'), ('S03396400186', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41715, 'Smirnoff Cranberry Vodka', 12, '750', 7.59, 11.56, 1, 11.56, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03388500019', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48106, 'Hennessy Vs Cognac', 12, '750', 17.06, 25.59, 12, 307.08, 9.0, 2.38, -90.668138, 42.504959, '88275a2119fffff'), ('S03401400011', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82847, 'Dekuyper Peachtree', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -95.126786, 43.416433, '88262ac237fffff'), ('S03392100024', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26820, 'Jack Daniels Old #7 Black Lbl Flat', 24, '375', 6.06, 9.09, 6, 54.54, 2.25, 0.59, -90.204748, 41.827845, '88275b7891fffff'), ('S03387300005', Timestamp('2012-01-03 00:00:00'), 3549, 'Quicker Liquor Store', '1414 48TH ST', 'FORT MADISON', '52627', 56, 'Lee', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69611, "Dr. Mcgillicuddy's Apple Pie Schnapps", 12, '750', 8.32, 12.48, 12, 149.76, 9.0, 2.38, -91.373211, 40.62422600000001, '8826703ac5fffff'), ('S03402100035', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69636, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -92.475403, 42.712263, '8826291801fffff'), ('S03395800151', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36901, 'McCormick Vodka Mini', 6, '500', 3.5, 5.25, 2, 10.5, 1.0, 0.26, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03399600003', Timestamp('2012-01-03 00:00:00'), 4229, 'Fareway Stores #790 / Harlan', '1910  23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76486, 'Paramount Amaretto', 12, '750', 3.37, 5.06, 12, 60.72, 9.0, 2.38, -95.339845, 41.650132, '88260c60e9fffff'), ('S03395700035', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52599, 'E & J Vs Brandy Round', 12, '1000', 6.94, 10.41, 3, 31.23, 3.0, 0.79, -95.147741, 43.14521, '88262a1925fffff'), ('S03408200067', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 46351, 'Hawkeye Light Rum', 12, '1000', 4.22, 6.33, 3, 18.99, 3.0, 0.79, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03408300260', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36903, 'Mccormick Vodka', 48, '200', 1.12, 1.68, 6, 10.08, 1.2, 0.32, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03395800075', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41694, 'Uv Blue (raspberry) Vodka', 12, '1000', 7.25, 10.87, 12, 130.44, 12.0, 3.17, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03411600003', Timestamp('2012-01-03 00:00:00'), 4001, 'Wal-Mart 1787 / Carroll', '2014 KITTY HAWK AVE', 'CARROLL', '51401', 14, 'Carroll', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11296, 'Crown Royal', 12, '750', 14.25, 21.37, 12, 256.44, 9.0, 2.38, -94.893501, 42.081312, '88260c144bfffff'), ('S03403800002', Timestamp('2012-01-03 00:00:00'), 3774, 'Eagle Country Market / Dubuque', '1800 ELM ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86916, 'Southern Comfort 100 Prf', 12, '750', 11.65, 17.47, 12, 209.64, 9.0, 2.38, -90.665763, 42.511105, '88275a2153fffff'), ('S03396400077', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 71446, 'After Shock Cinnamon Liqueur', 12, '750', 11.54, 17.31, 1, 17.31, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03393200009', Timestamp('2012-01-03 00:00:00'), 3895, "Tom's Market and Meats", '550 DIVISION ST', 'BURLINGTON', '52601', 29, 'Des Moines', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41806, 'Smirnoff Vanilla Vodka', 12, '750', 7.59, 11.56, 12, 138.72, 9.0, 2.38, -91.106472, 40.807275000000004, '882670a131fffff'), ('S03384600023', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 36, 804.24, 36.0, 9.51, -93.619787, 41.60566, '88260d8769fffff'), ('S03391000014', Timestamp('2012-01-03 00:00:00'), 3654, 'The Beverage Shop / Belmond', '207 E MAIN ST', 'BELMOND', '50421', 99, 'Wright', 1081200, 'CREAM LIQUEURS', 330, 'Gemini Spirits', 80457, "Ryan's Cream Liqueur", 12, '1000', 7.22, 10.83, 12, 129.96, 12.0, 3.17, -93.615247, 42.845901, '8826286587fffff'), ('S03371300005', Timestamp('2012-01-03 00:00:00'), 4443, 'Palo Mini Mart', '1204 1ST ST', 'PALO', '52324', 57, 'Linn', 1071100, 'AMERICAN COCKTAILS', 260, 'Diageo Americas', 58801, 'Captain Morgan Long Island Iced Tea', 6, '1750', 10.5, 15.74, 6, 94.44, 10.5, 2.77, -91.795808, 42.070316, '8826745323fffff'), ('S03385900010', Timestamp('2012-01-03 00:00:00'), 4356, 'Kum & Go #396 / Nevada', '136 LINCOLN HWY', 'NEVADA', '50201', 85, 'Story', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19064, 'Jim Beam', 24, '375', 4.53, 6.8, 3, 20.4, 1.12, 0.3, -93.458003, 42.022725, '88262b235bfffff'), ('S03411000076', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1012210, 'SINGLE MALT SCOTCH', 370, 'Pernod Ricard USA/Austin Nichols', 5034, 'Glenlivet 12YR', 12, '375', 10.6, 15.91, 3, 47.73, 1.12, 0.3, 0.0, 0.0, '0'), ('S03383400023', Timestamp('2012-01-03 00:00:00'), 3488, "Bender's Foods", '400 S SECOND', 'BELLEVUE', '52031', 49, 'Jackson', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11586, 'Black Velvet Reserve', 12, '750', 6.75, 10.12, 3, 30.36, 2.25, 0.59, -90.42292, 42.255449, '88275b5ad5fffff'), ('S03395800170', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1012200, 'SCOTCH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 4716, 'Chivas Regal 12 Yr', 12, '750', 17.7, 26.55, 3, 79.65, 2.25, 0.59, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03393100009', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23824, 'Five Star', 24, '375', 1.98, 2.96, 24, 71.04, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03381400071', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34422, 'Grey Goose Vodka', 6, '1000', 21.77, 32.66, 84, 2743.44, 84.0, 22.19, -90.527081, 41.559731, '882675daa5fffff'), ('S03410300110', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25603, 'Seagrams 7 Crown Bl Whiskey', 48, '200', 1.85, 2.77, 2, 5.54, 0.4, 0.11, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03397600058', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1071100, 'AMERICAN COCKTAILS', 434, 'Luxco-St Louis', 63528, "Salvador's Margarita Pet", 6, '1750', 5.91, 8.86, 6, 53.16, 10.5, 2.77, -95.825137, 41.242732, '882608c9d1fffff'), ('S03407900035', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1081200, 'CREAM LIQUEURS', 300, 'Mccormick Distilling Company', 65199, 'Tequila Rose Liqueur', 12, '1000', 12.0, 17.99, 12, 215.88, 12.0, 3.17, 0.0, 0.0, '0'), ('S03412100073', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82786, 'Dekuyper Buttershots Schnapps', 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03410900081', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 37937, 'Skol Vodka', 12, '1000', 4.42, 6.63, 4, 26.52, 4.0, 1.06, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03407600015', Timestamp('2012-01-03 00:00:00'), 3465, 'Hometown Foods / Panora', '601 E MAIN', 'PANORA', '50216', 39, 'Guthrie', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54444, 'Paramount Apricot Flavored Brandy', 24, '375', 3.34, 5.0, 6, 30.0, 2.25, 0.59, -94.357201, 41.69182, '88260dd89bfffff'), ('S03395800203', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34030, 'Absolut Citron (lemon Vodka)', 12, '750', 10.89, 16.33, 4, 65.32, 3.0, 0.79, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03395700010', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 22788, "Beam's 8 Star Bl Whiskey", 6, '1750', 7.24, 10.86, 3, 32.58, 5.25, 1.39, -95.147741, 43.14521, '88262a1925fffff'), ('S03401100001', Timestamp('2012-01-03 00:00:00'), 4239, 'Fareway Stores #850 / Spirit Lake', '1906 KEOKUK', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -95.106219, 43.42157, '88262ac23bfffff'), ('S03383800077', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 330, 'Gemini Spirits', 66936, 'Grangala Triple Orange Liqueur', 12, '750', 10.99, 16.49, 1, 16.49, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03403800021', Timestamp('2012-01-03 00:00:00'), 3774, 'Eagle Country Market / Dubuque', '1800 ELM ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67526, 'Kahlua Coffee Liqueur', 12, '750', 11.61, 17.42, 4, 69.68, 3.0, 0.79, -90.665763, 42.511105, '88275a2153fffff'), ('S03412100097', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1012210, 'SINGLE MALT SCOTCH', 420, 'Moet Hennessy USA, Inc.', 5133, 'Glenmorangie 10 Yr Single Malt Scotch', 6, '750', 19.96, 29.94, 6, 179.64, 4.5, 1.19, -90.483701, 41.552338, '882675da95fffff'), ('S03395800083', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1032080, 'IMPORTED VODKA', 267, 'Imperial Brands, Inc.', 34690, 'Sobieski Vodka', 12, '750', 6.49, 9.74, 12, 116.88, 9.0, 2.38, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03391300098', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1031080, 'VODKA 80 PROOF', 300, 'Mccormick Distilling Company', 36907, 'Mccormick Vodka PET', 12, '1000', 3.98, 5.97, 3, 17.91, 3.0, 0.79, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03385300003', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43025, 'Admiral Nelson Spiced Rum Traveler', 12, '750', 5.77, 8.66, 12, 103.92, 9.0, 2.38, -91.136655, 40.814666, '882670aacdfffff'), ('S03384000008', Timestamp('2012-01-03 00:00:00'), 2327, 'West Side Spirits', '801 HULL ST', 'CORNING', '50841', 2, 'Adams', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25607, 'Seagrams 7 Crown Bl Whiskey', 12, '1000', 7.5, 11.24, 3, 33.72, 3.0, 0.79, -94.74369200000001, 40.990378, '882608a233fffff'), ('S03398400026', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1011100, 'BLENDED WHISKIES', 434, 'Luxco-St Louis', 23277, 'Calvert Extra Bl Whiskey', 12, '1000', 6.88, 10.31, 4, 41.24, 4.0, 1.06, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03396000042', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41692, 'Uv Blue (raspberry) Vodka', 6, '1750', 10.63, 15.95, 12, 191.4, 21.0, 5.55, -90.705328, 42.491862, '88275a2131fffff'), ('S03397800013', Timestamp('2012-01-03 00:00:00'), 4421, 'KUM & GO #1 / HAMPTON', '609 CENTRAL AVE W', 'HAMPTON', '50441', 35, 'Franklin', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11771, 'Black Velvet Mini', 10, '600', 4.96, 7.44, 1, 7.44, 0.6, 0.16, -93.21508, 42.741741, '88262b9605fffff'), ('S03385300109', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081380, 'MISCELLANEOUS SCHNAPPS', 65, 'Jim Beam Brands', 82626, 'Dekuyper Cherry Pucker Schnapps', 12, '750', 6.3, 9.45, 6, 56.7, 4.5, 1.19, -91.136655, 40.814666, '882670aacdfffff'), ('S03407900062', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15656, 'Jameson 12YR HA', 12, '750', 21.4, 32.1, 4, 128.4, 3.0, 0.79, 0.0, 0.0, '0'), ('S03388000022', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1022100, 'TEQUILA', 434, 'Luxco-St Louis', 87937, 'Juarez Tequila Silver', 12, '1000', 6.67, 10.0, 12, 120.0, 12.0, 3.17, -93.633306, 43.261538, '88262808e7fffff'), ('S03392100031', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1012100, 'CANADIAN WHISKIES', 85, 'Brown-Forman Corporation', 12466, 'Canadian Mist Whisky', 12, '750', 5.44, 8.15, 2, 16.3, 1.5, 0.4, -90.204748, 41.827845, '88275b7891fffff'), ('S03388000014', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15247, 'Windsor Canadian', 12, '1000', 6.55, 9.83, 12, 117.96, 12.0, 3.17, -93.633306, 43.261538, '88262808e7fffff'), ('S03407900066', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 305, 'MHW Ltd', 66050, 'Nuvo Sparkling Liqueur', 12, '375', 9.93, 14.9, 2, 29.8, 0.75, 0.2, 0.0, 0.0, '0'), ('S03411900007', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45275, 'Paramount White Rum Traveler', 12, '750', 3.85, 5.77, 1, 5.77, 0.75, 0.2, -95.147741, 43.14521, '88262a1925fffff'), ('S03393400111', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58868, 'Jose Cuervo Authentic Strawberry Margarita', 6, '1750', 7.7, 11.8, 36, 424.8, 63.0, 16.64, -90.741064, 42.516633, '88275a212dfffff'), ('S03410200002', Timestamp('2012-01-03 00:00:00'), 3889, 'Wal-Mart 1005 / Waverly', '700  29 ST SW', 'WAVERLY', '50677', 9, 'Bremer', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86886, 'Southern Comfort', 12, '750', 9.36, 14.04, 12, 168.48, 9.0, 2.38, -92.478899, 42.70032100000001, '8826291807fffff'), ('S03396400173', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081317, 'GRAPE SCHNAPPS', 65, 'Jim Beam Brands', 82636, 'Dekuyper Grape Pucker Schnapps', 12, '750', 6.3, 9.45, 1, 9.45, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03408300019', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37348, 'Phillips Vodka', 6, '1750', 7.31, 10.97, 30, 329.1, 52.5, 13.87, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03383600011', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34821, 'Svedka Vodka', 6, '1750', 13.89, 20.84, 6, 125.04, 10.5, 2.77, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03408400003', Timestamp('2012-01-03 00:00:00'), 2650, 'Hy-Vee Wine and Spirits / Harlan', '1808 23RD ST', 'HARLAN', '51537', 83, 'Shelby', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 12, 295.68, 12.0, 3.17, -95.339881, 41.650658, '88260c60e9fffff'), ('S03402500020', Timestamp('2012-01-03 00:00:00'), 4184, 'Fareway Stores #998 / Muscatine', '2100 CEDAR PLAZA DR', 'MUSCATINE', '52761', 70, 'Muscatine', 1081317, 'GRAPE SCHNAPPS', 65, 'Jim Beam Brands', 82636, 'Dekuyper Grape Pucker Schnapps', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -91.08192400000001, 41.435264, '8826742a97fffff'), ('S03397500021', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43127, 'Bacardi Superior Rum', 12, '1000', 9.43, 14.14, 24, 339.36, 24.0, 6.34, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03391600002', Timestamp('2012-01-03 00:00:00'), 4464, 'Perfect Value Liquor Mart', '5B  LINCOLN AVE', 'ELDRIDGE', '52748', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 85, 'Brown-Forman Corporation', 17826, 'Early Times 36 Mo Ky Str Bourbon', 12, '750', 4.76, 7.51, 12, 90.12, 9.0, 2.38, -90.547796, 41.695329, '882674a5c9fffff'), ('S03393900010', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 395, 'Proximo', 46504, 'Kraken Black Spiced Rum', 12, '750', 10.0, 14.99, 3, 44.97, 2.25, 0.59, -95.873073, 41.261866, '882608c917fffff'), ('S03401400037', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1062050, 'BARBADOS RUM', 389, 'REMY COINTREAU USA         .', 42666, 'Mt. Gay Eclipse Rum', 12, '750', 9.0, 13.5, 3, 40.5, 2.25, 0.59, -95.126786, 43.416433, '88262ac237fffff'), ('S03392100040', Timestamp('2012-01-03 00:00:00'), 3920, "Smokin' Joe's #11 Tobacco and Liquor", '1606 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11773, 'Black Velvet', 48, '200', 1.56, 2.34, 4, 9.36, 0.8, 0.21, -90.204748, 41.827845, '88275b7891fffff'), ('S03382100005', Timestamp('2012-01-03 00:00:00'), 3477, "Sam's Club 6472 / Council Bluffs", '3221 MANAWA CENTRE DR', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1051010, 'AMERICAN GRAPE BRANDIES', 259, 'Heaven Hill Brands', 52317, 'Christian Bros Brandy-Square', 12, '1000', 6.94, 10.41, 24, 249.84, 24.0, 6.34, -95.847163, 41.225208, '882608c983fffff'), ('S03392300081', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42703, 'Malibu Pineapple Rum', 12, '750', 6.9, 10.35, 3, 31.05, 2.25, 0.59, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03395700031', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34136, 'Absolut Vodka Glimmer', 12, '750', 10.89, 16.33, 2, 32.66, 1.5, 0.4, -95.147741, 43.14521, '88262a1925fffff'), ('S03385300048', Timestamp('2012-01-03 00:00:00'), 2506, 'Hy-Vee #1044 / Burlington', '3140 AGENCY', 'BURLINGTON', '52601', 29, 'Des Moines', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82606, 'Dekuyper Sour Apple Pucker', 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -91.136655, 40.814666, '882670aacdfffff'), ('S03403400037', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1071100, 'AMERICAN COCKTAILS', 285, 'Charles Jacquin Et. Cie., Inc.', 56957, 'Bartenders Hot Sex', 12, '1000', 5.11, 7.66, 2, 15.32, 2.0, 0.53, -93.599362, 41.59878, '88260d8743fffff'), ('S03398600013', Timestamp('2012-01-03 00:00:00'), 2584, 'Hy-Vee Food Store / Fort Madison', '2606 AVENUE L', 'FORT MADISON', '52627', 56, 'Lee', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -91.343544, 40.626988, '8826703131fffff'), ('S03393100033', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 46350, 'Hawkeye Light Rum', 6, '1750', 7.56, 11.33, 6, 67.98, 10.5, 2.77, -95.855054, 41.261673, '882608c9cdfffff'), ('S03397500052', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1081600, 'WHISKEY LIQUEUR', 260, 'Diageo Americas', 67267, 'Yukon Jack Canadian Liqueur', 12, '1000', 11.21, 16.81, 12, 201.72, 12.0, 3.17, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03396600118', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24458, 'Kessler Blend Whiskey', 6, '1750', 11.01, 16.52, 1, 16.52, 1.75, 0.46, -93.378772, 43.142868, '8826283b29fffff'), ('S03400700034', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1101100, 'AMERICAN ALCOHOL', 434, 'Luxco-St Louis', 41846, 'Everclear Alcohol', 12, '750', 8.02, 12.03, 12, 144.36, 9.0, 2.38, 0.0, 0.0, '0'), ('S03389300007', Timestamp('2012-01-03 00:00:00'), 3858, 'Harolds Jack N Jill / Davenport', '6723 NW BLVD UNIT 2', 'DAVENPORT', '52806', 82, 'Scott', 1052010, 'IMPORTED GRAPE BRANDIES', 420, 'Moet Hennessy USA, Inc.', 48104, 'Hennessy Vs Cognac', 24, '375', 8.38, 12.58, 24, 301.92, 9.0, 2.38, -90.609747, 41.589369, '882675d959fffff'), ('S03387200010', Timestamp('2012-01-03 00:00:00'), 4296, 'Circle B Market', '111 MAIN ST', 'KEOSAUQUA', '52565', 89, 'Van Buren', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69947, 'Rumple Minze Peppermint Schnapps Liqueur', 12, '1000', 15.0, 22.49, 12, 269.88, 12.0, 3.17, -91.962466, 40.7294, '882670458bfffff'), ('S03391300151', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1081330, 'PEACH SCHNAPPS', 434, 'Luxco-St Louis', 84456, 'Paramount Peach Schnapps', 12, '750', 3.97, 5.95, 2, 11.9, 1.5, 0.4, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03412100186', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86881, 'Southern Comfort Mini', 12, '500', 4.7, 7.06, 2, 14.12, 1.0, 0.26, -90.483701, 41.552338, '882675da95fffff'), ('S03393100027', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43244, 'Captain Morgan 100 Proof Spiced Rum', 12, '750', 10.0, 14.99, 12, 179.88, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03401400018', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 41078, 'Jeremiah Weed Sweet Tea Vodka', 6, '1750', 14.97, 22.46, 6, 134.76, 10.5, 2.77, -95.126786, 43.416433, '88262ac237fffff'), ('S03397500002', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65257, 'Jagermeister Liqueur', 12, '1000', 14.93, 22.39, 36, 806.04, 36.0, 9.51, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03395800048', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 434, 'Luxco-St Louis', 63528, "Salvador's Margarita Pet", 6, '1750', 5.91, 8.86, 150, 1329.0, 262.5, 69.35, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03392900030', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45248, 'Paramount Gold Rum', 6, '1750', 7.58, 11.37, 6, 68.22, 10.5, 2.77, 0.0, 0.0, '0'), ('S03385800041', Timestamp('2012-01-03 00:00:00'), 3554, "Dick's Beverage Depot", '102 N MAIN', 'TRIPOLI', '50676', 9, 'Bremer', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10628, 'Canadian Club Whisky', 6, '1750', 15.75, 23.62, 3, 70.86, 5.25, 1.39, -92.258152, 42.809882, '8826291267fffff'), ('S03383800036', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37336, 'Uv Vodka', 12, '750', 6.22, 9.33, 1, 9.33, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03396600008', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19067, 'Jim Beam', 12, '1000', 11.54, 17.31, 12, 207.72, 12.0, 3.17, -93.378772, 43.142868, '8826283b29fffff'), ('S03396400066', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11298, 'Crown Royal Canadian Whisky', 6, '1750', 30.5, 45.74, 12, 548.88, 21.0, 5.55, -95.8792, 41.238092, '882608c9e7fffff'), ('S03390800017', Timestamp('2012-01-03 00:00:00'), 2806, 'Osco #881 / Clinton', '1307 N SECOND', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57148, "Chi-Chi's Margarita W/tequila", 6, '1750', 5.99, 9.24, 6, 55.44, 10.5, 2.77, -90.184797, 41.858932, '88275b7ab3fffff'), ('S03396000014', Timestamp('2012-01-03 00:00:00'), 2465, "Sid's Beverage Shop", '2727 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10627, 'Canadian Club Whisky', 12, '1000', 9.34, 14.01, 24, 336.24, 24.0, 6.34, -90.705328, 42.491862, '88275a2131fffff'), ('S03408600052', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37993, 'Smirnoff Vodka 80 Prf', 48, '200', 2.46, 3.69, 2, 7.38, 0.4, 0.11, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03394400007', Timestamp('2012-01-03 00:00:00'), 4334, 'Fareway Stores #114 / Dubuque', '2050 JOHN F KENNEDY RD', 'DUBUQUE', '52001', 31, 'Dubuque', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 6, 90.0, 4.5, 1.19, 0.0, 0.0, '0'), ('S03388300005', Timestamp('2012-01-03 00:00:00'), 4039, 'No Frills Supermarkets / Council Blu', '1817 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1022100, 'TEQUILA', 395, 'Proximo', 89198, 'Jose Cuervo Especial Reposado Tequila', 6, '1750', 20.25, 30.37, 6, 182.22, 10.5, 2.77, -95.870993, 41.261863, '882608c917fffff'), ('S03403900042', Timestamp('2012-01-03 00:00:00'), 2628, 'Hy-Vee Food Store / Eldora', '1616 EDGINGTON', 'ELDORA', '50627', 42, 'Hardin', 1081330, 'PEACH SCHNAPPS', 65, 'Jim Beam Brands', 82846, 'Dekuyper Luscious Peachtree Schnapps', 12, '750', 6.3, 9.45, 2, 18.9, 1.5, 0.4, -93.103764, 42.360631, '88262ba25dfffff'), ('S03407800024', Timestamp('2012-01-03 00:00:00'), 4181, 'Puff N Snuff', '1101 RHOMBERG AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19064, 'Jim Beam', 24, '375', 4.53, 6.8, 1, 6.8, 0.38, 0.1, -90.660269, 42.519502, '88275a215bfffff'), ('S03411100040', Timestamp('2012-01-03 00:00:00'), 3631, 'Audubon Food Pride', '104 MARKET ST PO BOX 206', 'AUDUBON', '50025', 5, 'Audubon', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5347, 'Johnnie Walker Red Label Scotch', 12, '1000', 16.99, 25.49, 1, 25.49, 1.0, 0.26, -94.93625, 41.723057, '88260c388dfffff'), ('S03383600003', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1022100, 'TEQUILA', 395, 'Proximo', 89198, 'Jose Cuervo Especial Reposado Tequila', 6, '1750', 20.25, 30.37, 6, 182.22, 10.5, 2.77, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03401000061', Timestamp('2012-01-03 00:00:00'), 3697, 'Oky Doky #2 / Dubuque', '51 W 32ND ST PO BOX 300', 'DUBUQUE', '52004', 31, 'Dubuque', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45276, 'Paramount White Rum', 12, '750', 3.85, 5.77, 1, 5.77, 0.75, 0.2, -90.683356, 42.52717, '88275a215dfffff'), ('S03388800007', Timestamp('2012-01-03 00:00:00'), 3811, 'Wal-Mart 1431 / Keokuk', '300 NORTH PARK DR', 'KEOKUK', '52632', 56, 'Lee', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69637, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '1000', 11.0, 16.5, 12, 198.0, 12.0, 3.17, -91.407263, 40.418589, '88267028d9fffff'), ('S03403600022', Timestamp('2012-01-03 00:00:00'), 3744, 'Payless Foods / Dyersville', '733 16TH AVE SE', 'DYERSVILLE', '52040', 31, 'Dubuque', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43316, 'Captain Morgan Private Stock Rum', 12, '750', 11.45, 17.18, 3, 51.54, 2.25, 0.59, -91.115769, 42.470204, '88275a654dfffff'), ('S03393900027', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36305, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 5, 24.4, 3.75, 0.99, -95.873073, 41.261866, '882608c917fffff'), ('S03395700068', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 305, 'MHW Ltd', 77570, 'Travis Hasse Apple Pie Liqueur', 12, '750', 7.94, 11.91, 3, 35.73, 2.25, 0.59, -95.147741, 43.14521, '88262a1925fffff'), ('S03397600049', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37997, 'Smirnoff Vodka 80 Prf', 12, '1000', 8.49, 12.74, 24, 305.76, 24.0, 6.34, -95.825137, 41.242732, '882608c9d1fffff'), ('S03390900015', Timestamp('2012-01-03 00:00:00'), 3911, 'Burg Spirits, Llc.', '813 1/2 HIGHLAND ST', 'WILLIAMSBURG', '52361', 48, 'Iowa', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43335, 'Captain Morgan Spiced Rum Pet', 6, '1750', 16.99, 25.48, 12, 305.76, 21.0, 5.55, 0.0, 0.0, '0'), ('S03397600001', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11297, 'Crown Royal Canadian Whisky', 12, '1000', 17.46, 26.19, 12, 314.28, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03410300101', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1031080, 'VODKA 80 PROOF', 297, 'Laird And Company', 35926, "Five O'clock PET Vodka", 12, '750', 3.31, 4.96, 2, 9.92, 1.5, 0.4, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03400000010', Timestamp('2012-01-03 00:00:00'), 2459, 'Reinhart Foods', '200 STATE PO BOX 98', 'GUTHRIE CENTER', '50115', 39, 'Guthrie', 1031080, 'VODKA 80 PROOF', 260, 'Diageo Americas', 37997, 'Smirnoff Vodka 80 Prf', 12, '1000', 8.49, 12.74, 4, 50.96, 4.0, 1.06, 0.0, 0.0, '0'), ('S03404600002', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11786, 'Black Velvet Traveler', 12, '750', 4.83, 7.44, 12, 89.28, 9.0, 2.38, -95.027238, 40.739231, '88260831cbfffff'), ('S03404600047', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1051010, 'AMERICAN GRAPE BRANDIES', 259, 'Heaven Hill Brands', 52317, 'Christian Bros Brandy-Square', 12, '1000', 6.94, 10.41, 2, 20.82, 2.0, 0.53, -95.027238, 40.739231, '88260831cbfffff'), ('S03398800026', Timestamp('2012-01-03 00:00:00'), 2636, 'Hy-Vee Wine and Spirits / Hubbell', '2310 HUBBELL AVE', 'DES MOINES', '50317', 77, 'Polk', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34423, 'Grey Goose Vodka', 12, '375', 8.48, 12.72, 2, 25.44, 0.75, 0.2, -93.573845, 41.604312, '88260d95b3fffff'), ('S03400700008', Timestamp('2012-01-03 00:00:00'), 2556, 'Hy-Vee Wine and Spirits / Esthervill', '2608 E. CENTRAL AVENUE', 'ESTHERVILLE', '51334', 32, 'Emmet', 1022100, 'TEQUILA', 395, 'Proximo', 89196, 'Jose Cuervo Especial Reposado Tequila', 12, '750', 10.49, 15.73, 12, 188.76, 9.0, 2.38, 0.0, 0.0, '0'), ('S03396400184', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32234, 'Seagrams Extra Dry Gin', 24, '375', 3.4, 5.1, 1, 5.1, 0.38, 0.1, -95.8792, 41.238092, '882608c9e7fffff'), ('S03396400043', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43338, 'Captain Morgan Spiced Rum', 6, '1750', 16.99, 25.48, 6, 152.88, 10.5, 2.77, -95.8792, 41.238092, '882608c9e7fffff'), ('S03409700124', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1081230, 'WHITE CREME DE CACAO', 434, 'Luxco-St Louis', 78866, 'Paramount Creme De Cacao/white', 12, '750', 4.44, 6.66, 3, 19.98, 2.25, 0.59, -93.737447, 41.62941, '88260d802bfffff'), ('S03397300087', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41704, 'Uv Red (cherry) Vodka', 12, '750', 6.22, 9.33, 2, 18.66, 1.5, 0.4, -91.044008, 41.427202, '8826742f4bfffff'), ('S03408200063', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1022100, 'TEQUILA', 55, 'Sazerac North America', 89577, 'Montezuma Gold Tequila', 12, '1000', 7.02, 10.53, 3, 31.59, 3.0, 0.79, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03406400020', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 68036, "Bailey's Original Irish Cream", 12, '750', 12.78, 19.18, 1, 19.18, 0.75, 0.2, -90.766126, 41.509119, '88267434edfffff'), ('S03408300037', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15248, 'Windsor Canadian Pet', 6, '1750', 8.92, 13.38, 6, 80.28, 10.5, 2.77, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03403800014', Timestamp('2012-01-03 00:00:00'), 3774, 'Eagle Country Market / Dubuque', '1800 ELM ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36308, 'Hawkeye Vodka', 6, '1750', 7.13, 10.7, 6, 64.2, 10.5, 2.77, -90.665763, 42.511105, '88275a2153fffff'), ('S03410700069', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1031080, 'VODKA 80 PROOF', 301, 'Fifth Generation Inc.', 38176, 'Titos Vodka', 12, '750', 9.64, 14.46, 12, 173.52, 9.0, 2.38, -94.853591, 42.064155, '88260c16a9fffff'), ('S03403200051', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1071100, 'AMERICAN COCKTAILS', 55, 'Sazerac North America', 57174, "Chi-Chi's Skinny Margarita", 6, '1750', 5.99, 9.24, 1, 9.24, 1.75, 0.46, -94.218, 43.29355, '8826284833fffff'), ('S03391300026', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1081200, 'CREAM LIQUEURS', 330, 'Gemini Spirits', 80457, "Ryan's Cream Liqueur", 12, '1000', 7.22, 10.83, 12, 129.96, 12.0, 3.17, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03411000071', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86887, 'Southern Comfort', 12, '1000', 11.34, 17.01, 24, 408.24, 24.0, 6.34, 0.0, 0.0, '0'), ('S03407700003', Timestamp('2012-01-03 00:00:00'), 4602, 'Cheap Smokes / Beer City', '58231 190th ST', 'PACIFIC JUNCTION', '51561', 65, 'Mills', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65195, 'Hpnotiq', 6, '750', 11.68, 17.52, 6, 105.12, 4.5, 1.19, -95.822983, 41.041739, '8826084255fffff'), ('S03397300032', Timestamp('2012-01-03 00:00:00'), 2645, 'Hy-Vee Drugstore / Muscatine', '510 E 6TH ST', 'MUSCATINE', '52761', 70, 'Muscatine', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58875, 'Jose Cuervo Golden Margarita', 6, '1750', 10.0, 15.24, 6, 91.44, 10.5, 2.77, -91.044008, 41.427202, '8826742f4bfffff'), ('S03381400030', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 192, 'Sidney Frank Importing Co.', 65257, 'Jagermeister Liqueur', 12, '1000', 14.93, 22.39, 72, 1612.08, 72.0, 19.02, -90.527081, 41.559731, '882675daa5fffff'), ('S03382100002', Timestamp('2012-01-03 00:00:00'), 3477, "Sam's Club 6472 / Council Bluffs", '3221 MANAWA CENTRE DR', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 260, 'Diageo Americas', 25607, 'Seagrams 7 Crown Bl Whiskey', 12, '1000', 7.5, 11.24, 24, 269.76, 24.0, 6.34, -95.847163, 41.225208, '882608c983fffff'), ('S03395700049', Timestamp('2012-01-03 00:00:00'), 3480, "C's Spencer Liquor", '719  2ND AVE W PO BOX 401', 'SPENCER', '51301', 21, 'Clay', 1062310, 'SPICED RUM', 434, 'Luxco-St Louis', 44657, 'Lady Bligh Spiced Rum', 12, '1000', 6.33, 9.49, 4, 37.96, 4.0, 1.06, -95.147741, 43.14521, '88262a1925fffff'), ('S03384600044', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 46351, 'Hawkeye Light Rum', 12, '1000', 4.22, 6.33, 12, 75.96, 12.0, 3.17, -93.619787, 41.60566, '88260d8769fffff'), ('S03411500046', Timestamp('2012-01-03 00:00:00'), 3700, 'Avoca Liquor', '158 SOUTH ELM', 'AVOCA', '51521', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43316, 'Captain Morgan Private Stock Rum', 12, '750', 11.45, 17.18, 1, 17.18, 0.75, 0.2, -95.339412, 41.476306, '88260c6c8bfffff'), ('S03402600074', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41989, 'Uv Cake Vodka', 12, '750', 6.22, 9.33, 1, 9.33, 0.75, 0.2, 0.0, 0.0, '0'), ('S03391300086', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1081380, 'MISCELLANEOUS SCHNAPPS', 380, 'Phillips Beverage Company', 84568, 'Phillips Black 100 Herbal Schnapps', 12, '750', 6.94, 10.41, 1, 10.41, 0.75, 0.2, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03403900019', Timestamp('2012-01-03 00:00:00'), 2628, 'Hy-Vee Food Store / Eldora', '1616 EDGINGTON', 'ELDORA', '50627', 42, 'Hardin', 1101100, 'AMERICAN ALCOHOL', 434, 'Luxco-St Louis', 41846, 'Everclear Alcohol', 12, '750', 8.02, 12.03, 12, 144.36, 9.0, 2.38, -93.103764, 42.360631, '88262ba25dfffff'), ('S03406900022', Timestamp('2012-01-03 00:00:00'), 4114, 'After 5 Somewhere', '704 W 7TH ST PO BOX 372', 'ATLANTIC', '50022', 15, 'Cass', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 34116, 'Absolut Mandrin', 12, '750', 10.89, 16.33, 2, 32.66, 1.5, 0.4, -95.023116, 41.40406200000001, '88260890a3fffff'), ('S03410900002', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11777, 'Black Velvet', 12, '1000', 6.63, 9.94, 12, 119.28, 12.0, 3.17, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03383800023', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1032080, 'IMPORTED VODKA', 277, 'Stoli Group', 34746, 'Stolichnaya Russian Vodka 80 Prf', 12, '750', 12.24, 18.37, 1, 18.37, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03389200004', Timestamp('2012-01-03 00:00:00'), 4247, 'Fareway Stores #879 / Belmond', '115 SECOND  AVE SE', 'BELMOND', '50421', 0, 0, 1071100, 'AMERICAN COCKTAILS', 65, 'Jim Beam Brands', 63355, 'Skinnygirl Margarita', 12, '750', 7.69, 11.53, 6, 69.18, 4.5, 1.19, -93.614197, 42.845742, '8826286587fffff'), ('S03398500012', Timestamp('2012-01-03 00:00:00'), 4249, 'Fareway Stores #048 / Clarinda', '115 ESSIE DAVISON DR', 'CLARINDA', '51632', 73, 'Page', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19066, 'Jim Beam', 12, '750', 8.42, 13.41, 3, 40.23, 2.25, 0.59, -95.040293, 40.724008, '8826083189fffff'), ('S03411000022', Timestamp('2012-01-03 00:00:00'), 2625, 'Hy-Vee Wine and Spirits #2', '3301 W KIMBERLY RD', 'DAVENPORT', '52804', 82, 'Scott', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 12, 268.08, 12.0, 3.17, 0.0, 0.0, '0'), ('S03396500024', Timestamp('2012-01-03 00:00:00'), 3587, 'Save A Lot  #482', '2122 MAIN ST', 'KEOKUK', '52632', 56, 'Lee', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43116, 'Bacardi O', 12, '750', 8.33, 12.49, 4, 49.96, 3.0, 0.79, -91.39688800000002, 40.408821, '8826702f67fffff'), ('S03404800097', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 461, 'Campari(skyy)', 64996, 'Frangelico Liqueur', 12, '750', 13.01, 19.52, 6, 117.12, 4.5, 1.19, -91.035138, 41.451349, '88267421abfffff'), ('S03391300033', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19477, "Maker's Mark", 12, '1000', 18.89, 28.34, 1, 28.34, 1.0, 0.26, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03395500001', Timestamp('2012-01-03 00:00:00'), 3976, 'North English Family Food', '119 NORTH MAIN ST', 'NORTH ENGLISH', '52316', 48, 'Iowa', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -92.076452, 41.515209, '882676a0adfffff'), ('S03388500010', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23823, 'Five Star', 48, '200', 1.21, 1.81, 48, 86.88, 9.6, 2.54, -90.668138, 42.504959, '88275a2119fffff'), ('S03397600060', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1022100, 'TEQUILA', 434, 'Luxco-St Louis', 89387, 'Juarez Tequila Gold', 12, '1000', 6.67, 10.0, 48, 480.0, 48.0, 12.68, -95.825137, 41.242732, '882608c9d1fffff'), ('S03404000010', Timestamp('2012-01-03 00:00:00'), 3841, 'Hartig Drug Company #2 / Locust', '157 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 3, 27.99, 2.25, 0.59, -90.665439, 42.495308, '88275a211bfffff'), ('S03393400136', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 73482, 'Dekuyper Washington Apple', 12, '750', 6.3, 9.45, 4, 37.8, 3.0, 0.79, -90.741064, 42.516633, '88275a212dfffff'), ('S03409700026', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1081600, 'WHISKEY LIQUEUR', 421, 'Sazerac Co., Inc.', 64866, 'Fireball Cinnamon Whiskey', 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -93.737447, 41.62941, '88260d802bfffff'), ('S03409700126', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24466, 'Kessler Blend Whiskey Traveler', 12, '750', 5.51, 8.26, 5, 41.3, 3.75, 0.99, -93.737447, 41.62941, '88260d802bfffff'), ('S03406300086', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1022100, 'TEQUILA', 395, 'Proximo', 89191, 'Jose Cuervo Especial Reposado Tequila Mini', 12, '500', 11.5, 17.25, 1, 17.25, 0.5, 0.13, -91.281344, 42.108275, '882674c6a9fffff'), ('S03388000078', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 259, 'Heaven Hill Brands', 65195, 'Hpnotiq', 6, '750', 11.68, 17.52, 2, 35.04, 1.5, 0.4, -93.633306, 43.261538, '88262808e7fffff'), ('S03388100097', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67536, 'Kahlua Especial', 12, '750', 11.73, 17.59, 3, 52.77, 2.25, 0.59, -95.879662, 41.261925, '882608c93bfffff'), ('S03406400051', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41604, 'Uv Grape Vodka', 12, '750', 6.22, 9.33, 2, 18.66, 1.5, 0.4, -90.766126, 41.509119, '88267434edfffff'), ('S03409700060', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 461, 'Campari(skyy)', 37986, 'Skyy Vodka', 12, '750', 10.01, 15.01, 4, 60.04, 3.0, 0.79, -93.737447, 41.62941, '88260d802bfffff'), ('S03410700025', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26827, 'Jack Daniels Old #7 Black Lbl', 12, '1000', 16.43, 24.64, 12, 295.68, 12.0, 3.17, -94.853591, 42.064155, '88260c16a9fffff'), ('S03391000015', Timestamp('2012-01-03 00:00:00'), 3654, 'The Beverage Shop / Belmond', '207 E MAIN ST', 'BELMOND', '50421', 99, 'Wright', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750', 9.97, 14.95, 4, 59.8, 3.0, 0.79, -93.615247, 42.845901, '8826286587fffff'), ('S03395800074', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41966, 'UV Espresso Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03408300176', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19026, 'Jim Beam Black', 12, '750', 13.12, 19.68, 1, 19.68, 0.75, 0.2, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03407200036', Timestamp('2012-01-03 00:00:00'), 4146, 'Fareway Stores #561 / Waverly', '220 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1011200, 'STRAIGHT BOURBON WHISKIES', 85, 'Brown-Forman Corporation', 17826, 'Early Times 36 Mo Ky Str Bourbon', 12, '750', 4.76, 7.51, 3, 22.53, 2.25, 0.59, -92.473476, 42.725748, '8826291847fffff'), ('S03410900134', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1051110, 'BLACKBERRY BRANDIES', 380, 'Phillips Beverage Company', 55106, 'Phillips Blackberry Flav Brandy', 12, '750', 4.75, 7.13, 4, 28.52, 3.0, 0.79, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03392300025', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11786, 'Black Velvet Traveler', 12, '750', 4.83, 7.44, 24, 178.56, 18.0, 4.76, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03408300233', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1022100, 'TEQUILA', 65, 'Jim Beam Brands', 89796, 'Sauza Conmemorativo', 12, '750', 14.17, 21.25, 1, 21.25, 0.75, 0.2, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03402900039', Timestamp('2012-01-03 00:00:00'), 2591, 'Hy-Vee Wine and Spirits / Atlantic', '1602 E. 7TH ST.', 'ATLANTIC', '50022', 15, 'Cass', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21594, 'Ten High', 24, '375', 2.65, 3.97, 4, 15.88, 1.5, 0.4, -94.990031, 41.403856, '8826089095fffff'), ('S03404100038', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52599, 'E & J Vs Brandy Round', 12, '1000', 6.94, 10.41, 6, 62.46, 6.0, 1.59, -90.993859, 42.3036, '8826749ac1fffff'), ('S03392500017', Timestamp('2012-01-03 00:00:00'), 3908, 'County Market #214 / Fort Madison', '11802 AVENUE H', 'FORT MADISON', '52627', 56, 'Lee', 1011200, 'STRAIGHT BOURBON WHISKIES', 55, 'Sazerac North America', 21596, 'Ten High', 12, '750', 4.0, 5.99, 12, 71.88, 9.0, 2.38, 0.0, 0.0, '0'), ('S03382700031', Timestamp('2012-01-03 00:00:00'), 2607, 'Hy-Vee Wine and Spirits / Shenandoah', '520 SO FREMONT', 'SHENANDOAH', '51601', 73, 'Page', 1012200, 'SCOTCH WHISKIES', 35, 'Bacardi U.S.A., Inc.', 4867, 'Dewars White Label Scotch', 12, '1000', 16.86, 25.29, 2, 50.58, 2.0, 0.53, -95.385111, 40.761736, '8826080d95fffff'), ('S03410300035', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1041100, 'AMERICAN DRY GINS', 434, 'Luxco-St Louis', 30527, 'Hawkeye Gin', 12, '1000', 3.89, 5.83, 2, 11.66, 2.0, 0.53, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03410300085', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19066, 'Jim Beam', 12, '750', 8.42, 13.41, 2, 26.82, 1.5, 0.4, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03396600112', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1081365, 'TROPICAL FRUIT SCHNAPPS', 65, 'Jim Beam Brands', 82857, 'Dekuyper Island Punch Pucker', 12, '1000', 7.35, 11.02, 3, 33.06, 3.0, 0.79, -93.378772, 43.142868, '8826283b29fffff'), ('S03389000027', Timestamp('2012-01-03 00:00:00'), 3583, "Mac's Liquor And Videos", '220 EAST JEFFERSON', 'CORYDON', '50060', 93, 'Wayne', 1012210, 'SINGLE MALT SCOTCH', 240, 'WILLIAM GRANT AND SONS, INC.', 5006, 'Glenfiddich 12yr Special Reserve', 12, '750', 21.17, 31.75, 1, 31.75, 0.75, 0.2, -93.315977, 40.757158, '88267252cdfffff'), ('S03388500038', Timestamp('2012-01-03 00:00:00'), 4167, 'Iowa Street Market, Inc.', '1256 IOWA ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -90.668138, 42.504959, '88275a2119fffff'), ('S03381300010', Timestamp('2012-01-03 00:00:00'), 3944, "Sam's Club 4973 / Dubuque", '4400 ASBURY RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1041100, 'AMERICAN DRY GINS', 55, 'Sazerac North America', 29288, 'Barton Gin', 6, '1750', 6.57, 9.85, 12, 118.2, 21.0, 5.55, -90.737821, 42.515289, '88275a212dfffff'), ('S03403000022', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15247, 'Windsor Canadian', 12, '1000', 6.55, 9.83, 4, 39.32, 4.0, 1.06, -94.480204, 43.39966, '88262a82cbfffff'), ('S03398400015', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1012100, 'CANADIAN WHISKIES', 85, 'Brown-Forman Corporation', 12478, 'Canadian Mist Canadian Why Pet', 6, '1750', 10.5, 15.75, 3, 47.25, 5.25, 1.39, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03390000010', Timestamp('2012-01-03 00:00:00'), 3606, "Scotty's Liquor Store", '804 G AVE', 'GRUNDY CENTER', '50638', 38, 'Grundy', 1081390, 'IMPORTED SCHNAPPS', 260, 'Diageo Americas', 69947, 'Rumple Minze Peppermint Schnapps Liqueur', 12, '1000', 15.0, 22.49, 12, 269.88, 12.0, 3.17, -92.77421500000001, 42.361514, '8826297831fffff'), ('S03407900057', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1062310, 'SPICED RUM', 259, 'Heaven Hill Brands', 43025, 'Admiral Nelson Spiced Rum Traveler', 12, '750', 5.77, 8.66, 12, 103.92, 9.0, 2.38, 0.0, 0.0, '0'), ('S03389900006', Timestamp('2012-01-03 00:00:00'), 2532, 'Hy-Vee Food Store #2 / Des Moines', '2540 E EUCLID', 'DES MOINES', '50317', 77, 'Polk', 1022100, 'TEQUILA', 395, 'Proximo', 89197, 'Jose Cuervo Especial Reposado Tequila', 12, '1000', 12.5, 18.74, 12, 224.88, 12.0, 3.17, -93.569452, 41.627838, '88260d9581fffff'), ('S03393200013', Timestamp('2012-01-03 00:00:00'), 3895, "Tom's Market and Meats", '550 DIVISION ST', 'BURLINGTON', '52601', 29, 'Des Moines', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35948, "Fleischmann's Royal Vodka 80 Prf", 6, '1750', 7.14, 10.7, 6, 64.2, 10.5, 2.77, -91.106472, 40.807275000000004, '882670a131fffff'), ('S03387800087', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 11294, 'Crown Royal Canadian Whisky', 24, '375', 7.24, 10.85, 2, 21.7, 0.75, 0.2, -91.387797, 40.400038, '8826702f29fffff'), ('S03407900084', Timestamp('2012-01-03 00:00:00'), 2549, 'Hy-Vee Food Store / Indianola', '910 N JEFFERSON', 'INDIANOLA', '50125', 91, 'Warren', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82604, 'Dekuyper Sour Apple Pucker', 24, '375', 3.06, 4.59, 2, 9.18, 0.75, 0.2, 0.0, 0.0, '0'), ('S03390400028', Timestamp('2012-01-03 00:00:00'), 3738, 'Hartig Drug Company #4 / Dubuque', '2225 CENTRAL AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86916, 'Southern Comfort 100 Prf', 12, '750', 11.65, 17.47, 2, 34.94, 1.5, 0.4, -90.671531, 42.51384, '88275a2153fffff'), ('S03396600037', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26824, 'Jack Daniels Old #7 Black Lbl Square', 24, '375', 6.06, 9.09, 3, 27.27, 1.12, 0.3, -93.378772, 43.142868, '8826283b29fffff'), ('S03411200168', Timestamp('2012-01-03 00:00:00'), 2587, 'Hy-Vee Food Store / Johnston', '5750 MERLE HAY ROAD', 'JOHNSTON', '50131', 77, 'Polk', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 65, 'Jim Beam Brands', 44486, 'Cruzan Dark Rum', 12, '750', 6.82, 10.23, 3, 30.69, 2.25, 0.59, -93.697731, 41.665172, '88260d8231fffff'), ('S03404000015', Timestamp('2012-01-03 00:00:00'), 3841, 'Hartig Drug Company #2 / Locust', '157 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1011200, 'STRAIGHT BOURBON WHISKIES', 259, 'Heaven Hill Brands', 17956, 'Evan Williams Str Bourbon', 12, '750', 6.37, 9.56, 3, 28.68, 2.25, 0.59, -90.665439, 42.495308, '88275a211bfffff'), ('S03394100019', Timestamp('2012-01-03 00:00:00'), 3451, 'Barnes Foodland', '622 10TH ST', 'DEWITT', '52742', 23, 'Clinton', 1081340, 'ROOT BEER SCHNAPPS', 65, 'Jim Beam Brands', 82826, "Dekuyper Ragin' Root Beer Schnapps", 12, '750', 6.3, 9.45, 12, 113.4, 9.0, 2.38, -90.540132, 41.825139, '882674b5e7fffff'), ('S03412100079', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1081010, 'AMERICAN AMARETTO', 55, 'Sazerac North America', 73986, 'Di Amore Amaretto', 12, '750', 6.0, 8.99, 12, 107.88, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03412100196', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41601, 'Uv Blue (raspberry) Vodka', 12, '375', 3.44, 5.16, 2, 10.32, 0.75, 0.2, -90.483701, 41.552338, '882675da95fffff'), ('S03402100017', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42699, 'Malibu Mango Rum', 12, '750', 6.9, 10.35, 12, 124.2, 9.0, 2.38, -92.475403, 42.712263, '8826291801fffff'), ('S03402700022', Timestamp('2012-01-03 00:00:00'), 3907, "Smokin' Joe's #16 Tobacco and Liquor", '1504 PARK AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1062300, 'FLAVORED RUM', 35, 'Bacardi U.S.A., Inc.', 43136, 'Bacardi Limon', 12, '750', 8.33, 12.49, 2, 24.98, 1.5, 0.4, -91.030569, 41.442458, '88267421bdfffff'), ('S03408200004', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1011200, 'STRAIGHT BOURBON WHISKIES', 259, 'Heaven Hill Brands', 17958, 'Evan Williams Yr Str Bourbon', 6, '1750', 13.4, 20.1, 6, 120.6, 10.5, 2.77, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03395800224', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081365, 'TROPICAL FRUIT SCHNAPPS', 65, 'Jim Beam Brands', 82857, 'Dekuyper Island Punch Pucker', 12, '1000', 7.35, 11.02, 3, 33.06, 3.0, 0.79, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03405000011', Timestamp('2012-01-03 00:00:00'), 4158, 'Fareway Stores #409 / Carroll', '709 MONTEREY DR', 'CARROLL', '51401', 14, 'Carroll', 1062300, 'FLAVORED RUM', 65, 'Jim Beam Brands', 44520, 'Cruzan Vanilla Rum', 12, '750', 6.38, 9.57, 3, 28.71, 2.25, 0.59, -94.850616, 42.065252, '88260c16e3fffff'), ('S03396400131', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031080, 'VODKA 80 PROOF', 301, 'Fifth Generation Inc.', 38176, 'Titos Vodka', 12, '750', 9.64, 14.46, 1, 14.46, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03395800060', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43336, 'Captain Morgan Original Spiced', 12, '750', 8.5, 12.74, 48, 611.52, 36.0, 9.51, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03408300146', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1031200, 'VODKA FLAVORED', 260, 'Diageo Americas', 40997, 'Smirnoff Pomegranate Vodka', 12, '750', 7.59, 11.56, 4, 46.24, 3.0, 0.79, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03408300161', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10627, 'Canadian Club Whisky', 12, '1000', 9.34, 14.01, 1, 14.01, 1.0, 0.26, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03383800083', Timestamp('2012-01-03 00:00:00'), 4344, 'Spirits / Grimes', '109 EAST 1ST ST STE B', 'GRIMES', '50111', 77, 'Polk', 1081200, 'CREAM LIQUEURS', 260, 'Diageo Americas', 74086, 'Godiva Liqueur', 12, '750', 16.42, 24.62, 1, 24.62, 0.75, 0.2, -93.793827, 41.68840000000001, '88260d8329fffff'), ('S03405500007', Timestamp('2012-01-03 00:00:00'), 4203, 'Liquor and Tobacco Outlet /', '620 W BREMER AVE', 'WAVERLY', '50677', 9, 'Bremer', 1081300, 'PEPPERMINT SCHNAPPS', 65, 'Jim Beam Brands', 80686, 'Dekuyper Blustery Peppermint Burst Schnapps', 12, '750', 3.44, 5.15, 12, 61.8, 9.0, 2.38, -92.47898, 42.725694, '8826291847fffff'), ('S03381400062', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1071100, 'AMERICAN COCKTAILS', 65, 'Jim Beam Brands', 63355, 'Skinnygirl Margarita', 12, '750', 7.69, 11.53, 48, 553.44, 36.0, 9.51, -90.527081, 41.559731, '882675daa5fffff'), ('S03394100023', Timestamp('2012-01-03 00:00:00'), 3451, 'Barnes Foodland', '622 10TH ST', 'DEWITT', '52742', 23, 'Clinton', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 330, 'Gemini Spirits', 66936, 'Grangala Triple Orange Liqueur', 12, '750', 10.99, 16.49, 12, 197.88, 9.0, 2.38, -90.540132, 41.825139, '882674b5e7fffff'), ('S03410700108', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35944, "Fleischmann's Royal Vodka", 24, '375', 1.74, 2.6, 3, 7.8, 1.12, 0.3, -94.853591, 42.064155, '88260c16a9fffff'), ('S03406400023', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 54647, 'Arrow Blackberry Flav Brandy', 12, '1000', 6.11, 9.17, 3, 27.51, 3.0, 0.79, -90.766126, 41.509119, '88267434edfffff'), ('S03411600010', Timestamp('2012-01-03 00:00:00'), 4001, 'Wal-Mart 1787 / Carroll', '2014 KITTY HAWK AVE', 'CARROLL', '51401', 14, 'Carroll', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45278, 'Paramount White Rum', 6, '1750', 7.58, 11.37, 12, 136.44, 21.0, 5.55, -94.893501, 42.081312, '88260c144bfffff'), ('S03389400012', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 420, 'Moet Hennessy USA, Inc.', 65126, 'Grand Marnier Cordon Rouge', 12, '750', 23.31, 34.96, 12, 419.52, 9.0, 2.38, -95.909384, 41.242291, '88260e26d5fffff'), ('S03404600058', Timestamp('2012-01-03 00:00:00'), 2205, "Ding's Honk And Holler", '900 E WASHINGTON', 'CLARINDA', '51632', 73, 'Page', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 305, 'MHW Ltd', 64000, 'Absente', 12, '750', 21.67, 32.5, 1, 32.5, 0.75, 0.2, -95.027238, 40.739231, '88260831cbfffff'), ('S03407600014', Timestamp('2012-01-03 00:00:00'), 3465, 'Hometown Foods / Panora', '601 E MAIN', 'PANORA', '50216', 39, 'Guthrie', 1012100, 'CANADIAN WHISKIES', 55, 'Sazerac North America', 12406, 'Canadian Ltd Whisky', 12, '750', 3.75, 5.87, 3, 17.61, 2.25, 0.59, -94.357201, 41.69182, '88260dd89bfffff'), ('S03395200107', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 46351, 'Hawkeye Light Rum', 12, '1000', 4.22, 6.33, 4, 25.32, 4.0, 1.06, -91.110752, 40.801889, '882670a137fffff'), ('S03390300038', Timestamp('2012-01-03 00:00:00'), 3728, "Brewski's Beverage", '726 CREEK TOP', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081305, 'APPLE SCHNAPPS', 65, 'Jim Beam Brands', 82607, 'Dekuyper Sour Apple', 12, '1000', 7.35, 11.02, 12, 132.24, 12.0, 3.17, -95.854809, 41.262316, '882608c9cdfffff'), ('S03397600080', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69666, "Dr. Mcgillicuddy's Vanilla", 12, '750', 8.66, 12.99, 12, 155.88, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03412100062', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52596, 'E & J Vs', 12, '750', 5.51, 8.26, 12, 99.12, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03405800010', Timestamp('2012-01-03 00:00:00'), 3761, "Cheese's Food Center", '200 N WALNUT ST', 'LENOX', '50851', 87, 'Taylor', 1081315, 'CINNAMON SCHNAPPS', 65, 'Jim Beam Brands', 82611, 'Dekuyper Hot Damn!', 12, '750', 6.3, 9.45, 3, 28.35, 2.25, 0.59, -94.560455, 40.882645, '882608a6e9fffff'), ('S03406400022', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1051100, 'APRICOT BRANDIES', 434, 'Luxco-St Louis', 54057, 'Arrow Apricot Flav Brandy', 12, '1000', 6.11, 9.17, 2, 18.34, 2.0, 0.53, -90.766126, 41.509119, '88267434edfffff'), ('S03404500034', Timestamp('2012-01-03 00:00:00'), 2518, 'Hy-Vee Wine and Spirits / Red Oak', '1607 NORTH BROADWAY', 'RED OAK', '51566', 69, 'Montgomery', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750', 9.97, 14.95, 3, 44.85, 2.25, 0.59, -95.83539, 41.279189, '882608c829fffff'), ('S03395800014', Timestamp('2012-01-03 00:00:00'), 2616, 'Hy-Vee Food and Drug / Clinton', '901 SOUTH 4TH ST', 'CLINTON', '52732', 23, 'Clinton', 1071100, 'AMERICAN COCKTAILS', 65, 'Jim Beam Brands', 63355, 'Skinnygirl Margarita', 12, '750', 7.69, 11.53, 12, 138.36, 9.0, 2.38, -90.193204, 41.836268000000004, '88275b6325fffff'), ('S03399000008', Timestamp('2012-01-03 00:00:00'), 3762, 'Wine and Spirits Gallery', '7690 HICKMAN RD', 'WINDSOR HEIGHTS', '50322', 77, 'Polk', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43331, 'Captain Morgan Spiced Rum Mini', 12, '500', 4.97, 7.25, 1, 7.25, 0.5, 0.13, 0.0, 0.0, '0'), ('S03381400035', Timestamp('2012-01-03 00:00:00'), 3354, "Sam's Club 8238 / Davenport", '3845 ELMORE AVE.', 'DAVENPORT', '52807', 82, 'Scott', 1012200, 'SCOTCH WHISKIES', 55, 'Sazerac North America', 8827, "Lauder's", 12, '1000', 6.13, 9.2, 24, 220.8, 24.0, 6.34, -90.527081, 41.559731, '882675daa5fffff'), ('S03383600001', Timestamp('2012-01-03 00:00:00'), 3886, 'Wal-Mart 0646 / Anamosa', '101  115 ST', 'ANAMOSA', '52205', 53, 'Jones', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -91.26496500000002, 42.101677, '882674c685fffff'), ('S03393400067', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1081600, 'WHISKEY LIQUEUR', 85, 'Brown-Forman Corporation', 86888, 'Southern Comfort PET', 6, '1750', 18.14, 27.21, 6, 163.26, 10.5, 2.77, -90.741064, 42.516633, '88275a212dfffff'), ('S03408200108', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 420, 'Moet Hennessy USA, Inc.', 65124, 'Grand Marnier Cordon Rouge', 24, '375', 13.36, 20.03, 3, 60.09, 1.12, 0.3, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03408300319', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1071100, 'AMERICAN COCKTAILS', 395, 'Proximo', 58863, 'Jose Cuervo Authentic Pomegranate Margarita', 6, '1750', 8.2, 12.3, 5, 61.5, 8.75, 2.31, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03393600013', Timestamp('2012-01-03 00:00:00'), 4040, 'No Frills Supermarkets / Council Blu', '1801 VALLEY VIEW DR', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43127, 'Bacardi Superior Rum', 12, '1000', 9.43, 14.14, 12, 169.68, 12.0, 3.17, -95.81877500000002, 41.24394, '882608c9d9fffff'), ('S03405200036', Timestamp('2012-01-03 00:00:00'), 2954, "Dahl's / Ingersoll", '3425 INGERSOLL', 'DES MOINES', '50300', 77, 'Polk', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11774, 'Black Velvet', 24, '375', 3.07, 4.6, 24, 110.4, 9.0, 2.38, -93.663337, 41.586313, '88260d80d7fffff'), ('S03384600025', Timestamp('2012-01-03 00:00:00'), 2190, 'Central City Liquor, Inc.', '1460 2ND AVE', 'DES MOINES', '50314', 77, 'Polk', 1031080, 'VODKA 80 PROOF', 380, 'Phillips Beverage Company', 37347, 'Phillips Vodka', 12, '1000', 4.0, 6.01, 60, 360.6, 60.0, 15.85, -93.619787, 41.60566, '88260d8769fffff'), ('S03387800056', Timestamp('2012-01-03 00:00:00'), 2191, 'Keokuk Spirits', '1013 MAIN', 'KEOKUK', '52632', 56, 'Lee', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750', 9.97, 14.95, 3, 44.85, 2.25, 0.59, -91.387797, 40.400038, '8826702f29fffff'), ('S03407400011', Timestamp('2012-01-03 00:00:00'), 2962, "Dahl's / Johnston", '5440 NW 86TH ST', 'JOHNSTON', '50131', 77, 'Polk', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10008, 'Scoresby Rare Scotch', 6, '1750', 10.22, 15.33, 6, 91.98, 10.5, 2.77, -93.736517, 41.659057, '88260d8045fffff'), ('S03386600029', Timestamp('2012-01-03 00:00:00'), 3647, 'Wal-Mart 2889 / Clinton', '2715 SOUTH 25TH ST', 'CLINTON', '52732', 23, 'Clinton', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76486, 'Paramount Amaretto', 12, '750', 3.37, 5.06, 12, 60.72, 9.0, 2.38, -90.245005, 41.818874, '88275b78b7fffff'), ('S03411500063', Timestamp('2012-01-03 00:00:00'), 3700, 'Avoca Liquor', '158 SOUTH ELM', 'AVOCA', '51521', 78, 'Pottawattamie', 1081200, 'CREAM LIQUEURS', 300, 'Mccormick Distilling Company', 74763, 'Hot Rose Cinnamon Cream Liqueur', 12, '750', 10.05, 15.07, 2, 30.14, 1.5, 0.4, -95.339412, 41.476306, '88260c6c8bfffff'), ('S03392900084', Timestamp('2012-01-03 00:00:00'), 2524, 'Hy-Vee Food Store / Dubuque', '3500 DODGE ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1032200, 'IMPORTED VODKA - MISC', 395, 'Proximo', 35469, 'Three Olives Bubble Vodka', 12, '750', 11.16, 16.74, 3, 50.22, 2.25, 0.59, 0.0, 0.0, '0'), ('S03401400016', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34433, 'Grey Goose Vodka', 12, '750', 17.97, 26.96, 12, 323.52, 9.0, 2.38, -95.126786, 43.416433, '88262ac237fffff'), ('S03410900038', Timestamp('2012-01-03 00:00:00'), 2649, 'Hy-Vee #3 / Dubuque', '400 LOCUST ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 322, 'Prestige Wine and Spirits Group', 75210, 'Kinky Liqueur', 6, '750', 10.0, 15.0, 6, 90.0, 4.5, 1.19, -90.666497, 42.49721900000001, '88275a211bfffff'), ('S03393100035', Timestamp('2012-01-03 00:00:00'), 2641, 'Hy-Vee Drugstore / Council Bluffs', '757 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1041100, 'AMERICAN DRY GINS', 260, 'Diageo Americas', 30316, "Gordon's Gin London Dry", 12, '750', 5.0, 7.49, 12, 89.88, 9.0, 2.38, -95.855054, 41.261673, '882608c9cdfffff'), ('S03377200001', Timestamp('2012-01-03 00:00:00'), 4438, 'Kum & Go #214  Ames', '111 DUFF AVE', 'AMES', '50010', 85, 'Story', 1031080, 'VODKA 80 PROOF', 421, 'Sazerac Co., Inc.', 36978, 'Nikolai Vodka', 6, '1750', 6.87, 10.3, 6, 61.8, 10.5, 2.77, -93.610396, 42.023592, '88262b38adfffff'), ('S03402100011', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36307, 'Hawkeye Vodka', 12, '1000', 3.85, 5.78, 12, 69.36, 12.0, 3.17, -92.475403, 42.712263, '8826291801fffff'), ('S03402100051', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1012300, 'IRISH WHISKIES', 395, 'Proximo', 15776, 'Bushmills Irish Whiskey', 12, '750', 12.99, 19.49, 1, 19.49, 0.75, 0.2, -92.475403, 42.712263, '8826291801fffff'), ('S03386400002', Timestamp('2012-01-03 00:00:00'), 4437, 'Kum & Go #215  Ames', '4506 LINCOLN WAY', 'AMES', '50010', 85, 'Story', 1031080, 'VODKA 80 PROOF', 434, 'Luxco-St Louis', 36306, 'Hawkeye Vodka', 12, '750', 3.25, 4.88, 12, 58.56, 9.0, 2.38, 0.0, 0.0, '0'), ('S03401400030', Timestamp('2012-01-03 00:00:00'), 2233, 'Forbes Liquor Locker / remi', '2508 ENTERPRISE AVE', 'SPIRIT LAKE', '51360', 30, 'Dickinson', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52597, 'E & J Vs Brandy Square', 12, '1000', 6.94, 10.41, 3, 31.23, 3.0, 0.79, -95.126786, 43.416433, '88262ac237fffff'), ('S03389400033', Timestamp('2012-01-03 00:00:00'), 3521, 'Ameristar Casino / Council Bluffs', '2200 RIVER ROAD', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 15248, 'Windsor Canadian Pet', 6, '1750', 8.92, 13.38, 6, 80.28, 10.5, 2.77, -95.909384, 41.242291, '88260e26d5fffff'), ('S03406300016', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11777, 'Black Velvet', 12, '1000', 6.63, 9.94, 24, 238.56, 24.0, 6.34, -91.281344, 42.108275, '882674c6a9fffff'), ('S03397600091', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55086, 'Paramount Blackberry Brandy', 12, '750', 5.21, 7.82, 12, 93.84, 9.0, 2.38, -95.825137, 41.242732, '882608c9d1fffff'), ('S03408300283', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1051010, 'AMERICAN GRAPE BRANDIES', 115, 'Constellation Wine Company, Inc.', 53214, 'Paul Masson Grande Amber Brandy', 24, '375', 3.05, 4.57, 12, 54.84, 4.5, 1.19, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03405900054', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1081390, 'IMPORTED SCHNAPPS', 421, 'Sazerac Co., Inc.', 69637, "Dr. Mcgillicuddy's Cherry Schnapps", 12, '1000', 11.0, 16.5, 48, 792.0, 48.0, 12.68, -90.48065, 41.529655, '882675c361fffff'), ('S03403700018', Timestamp('2012-01-03 00:00:00'), 4547, 'Fareway Stores #902 / Hampton', '309 CENTRAL AVE WEST', 'HAMPTON', '50441', 35, 'Franklin', 1022100, 'TEQUILA', 421, 'Sazerac Co., Inc.', 88766, 'Tortilla White Tequila', 12, '750', 6.34, 9.52, 3, 28.56, 2.25, 0.59, -93.211676, 42.741748, '88262b9605fffff'), ('S03407100013', Timestamp('2012-01-03 00:00:00'), 2657, 'Hy-Vee Food Store / Bedford', '1604 BENT', 'BEDFORD', '50833', 87, 'Taylor', 1012100, 'CANADIAN WHISKIES', 260, 'Diageo Americas', 10836, 'Crown Royal Black', 12, '750', 16.47, 24.7, 1, 24.7, 0.75, 0.2, -94.72557800000001, 40.676171, '882609d9e5fffff'), ('S03396600027', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 12, 123.6, 21.0, 5.55, -93.378772, 43.142868, '8826283b29fffff'), ('S03410300104', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1012200, 'SCOTCH WHISKIES', 55, 'Sazerac North America', 8827, "Lauder's", 12, '1000', 6.13, 9.2, 1, 9.2, 1.0, 0.26, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03397600034', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23828, 'Five Star', 6, '1750', 7.36, 11.04, 6, 66.24, 10.5, 2.77, -95.825137, 41.242732, '882608c9d1fffff'), ('S03402100075', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 125, 'Cedar Ridge Vineyards,LLC', 72950, 'Cedar Ridge Lemoncella', 12, '375', 8.0, 12.0, 2, 24.0, 0.75, 0.2, -92.475403, 42.712263, '8826291801fffff'), ('S03404800013', Timestamp('2012-01-03 00:00:00'), 2573, 'Hy-Vee Food Store / Muscatine', '2400 2ND AVE', 'MUSCATINE', '52761', 70, 'Muscatine', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 10008, 'Scoresby Rare Scotch', 6, '1750', 10.22, 15.33, 6, 91.98, 10.5, 2.77, -91.035138, 41.451349, '88267421abfffff'), ('S03386400001', Timestamp('2012-01-03 00:00:00'), 4437, 'Kum & Go #215  Ames', '4506 LINCOLN WAY', 'AMES', '50010', 85, 'Story', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, 0.0, 0.0, '0'), ('S03405200033', Timestamp('2012-01-03 00:00:00'), 2954, "Dahl's / Ingersoll", '3425 INGERSOLL', 'DES MOINES', '50300', 77, 'Polk', 1012100, 'CANADIAN WHISKIES', 434, 'Luxco-St Louis', 13036, 'Canadian Reserve Whisky', 12, '750', 4.25, 6.38, 12, 76.56, 9.0, 2.38, -93.663337, 41.586313, '88260d80d7fffff'), ('S03398200005', Timestamp('2012-01-03 00:00:00'), 2633, 'Hy-Vee #3 / BDI / Des Moines', '3221 SE 14TH ST', 'DES MOINES', '50320', 77, 'Polk', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41698, 'Uv Ivory (vanilla) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -93.596754, 41.554101, '88260d846dfffff'), ('S03397500044', Timestamp('2012-01-03 00:00:00'), 3390, 'Okoboji Avenue Liquor', '1610 OKOBOJI AVENUE', 'MILFORD', '51351', 30, 'Dickinson', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -95.149955, 43.331525, '88262ac0a7fffff'), ('S03408100036', Timestamp('2012-01-03 00:00:00'), 3162, 'Nash Finch / Wholesale Food', '807 GRANDVIEW', 'MUSCATINE', '52761', 70, 'Muscatine', 1042100, 'IMPORTED DRY GINS', 260, 'Diageo Americas', 28867, 'Tanqueray Gin', 12, '1000', 14.99, 22.48, 1, 22.48, 1.0, 0.26, -91.06411300000002, 41.408437, '8826742f05fffff'), ('S03391300127', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43124, 'Bacardi Superior Rum', 24, '375', 4.3, 6.45, 2, 12.9, 0.75, 0.2, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03388000070', Timestamp('2012-01-03 00:00:00'), 4073, 'Uptown Liquor, Llc', '306 HWY 69 SOUTH', 'FOREST CITY', '50436', 95, 'Winnebago', 1022100, 'TEQUILA', 410, 'The Patron Spirits Company', 89626, 'Patron Reposado Tequila', 12, '750', 27.0, 40.5, 1, 40.5, 0.75, 0.2, -93.633306, 43.261538, '88262808e7fffff'), ('S03385900015', Timestamp('2012-01-03 00:00:00'), 4356, 'Kum & Go #396 / Nevada', '136 LINCOLN HWY', 'NEVADA', '50201', 85, 'Story', 1041100, 'AMERICAN DRY GINS', 370, 'Pernod Ricard USA/Austin Nichols', 32234, 'Seagrams Extra Dry Gin', 24, '375', 3.4, 5.1, 3, 15.3, 1.12, 0.3, -93.458003, 42.022725, '88262b235bfffff'), ('S03396600075', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34820, 'Svedka Vodka', 12, '750', 7.97, 11.95, 4, 47.8, 3.0, 0.79, -93.378772, 43.142868, '8826283b29fffff'), ('S03384700015', Timestamp('2012-01-03 00:00:00'), 4138, 'Fareway Stores #657 / Indianola', '1309 W 2ND', 'INDIANOLA', '50125', 91, 'Warren', 1032080, 'IMPORTED VODKA', 35, 'Bacardi U.S.A., Inc.', 34433, 'Grey Goose Vodka', 12, '750', 17.97, 26.96, 3, 80.88, 2.25, 0.59, -93.579252, 41.356616, '88260db8e9fffff'), ('S03403400035', Timestamp('2012-01-03 00:00:00'), 4012, 'Fast Mart', '1401 BUCHANAN ST', 'DES MOINES', '50316', 77, 'Polk', 1032080, 'IMPORTED VODKA', 115, 'Constellation Wine Company, Inc.', 34820, 'Svedka Vodka', 12, '750', 7.97, 11.95, 3, 35.85, 2.25, 0.59, -93.599362, 41.59878, '88260d8743fffff'), ('S03395200088', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1011100, 'BLENDED WHISKIES', 65, 'Jim Beam Brands', 24466, 'Kessler Blend Whiskey Traveler', 12, '750', 5.51, 8.26, 2, 16.52, 1.5, 0.4, -91.110752, 40.801889, '882670a137fffff'), ('S03393900030', Timestamp('2012-01-03 00:00:00'), 3963, 'Tobacco Hut #14 / Council Bluffs', '1925 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55087, 'Paramount Blackberry Brandy', 12, '1000', 6.21, 9.32, 2, 18.64, 2.0, 0.53, -95.873073, 41.261866, '882608c917fffff'), ('S03412100072', Timestamp('2012-01-03 00:00:00'), 2603, 'Hy-Vee Wine and Spirits / Bettendorf', '2890 DEVILS GLEN ROAD', 'BETTENDORF', '52722', 82, 'Scott', 1081312, 'BUTTERSCOTCH SCHNAPPS', 434, 'Luxco-St Louis', 81966, "Arrow Mcdale's Butterscotch Schnapps", 12, '750', 3.97, 5.95, 12, 71.4, 9.0, 2.38, -90.483701, 41.552338, '882675da95fffff'), ('S03397600015', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1012200, 'SCOTCH WHISKIES', 260, 'Diageo Americas', 5289, 'J & B Rare Scotch', 12, '1000', 14.98, 22.47, 12, 269.64, 12.0, 3.17, -95.825137, 41.242732, '882608c9d1fffff'), ('S03388100093', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1081600, 'WHISKEY LIQUEUR', 260, 'Diageo Americas', 67267, 'Yukon Jack Canadian Liqueur', 12, '1000', 11.21, 16.81, 2, 33.62, 2.0, 0.53, -95.879662, 41.261925, '882608c93bfffff'), ('S03410300088', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1041150, 'FLAVORED GINS', 370, 'Pernod Ricard USA/Austin Nichols', 33266, "Seagram's Orange Twisted Gin", 12, '750', 5.9, 8.85, 1, 8.85, 0.75, 0.2, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03410300124', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1022100, 'TEQUILA', 395, 'Proximo', 89194, 'Jose Cuervo Especial Reposado Flask', 24, '375', 5.49, 8.24, 2, 16.48, 0.75, 0.2, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03402100082', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1081300, 'PEPPERMINT SCHNAPPS', 421, 'Sazerac Co., Inc.', 80826, 'Ice 101, Peppermint Schnapps', 12, '750', 9.34, 14.01, 2, 28.02, 1.5, 0.4, -92.475403, 42.712263, '8826291801fffff'), ('S03394500040', Timestamp('2012-01-03 00:00:00'), 3443, 'Super Saver Iv', '1141 N BROADWAY', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43296, 'Captain Morgan Silver Spiced Rum', 12, '750', 8.0, 11.99, 12, 143.88, 9.0, 2.38, -95.836515, 41.270824, '882608c823fffff'), ('S03396400162', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1051110, 'BLACKBERRY BRANDIES', 434, 'Luxco-St Louis', 55086, 'Paramount Blackberry Brandy', 12, '750', 5.21, 7.82, 1, 7.82, 0.75, 0.2, -95.8792, 41.238092, '882608c9e7fffff'), ('S03409700036', Timestamp('2012-01-03 00:00:00'), 2534, 'Hy-Vee Drugtown / Urbandale', '8601 DOUGLAS', 'URBANDALE', '50322', 77, 'Polk', 1012300, 'IRISH WHISKIES', 240, 'WILLIAM GRANT AND SONS, INC.', 15940, 'Tullamore Dew Irish Whisky', 12, '750', 12.37, 18.56, 3, 55.68, 2.25, 0.59, -93.737447, 41.62941, '88260d802bfffff'), ('S03403200038', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1081600, 'WHISKEY LIQUEUR', 260, 'Diageo Americas', 86666, 'Jeremiah Weed 100prf Bourbon Liqueur', 12, '750', 6.43, 9.64, 3, 28.92, 2.25, 0.59, -94.218, 43.29355, '8826284833fffff'), ('S03384000024', Timestamp('2012-01-03 00:00:00'), 2327, 'West Side Spirits', '801 HULL ST', 'CORNING', '50841', 2, 'Adams', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 3, 49.47, 3.0, 0.79, -94.74369200000001, 40.990378, '882608a233fffff'), ('S03396100021', Timestamp('2012-01-03 00:00:00'), 2656, 'Hy-Vee Food Store / Corning', '300  10TH ST', 'CORNING', '51632', 22, 'Clayton', 1081010, 'AMERICAN AMARETTO', 434, 'Luxco-St Louis', 76486, 'Paramount Amaretto', 12, '750', 3.37, 5.06, 2, 10.12, 1.5, 0.4, -94.731809, 40.991861, '882608a217fffff'), ('S03402100044', Timestamp('2012-01-03 00:00:00'), 2651, 'Hy-Vee / Waverly', '1311 4 STREET SW', 'WAVERLY', '50677', 9, 'Bremer', 1022100, 'TEQUILA', 434, 'Luxco-St Louis', 89387, 'Juarez Tequila Gold', 12, '1000', 6.67, 10.0, 12, 120.0, 12.0, 3.17, -92.475403, 42.712263, '8826291801fffff'), ('S03396400118', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28206, 'Bombay Dry Gin', 12, '750', 10.26, 15.39, 10, 153.9, 7.5, 1.98, -95.8792, 41.238092, '882608c9e7fffff'), ('S03388100009', Timestamp('2012-01-03 00:00:00'), 2613, 'Hy-Vee Food Store #1 / Council Bluff', '2323 W BROADWAY', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1031200, 'VODKA FLAVORED', 380, 'Phillips Beverage Company', 41693, 'Uv Blue (raspberry) Vodka', 12, '750', 6.22, 9.33, 12, 111.96, 9.0, 2.38, -95.879662, 41.261925, '882608c93bfffff'), ('S03403800013', Timestamp('2012-01-03 00:00:00'), 3774, 'Eagle Country Market / Dubuque', '1800 ELM ST', 'DUBUQUE', '52001', 31, 'Dubuque', 1031080, 'VODKA 80 PROOF', 421, 'Sazerac Co., Inc.', 36976, 'Nikolai Vodka', 12, '750', 3.26, 4.88, 24, 117.12, 18.0, 4.76, -90.665763, 42.511105, '88275a2153fffff'), ('S03392500014', Timestamp('2012-01-03 00:00:00'), 3908, 'County Market #214 / Fort Madison', '11802 AVENUE H', 'FORT MADISON', '52627', 56, 'Lee', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 27544, 'Red Stag By Jim Beam', 12, '750', 9.97, 14.95, 12, 179.4, 9.0, 2.38, 0.0, 0.0, '0'), ('S03405900044', Timestamp('2012-01-03 00:00:00'), 3952, 'Lot-A-Spirits', '3780 STATE ST', 'BETTENDORF', '52722', 82, 'Scott', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43156, 'Bacardi 151 Proof Rum', 12, '750', 12.66, 18.99, 12, 227.88, 9.0, 2.38, -90.48065, 41.529655, '882675c361fffff'), ('S03396400009', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1011100, 'BLENDED WHISKIES', 297, 'Laird And Company', 23827, 'Five Star', 12, '1000', 4.36, 6.53, 12, 78.36, 12.0, 3.17, -95.8792, 41.238092, '882608c9e7fffff'), ('S03391300062', Timestamp('2012-01-03 00:00:00'), 3612, 'B and C Liquor / Maquoketa', '509 E PLATT', 'MAQUOKETA', '52060', 49, 'Jackson', 1062300, 'FLAVORED RUM', 370, 'Pernod Ricard USA/Austin Nichols', 42703, 'Malibu Pineapple Rum', 12, '750', 6.9, 10.35, 1, 10.35, 0.75, 0.2, -90.659692, 42.06922000000001, '88267486d3fffff'), ('S03410700011', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1012100, 'CANADIAN WHISKIES', 65, 'Jim Beam Brands', 10626, 'Canadian Club Whisky', 12, '750', 7.34, 11.01, 12, 132.12, 9.0, 2.38, -94.853591, 42.064155, '88260c16a9fffff'), ('S03389600007', Timestamp('2012-01-03 00:00:00'), 3632, 'Wal-Mart 2004 / Dubuque', '4200 DODGE ST', 'DUBUQUE', '52003', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34006, 'Absolut Swedish Vodka 80 Prf', 12, '750', 10.89, 16.33, 12, 195.96, 9.0, 2.38, -90.736955, 42.489041, '88275a2ac1fffff'), ('S03406400055', Timestamp('2012-01-03 00:00:00'), 4347, "Jeff's Market / Blue Grass", '102 W MAYNE', 'BLUE GRASS', '52726', 82, 'Scott', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36908, 'Mccormick Vodka Pet', 6, '1750', 7.46, 11.19, 1, 11.19, 1.75, 0.46, -90.766126, 41.509119, '88267434edfffff'), ('S03406300055', Timestamp('2012-01-03 00:00:00'), 2487, 'Anamosa Family Foods', '402 EAST MAIN', 'ANAMOSA', '52205', 53, 'Jones', 1081900, 'MISC. AMERICAN CORDIALS & LIQUEURS', 305, 'MHW Ltd', 77570, 'Travis Hasse Apple Pie Liqueur', 12, '750', 7.94, 11.91, 1, 11.91, 0.75, 0.2, -91.281344, 42.108275, '882674c6a9fffff'), ('S03396400058', Timestamp('2012-01-03 00:00:00'), 4312, 'I-80 Liquor / Council Bluffs', '2411 S 24TH ST #1', 'COUNCIL BLUFFS', '51501', 78, 'Pottawattamie', 1042100, 'IMPORTED DRY GINS', 35, 'Bacardi U.S.A., Inc.', 28233, 'Bombay Sapphire Gin', 12, '1000', 17.61, 26.41, 12, 316.92, 12.0, 3.17, -95.8792, 41.238092, '882608c9e7fffff'), ('S03400900006', Timestamp('2012-01-03 00:00:00'), 3803, 'Strawberry Foods and Deli', '128 W MISSION', 'STRAWBERRY POINT', '52076', 22, 'Clayton', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11788, 'Black Velvet', 6, '1750', 10.45, 15.67, 6, 94.02, 10.5, 2.77, -91.535736, 42.683731, '882629b66dfffff'), ('S03408300294', Timestamp('2012-01-03 00:00:00'), 2614, 'Hy-Vee #3 Food and Drugstore', '1823 E KIMBERLY RD', 'DAVENPORT', '52807', 82, 'Scott', 1082900, 'MISC. IMPORTED CORDIALS & LIQUEURS', 305, 'MHW Ltd', 66052, 'Nuvo', 12, '750', 15.0, 22.49, 2, 44.98, 1.5, 0.4, -90.548919, 41.55678100000001, '882675d811fffff'), ('S03395200110', Timestamp('2012-01-03 00:00:00'), 2507, 'Hy-Vee Food Store #1 / Burlington', '939 ANGULAR', 'BURLINGTON', '52601', 29, 'Des Moines', 1101100, 'AMERICAN ALCOHOL', 434, 'Luxco-St Louis', 41846, 'Everclear Alcohol', 12, '750', 8.02, 12.03, 2, 24.06, 1.5, 0.4, -91.110752, 40.801889, '882670a137fffff'), ('S03396300010', Timestamp('2012-01-03 00:00:00'), 4041, 'Mandm Pantry', '2344 CAMANCHE AVE', 'CLINTON', '52732', 23, 'Clinton', 1022100, 'TEQUILA', 395, 'Proximo', 89199, 'Jose Cuervo Especial Reposado Square', 12, '375', 5.49, 8.24, 4, 32.96, 1.5, 0.4, -90.213015, 41.821777, '88275b7897fffff'), ('S03393400153', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1012300, 'IRISH WHISKIES', 370, 'Pernod Ricard USA/Austin Nichols', 15656, 'Jameson 12YR HA', 12, '750', 21.4, 32.1, 2, 64.2, 1.5, 0.4, -90.741064, 42.516633, '88275a212dfffff'), ('S03391800006', Timestamp('2012-01-03 00:00:00'), 4480, 'Circle K #6602 / Clinton', '1530 NORTH 2ND ST', 'CLINTON', '52732', 23, 'Clinton', 1031100, '100 PROOF VODKA', 300, 'Mccormick Distilling Company', 36908, 'Mccormick Vodka Pet', 6, '1750', 7.46, 11.19, 6, 67.14, 10.5, 2.77, -90.183722, 41.861892, '88275b7ab3fffff'), ('S03397600141', Timestamp('2012-01-03 00:00:00'), 2629, 'Hy-Vee Food Store #2 / Council Bluff', '1745 MADISON AVE', 'COUNCIL BLUFFS', '51503', 78, 'Pottawattamie', 1012100, 'CANADIAN WHISKIES', 115, 'Constellation Wine Company, Inc.', 11774, 'Black Velvet', 24, '375', 3.07, 4.6, 12, 55.2, 4.5, 1.19, -95.825137, 41.242732, '882608c9d1fffff'), ('S03410700051', Timestamp('2012-01-03 00:00:00'), 2593, 'Hy-Vee Food Store / Carroll', '905 US HIGHWAY 30 WEST', 'CARROLL', '51401', 14, 'Carroll', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 35, 'Bacardi U.S.A., Inc.', 43126, 'Bacardi Superior Rum', 12, '750', 7.53, 11.3, 12, 135.6, 9.0, 2.38, -94.853591, 42.064155, '88260c16a9fffff'), ('S03398400002', Timestamp('2012-01-03 00:00:00'), 3717, 'The Liquor Stop / Sumner', '201 W 1st ST', 'SUMNER', '50674', 9, 'Bremer', 1062310, 'SPICED RUM', 260, 'Diageo Americas', 43337, 'Captain Morgan Spiced Rum', 12, '1000', 10.99, 16.49, 12, 197.88, 12.0, 3.17, -92.09922100000001, 42.847479, '8826298f09fffff'), ('S03402600062', Timestamp('2012-01-03 00:00:00'), 4262, 'Cork and Bottle / Carroll', '1004 HIGHWAY 30 W', 'CARROLL', '51401', 14, 'Carroll', 1022100, 'TEQUILA', 395, 'Proximo', 89212, 'Jose Cuervo Black Medallion', 12, '750', 12.5, 18.74, 1, 18.74, 0.75, 0.2, 0.0, 0.0, '0'), ('S03396200003', Timestamp('2012-01-03 00:00:00'), 3732, 'North Scott Foods', '425 E LECLAIRE RD', 'ELDRIDGE', '52748', 82, 'Scott', 1031080, 'VODKA 80 PROOF', 55, 'Sazerac North America', 35318, 'Barton Vodka', 6, '1750', 6.87, 10.3, 6, 61.8, 10.5, 2.77, -90.577311, 41.654695000000004, '882674a5bdfffff'), ('S03410300015', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1051120, 'CHERRY BRANDIES', 434, 'Luxco-St Louis', 55506, 'Paramount Cherry Flavored Brandy', 12, '750', 5.21, 7.82, 12, 93.84, 9.0, 2.38, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03377200014', Timestamp('2012-01-03 00:00:00'), 4438, 'Kum & Go #214  Ames', '111 DUFF AVE', 'AMES', '50010', 85, 'Story', 1011200, 'STRAIGHT BOURBON WHISKIES', 65, 'Jim Beam Brands', 19064, 'Jim Beam', 24, '375', 4.53, 6.8, 7, 47.6, 2.62, 0.69, -93.610396, 42.023592, '88262b38adfffff'), ('S03390400001', Timestamp('2012-01-03 00:00:00'), 3738, 'Hartig Drug Company #4 / Dubuque', '2225 CENTRAL AVE', 'DUBUQUE', '52001', 31, 'Dubuque', 1051010, 'AMERICAN GRAPE BRANDIES', 205, 'E AND J GALLO WINERY', 52591, 'E & J Vs Brandy Mini', 12, '500', 5.91, 8.87, 12, 106.44, 6.0, 1.59, -90.671531, 42.51384, '88275a2153fffff'), ('S03403000032', Timestamp('2012-01-03 00:00:00'), 3725, 'Armstrong Liquor Store', '101  6TH ST', 'ARMSTRONG', '50514', 32, 'Emmet', 1062200, 'PUERTO RICO & VIRGIN ISLANDS RUM', 434, 'Luxco-St Louis', 45248, 'Paramount Gold Rum', 6, '1750', 7.58, 11.37, 2, 22.74, 3.5, 0.92, -94.480204, 43.39966, '88262a82cbfffff'), ('S03382700034', Timestamp('2012-01-03 00:00:00'), 2607, 'Hy-Vee Wine and Spirits / Shenandoah', '520 SO FREMONT', 'SHENANDOAH', '51601', 73, 'Page', 1081030, 'COFFEE LIQUEURS', 370, 'Pernod Ricard USA/Austin Nichols', 67540, 'Kahlua French Vanilla Liqueur', 12, '750', 11.61, 17.42, 1, 17.42, 0.75, 0.2, -95.385111, 40.761736, '8826080d95fffff'), ('S03396600047', Timestamp('2012-01-03 00:00:00'), 3456, 'Quick Shop / Clear Lake', '904 N 8TH ST', 'CLEAR LAKE', '50428', 17, 'Cerro Gordo', 1081312, 'BUTTERSCOTCH SCHNAPPS', 65, 'Jim Beam Brands', 82784, 'Dekuyper Buttershots Burst Schnapps', 24, '375', 3.06, 4.59, 2, 9.18, 0.75, 0.2, -93.378772, 43.142868, '8826283b29fffff'), ('S03403600002', Timestamp('2012-01-03 00:00:00'), 3744, 'Payless Foods / Dyersville', '733 16TH AVE SE', 'DYERSVILLE', '52040', 31, 'Dubuque', 1011500, 'STRAIGHT RYE WHISKIES', 255, 'Wilson Daniels Ltd.', 27102, 'Templeton Rye', 6, '750', 18.08, 27.13, 6, 162.78, 4.5, 1.19, -91.115769, 42.470204, '88275a654dfffff'), ('S03408200068', Timestamp('2012-01-03 00:00:00'), 2619, 'Hy-Vee Wine and Spirits / WDM', '1725  74TH ST', 'WEST DES MOINES', '50266', 77, 'Polk', 1081315, 'CINNAMON SCHNAPPS', 421, 'Sazerac Co., Inc.', 82956, 'Firewater Cinnamon Schnapps', 12, '750', 9.34, 14.01, 3, 42.03, 2.25, 0.59, -93.808855, 41.598515, '88260d8ad1fffff'), ('S03392300020', Timestamp('2012-01-03 00:00:00'), 2413, 'Beecher Beverage', '1691 ASBURY ROAD', 'DUBUQUE', '52001', 31, 'Dubuque', 1011300, 'TENNESSEE WHISKIES', 85, 'Brown-Forman Corporation', 26821, 'Jack Daniels Old #7 Black Lbl Mini', 12, '500', 8.62, 12.94, 12, 155.28, 6.0, 1.59, -90.696886, 42.500775000000004, '88275a2115fffff'), ('S03403200046', Timestamp('2012-01-03 00:00:00'), 3842, 'Bancroft Liquor Store', '107 N PORTLAND ST PO BX 222', 'BANCROFT', '50517', 55, 'Kossuth', 1081240, 'GREEN CREME DE MENTHE', 434, 'Luxco-St Louis', 79336, 'Paramount Creme De Menthe/green', 12, '750', 4.44, 6.66, 1, 6.66, 0.75, 0.2, -94.218, 43.29355, '8826284833fffff'), ('S03408600079', Timestamp('2012-01-03 00:00:00'), 2448, 'Bottle Boutique', '705 E. SECOND', 'INDIANOLA', '50125', 91, 'Warren', 1022100, 'TEQUILA', 55, 'Sazerac North America', 89577, 'Montezuma Gold Tequila', 12, '1000', 7.02, 10.53, 2, 21.06, 2.0, 0.53, -93.55261600000001, 41.358176, '88260db8c9fffff'), ('S03393400094', Timestamp('2012-01-03 00:00:00'), 2624, 'Hy-Vee #2 / Dubuque', '2395 NW ARTERIAL RD', 'DUBUQUE', '52002', 31, 'Dubuque', 1051010, 'AMERICAN GRAPE BRANDIES', 85, 'Brown-Forman Corporation', 52806, 'Korbel Brandy', 12, '750', 5.69, 8.54, 12, 102.48, 9.0, 2.38, -90.741064, 42.516633, '88275a212dfffff'), ('S03410300064', Timestamp('2012-01-03 00:00:00'), 3495, 'Great Pastimes', '228 N MAIN ST', 'MONTICELLO', '52310', 53, 'Jones', 1032200, 'IMPORTED VODKA - MISC', 370, 'Pernod Ricard USA/Austin Nichols', 33987, 'Absolut Wild Tea', 12, '750', 10.89, 16.33, 1, 16.33, 0.75, 0.2, -91.183413, 42.24013200000001, '882674d69dfffff'), ('S03404100024', Timestamp('2012-01-03 00:00:00'), 4171, 'Brothers Market, Inc. / Cascade', '105 SE NIXON ST', 'CASCADE', '52033', 31, 'Dubuque', 1032080, 'IMPORTED VODKA', 370, 'Pernod Ricard USA/Austin Nichols', 34007, 'Absolut Swedish Vodka 80 Prf', 12, '1000', 14.89, 22.34, 12, 268.08, 12.0, 3.17, -90.993859, 42.3036, '8826749ac1fffff')]

[('S03394500034',
  Timestamp('2012-01-03 00:00:00'),
  3443,
  'Super Saver Iv',
  '1141 N BROADWAY',
  'COUNCIL BLUFFS',
  '51503',
  78,
  'Pottawattamie',
  1031080,
  'VODKA 80 PROOF',
  55,
  'Sazerac North America',
  37938,
  'Skol Vodka',
  6,
  '1750',
  7.1,
  10.65,
  6,
  63.9,
  10.5,
  2.77,
  -95.836515,
  41.270824,
  '882608c823fffff'),
 ('S03388700010',
  Timestamp('2012-01-03 00:00:00'),
  2367,
  "Spencer's Grocery",
  '101 SOUTH BURNETT',
  'SHENANDOAH',
  '51601',
  73,
  'Page',
  1081365,
  'TROPICAL FRUIT SCHNAPPS',
  421,
  'Sazerac Co., Inc.',
  83906,
  'Maui Blue Hawaiian Schnapps',
  12,
  '750',
  3.67,
  5.5,
  12,
  66.0,
  9.0,
  2.38,
  -95.381935,
  40.765934,
  '8826080d9dfffff'),
 ('S03408600047',
  Timestamp('2012-01-03 00:00:00'),
  2448,
  'Bottle Boutique',
  '705 E. SECOND',
  'INDIANOLA',
  '50125',
  91,
  'Warren',
  1081390,
  'IMPORTED SCHNAPPS',
  260,
  'Diageo Americas',
  65066,
  'Goldschlager Cinnamon Schnapps',
  12,
  '750',
  12

In [4]:
len(lst)

1000

In [7]:
import pandas as pd
df = pd.DataFrame(lst)
df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,S03407900038,2012-01-03,2549,Hy-Vee Food Store / Indianola,910 N JEFFERSON,INDIANOLA,50125,91,Warren,1031080,...,1750,8.49,12.74,12,152.88,21.00,5.55,0.000000,0.000000,0
1,S03411000067,2012-01-03,2625,Hy-Vee Wine and Spirits #2,3301 W KIMBERLY RD,DAVENPORT,52804,82,Scott,1011200,...,750,9.97,14.95,12,179.40,9.00,2.38,0.000000,0.000000,0
2,S03411000030,2012-01-03,2625,Hy-Vee Wine and Spirits #2,3301 W KIMBERLY RD,DAVENPORT,52804,82,Scott,1062310,...,1000,10.99,16.49,12,197.88,12.00,3.17,0.000000,0.000000,0
3,S03410300045,2012-01-03,3495,Great Pastimes,228 N MAIN ST,MONTICELLO,52310,53,Jones,1032080,...,1000,21.77,32.66,2,65.32,2.00,0.53,-91.183413,42.240132,882674d69dfffff
4,S03400700021,2012-01-03,2556,Hy-Vee Wine and Spirits / Esthervill,2608 E. CENTRAL AVENUE,ESTHERVILLE,51334,32,Emmet,1081400,...,1000,3.75,5.62,12,67.44,12.00,3.17,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,S03404800133,2012-01-03,2573,Hy-Vee Food Store / Muscatine,2400 2ND AVE,MUSCATINE,52761,70,Muscatine,1032200,...,1000,16.18,24.27,6,145.62,6.00,1.59,-91.035138,41.451349,88267421abfffff
996,S03412100012,2012-01-03,2603,Hy-Vee Wine and Spirits / Bettendorf,2890 DEVILS GLEN ROAD,BETTENDORF,52722,82,Scott,1031200,...,750,7.59,11.56,12,138.72,9.00,2.38,-90.483701,41.552338,882675da95fffff
997,S03390000016,2012-01-03,3606,Scotty's Liquor Store,804 G AVE,GRUNDY CENTER,50638,38,Grundy,1081380,...,1000,7.35,11.02,2,22.04,2.00,0.53,-92.774215,42.361514,8826297831fffff
998,S03395200040,2012-01-03,2507,Hy-Vee Food Store #1 / Burlington,939 ANGULAR,BURLINGTON,52601,29,Des Moines,1032080,...,750,17.97,26.96,4,107.84,3.00,0.79,-91.110752,40.801889,882670a137fffff


In [8]:
df.dtypes

0             object
1     datetime64[ns]
2              int64
3             object
4             object
5             object
6             object
7              int64
8             object
9              int64
10            object
11             int64
12            object
13             int64
14            object
15             int64
16            object
17           float64
18           float64
19             int64
20           float64
21           float64
22           float64
23           float64
24           float64
25            object
dtype: object